In [ ]:
from xml.etree import ElementTree as ET
import pandas as pd
import xml.dom.minidom
import codecs
import numpy as np
%matplotlib inline
import subprocess
import datetime
import time
from sklearn.cross_validation import train_test_split
from IPython.lib import backgroundjobs as bg

In [ ]:
def xmltodf(xmldir):
    tree = ET.parse(xmldir)
    root = tree.getroot()
    li = []
    for d in root.iterfind("document"):
        docid = int(d.find("id").text)
        nannotators = int(d.find("infon[@key='n_annotators']").text)
        annotatorids = d.find("infon[@key='annotator_ids']").text
        for p in d.iterfind("passage"):
            doctype = p.find("infon[@key='type']").text
            docoffset = int(p.find("offset").text)
            doctext = p.find("text").text
            for a in p.iterfind("annotation"):
                annotationid = int(a.items()[0][1])
                annotatorid = a.find("infon[@key='annotator_id']").text
                annotationoffset = int(dict(a.find("location").items())['offset'])
                annotationlen = int(dict(a.find("location").items())['length'])
                annotationtext = a.find("text").text
                li.append({'doc_id':docid,'annotator_ids':annotatorids,'n_annotator':nannotators,'doc_type':doctype,'doc_offset':docoffset,'doc_text':doctext,'annotation_id':annotationid,'annotator_id':annotatorid,'annotation_offset':annotationoffset,'annotation_len':annotationlen,'annotation_text':annotationtext})
    return pd.DataFrame(li)

def appendelem(collection,parent,tag,text,attr1,attrvalue1,attr2=None,attrvalue2=None):
    elem = collection.createElement(tag)
    if text != None:
        elem.appendChild(collection.createTextNode(text))
    if attr1 != None:
        elem_attr1 = collection.createAttribute(attr1)
        elem_attr1.value = attrvalue1
        elem.setAttributeNode(elem_attr1)
    if attr2 != None:
        elem_attr2 = collection.createAttribute(attr2)
        elem_attr2.value = attrvalue2
        elem.setAttributeNode(elem_attr2)
    parent.appendChild(elem)

def dftoxml(df,output='sample.xml'):
    impl = xml.dom.minidom.getDOMImplementation()
    collection = impl.createDocument(None, 'collection', None)
    top_element = collection.documentElement
    groups1 = df.groupby(['doc_id'])
    for did,grp1 in groups1:
        doc = collection.createElement('document')
        appendelem(collection,doc,'id',str(did),None,None)
        appendelem(collection,doc,'infon',str(np.array(grp1['n_annotator'])[0]),'key','n_annotators')
        appendelem(collection,doc,'infon',np.array(grp1['annotator_ids'])[0],'key','annotator_ids')
        groups2 = grp1.groupby(['doc_type'])
        for dtype,grp2 in groups2:
            pas = collection.createElement('passage')
            appendelem(collection,pas,'infon',str(dtype),'key','type')
            appendelem(collection,pas,'offset',str(np.array(grp2['doc_offset'])[0]),None,None)
            appendelem(collection,pas,'text',np.array(grp2['doc_text'])[0],None,None)
            groups3 = grp2.groupby(['annotation_id'])
            for aid,grp3 in groups3:
                annotation = collection.createElement('annotation')
                annotation_attr = collection.createAttribute('id')
                annotation_attr.value = str(aid)
                annotation.setAttributeNode(annotation_attr)
                appendelem(collection,annotation,'infon',str(np.array(grp3['annotator_id'])[0]),'key','annotator_id')
                appendelem(collection,annotation,'infon','Disease','key','type')
                appendelem(collection,annotation,'location',None,'offset',str(np.array(grp3['annotation_offset'])[0]),'length',str(np.array(grp3['annotation_len'])[0]))
                appendelem(collection,annotation,'text',np.array(grp3['annotation_text'])[0],None,None)
                pas.appendChild(annotation)
            doc.appendChild(pas)
        top_element.appendChild(doc)
    f = codecs.open(output, 'w', 'utf-8')
    collection.writexml(writer=f, encoding='UTF-8', newl='\n', addindent='\t')
    f.close()
    
def dfappendvoterate(df):
    dfvote = df.groupby(['annotation_text','annotation_offset','doc_id','annotation_len']).apply(lambda x:len(x)).reset_index().sort(0)
    dfvote.columns = ['annotation_text', 'annotation_offset', 'doc_id', 'annotation_len', 'annotation_vote']
    df = pd.merge(df,dfvote)
    df['annotation_rate'] = df['annotation_vote']/df['n_annotator']
    return df

def runBANNER(df):
    d = datetime.datetime.today()
    now = d.strftime("%Y-%m-%d-%H-%M-%S")
    
    dftoxml(df,'banner_source/data/'+now+'.xml')

    tree = ET.parse('banner_source/config/banner_bioc.xml')
    root = tree.getroot()

    filename = root.find('.//filename')
    idInputFilename = root.find('.//idInputFilename')
    rawInputFilename = root.find('.//rawInputFilename')
    trainingInputFilename = root.find('.//trainingInputFilename')
    outputFilename = root.find('.//outputFilename')
    inContextAnalysisFilename = root.find('.//inContextAnalysisFilename')
    mentionFilename = root.find('.//mentionFilename')
    modelFilename = root.find('.//modelFilename')

    filename.text = 'data/'+now+'.xml'
    idInputFilename.text = idInputFilename.text.replace('output','output/'+now)
    rawInputFilename.text = rawInputFilename.text.replace('output','output/'+now)
    trainingInputFilename.text = trainingInputFilename.text.replace('output','output/'+now)
    outputFilename.text = outputFilename.text.replace('output','output/'+now)
    inContextAnalysisFilename.text = inContextAnalysisFilename.text.replace('output','output/'+now)
    mentionFilename.text = mentionFilename.text.replace('output','output/'+now)
    modelFilename.text = modelFilename.text.replace('output','output/'+now)

    tree.write('banner_source/config/'+now+'.xml')
    
    p1 = subprocess.Popen(['mkdir',now],cwd='banner_source/output')
    p1.wait()
    p2 = subprocess.Popen(['java','-Xmx1024m', '-cp','lib/*','banner.eval.BANNER','train','config/'+now+'.xml'],cwd='banner_source',stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    p2.wait()
    p3 = subprocess.Popen(['java','-cp','lib/*','BANNER_BioC','config/'+now+'.xml','data/test_file.xml','output/'+now+'/out.xml'],cwd='banner_source', stderr=subprocess.PIPE)
    p3.wait()
    p4 = subprocess.Popen(['cp','BannerAnnotate.java','output/'+now+'/BannerAnnotate.java'],cwd='banner_source',stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    p4.wait()
    p5 = subprocess.Popen(['cp','banner_source/BannerAnnotate.java','mm_tester/BannerAnnotate.java'],stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    p5.wait()
    p6 = subprocess.Popen(['javac','BannerAnnotatorVis.java'],cwd='mm_tester',stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    p6.wait()
    p = subprocess.Popen(['java','BannerAnnotatorVis'],cwd='mm_tester',stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(p.communicate())
    return p

In [ ]:
dfex = xmltodf('expert1_bioc.xml')

dfmt1 = xmltodf('mturk1_bioc.xml')
dfmt1 = dfmt1[dfmt1.doc_id.isin(dfex.doc_id)]
dfmt1 = dfappendvoterate(dfmt1)

dfex['annotation_correct'] = 1
dfmt1 = pd.merge(dfex[['annotation_text', 'annotation_offset', 'doc_id', 'annotation_len','annotation_correct']],dfmt1,how='right')
dfmt1 = dfmt1.fillna(0)
dfmt1 = dfmt1.drop_duplicates(['annotation_text','annotation_offset','doc_id','annotation_len'])

dfmt2 = xmltodf('mturk2_bioc.xml')
dfmt2 = dfmt2[~dfmt2.doc_id.isin(dfex.doc_id)]
dfmt2 = dfappendvoterate(dfmt2)
dfmt2 = dfmt2.drop_duplicates(['annotation_text','annotation_offset','doc_id','annotation_len'])

dfmt2 = dfmt2[~dfmt2.annotation_id.isin([389136,296499,298346,305348,312170,388906,334987,369579,375145,
                                         379106,289785,296796,326778,354107,289786,296797,326780,354109,388908,414501,356269,
                                         289364,372648,306575,332871,289371,289373,306580,332878,292506,317072])]

In [ ]:
for x in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    print(x,dfmt1[(x-0.1 < dfmt1['annotation_rate']) &  (dfmt1['annotation_rate'] <= x)]['annotation_correct'].sum()/len(dfmt1[(x-0.1 < dfmt1['annotation_rate']) &  (dfmt1['annotation_rate'] <= x)]))
for x in range(1,150):
    df = dfmt1[(x-1 < dfmt1['annotation_vote']) &  (dfmt1['annotation_vote'] <= x)]
    if(len(df) > 0):
        print(x,len(df),df['annotation_correct'].sum()/len(df))

In [ ]:
train = pd.DataFrame(train_test_split(dfex,test_size=0.95)[1])
train.columns = ['annotation_id', 'annotation_len', 'annotation_offset', 'annotation_text', 'annotator_id', 'annotator_ids', 'doc_id', 'doc_offset', 'doc_text', 'doc_type', 'n_annotator', 'annotation_correct']

In [ ]:
jobs = bg.BackgroundJobManager()
jobs.new('runBANNER(dfex)')
jobs.new('runBANNER(train),runBANNER(dfex.ix[:4928,:])')

In [ ]:
train1 = pd.DataFrame(train_test_split(dfex.ix[4929:,:],test_size=0.95)[1])
train1.columns = ['annotation_id', 'annotation_len', 'annotation_offset', 'annotation_text', 'annotator_id', 'annotator_ids', 'doc_id', 'doc_offset', 'doc_text', 'doc_type', 'n_annotator', 'annotation_correct']

In [ ]:
jobs.new('runBANNER(dfex.ix[:4928,:].append(train1))')

In [ ]:
dfmt1[(dfmt1.annotation_vote >= 10) & (dfmt1.annotation_rate >= 0.9)]['annotation_correct'].mean()

In [ ]:
a = dfmt1[(dfmt1.annotation_vote >= 10) & (dfmt1.annotation_rate >= 0.9) & (dfmt1.annotation_correct == 0)].annotation_text.values

In [ ]:
train1

In [ ]:
dfmt2[dfmt2.annotation_text.isin(a)]

In [ ]:
jobs.new('runBANNER(dfex.append(dfmt1[(dfmt1.annotation_correct == 0) & (dfmt1.annotation_text.isin(dfex.annotation_text))]))')

In [ ]:
jobs.new('runBANNER(dfex)')

In [ ]:
bad_annotations = dfmt1[(dfmt1.annotation_correct == 0) & (~dfmt1.annotation_text.isin(dfex.annotation_text))]

In [ ]:
runBANNER(dfex.append(dfmt2[(~dfmt2.annotation_text.isin(bad_annotations.annotation_text)) & (dfmt2.annotation_vote >= 10) & (dfmt2.annotation_rate >= 0.9)]))

In [ ]:
dfex['annotation_end'] = dfex['annotation_offset'] + dfex['annotation_text'].map(len) -1

In [ ]:
dfmt1[(dfmt1.annotation_correct == 0) & (dfmt1.annotation_text.isin(dfex.annotation_text))]

In [ ]:
dfmt1[dfmt1.n_annotator == 15].sort(['doc_id','doc_type','annotation_offset'])

In [ ]:
dfmt2

In [ ]:
dfmt2

In [ ]:
trainlist = []
for x in range(9):
    train = pd.DataFrame(train_test_split(dfex,test_size=0.95)[1])
    train.columns = ['annotation_id', 'annotation_len', 'annotation_offset', 'annotation_text', 'annotator_id', 'annotator_ids', 'doc_id', 'doc_offset', 'doc_text', 'doc_type', 'n_annotator', 'annotation_correct']
    trainlist.append(train)

In [ ]:
jobs.new('runBANNER(trainlist[0]),runBANNER(trainlist[1]),runBANNER(trainlist[2]),runBANNER(trainlist[3]),runBANNER(trainlist[4]),runBANNER(trainlist[5]),runBANNER(trainlist[6]),runBANNER(trainlist[7]),runBANNER(trainlist[8])')

In [ ]:
len(dfex)

In [ ]:
dfex.ix[train_test_split(range(5149),test_size=0.95)[1]]

In [ ]:
dfex[dfex.doc_id.isin(train_test_split(dfex.doc_id.drop_duplicates().values,test_size=0.95)[1])]

In [ ]:
def xmlouttodf(xmldir):
    tree = ET.parse(xmldir)
    root = tree.getroot()
    li = []
    for d in root.iterfind("document"):
        docid = int(d.find("id").text)
        for p in d.iterfind("passage"):
            doctype = p.find("infon[@key='type']").text
            docoffset = int(p.find("offset").text)
            for a in p.iterfind("annotation"):
                annotationoffset = int(dict(a.find("location").items())['offset'])
                annotationlen = int(dict(a.find("location").items())['length'])
                annotationtext = a.find("text").text
                li.append({'doc_id':docid,'doc_type':doctype,'doc_offset':docoffset,'annotation_offset':annotationoffset,'annotation_len':annotationlen,'annotation_text':annotationtext})
    return pd.DataFrame(li)
dfout = xmlouttodf('banner_source/output/3/out.xml')

In [ ]:
f = open("output_testtesttesttest.py","w")
for x in np.array(dfout[['doc_id','annotation_offset','annotation_len']]):
    f.write(str(x[0])+','+str(x[1])+','+str(x[2])+',\\\n')
    #f.write(str(x[0])+','+str(x[1])+','+str(x[2])+',')

In [ ]:
dfout.doc_type.value_counts()

In [ ]:
dfout.dropna()

In [ ]:
np.array(dfout[dfout.doc_type == 'abstract'].sort('doc_id')[['doc_id','annotation_offset','annotation_len']])

In [ ]:
test.testtest()

In [ ]:
class BannerAnnotate:
    def annotate():
        return [0,54,380,159,181,976,101,1232,361,1359,183,953,143,913,63,510,93,470,163,1554,33,343,33,91,33,68,213,392,136,641,206,612,236,585,256,568,156,522,156,152,186,56,166,539,279,1160,79,1614,139,1593,59,1357,59,920,59,564,59,222,329,156,69,104,69,74,610,167,2610,278,1710,355,1711,170,311,215,4011,870,313,1430,313,400,313,348,313,213,313,181,3013,102,1713,137,1813,91,313,68,2113,157,1915,1819,1215,2711,1515,2108,1515,2083,615,1770,1215,370,1515,1518,515,708,1415,1737,515,1021,1518,665,1218,1526,1518,1498,1818,1242,1518,1134,318,995,318,944,418,844,818,383,1218,374,318,345,2718,306,3018,276,918,145,2718,72,918,584,819,1293,1519,1547,1419,525,1419,162,5119,1017,1320,1377,4321,1154,1221,398,1521,380,1621,369,921,418,3721,200,321,166,1321,90,3921,310,1322,180,1422,262,3122,1424,3123,466,1623,717,1623,640,323,547,1923,507,1723,444,323,763,1923,331,1823,300,2623,244,1623,229,323,197,3023,100,5623,414,325,1708,325,1691,1225,1290,1225,1222,325,1169,1225,985,1825,971,925,358,2625,432,325,1765,1125,301,2725,250,1925,149,325,119,2825,580,325,1819,326,412,2326,233,1828,61,1528,78,228,269,1928,298,2428,324,228,491,1528,759,1528,840,1528,981,229,184,1429,2174,1430,1315,1931,155,2731,184,331,190,331,349,2831,963,331,1088,332,1238,1532,1552,3232,990,332,1769,332,1587,1832,893,332,1887,332,326,332,304,1832,264,332,240,2132,167,332,137,2732,553,334,231,2734,268,2034,666,335,1864,635,806,635,1894,735,142,735,719,638,158,2938,189,338,226,338,261,338,396,338,405,2038,499,2138,927,341,1389,3241,1369,1441,1350,1741,1325,2341,1296,1741,1247,4041,1289,441,781,1641,747,2941,126,2641,103,1841,68,341,48,1841,1051,2242,1419,1642,1274,1142,1221,2042,1166,642,1062,1142,1300,2042,547,942,307,1642,245,1842,190,2342,847,643,1410,1743,1349,2243,1159,1043,1068,2243,681,2243,497,1443,457,1943,361,943,339,1743,215,2943,625,1844,724,2444,1139,3144,979,3244,676,1044,263,244,358,2144,243,1844,150,2346,217,1046,313,2746,1567,1847,1252,5447,992,1847,508,1647,349,2847,1042,2547,197,1347,177,1847,157,1447,139,1647,329,1348,2247,648,1472,648,1573,648,537,2048,721,549,239,1649,260,950,649,2150,1034,951,476,351,997,351,614,1451,369,351,181,351,211,3051,153,2651,243,352,257,2252,297,2052,378,1452,413,2252,825,1452,935,1452,1914,1453,979,353,728,4153,521,1153,324,654,1324,254,105,2154,128,254,521,254,847,2554,1026,654,1184,654,1299,656,536,2756,104,1856,127,2758,851,2058,1205,2758,1154,2058,1010,4258,940,2058,817,3058,791,1458,700,2758,665,2458,634,2058,571,2858,376,6058,339,1058,245,2058,215,1858,1313,2060,1466,360,1430,1560,1709,360,1380,360,1594,1960,1194,360,151,2060,450,360,257,4260,141,360,104,3560,877,361,261,2961,293,5061,467,5061,1026,2162,1463,1862,2758,2762,2567,1362,2208,1862,2134,1862,988,1462,2930,1862,743,1762,645,1862,599,1862,333,1762,274,5362,131,1862,835,1863,1283,1363,320,2263,298,1763,275,2163,217,2863,160,463,117,4163,181,1564,126,1564,299,364,1193,365,799,2665,519,365,292,365,849,365,170,365,140,2865,180,1466,99,3166,132,366,162,366,172,2866,274,367,158,5867,218,267,376,367,384,267,443,1668,671,1068,861,3168,467,768,148,4068,120,1568,90,468,57,3169,124,4469,189,2270,683,1271,1623,1371,1567,371,1347,2171,1297,371,1117,371,829,371,755,371,687,371,528,371,379,371,347,371,312,2971,191,371,174,1572,1456,672,1366,672,1264,1272,1168,672,1127,672,1017,1472,187,1072,842,572,655,1472,330,572,243,672,945,673,1644,373,1193,373,988,373,438,373,374,373,581,373,160,2373,103,3773,94,373,79,1373,220,375,95,3175,446,3975,493,2175,525,1577,631,277,1440,277,1375,2077,1037,277,559,277,52,2777,277,577,239,3677,193,277,171,2077,303,278,232,2078,254,278,263,2678,407,278,1138,1378,1607,280,1459,280,1149,280,1161,280,1258,280,1523,480,1042,480,1624,280,1792,280,1901,280,2038,2480,1582,280,998,2780,882,280,905,380,86,1880,106,280,179,3280,578,2480,123,2180,747,2180,770,380,844,480,809,3380,688,281,989,2381,1322,2381,1275,1581,1155,2481,1052,1481,858,2381,408,1481,633,1481,578,2381,329,2381,262,2381,121,2381,749,1483,646,2383,894,2283,876,1683,691,1183,496,383,90,1683,365,683,326,3783,163,3983,414,1684,110,2684,172,3084,328,2084,369,2784,519,2086,1213,1786,1092,4286,459,1886,131,5486,188,586,202,1787,935,2387,1165,1487,1077,1687,500,987,542,1687,464,1287,147,6687,367,1687,479,1487,434,2788,905,588,193,588,228,1388,348,1788,418,588,491,1788,690,1788,758,588,803,1788,926,1788,997,688,1078,1788,1140,1788,1348,588,1465,1389,1044,389,973,389,413,3389,659,389,284,389,261,2189,337,2890,857,1090,230,1490,246,290,401,2390,483,3990,768,290,888,2191,781,2491,1090,391,1622,391,1502,391,745,391,1141,391,677,391,248,1691,310,391,277,1791,266,691,206,391,180,2491,338,2592,1455,2192,320,292,641,1692,1515,292,475,2192,153,292,249,2292,133,1892,386,293,86,593,162,2393,420,1393,445,2393,573,593,660,593,1860,594,292,1694,400,1394,384,1494,376,694,361,1394,281,994,324,1494,310,1294,422,1894,340,1994,446,2295,995,395,1244,2295,703,395,141,595,99,3195,342,396,68,1696,97,1696,320,1396,673,1698,1183,1398,1248,3798,1206,798,733,598,220,3798,183,698,159,1399,250,2899,280,399,288,1899,510,399,1356,399,1876,15100,1438,6100,1677,18100,467,27100,1884,17100,1035,6100,1744,5100,915,6100,414,17100,683,18100,245,18100,378,5100,718,6100,496,3101,88,29101,119,5101,143,26101,394,5101,598,31101,788,5101,847,5101,867,39101,1301,5101,1721,12101,1853,5101,1869,36101,587,5103,620,35103,1244,8103,147,4103,126,19103,194,46104,144,31104,237,24104,530,29105,422,10105,208,13105,850,10105,143,10105,159,15106,1908,38106,285,23106,356,23106,408,14106,1061,5106,1084,5106,1115,5106,1180,16106,1207,5106,2238,14107,1792,5107,2040,22107,2394,6107,1582,6107,2142,5107,1332,5107,2078,39107,1009,6107,897,21107,847,6107,1239,19107,604,5107,240,6109,1392,3109,1297,3109,1229,14109,984,41109,921,12109,808,22109,751,3109,735,12109,392,10109,417,18109,368,3109,341,24109,230,18109,217,8109,208,3109,181,24109,679,3110,701,2110,819,2110,414,23110,126,2110,297,24110,98,26110,399,13112,1540,24112,1414,3112,1083,3112,1075,3112,441,21112,745,3112,392,3112,364,26112,337,10112,311,6112,288,17112,402,12113,172,19113,795,17114,623,19116,558,37119,790,33119,580,16119,2413,34120,177,15120,146,21121,234,9121,267,32122,506,45122,553,3123,137,10123,187,7123,202,42123,772,18124,1119,12124,1519,22124,1511,3124,1278,17124,1221,9124,1204,3124,775,62124,1160,9124,1133,3124,1702,20124,345,28124,1190,12124,1990,12126,423,19126,329,3126,171,3126,142,27128,1886,3128,1856,25128,1698,3128,1394,3128,1266,3128,1132,3128,982,22128,960,3128,769,3128,748,16128,688,3128,566,3128,475,43128,314,25128,276,3128,265,3128,231,31128,182,3128,140,23129,1287,14129,2115,17129,1200,14129,1975,14129,1837,14129,1032,14129,1733,21129,760,5129,632,15129,517,14129,424,17129,249,7129,168,51129,928,3130,231,2130,165,47130,1262,2130,121,26130,149,2131,96,32131,144,37131,587,52131,641,4131,646,3131,895,16132,101,14132,1007,8134,1608,22134,1406,23134,1297,23134,335,14134,374,23134,181,18134,143,19134,109,13134,78,16134,229,14135,68,28135,98,3135,123,48135,344,3135,439,10136,1566,22136,341,18136,588,26136,188,12136,270,32137,99,24138,126,16138,143,21138,167,3138,597,3138,988,3139,82,36139,565,36139,811,36139,852,24140,181,16141,1657,33141,930,19141,1448,3141,305,19141,757,52142,234,3142,2260,6142,2052,17142,1845,6142,1677,21142,1125,6142,2383,6142,736,6142,937,25142,301,6142,2302,23142,603,6142,525,6143,346,24143,1774,15143,1849,15144,110,25146,984,12146,247,29146,281,22146,494,9146,540,14146,568,5146,665,35146,746,14147,304,40147,101,34147,202,24149,119,5149,206,20149,492,8149,504,24149,780,44149,1115,6149,1143,15149,1275,14149,1435,27150,376,48150,1667,34150,1437,26150,72,15150,289,15151,105,18151,261,28153,332,6155,987,6155,1147,34155,538,6155,1531,34155,444,8155,331,6155,454,27157,1429,8157,863,19157,1672,21157,1566,8157,807,3157,358,3157,640,14157,715,23157,278,3157,199,21157,112,3158,201,2158,181,18158,958,8159,774,2159,749,3159,508,3159,397,3159,353,6159,257,15159,211,28159,135,27159,109,15160,171,24160,197,3160,215,33160,276,24160,368,3160,1078,24160,1164,8161,1332,15161,806,17161,430,14163,195,26163,420,15165,1340,18166,188,3166,208,3166,263,8166,177,9166,654,3166,1217,3167,1205,21168,1861,17168,1730,19168,1659,17168,638,25168,238,25168,380,26168,340,17168,143,25168,554,6170,440,18170,143,40170,331,3170,381,20170,665,21172,734,39172,775,3173,144,17173,231,16174,156,15174,257,15174,2045,26176,175,16176,1258,34177,1108,18177,1314,2177,468,2177,453,13177,348,18177,254,13177,161,20178,151,28178,180,21178,204,3178,288,3178,1045,3178,1104,3178,1559,3179,818,21179,848,17179,614,27179,1138,27179,989,6179,782,9179,261,23179,435,15179,324,5179,667,34179,239,6180,1034,3180,1001,3180,1412,3180,186,26180,227,3181,80,42183,553,12184,158,17184,1212,20185,440,25186,1094,3186,996,18186,1053,6186,1272,3186,951,3186,1320,3186,1325,3186,1432,28186,1681,3186,1288,17186,892,17186,767,3186,823,3186,730,7186,694,17186,635,3186,505,10186,416,3186,384,30186,323,5186,159,18186,116,3187,1213,3187,786,3187,756,3187,563,18187,473,3187,134,27187,189,20187,123,3187,93,28187,372,3188,930,2188,126,18188,146,2188,787,2189,1200,6189,680,13189,239,5190,127,40190,196,14190,212,2190,422,2190,429,14190,478,2190,485,14190,1575,35190,1678,2191,159,5191,223,43192,777,6192,534,3192,370,39192,286,3192,103,14192,142,23192,119,2192,258,26193,800,41193,1334,20193,1150,12193,1073,26193,861,12193,526,9193,189,24193,348,20193,247,26193,614,24193,154,3193,126,26194,1938,13194,1797,14194,1664,13194,1009,20194,847,20194,427,24194,333,13194,300,13194,179,23194,703,21195,1194,25195,1237,18195,920,20195,1349,20195,406,13195,137,13195,649,6196,516,36197,1259,13197,1410,28198,1766,4198,420,13198,220,4198,1703,4198,308,13198,244,26198,175,43198,333,21200,269,6200,834,6201,67,3201,72,21201,124,21201,742,28202,111,16203,798,3203,547,5203,670,35203,1097,18203,485,12203,1163,3203,1336,17203,1428,7203,1145,3203,442,3203,263,3203,370,12203,319,10203,295,19203,413,5203,245,3203,208,35203,171,12203,149,18203,92,3203,64,26204,768,32204,694,21204,643,31204,437,17204,343,19204,290,32204,134,32204,95,24204,388,24205,693,33205,729,4205,879,4206,969,9206,1595,48206,1399,9206,1095,12206,1067,9206,957,3206,565,24206,754,14206,450,28206,100,3206,80,18206,932,23207,771,6207,890,6207,964,6207,1067,18207,1090,19209,1934,33209,1582,24209,1503,15209,1374,15209,102,15209,604,15209,250,18209,145,43209,714,15210,898,3210,1101,19210,409,20210,379,21210,191,21210,402,2211,1481,10211,295,28211,325,3211,893,3211,1062,3211,1308,19211,1395,17211,1598,22212,929,5212,869,5212,199,13212,503,14213,239,12214,267,27214,296,37214,191,36214,338,18214,147,4214,119,26214,171,12215,1556,13215,1520,28215,1429,13215,900,24215,184,13215,326,28215,123,13215,394,13216,173,7216,286,25216,339,7216,658,7216,1043,2216,1309,17217,112,6217,151,26217,1450,15217,1581,24217,1698,15218,803,3218,513,11218,406,11218,757,3218,530,3218,402,2218,390,10218,290,5218,244,3218,179,17218,146,3218,116,28218,372,17219,1285,26219,727,3219,597,3219,698,27220,91,14220,107,3220,116,28220,220,36221,1300,13221,1710,19221,1393,13221,1216,13221,1680,12221,335,10221,245,28221,136,91221,347,23222,203,20222,225,2222,570,2223,380,16224,1195,5224,572,3224,357,3224,455,3224,230,3224,212,16224,236,12225,2035,5226,1195,13226,1778,14226,1924,25226,1625,13226,1537,25226,1456,13226,1364,25226,1281,14226,1092,14226,1687,24226,1011,18226,789,14226,681,13226,620,24226,509,25226,439,6226,265,13226,99,26226,1049,20227,188,37227,229,36227,267,4227,281,4228,319,53228,278,18228,177,17228,118,20228,140,2229,162,15229,213,19229,383,20230,142,17230,161,3230,183,18230,216,27230,357,3230,886,3230,1000,3231,1317,19231,1196,18231,97,18231,142,28233,169,18233,1315,18233,1573,9235,174,20235,257,33235,300,20236,600,5236,1701,5236,1681,2236,1611,2236,1470,22236,1362,5236,306,2236,311,2236,286,18236,238,18236,148,17236,338,19237,955,3237,741,17237,1677,17237,722,3237,1636,3237,1661,3237,1237,3237,683,3237,1085,17237,625,3237,484,3237,413,3237,384,26237,333,3237,300,30237,227,17237,650,17238,259,20238,878,19238,66,20238,111,27239,175,14239,191,3239,974,31239,1582,3239,1703,3240,1149,3240,1582,3240,1461,3240,1404,3240,1221,3240,1176,12240,952,3240,1083,3240,461,21240,349,9240,240,4240,224,14240,189,3240,156,31240,621,3241,138,25241,165,6241,561,3241,689,3242,241,23242,1209,14242,696,14242,196,19242,112,51243,154,21243,177,3243,186,17243,205,2243,280,3243,354,2243,441,20243,466,18243,838,3244,1663,4244,519,4244,278,4244,242,34245,152,39245,193,6245,363,4245,573,26245,616,8245,1306,15245,1604,8245,1707,8248,1185,3248,851,3248,573,3248,170,3248,147,21248,130,16248,442,3249,188,41249,1213,16250,964,31250,1000,23250,1276,19251,536,16252,1059,18252,424,32252,123,39252,175,19253,1535,6253,202,13253,235,14253,613,6253,722,7253,742,13253,852,36253,999,14253,1302,25253,1354,13255,1261,18255,764,2255,447,2255,128,18255,149,2255,270,2256,581,7256,134,13256,385,5256,502,3257,844,5258,738,2258,856,32258,332,2258,793,2258,145,27258,135,2258,114,19258,278,26259,853,3259,776,3259,652,6259,631,3259,700,3259,361,4259,331,28259,248,3259,126,38259,395,3260,209,8260,260,39260,470,12260,1115,12260,1231,12262,947,20262,2015,5262,1360,5262,772,21262,715,18262,620,30262,580,28262,258,5262,214,42262,694,5263,1402,11263,1589,11263,1944,11264,561,2264,1615,2264,1478,3264,1324,3264,591,3264,387,2264,307,25264,271,3264,215,53265,354,3265,381,24265,490,31265,523,3266,117,35266,733,6267,891,3267,1191,6267,336,3267,1226,3267,286,26267,222,3267,185,35267,314,3268,259,31268,292,3268,314,24268,340,3268,404,8269,1406,3269,1384,18269,752,3269,494,44269,255,42269,145,3269,108,35270,113,25270,141,2270,362,24270,458,19270,1461,3270,2019,16271,1499,15271,606,13271,556,43271,641,19271,210,27271,168,13271,88,64271,264,20272,1621,3272,1570,3272,1404,3272,1373,3272,1266,3272,663,3272,452,14272,206,3272,182,22272,870,3273,607,4273,957,2273,1110,3273,824,4273,577,28273,248,2273,401,3273,375,24273,219,27273,470,2274,116,32274,150,3274,825,32274,973,3275,1296,2275,1240,52275,1164,43275,1134,15275,613,9275,795,48275,94,18275,845,2275,128,27276,198,20276,283,7276,417,18276,554,7276,729,10276,965,6276,1497,21276,1609,3277,477,3277,1231,3277,1129,51277,111,28277,141,4278,199,21278,234,27280,353,16280,381,14280,400,25280,1159,3281,970,21281,833,24282,743,11282,286,26282,726,15283,1400,17283,1577,19283,1486,15283,1155,26283,661,18283,636,10283,1323,17283,479,26283,343,30283,254,29283,175,19283,142,25283,569,14284,445,13284,611,4284,812,4284,2104,4284,2412,4284,2612,5285,221,23285,1861,16286,587,15286,413,20286,578,2286,134,20286,156,2287,421,33287,579,39287,687,15287,1392,15288,170,15288,255,18289,1392,39289,1324,20289,1288,3289,1135,40289,1090,3289,1017,6289,936,15289,696,3289,533,15289,516,13289,433,3289,265,3289,237,26289,732,40289,566,26290,1748,13290,1458,13290,1599,13290,1480,22290,1070,7290,974,23290,324,32290,218,21290,1201,13291,202,12291,345,37291,1050,25292,91,15292,108,3292,117,28292,158,47292,507,15294,889,19294,842,4294,603,19294,453,4294,86,19294,279,19294,119,46294,107,4294,394,19295,2185,3295,1735,3295,1425,12295,1180,3295,995,3295,504,3295,123,37295,162,3295,876,3295,202,16296,112,39296,153,4297,997,15297,973,22297,817,14297,749,16297,526,27297,919,14297,320,14297,289,22297,189,16297,39,38297,437,15298,163,25298,241,5298,1422,5299,62,3299,71,19299,184,11299,233,14299,299,3299,471,11301,808,6301,92,36301,726,13302,200,57302,266,25302,333,12302,350,12302,527,16302,574,49302,853,12303,141,3303,975,17303,883,29303,788,18303,107,32303,257,13303,272,2303,594,8304,1182,6304,764,6304,1266,17304,303,10304,164,18304,624,18305,978,6305,172,36305,319,21307,885,12307,962,19307,774,12307,649,9307,559,19307,611,19307,377,9307,226,23307,178,22307,104,10307,80,19307,509,10308,318,3308,643,21308,612,3308,179,9308,30,28308,86,31308,60,3308,814,11308,169,5308,1052,17309,286,38309,1344,18309,1400,38309,932,56309,202,3309,175,25309,505,3310,1690,2310,1606,5310,1423,2310,1394,27310,1144,6310,1056,6310,984,3310,842,23310,818,3310,812,3310,780,30310,722,3310,302,3310,277,23310,1763,5310,827,3311,1210,3311,1095,3311,1348,3311,1069,3311,1606,14311,880,3311,156,16311,130,14311,862,16312,171,26312,199,3312,570,15312,767,3312,805,15312,1636,27313,925,25313,810,6313,135,25313,239,36314,257,25314,419,16314,1583,24314,1652,45315,1652,2315,1982,2315,1971,6315,1282,2315,1154,12315,397,2315,405,25315,370,25315,1168,2316,239,31316,1052,10316,1735,21317,955,18317,1048,18317,984,29317,928,14317,657,23317,453,27317,207,23317,685,30318,111,52318,165,6318,739,52318,793,7319,739,2319,353,5319,716,21319,776,2320,1347,24320,1143,24320,1743,12320,211,22320,323,31321,158,26321,191,46321,253,35321,305,13321,1121,16321,1327,25321,1529,51322,1807,5322,1501,13322,204,16322,893,9322,1288,15323,771,19323,1266,22323,1219,20323,1176,17323,752,15323,138,27323,287,14323,263,18323,213,42323,98,33323,382,3324,816,22324,1527,13324,1679,23324,1483,6324,1146,13324,976,13324,649,21324,153,13324,493,7324,371,13324,317,23324,272,13324,69,23324,533,23326,906,12326,398,24326,311,13326,156,23326,84,24326,208,13327,90,18327,111,2327,229,2327,748,2328,215,31328,1593,43329,1013,17329,313,18329,558,17329,112,26329,286,18329,91,17330,2902,2330,2295,18330,2185,14330,2168,2330,2024,2330,1956,3330,1874,3330,1854,2330,1625,3330,1338,2330,1249,2330,2472,24330,847,3330,155,6330,436,43330,530,22330,555,38330,514,7330,686,14330,703,2330,752,30330,825,2330,665,9330,955,2332,428,9332,86,25332,113,2332,162,4332,126,34334,1045,13334,1134,18334,971,18334,1072,13334,905,18334,84,23334,171,27334,146,18334,126,18334,545,18335,2497,31335,232,18335,335,23335,396,21335,419,3335,438,29335,360,3335,1587,3336,165,12337,1414,8337,1266,22338,2203,15338,2093,15338,1939,15338,1484,15338,1336,15338,1035,3338,2359,15338,562,3338,441,3338,349,3338,321,26338,162,15338,585,24339,1108,16339,757,16339,958,21339,275,16339,316,7340,171,30340,321,24340,491,35340,1021,16340,1701,24341,1302,24341,1014,8341,790,8341,308,24341,1453,19341,714,15341,1343,8341,648,8341,135,24341,482,8341,425,8341,384,24341,349,8341,165,22341,560,24344,79,18345,1455,9345,1421,21345,1062,6345,416,7346,588,22347,342,3347,91,24347,117,3347,128,25348,1500,25348,891,14348,1809,16348,1770,14348,852,13348,1632,13348,507,26348,399,36348,544,13348,197,35348,133,13349,109,18349,129,2349,226,2349,848,46349,896,13349,1084,2350,946,4350,773,20350,705,17350,661,4350,553,4350,1520,4350,269,9350,254,5350,188,4350,159,26350,415,4351,161,3351,202,23351,137,22352,77,26352,202,15352,647,27353,469,15353,1369,15353,426,15354,205,5354,246,6354,435,6354,640,6354,1786,6354,2148,5355,879,25355,2060,5355,1866,16355,1819,17355,1718,16355,1230,28355,701,3355,1807,5355,584,15355,204,49355,256,6355,271,26355,624,13355,420,15355,457,10355,471,21356,268,15356,285,2356,915,2356,1681,2357,971,24357,1287,24357,549,39357,250,24358,240,23358,270,35358,514,41358,557,2358,611,4358,1009,20358,1142,2358,1216,20359,908,2359,1012,2359,855,2359,365,2359,334,29359,418,30360,44,16360,62,3360,108,29360,172,12360,205,3360,692,24360,721,23360,853,18360,911,18363,954,17363,905,6363,467,8363,255,6363,213,25363,310,17364,1318,41365,1752,3365,2221,23365,2250,3365,2067,3365,1996,27365,1885,3365,1793,3365,1768,21365,1210,2365,2086,15365,1165,3365,1075,2365,1058,15365,1016,3365,925,3365,582,23365,221,3365,184,35365,1193,15366,1094,2366,939,2366,524,2366,486,2366,414,2366,1004,2366,251,3366,219,29366,78,26366,67,2366,283,2366,46,18367,1516,3367,1454,3367,2197,3367,312,21367,335,3367,828,15368,1132,26368,1578,35368,1491,26368,1834,26368,1385,26368,1357,10368,1317,26368,1292,10368,1236,2368,1208,26368,1180,10368,1041,26368,305,35368,724,17368,1016,10368,228,26368,363,10368,433,26368,182,15368,519,26368,558,10368,591,26368,641,17368,475,17370,172,40370,767,12371,152,15371,207,27371,489,18372,1837,32372,667,22372,227,2372,179,3372,149,28372,207,18373,533,5373,849,13373,988,36373,1080,26373,1245,4373,1639,5373,1947,11373,1964,5374,767,15374,438,4374,389,3374,179,46374,132,33374,86,4374,65,19374,167,3375,242,33375,1744,36376,81,15376,98,3376,129,36376,176,3376,188,18376,532,3376,621,3378,418,18378,438,3378,199,61378,116,22378,140,3378,155,25379,63,18379,92,17379,271,14379,287,9379,298,15379,457,29379,526,9379,537,23379,562,15380,1557,19380,1594,2380,1631,8380,1763,28380,1880,19380,2745,2380,2047,29380,2226,30380,2501,21380,2570,6380,1467,15380,1990,40380,1435,2380,476,9380,1298,2380,64,21380,87,2380,114,28380,160,21380,257,2380,228,6380,319,21380,358,2380,411,18380,445,8380,1349,13380,280,6381,1061,16381,1880,16381,1582,16381,858,16381,1900,12381,709,12381,461,15381,413,12381,299,15381,766,16382,1934,12382,1728,12382,1512,21382,1304,8382,2207,10382,833,18382,707,19382,358,17382,877,11383,1394,15383,1366,18383,1229,24383,748,13383,731,15383,422,15383,130,16383,88,15383,680,5384,1246,3384,1957,15384,1324,3384,735,15384,1423,24384,604,26384,337,15384,632,3384,483,15385,1424,9385,955,3385,804,3385,416,24385,1696,3385,171,45385,160,3385,136,22385,347,22387,1792,12387,152,5387,137,12388,442,53388,899,8390,164,32390,320,13390,448,20390,552,24390,633,21390,949,9391,1858,21391,1939,20391,1174,21391,631,18392,215,75393,854,3393,1799,3393,1526,14393,1338,3393,1084,3393,970,2393,966,3393,861,2393,516,21393,495,2393,488,3393,375,5393,397,14393,337,36393,266,2393,247,17393,231,3393,208,21393,1803,2394,1521,15394,1040,4394,1606,6394,1031,5394,1750,15394,656,18394,599,15394,293,15394,131,15394,733,15395,2372,8395,1878,15395,1454,8395,1232,8395,1143,8395,818,2395,538,38395,170,28395,162,2395,145,15396,227,37396,690,14397,925,6397,717,27397,529,3397,968,6397,215,5397,151,7397,114,5397,302,6398,181,29398,212,3398,376,3398,419,3398,595,10398,878,31398,1138,3398,1288,3399,1507,14399,135,28399,81,26400,494,28400,523,21400,546,2400,940,3400,1419,2401,256,18401,310,7401,785,7402,683,8402,842,18402,829,8402,721,3402,575,3402,1168,8402,394,8402,133,8402,470,17402,175,29402,147,17403,72,3403,312,3403,477,3403,1522,3404,618,53405,1165,10405,1011,2405,901,2405,853,10405,782,2405,545,10405,201,45405,160,26405,129,2405,112,15405,330,19406,141,16406,159,5406,807,5407,283,31408,1643,19408,1591,18408,1010,19408,905,19408,789,18408,302,19408,275,17408,147,17408,612,19409,917,5409,1698,4410,1708,18410,914,2410,784,17410,708,39410,478,2410,803,2410,295,22410,280,13410,173,2410,156,15410,326,20412,194,14412,305,20414,1302,22414,1021,15414,480,23414,288,5414,591,14415,1271,8415,1315,11416,663,24416,613,3416,442,3416,523,24416,253,3416,124,15416,349,24416,141,3417,255,21418,442,22418,466,3418,774,3419,121,29419,367,31419,737,18419,841,18421,206,3421,941,16421,922,3421,874,3421,739,15421,515,3421,73,3421,260,28421,90,39421,1081,3421,49,22421,350,11422,368,19422,1983,13422,310,27423,777,2423,576,18423,167,14423,286,2423,266,18424,156,20424,234,20424,772,27425,216,29425,290,14425,1714,29425,1956,43427,1716,13427,985,24427,853,34427,822,22427,315,31427,247,13427,175,19427,371,28428,274,22428,766,20428,790,8429,453,3429,187,11430,680,32430,714,3430,565,8431,309,13431,447,13431,625,13432,102,16432,287,37432,368,23432,733,18432,780,18432,810,10433,949,15433,2045,16433,1937,17433,1707,17433,1565,25433,409,16433,128,17433,106,16433,610,16434,387,20434,86,27434,115,3434,152,19434,269,3435,1292,17435,300,16435,1189,10435,1411,23435,1127,40435,242,16435,484,15435,455,26435,1014,19435,152,36436,170,12436,184,2436,1263,17436,1360,17436,1385,26437,692,26437,598,23438,124,28438,202,24438,528,18439,339,31439,70,27439,99,3439,226,24439,269,24439,315,19439,551,21440,1191,2440,1036,37440,862,6440,262,2440,408,14440,254,6440,661,14441,2839,14441,2573,19441,2516,14441,2420,14441,2147,19441,2053,11441,2033,2441,1960,2441,1810,18441,1538,18441,1472,4441,1416,54441,1830,2441,1326,3441,1352,18441,736,28441,766,3441,826,4441,964,3441,356,33441,1079,33441,1118,21441,1256,19441,1280,16441,1074,3442,1163,13442,1522,11442,1570,3442,1627,13442,1676,11442,1778,18442,1815,11442,2023,11442,2008,3442,1037,11442,1702,18442,990,18442,1987,3442,737,18442,774,19442,122,28442,163,28442,233,5442,243,9442,152,3442,382,3442,400,11442,522,11442,559,3442,675,11442,254,17443,1815,6443,1578,2443,1600,3443,1605,9443,1619,6443,1669,3443,1737,2443,1798,2443,1938,28443,2418,2443,2166,21443,2203,29443,2362,9443,2376,6443,2473,2443,2493,9443,2507,3443,2575,2443,2058,2443,1533,2443,1506,9443,1517,3443,206,12443,248,22443,272,2443,277,9443,1522,6443,296,26443,324,3443,703,2443,744,2443,288,6443,836,2443,886,2443,988,2443,997,18443,1135,6443,1301,14443,1367,15443,1434,18443,778,11444,164,21444,256,6444,318,5444,422,15447,1352,10447,1219,4447,1645,4447,1445,5447,1116,20447,71,34447,559,29447,118,16447,1109,3447,107,4448,274,24448,369,36448,407,4449,159,15449,695,5449,1079,13450,561,3450,451,27450,439,6450,422,15450,367,3450,313,3450,177,19450,210,21450,198,6450,134,25450,116,3450,90,24450,233,3451,1486,13451,1340,13451,1177,25451,476,28451,395,13453,458,16453,477,3453,748,3453,1186,3454,1444,24454,1339,24454,1129,24454,937,24454,890,15454,476,24454,308,15454,236,21454,222,8454,196,24454,143,27454,86,24454,388,24455,536,3455,1462,3455,1264,25455,319,16455,1732,3455,221,35455,164,27455,155,3455,129,24455,307,6456,269,36456,307,4457,1114,40457,995,2457,700,27457,593,2457,234,2457,176,2457,159,14457,279,2458,677,3458,1621,14458,1544,7458,1303,6458,1235,9458,1183,6458,1090,3458,1668,6458,561,3458,728,9458,488,17458,536,16458,123,3458,241,6458,250,30458,99,21458,359,23458,398,6458,428,40458,312,3459,1098,23459,976,16459,914,23459,810,36459,778,17459,728,5459,700,17459,534,17459,477,25459,412,17459,343,27459,281,17459,218,16459,587,25460,317,34460,374,2460,678,2461,298,17461,612,13462,1458,3462,1221,18462,892,9462,737,9463,777,17463,747,10463,565,10463,1102,23463,329,26463,244,26463,113,17463,451,17464,1324,22464,1845,26464,1825,7464,1759,3464,1643,3464,1586,3464,1476,2464,1148,3464,1836,3464,941,2464,884,3464,785,3464,593,3464,519,3464,257,58464,234,3464,925,14464,211,21465,123,25465,151,7465,167,46465,226,14465,497,17465,786,7466,308,21466,114,5467,965,15467,66,15467,127,18467,210,16467,320,15467,599,15467,704,18467,834,15467,1268,18467,1534,18468,82,26468,115,44469,733,3469,687,21469,803,19469,795,3469,529,3469,336,2469,362,3469,229,32469,131,3469,108,21470,1183,16470,1122,15470,949,15470,730,9470,540,15470,517,3470,566,3470,415,14470,373,14470,347,14470,148,30470,90,51470,491,24471,961,12471,1605,9471,1416,3471,1322,5471,1275,5471,1232,3471,1022,3471,780,3471,534,3471,657,3471,346,4471,364,3471,323,21471,199,3471,172,25471,850,3474,621,17474,330,17474,124,51474,178,5476,1567,13476,145,21476,168,3476,199,13476,229,3476,324,13476,584,13476,1039,13476,1670,13478,464,3478,653,3478,302,13478,169,46478,250,35478,143,3478,124,16479,95,26479,124,2479,825,2479,1077,6480,911,48481,480,8481,360,17481,243,21481,191,15481,150,23481,138,8483,2440,4483,2278,4483,1988,4483,1645,36483,1637,4483,1593,5483,1175,5483,1683,4483,1024,5483,983,39483,972,4483,930,40483,225,4483,236,18483,188,35483,1135,13484,1129,34484,1070,3484,983,7484,958,3484,812,21484,779,28484,738,21484,668,2484,516,33484,458,35484,386,2484,301,3484,214,43484,178,30484,147,3484,568,35485,601,18485,1670,18485,1650,16485,1306,3485,388,19485,310,19485,236,37485,207,11485,179,23485,145,16485,333,18486,494,14486,71,20486,127,29486,204,6486,369,3486,534,7487,252,3487,201,20487,142,53487,59,40487,24,22487,48,3488,388,12488,421,8488,157,18488,471,16488,101,40488,80,14488,143,8489,1348,3489,485,3489,463,3489,86,28489,131,23489,116,3489,433,26490,398,25491,190,21491,635,19491,1088,16492,350,17492,369,31493,1071,10493,1031,24493,682,21493,387,23493,163,39493,301,12493,284,15493,204,4493,1125,4493,329,20493,1324,4494,172,50495,1092,19495,145,31495,1366,16495,762,16495,95,32495,318,16495,626,24495,65,24496,372,2496,348,22496,588,16496,606,4496,836,13496,893,21497,1150,18497,1035,3497,222,3497,34,25497,74,28497,62,3497,115,47499,272,16499,300,16499,264,6499,520,3499,136,35499,125,3499,99,24499,212,3500,175,13500,189,21500,573,21500,650,15500,730,15500,903,14500,1306,16500,1557,5501,786,16501,127,11502,1641,20503,105,18503,200,33503,314,30503,435,23503,584,13503,720,13503,793,6503,819,14503,901,18503,945,23504,1115,3504,967,6504,1064,49504,220,34504,603,5505,1677,3505,969,3505,544,3505,1569,3505,258,3505,225,3505,199,24505,431,3506,654,12506,1034,14506,995,8506,959,6506,749,11506,716,13506,325,11506,146,8506,185,18506,156,23506,132,12506,117,13506,83,8506,286,13507,1549,16507,1639,17507,2238,14507,1445,17507,691,26507,1270,25507,974,27507,89,17507,1399,19509,152,45510,206,15510,251,15511,1099,6511,1069,3511,1064,3511,872,23511,800,3511,613,3511,428,3511,242,11511,316,3511,215,3511,168,13511,126,3511,94,25511,1297,3511,365,12511,1386,24512,971,5512,1017,5512,858,12512,62,3512,44,16512,82,18513,271,26513,299,2513,382,2513,1274,15513,1319,14515,821,13515,765,23515,706,28515,688,16515,666,15515,647,13515,580,19515,941,13515,526,7515,498,26515,473,23515,410,28515,382,17515,258,40515,80,17515,535,39516,148,11516,556,21516,581,20517,1513,35517,1422,25517,1103,15517,1048,19517,913,28517,662,19517,288,6517,145,22517,97,16517,491,19518,104,21518,128,2518,350,2518,1093,13518,1154,2520,894,13520,1641,43520,1522,14520,1164,13520,1050,14520,809,23520,1584,13520,630,13520,604,5520,495,13520,468,5520,729,13520,367,20521,962,3521,703,3521,300,11521,910,29521,252,13521,230,3521,200,28521,277,9523,643,17523,1380,5523,1237,17523,1214,5523,1113,4523,1104,5523,1085,3523,1043,3523,926,5523,1586,10523,691,3523,540,3523,662,27523,471,4523,496,3523,147,29523,178,5523,286,5523,304,32523,224,5523,380,3523,392,3523,438,3523,443,26523,357,21524,804,26524,1251,32524,1184,3524,1128,3524,1027,3524,718,3524,590,20524,667,19524,616,8524,544,12524,485,3524,461,22524,688,28525,694,13525,977,16525,1074,13526,1994,11527,666,13527,1007,13527,559,13527,500,23527,425,20527,341,44528,221,6528,279,26528,307,3528,365,6528,1386,6529,314,3529,296,16529,420,37529,1468,3529,1538,2531,240,46531,187,5531,131,54531,200,17533,1693,13533,1515,13533,1473,13533,743,13533,1344,26533,516,13533,456,13533,354,27533,257,48533,609,13534,770,19534,1419,7534,1300,7534,1253,14534,917,16534,711,13534,132,13534,491,14534,336,14534,235,17534,90,34534,620,13535,75,18536,1370,17536,1209,3536,1010,16536,740,3536,452,3536,227,27536,141,27536,129,3536,106,21536,271,3537,884,18537,1379,19537,1353,2537,1045,22537,956,10537,946,6537,859,18537,748,21537,816,10537,796,18537,339,10537,222,4537,193,27537,835,15538,1611,14538,1541,21538,1533,6538,1519,12538,1494,23538,1482,8538,1454,22538,1374,19538,1288,20538,1571,12538,903,21538,796,53538,596,14538,544,12538,530,12538,505,23538,491,12538,301,34538,196,46538,1273,5539,1393,7539,657,3539,562,3539,910,3539,218,32539,180,7539,153,25539,381,7540,1691,13540,1576,24540,922,9540,1324,13540,651,21540,458,13540,211,28540,794,28541,1057,3541,1581,3541,1703,3541,1390,3541,1304,3541,1145,22541,754,6541,233,31541,409,3541,322,9541,285,3541,184,3541,165,17541,611,6542,521,29542,385,15542,306,14542,150,3542,127,21542,165,39543,2377,23543,3729,24543,3651,3543,3441,20543,3156,15543,3044,23543,2495,54543,824,18543,1064,20543,710,32543,681,27543,673,3543,306,3543,227,3543,178,3543,153,23543,777,41546,422,33546,511,16546,457,3546,239,22546,299,16546,417,3547,1008,3547,914,3547,1099,12547,872,3547,1502,12547,1845,3547,2080,3547,1219,3547,824,3547,957,3547,684,5547,630,4547,643,39547,794,3547,593,35547,556,22547,435,3547,235,3547,211,22547,91,3548,1025,3548,462,9548,439,3548,396,3548,139,3548,186,29548,150,25548,109,28548,345,3551,1226,13551,1975,13551,1612,13551,2054,13551,1066,13551,1440,13551,589,13551,333,13551,275,13551,175,13551,718,14553,117,10553,652,14553,751,18553,1052,14554,116,63554,274,22554,544,15554,1209,14555,1787,15555,1732,15555,285,15555,142,52555,216,25555,243,4556,468,12556,482,26556,514,26557,574,15557,1029,11557,1004,15557,1078,15557,715,50557,535,20557,881,21557,388,24557,369,13557,293,14557,219,13557,134,64557,433,26558,453,33558,655,17558,1226,5559,1397,34559,1353,23559,1304,31559,1259,27559,717,23559,636,27560,70,33560,242,32560,483,12560,675,21560,791,21560,904,21561,1111,3561,836,17561,485,3561,1019,3561,131,3561,107,21561,266,3562,157,21562,510,16562,528,21562,719,16562,937,16562,1238,21562,1366,21562,1403,12563,959,7563,841,16563,421,7563,339,26563,71,18563,60,7563,214,18565,156,15565,621,15565,2199,15566,2020,26566,1965,4566,1897,4566,1851,9566,1781,3566,1269,9566,1096,3566,636,49566,557,3566,601,3566,111,22566,135,3566,146,38566,687,5566,340,3566,507,11566,238,5567,178,25567,208,21567,644,15568,1529,26568,860,6568,515,16568,161,26568,214,20568,188,21569,1311,3569,1988,3569,1970,13569,1802,3569,1598,9569,1588,9569,1449,9569,1388,3569,810,3569,784,24569,767,9569,745,17569,502,14569,490,9569,480,6569,450,13569,1235,13571,1592,16571,1372,6571,1235,6571,716,6571,708,3571,167,5571,123,42571,178,40572,277,14572,293,2572,329,2572,523,2572,698,2573,450,30573,556,12573,342,19573,862,21573,155,34573,87,14573,241,20574,1338,28574,990,14574,946,18574,901,43574,720,18574,701,17574,970,14574,639,28574,360,3574,246,3574,208,3574,167,28574,678,19575,185,22575,588,17576,936,2576,970,16576,1019,21576,1048,2576,1705,2576,1547,11576,1578,13576,830,13576,1430,3576,814,14576,719,13576,769,12576,745,13576,651,4576,619,30576,601,3576,580,19576,509,13576,266,25576,787,17578,1398,5578,1304,20578,1201,20578,358,3578,101,29578,142,27578,132,3578,254,3579,531,2579,1373,2579,506,21579,339,2579,185,18579,89,2579,69,18580,77,29580,112,26580,233,21580,260,15580,804,29580,1328,29581,1022,23581,978,2584,1826,3584,1725,3584,1646,3584,1546,3584,1220,3584,1128,3584,1049,3584,891,3584,1431,3584,504,17584,470,29584,446,3584,384,17584,359,20584,306,50584,254,3584,99,33584,622,3585,153,21585,379,20585,401,21585,1283,20586,1802,15586,1489,17586,147,24586,127,15586,601,15587,90,32587,124,4587,217,4588,124,17588,225,4589,358,15589,1508,15589,1822,15590,377,20590,830,4590,430,10590,201,25590,1115,15590,118,36590,107,4590,72,33590,182,14591,1887,6591,1600,3591,1497,18591,1327,31591,1275,4591,1238,35591,1152,51591,1074,15591,1041,29591,972,8591,925,3591,1546,3591,780,3591,194,26591,222,3591,255,30591,287,3591,860,22591,389,3591,508,3591,549,3591,615,3591,362,3592,516,33592,161,37592,220,33592,255,4592,347,32593,695,3593,245,3594,124,31594,157,2594,269,5594,821,6594,1078,5594,1191,25594,2155,24594,2181,12594,2201,24594,2287,2596,879,23596,912,21596,769,26596,722,30596,700,18596,840,37596,524,24596,119,14596,271,22596,646,48596,482,25596,389,22597,261,25597,348,7597,425,5597,689,16597,1694,7598,1756,19598,1635,19598,1567,2598,1268,19598,1243,2598,1032,19598,936,6598,646,17598,384,19598,433,2598,281,19598,180,19598,150,2598,128,20598,500,19598,227,20599,1387,15599,1625,8599,1039,8599,209,8600,92,18600,116,20600,139,2600,152,39600,359,2600,478,38600,1019,2600,1486,2601,1161,13601,1742,5601,1536,6601,1232,19601,1063,5601,1044,5601,648,35601,158,5603,767,15603,888,27604,763,17604,811,2605,53,17605,72,3605,83,46605,131,3607,1103,14607,1167,19607,916,3607,124,3607,378,20607,87,35607,829,18608,210,49608,1167,15609,2148,14609,2045,13609,1931,19609,1512,18609,1313,19609,1236,14609,1175,18609,1111,24609,1071,28609,1019,11609,2195,3609,894,15609,839,3609,772,3609,578,15609,483,3609,448,18609,409,15609,316,33609,244,3609,217,25609,980,15610,2086,22610,624,19610,2149,13610,476,14610,277,26610,557,3612,1114,25612,778,25612,596,6612,465,6612,678,6612,356,12612,221,18612,119,7612,373,13613,1149,4613,1793,12613,1691,3613,1254,4613,1237,4613,1184,4613,1159,20613,1079,65613,1061,13613,1047,9613,989,4613,971,6613,859,3613,779,3613,622,29613,294,3613,225,3613,201,22614,1698,3614,618,16614,274,15614,97,20614,119,3614,146,67615,896,3615,1218,17615,1135,9615,802,17615,1290,5615,490,5615,363,17615,240,20615,217,5615,725,40617,266,14618,154,5618,198,6620,130,6620,179,47620,65,63620,143,13621,133,21621,157,2622,539,32622,765,11622,663,15622,501,14622,71,15622,409,44622,130,8622,467,22623,242,50623,294,6623,396,6623,1195,6623,1361,6624,1851,3624,1967,3624,2034,3624,1740,3624,404,3624,162,3624,145,15624,798,24625,645,3625,452,15625,316,29625,305,4625,258,17625,419,21625,177,39625,151,4625,116,33625,107,3625,77,28625,248,3626,2199,2626,1899,2626,1182,2626,1032,2626,654,2626,847,22626,288,2626,264,22626,910,22628,1183,4628,195,16628,693,4628,153,17628,172,4629,329,21629,361,39629,402,4629,524,4630,108,37630,152,18631,655,3631,1316,5631,1214,29631,1105,3631,852,3631,476,6631,461,3631,241,28631,176,3631,152,21631,628,3632,283,28632,313,3632,657,3632,1693,3633,1314,3633,1560,3633,1306,3633,1585,17633,1896,15633,1568,3633,1127,10633,331,18633,497,3633,466,29633,456,3633,429,25633,423,3633,394,27633,917,18634,150,20634,1058,18635,857,3635,346,3635,139,39635,128,3635,104,22636,449,3636,491,3636,634,13636,842,3637,1240,16637,747,16637,737,8637,1261,6637,718,13637,699,17637,575,20637,597,16637,515,17637,334,65637,293,36637,679,18638,1800,14638,1849,14638,1879,10638,1932,3638,2034,20638,2151,6638,2417,3638,2566,6638,2590,3638,2700,9638,2716,12638,1647,3638,2361,18638,1413,32638,2251,5638,1206,18638,255,26638,283,3638,351,23638,422,6638,460,2638,1357,14638,437,21638,714,10638,773,6638,903,14638,1020,17638,1051,20638,637,31639,830,3639,1517,21639,1314,37639,1116,7639,948,18639,708,3639,274,27639,216,30639,180,3639,152,26639,65,18639,315,3640,977,4640,987,9640,1138,35640,1202,4640,1404,9640,1732,9640,1622,24640,1743,24640,1888,11640,2223,4640,1415,22640,931,20640,59,11640,840,18640,808,4640,778,28640,697,16640,542,16640,505,4640,427,9640,293,9640,108,4640,95,11640,891,24641,793,56642,1107,33642,975,42642,2011,14642,223,16642,559,3642,128,17643,379,20643,449,3643,491,39643,532,4646,560,30646,688,22646,629,36646,430,15646,410,17646,466,2646,313,13646,163,2646,148,12646,122,23646,387,20647,1216,17647,973,12647,863,3647,706,17647,266,14647,258,6647,137,3647,87,48647,59,14647,1450,7647,75,2648,541,8648,123,18648,316,17649,150,30649,185,6649,204,17649,335,6649,645,6649,697,13649,933,7649,1114,17650,161,30650,193,3651,635,21651,184,14651,215,30652,171,12652,185,2652,222,20652,261,8652,278,27652,412,5652,584,2652,743,5653,1130,6653,1215,3653,1069,15653,1260,3653,1220,26653,864,17653,340,30653,597,3653,569,18653,491,17653,779,6653,335,3653,312,17654,275,20654,498,25655,1218,13655,1152,12655,1101,25655,1054,5655,798,49655,775,17655,693,5655,672,13655,636,24655,576,14655,480,5655,147,16655,761,12656,164,27656,193,3656,197,3656,573,3656,577,3657,837,18657,770,20657,674,11657,33,28658,1128,15660,350,28660,825,15660,992,16662,103,27662,132,3663,1202,3663,1198,3663,1036,58663,1003,19663,985,16663,828,10663,708,21663,699,3663,261,3663,586,3663,257,3663,151,3663,124,25663,115,3663,86,27663,611,3665,690,3665,741,3665,459,3665,110,27665,176,28665,139,3665,403,16668,970,3668,663,26668,652,3668,561,12668,1276,3668,371,3668,333,35668,131,13668,399,3672,111,34673,233,18673,913,18674,79,18674,432,18674,703,18674,1045,17676,250,27676,118,33676,153,4676,228,17676,279,3677,149,14679,250,20680,129,8680,398,8680,577,8680,752,8680,827,8680,886,24681,188,15681,575,13681,676,15681,917,15682,1073,4682,544,4682,273,4682,241,25682,145,4682,174,22682,118,25682,214,22683,471,32683,858,3683,945,3683,790,18683,524,6683,440,25683,80,26683,308,14683,295,3683,210,40683,132,30683,75,3683,417,21684,254,3685,529,19685,235,3685,148,3685,119,27685,97,17691,1386,3691,1199,3691,1109,18691,960,28691,922,10691,847,3691,744,17691,1242,3691,492,18691,470,20691,439,13691,242,3691,159,3691,142,15691,622,31692,155,25693,502,13695,2254,24695,2034,12695,2012,12695,1566,6695,1277,12695,1219,17695,1114,23695,691,6695,630,6695,548,6695,445,6695,234,23695,808,13696,258,4696,290,21696,231,25696,486,22696,765,4696,933,4697,881,13697,948,13697,860,18697,939,6697,199,16697,131,24697,220,14698,143,33698,347,14698,379,18698,429,18698,555,14698,1524,33699,128,45699,334,26699,694,17701,592,18701,1004,28701,296,28702,134,18702,155,2702,571,2702,1257,2703,82,26703,110,3703,120,5703,202,3703,616,17703,793,3704,1084,22704,1171,33704,1110,13704,1226,19704,956,28704,187,30704,515,8704,496,2704,219,3704,140,13704,560,31705,501,18705,734,18705,1187,9705,1369,9707,301,20707,661,21708,1666,3708,217,35708,112,28708,142,3708,153,34709,155,25709,187,28709,228,55709,1156,24710,1051,3710,856,3710,759,25710,683,17710,549,3710,283,3710,251,30710,643,3711,193,5711,300,16712,937,3712,863,3712,653,12712,296,3712,243,3712,85,3712,57,26713,1754,18713,1702,16713,1684,16713,1374,3713,1370,3713,270,3713,1720,17713,174,29713,151,3713,124,25713,115,3713,221,16713,89,24714,166,21714,367,13714,452,25714,823,6714,1225,33714,1370,6715,513,14715,182,14716,193,24716,268,48716,351,9716,368,15716,410,37716,733,57716,802,16717,906,3717,1867,3717,1310,3717,1970,3717,1083,3717,929,3717,882,3717,1269,3717,583,26717,574,3717,564,3717,433,3717,367,3717,350,15717,611,3718,193,52718,249,26718,1672,25719,1507,23719,856,27719,574,26719,546,18719,527,3719,291,22719,457,16719,176,3719,152,22719,507,18720,1748,24720,1636,18720,1609,5720,1569,23720,1042,24720,1161,23720,504,19720,414,10720,324,19720,171,11720,124,15720,998,5721,102,14721,130,16721,163,10721,310,10721,367,14721,1288,27722,791,23722,386,58722,293,16722,239,27722,184,23722,218,16723,110,5723,325,19724,1090,3724,1552,9724,1490,10724,1430,3724,1391,27724,1618,3724,1353,9724,1199,10724,1072,7724,479,17724,806,11724,135,20724,157,3724,309,37724,997,3724,498,3724,581,12724,698,16724,348,4725,1653,3725,1582,3725,1466,3725,1426,3725,1347,3725,1320,3725,1316,3725,1212,3725,282,3725,163,3725,155,3725,115,38726,963,24726,945,15726,599,15726,1220,20726,169,20726,67,76726,401,20727,1269,3727,1178,3727,1035,3727,987,19727,824,21727,578,24727,878,3727,401,3727,328,25727,231,19727,197,7727,131,3727,108,21727,549,12728,129,6728,140,18728,160,38728,206,12728,746,42728,833,28728,874,52729,1745,28729,1017,24729,1531,28729,200,42729,731,28729,161,28731,154,18731,174,2731,314,2731,705,2731,857,2732,1059,18732,108,15732,237,15732,1004,18732,1297,31732,1489,15733,129,18733,173,17733,301,28734,1573,13734,1801,20734,1463,26734,1963,14734,778,19734,1491,3734,403,17734,325,6734,283,40734,187,17734,629,6735,114,30735,1177,23735,1324,12735,1475,12735,1699,12736,964,14736,930,29736,910,18736,860,29736,830,15736,721,30736,407,9736,473,10736,396,10736,326,32736,144,26736,633,23737,174,18737,216,26737,767,21738,537,3738,790,20738,1113,3738,382,3738,120,3738,98,20738,135,27739,1969,3739,1881,3739,1825,3739,1601,3739,1455,3739,1330,27739,831,3739,1208,3739,601,3739,457,3739,364,3739,298,25739,112,16739,82,3739,59,21739,701,3740,1179,21740,1578,3740,1520,16740,1398,18740,1347,3740,1242,21740,882,4740,940,8740,857,23740,849,3740,603,18740,496,3740,330,3740,243,3740,212,29740,910,26741,138,43741,229,17741,318,13742,1688,19742,1709,16742,1655,16742,171,2743,153,18743,185,5743,370,6743,449,18743,599,17743,770,18743,928,17743,1083,5743,1343,18744,1259,13744,1056,18744,1767,24744,988,13744,136,23744,357,33744,322,22744,234,13744,459,23745,1809,5745,1676,11745,1590,5745,1493,16745,1422,12745,1350,6745,1251,12745,1084,5745,780,15745,533,5745,512,2745,480,5745,350,20745,372,2745,232,15745,124,15745,569,14746,1168,11746,1126,13746,1071,13746,993,18746,253,20746,230,21746,151,13746,709,12748,1378,27749,1387,4749,877,4749,672,4749,1094,4749,152,17749,129,4749,110,17749,243,4751,156,22751,288,14755,2055,3755,1940,13755,1787,13755,1732,3755,1626,13755,1312,13755,903,3755,1214,13755,603,13755,576,13755,430,13755,338,13755,291,22755,244,29755,213,26755,744,3756,259,17757,455,21757,1600,12758,259,25758,1225,15759,646,28759,203,28759,271,9760,70,51760,134,31761,342,20762,1076,11762,1627,11762,1564,11762,1441,11762,1301,11762,1283,3762,1210,11762,945,11762,346,17762,905,11762,887,3762,568,3762,553,11762,178,29762,168,3762,138,28762,924,3763,654,3763,729,3763,745,33763,1270,3763,987,3763,1191,28763,509,3763,780,24763,483,3763,878,3763,302,32763,273,3763,188,31763,169,8763,141,23763,380,3763,99,27763,88,3763,64,22764,101,38764,141,3764,149,26764,183,26765,1651,3765,1421,19765,1402,3765,1474,12765,1257,12765,135,12765,1387,13766,1626,11766,1895,12766,1941,28766,2004,11766,2048,9766,2108,11766,2637,3766,2201,11766,2276,11766,2318,25766,2359,11766,2465,18766,2521,10766,2584,3766,1576,11766,2145,24766,1541,17766,946,21766,1398,22766,63,31766,100,38766,170,30766,255,11766,349,17766,429,11766,1517,10766,489,3766,1024,11766,1102,10766,1189,11766,1202,12766,1272,14766,1288,14766,1353,9766,701,11767,778,23767,362,20767,1098,21769,1324,14769,272,53770,1514,18770,844,12770,664,22770,609,14770,521,40770,1466,22770,421,18770,381,22770,110,29770,87,12770,478,14771,169,20771,259,21771,191,4772,140,18773,558,3773,1420,5773,1382,15773,978,10773,855,3773,670,3773,544,12773,416,3773,143,26773,205,19773,171,3773,484,3774,256,4774,179,50774,67,34774,103,4774,138,17775,2657,18775,4531,6775,3024,6775,3213,8775,3264,16775,3364,2775,3448,6775,4010,6775,4223,25775,4654,6775,2080,18775,4818,25775,4902,6775,5275,6775,5639,20775,5661,18775,5704,23775,6007,6775,6162,31775,6474,6775,6641,6775,2484,3775,4752,6775,2432,18775,2049,18775,2292,18775,2420,6775,24,6775,293,12775,307,6775,557,6775,635,23775,866,32775,900,3775,949,16775,1185,3775,1199,17775,1055,13775,1237,23775,1266,3775,1398,6775,1557,6775,1723,15775,1883,6775,1951,6775,1218,17775,2208,25775,2264,18776,953,17776,1035,3776,1753,17776,1528,3776,778,3776,1447,17776,639,30776,137,17776,461,3776,156,3776,276,5776,233,41776,224,3776,192,30776,580,3777,919,3777,100,22777,124,3777,245,45777,429,3777,491,18777,515,13777,837,3777,1001,3778,500,2778,312,2778,104,18778,124,2779,1627,13779,229,22779,318,16779,609,13779,738,17779,924,7779,1135,18779,1301,7779,1360,23779,1455,6779,1538,3780,863,10780,815,15780,798,12780,452,39780,494,4780,129,21780,598,4781,215,25781,242,3782,829,15782,428,15782,1695,15782,134,15782,170,16783,1732,14783,347,24783,194,5783,227,25784,304,31784,708,17784,740,16784,1010,20785,441,18785,1063,4785,312,4785,281,18785,169,18785,80,22785,50,18786,55,20786,77,3786,92,48786,299,24786,393,24786,506,18787,743,19788,1569,6788,1637,15788,1774,15788,745,15788,2136,14788,2300,14788,1453,15788,1863,14788,1281,14788,256,22788,1006,14788,943,15788,570,14788,528,5788,468,14788,1094,5788,215,14789,746,4789,489,36789,380,46789,123,16789,191,23789,87,17789,258,17790,753,3790,992,3790,369,3790,1136,3790,275,19790,204,8790,349,17791,1940,12791,1820,21791,1595,24791,1432,2791,907,4791,869,36791,620,32791,612,2791,598,12791,427,4791,402,23791,264,18791,230,14791,219,5791,202,15791,189,11791,155,2791,135,18791,493,12792,1295,14792,990,14792,890,14792,647,14792,370,14792,750,14793,568,3793,360,3793,139,58793,73,3793,48,22793,89,35794,1594,3794,1412,3794,1362,3794,1312,9794,1255,3794,1240,3794,1045,3794,1010,8794,985,3794,953,30794,758,9794,722,3794,647,3794,609,3794,553,9794,414,15794,307,3794,199,9794,176,3794,1626,3794,1686,20795,708,27795,145,16796,79,15796,383,15796,452,15796,520,6796,814,15796,1335,15797,805,23797,1068,14797,1032,2797,1020,2797,976,3797,938,21797,906,19797,712,2797,738,25797,327,3797,288,36797,206,19797,894,2799,2485,15799,1564,15799,1062,15799,2355,15799,2115,18799,962,18799,560,10799,781,20799,172,15799,899,18799,377,15799,219,21800,1004,13800,973,25800,844,31800,814,28800,800,12800,780,18800,707,15800,646,13800,598,13800,262,28800,198,25800,103,25800,664,21801,1480,13801,1451,12801,1255,3801,985,13801,163,3801,135,26801,93,3801,942,10802,151,35802,188,3802,505,16802,975,3802,1522,3803,1468,88803,1424,19803,1401,18803,1288,16803,236,12803,224,10803,203,19803,250,49804,1670,15804,234,14804,528,28804,1513,15804,1600,14805,596,3805,635,12805,141,12805,286,3807,1649,3807,1509,3807,1114,3807,1298,3807,1008,3807,173,3807,644,3807,388,3807,149,22807,819,3808,486,23809,907,17809,192,4809,616,17809,61,17809,85,48810,532,13810,572,27810,1071,17811,793,24811,1597,24811,1437,24811,985,17811,694,13811,261,24811,489,25811,406,24811,107,13811,591,35812,154,28812,191,39812,474,13813,654,21813,677,3813,320,6813,219,26813,184,17813,296,19815,274,36815,1265,3815,1514,27816,1585,4818,1073,36818,2248,5818,2104,28818,1925,5818,1793,28818,1556,36818,1443,28818,841,26818,877,6818,2285,35818,777,36818,400,5818,362,19818,218,35818,1122,36819,1287,6819,1236,6819,1354,6819,1069,6819,129,6819,707,6819,658,6819,890,24820,180,17820,1299,55821,2625,14821,2182,2821,2124,10821,1876,3821,1853,2821,1531,5821,1461,10821,1390,26821,1293,10821,2669,27821,1082,14821,362,27821,391,2821,399,23821,1138,2821,476,2821,517,2821,979,2821,424,3822,302,35822,339,4822,704,2823,992,13823,1636,13823,1197,27823,1066,24823,1788,13823,746,33823,491,13823,312,6823,182,13823,687,13824,147,3824,339,3824,420,2824,882,3825,1148,16825,1518,24825,1326,18825,1182,24825,1617,24825,1169,11825,1108,25825,569,24825,987,24825,855,24825,674,24825,284,24825,100,24825,45,24825,1079,24826,274,12826,808,24827,2130,20827,834,2827,409,23827,171,2827,142,27827,180,16828,173,17828,246,27829,1224,3829,160,20829,182,3829,284,3829,502,3829,632,3829,1089,3829,1318,3830,514,18831,125,13831,2261,18831,1945,13831,1783,18831,1505,18831,1108,18831,717,3831,680,18831,664,11831,1045,13831,509,20831,494,13831,459,18831,279,18831,262,13831,213,8831,185,16831,531,3832,256,2832,239,15833,791,2833,1305,2833,1454,2833,1083,2833,656,2833,336,2833,403,2833,364,2833,305,29833,429,2834,1430,2834,1310,13834,1093,23834,932,23834,901,20834,840,12834,668,2834,573,2834,505,6834,468,28834,440,8834,366,14834,309,26834,236,32834,181,18834,149,2834,131,16835,1308,20835,840,23835,748,48835,692,32835,658,8835,120,23835,413,21835,374,21835,275,22835,161,21835,485,22836,828,6837,1628,7837,906,21837,874,21837,650,22837,332,5837,365,7837,288,41837,111,7837,599,26839,102,18840,764,17840,880,15840,820,28840,717,23840,742,17840,250,16840,128,26840,676,15840,176,29841,914,10841,948,14842,140,16842,174,23842,208,42842,308,19842,853,18843,1125,17843,1198,3843,589,28843,430,7843,161,3843,185,34843,138,21843,118,19843,334,21844,817,8844,1062,15844,895,16844,855,35844,409,45844,944,3844,237,32844,206,26844,141,27844,118,16844,300,16846,420,14846,1667,21846,1465,14846,244,17846,195,3846,210,21846,177,15846,234,3847,123,3847,165,17847,249,20847,437,20847,2180,20848,745,24849,1366,2849,1056,2849,901,2849,184,2849,573,2849,162,20849,579,2850,121,8850,279,8851,862,21851,942,8851,1047,9851,379,16851,1128,15851,703,15851,1098,2851,681,16851,144,51851,443,17851,403,15851,552,16851,319,20851,231,16851,215,11852,1589,20852,1476,12852,1418,3852,1394,22852,1222,10852,1118,4852,1187,10852,1002,3852,980,20852,199,6852,349,10852,328,16852,160,37852,1051,10853,986,13853,1453,6853,1517,17853,1379,16853,1361,7853,1294,6853,1224,14853,1010,7853,893,6853,179,6853,721,22853,671,16853,541,35853,460,34853,325,16853,264,6853,249,6853,100,15853,801,13854,1129,18854,969,18854,764,20854,701,18854,808,9854,363,18854,226,17854,176,18854,448,20855,1538,14855,1740,3855,1463,3855,630,3855,503,24855,271,24856,2697,17856,2650,5856,2475,5856,2388,25856,2033,15856,1985,16856,1941,15856,1815,16856,1765,17856,1898,14856,1676,15856,1594,10856,1545,16856,1444,24856,754,3856,727,25856,246,25856,108,3856,1720,16857,80,17857,412,19857,570,28857,616,17857,714,34858,779,3858,484,3858,103,27858,132,3858,143,50859,112,16859,137,19859,196,17859,215,18859,235,13859,250,14859,425,18859,719,16861,1876,11861,1673,3861,1658,11861,1552,11861,1228,11861,144,3861,210,41861,155,27861,114,28861,612,3862,1436,6862,2224,14862,2084,3862,1242,24862,254,20862,633,24862,539,3862,276,3862,1090,5863,152,29863,193,17863,615,2863,2195,29864,1259,14864,779,14864,634,8864,230,18864,659,26865,94,18865,115,2865,234,2865,432,2866,1689,15866,716,14866,1896,15866,197,32866,158,26866,346,24867,1741,17867,1291,23867,1221,23867,1049,17867,825,5867,690,21867,1114,19867,558,17867,462,17867,310,21867,280,10867,84,17867,653,6868,96,15868,118,37868,236,15868,486,15869,1251,15869,1377,5869,1272,33869,1332,21869,1372,3869,1489,9869,264,19869,1615,3869,1683,6869,1699,63869,1873,15869,1972,2869,859,16869,1535,10869,769,27869,112,3869,744,3869,761,3869,124,14869,192,42869,93,17869,294,30869,236,5869,332,21869,355,3869,365,18869,385,2869,412,6869,326,3871,185,6871,248,51871,458,15872,779,22872,1358,11872,1168,22872,521,29872,658,24872,400,5872,317,13872,285,5872,242,11872,432,62874,2036,3874,1826,3874,1822,3874,883,3874,2032,3874,238,3874,234,3874,192,40874,879,3875,121,28875,151,3875,161,22875,423,26875,492,3875,1087,3875,1232,3876,1861,19876,2035,25876,1997,17876,1325,17876,697,17876,261,17876,114,57876,92,17877,1568,5877,1458,56877,1318,5877,886,5877,545,5877,497,27877,1156,15877,415,5877,383,15877,190,35877,158,5877,127,29877,437,31878,44,18878,63,2878,104,18878,268,2878,625,2878,740,2879,1286,3879,1393,3879,1375,3879,1125,3879,94,17879,253,3879,134,46879,113,3879,579,3880,663,12880,607,25880,483,33880,521,28880,137,32880,104,28880,278,8881,85,21882,747,33882,1453,4882,1639,4882,911,3882,782,4882,656,3882,970,4882,447,18882,438,3882,408,28882,376,30882,329,33882,620,3883,975,14883,558,27883,998,27883,490,19883,319,31883,521,25884,96,27884,134,40884,197,14884,255,16884,313,27884,1403,27887,1634,13890,657,6890,1103,14890,782,13890,581,6890,354,15890,318,13890,187,13890,134,14891,227,12891,1338,12892,168,22892,201,21892,1079,22892,1285,22892,1389,22893,1174,18893,1542,18893,1304,4893,1338,11893,1520,16893,1371,19893,1396,17893,1423,12893,1124,35893,1351,18893,1030,24893,1280,20893,840,34893,988,2893,154,33893,199,30893,268,20893,301,14893,189,4893,358,43893,475,4893,562,4893,757,23893,317,36894,627,2894,1177,2894,1071,2894,926,21894,857,2894,783,2894,481,2894,1190,39894,347,16894,284,2894,256,21894,166,27894,111,2894,89,20894,454,2895,1089,2895,1046,2895,1001,2895,411,2895,151,27895,78,2895,58,17895,181,3897,855,18897,940,15897,1150,18897,701,15897,342,28897,485,38897,112,16897,665,20898,49,24898,122,20898,156,10898,177,22898,1117,3898,1161,37899,1765,22899,1157,26899,1404,13899,1586,23899,1801,8899,631,8899,2074,4899,2123,24899,2225,8899,2306,9899,2353,26899,1084,4899,2006,27899,943,8899,523,25899,843,8899,87,8899,105,34899,184,19899,141,4899,280,8899,437,4899,922,19899,484,37899,474,8899,581,8899,644,22899,752,8900,1365,3900,1175,7900,1149,13900,1135,12900,1113,20900,1089,18900,2031,3900,656,3900,599,3900,366,27900,188,3900,152,21900,719,3901,789,15902,1761,13903,1418,20903,1099,3903,250,19903,135,25903,654,5904,977,19904,128,41904,471,9904,1061,12904,1157,17904,1425,16905,806,21905,465,37905,439,20905,135,3905,199,10905,148,29905,101,32905,214,22906,396,5906,637,32906,516,3906,426,3906,1072,3906,408,16906,331,3906,301,28906,231,31906,369,25907,1761,2907,1538,2907,1437,2907,1296,2907,1153,2907,984,2907,754,2907,663,2907,525,2907,369,2907,289,2907,265,22907,95,9908,2228,5908,2337,5908,1004,3908,934,5908,540,21908,1159,3908,199,73908,149,20908,123,5908,92,29908,418,5909,1529,33909,1463,17909,1440,11909,1032,2909,710,16909,1393,11910,896,15910,1519,12910,1165,23910,1652,23910,958,15910,794,11910,414,10910,494,28910,430,11910,147,42910,124,12910,757,20911,155,17911,396,33912,1275,3912,1180,3912,1048,20912,235,3912,328,3912,205,27912,484,3913,180,15913,979,2913,1097,2913,1357,2914,725,38914,1248,5914,1366,5915,170,13915,105,29915,85,14916,184,27916,212,21916,235,3916,805,18916,1024,16916,1398,29916,1636,3918,1406,15918,1090,15918,942,6918,918,16918,596,15918,343,17918,171,6918,717,5919,195,21919,713,2919,752,15920,395,3920,223,13920,238,3920,276,42920,651,3920,746,3920,923,13921,1476,18921,1958,18921,1454,17921,1734,14921,1936,17921,1695,35921,1238,18921,1621,6921,1018,36921,990,20921,607,17921,527,17921,451,5921,285,17921,100,36921,1133,17922,1627,6922,1546,15922,1310,15922,1506,15922,249,39922,214,26922,348,10923,58,24923,84,3923,95,33923,174,3923,577,3923,794,3923,993,3924,2150,20924,2055,4924,1223,17924,2036,15924,169,30924,113,26924,267,14925,2273,17925,2069,18925,1854,17925,1895,31925,1568,15925,1444,24925,202,17925,1790,19926,840,12926,1343,12926,1189,12926,1481,5926,888,5926,714,3926,221,23926,581,12926,519,12926,311,11926,187,13926,102,21926,675,12927,1126,9927,1057,23927,963,19927,843,24927,871,22927,514,38927,366,19927,315,9927,184,18927,812,9928,1288,5928,1063,6928,1010,5928,544,15928,471,15928,328,5928,234,6928,214,15929,237,20930,1227,28930,476,29930,328,9930,314,9930,424,29930,220,21930,121,27930,110,3930,87,21930,252,3931,65,18931,118,20931,143,24931,1971,18932,1371,2932,1097,2932,935,2932,688,4932,254,2932,233,19932,822,2933,846,7933,1395,3933,1373,3933,1120,3933,1088,3933,996,3933,871,9933,717,8933,1436,3933,270,36933,429,3933,417,8933,308,5933,206,3933,178,26933,110,3933,78,30933,705,10934,919,17934,480,13934,213,8934,223,2935,1057,18935,1735,13935,1612,3935,1490,30935,1391,17935,1762,3935,1304,39935,1258,3935,1117,3935,1077,13935,993,22935,1374,3935,901,27935,169,29935,211,9935,315,3935,356,14935,200,3935,513,3935,784,17935,842,3935,407,3935,951,3936,940,16936,1484,11936,1371,21936,1339,17936,1154,17936,1060,19936,963,6936,889,31936,707,5936,588,25936,413,20936,375,5936,272,17936,136,26936,730,5938,782,14938,546,14938,166,22938,95,51938,447,14939,1393,3939,942,12939,912,15939,773,35939,750,15939,641,3939,463,24939,440,18939,250,3939,107,3939,70,35939,578,12940,1674,3940,1265,3940,1714,25940,79,27940,47,25940,1220,9941,775,3941,1667,3941,1508,3941,1472,4941,1399,14941,1360,20941,1336,19941,1290,4941,1160,8941,582,4941,924,19941,437,4941,327,17941,289,16941,235,3941,202,30941,180,4941,140,37941,118,3941,101,5941,528,3942,1113,14942,1087,3942,1001,32942,661,14942,581,3942,562,14942,148,27942,296,10942,267,3942,137,3942,107,28942,353,3944,341,2944,618,23944,364,2944,188,2944,573,28944,85,2944,120,17944,63,20944,162,12945,769,30945,670,20945,656,12945,637,14945,581,8945,411,21945,388,21945,372,14945,187,22945,540,10946,358,6946,902,37946,808,6946,410,7946,387,12946,349,3946,132,30946,321,17946,192,27946,165,3946,96,3946,341,3947,1371,2947,1737,2947,1589,2947,1479,2947,1340,2947,245,16947,495,2947,315,2947,263,2947,1083,2948,1843,3948,1782,9948,1668,3948,1258,3948,585,13948,516,3948,489,2948,371,3948,292,7948,276,11948,209,25948,194,13948,141,23948,131,3948,108,21948,301,38949,803,19949,1025,2949,450,19949,323,28949,370,25950,131,13950,149,14950,203,16950,361,21951,1570,5951,1078,2951,939,2951,792,9951,803,2952,169,14952,297,14953,1547,16953,1104,9953,809,15953,709,26953,677,15953,243,7953,324,32953,263,34953,200,27953,538,15954,493,2954,1455,21954,627,5954,611,5954,502,41954,462,2954,360,16954,383,5954,110,16954,158,16954,128,2954,395,26955,1258,26955,742,24955,656,18955,596,16955,520,19955,415,16955,317,9955,305,3955,123,41955,112,3955,80,30955,356,18956,1283,2956,1658,2956,1524,2956,1496,2956,1357,2956,1060,2956,236,2956,264,2956,185,26956,213,3956,219,15956,244,18957,1439,17957,1375,2957,776,3957,652,2957,592,3957,478,2957,554,2957,408,18957,371,3957,344,25957,163,8957,173,2957,443,20958,847,3958,734,3958,482,3958,444,3958,363,3958,121,4958,150,39958,95,24958,330,31960,152,21960,693,21961,1964,3961,1359,24961,840,24961,1910,14961,325,3961,299,24961,639,3962,87,3962,53,32962,276,3962,585,3962,884,3963,804,13963,1352,3963,1090,3963,873,3963,781,3963,1013,3963,600,3963,531,17963,366,3963,570,28963,181,25963,115,6964,271,30964,304,3965,185,26965,2019,26967,1566,22967,1456,12967,1349,9967,1282,23967,572,17967,710,12967,205,17967,1122,8968,605,24969,609,7969,1626,15969,1443,15969,500,14969,678,7969,163,10969,129,7969,97,30969,180,32970,507,15971,1486,2971,1292,2971,1176,2971,648,2971,619,2971,878,2971,456,2971,387,2971,317,2971,232,2971,93,2971,489,2973,322,18974,238,21974,369,28974,399,3974,735,3974,1042,3974,1318,3974,1841,3975,828,15975,310,25975,1126,15975,108,26975,145,28976,146,14976,414,22976,1220,6977,2039,37977,1860,16977,1363,16977,1655,16977,430,24977,332,54977,829,16978,1453,13978,2048,13978,1851,13978,1767,6978,1731,6978,1694,12978,1279,22978,2125,22978,969,13978,866,23978,718,13978,576,13978,453,25978,395,13978,293,13978,1153,6979,2084,6979,1973,8979,1693,6979,195,28979,174,4979,147,25979,495,6980,1025,18980,813,21980,776,8980,494,33980,163,29980,277,14980,130,16980,103,14980,464,14981,626,3981,630,3981,1276,3982,69,30982,101,3982,116,28982,472,3982,528,3982,581,20982,616,14983,1001,20983,862,4983,1258,27983,171,4983,141,28983,755,10984,56,15984,231,32984,326,15984,543,6984,578,6986,874,22986,319,32986,653,3986,97,6986,187,32987,1014,14987,1586,3987,1501,11987,1407,3987,1376,14987,1318,17987,1268,17987,1167,14987,1101,3987,1032,3987,917,3987,892,6987,870,17987,708,3987,676,16987,657,14987,456,14987,322,17987,286,18987,230,3987,202,26987,175,3987,143,30988,1325,3988,1172,3988,842,3988,503,26988,531,3988,753,3989,156,5989,351,21989,637,14991,680,6991,1053,34991,912,18991,606,6991,761,5991,217,2991,500,5991,203,12991,130,5992,1963,12992,1731,16992,1687,12992,1504,16992,1467,12992,1379,34992,1167,12992,999,12992,662,12992,506,12992,419,15992,285,21992,104,23992,239,40992,224,13992,129,3992,730,12993,502,11993,1171,4993,412,3993,897,18993,265,70993,111,14993,380,29994,124,3994,416,5994,448,26994,1232,13994,1335,12995,944,4995,901,38995,666,19995,289,4995,764,23995,230,4995,189,39995,137,40995,249,38996,246,28996,533,25996,598,6996,750,22996,846,14996,909,6996,1566,13996,1635,14996,1705,13997,926,19998,416,5998,371,5998,1598,5998,153,5998,122,29998,172,26999,509,181000,811,151000,770,31000,987,71000,566,71000,651,71001,1110,111001,1764,201001,1633,191001,1387,161001,1370,31001,1313,31001,1172,201001,1132,251001,966,31001,1727,31001,762,31001,620,161001,422,91001,343,211001,269,261001,234,311001,158,221001,136,171001,905,51002,160,171002,179,31002,222,151002,709,31002,798,31002,953,31002,1275,31005,107,231006,128,131006,1664,181006,1407,171006,1248,181006,881,131006,658,181006,1024,131006,542,141006,520,141006,459,71006,396,141006,606,141007,1702,201007,1579,181007,1492,171007,1280,61007,624,181007,845,61007,566,181007,538,231007,497,351007,386,171007,589,51008,260,101008,284,101008,482,461008,603,201008,985,201008,1672,261008,1828,251009,1223,21009,1144,191009,1115,111009,707,21009,269,21009,313,271009,254,131009,342,31010,300,61010,686,51010,1308,31011,739,231011,211,281011,250,351012,404,211013,273,191014,181,131014,389,131015,705,221015,774,211015,797,221015,1183,21015,1673,221015,1905,161016,1322,51016,874,31016,791,151016,752,171016,645,201016,531,21016,523,31016,468,31016,380,61016,390,261016,294,31016,261,21016,241,181016,232,31016,196,341016,440,261017,1202,141017,1509,31017,1440,31017,557,31017,91,291017,214,31017,171,191017,122,31017,456,31018,937,151018,1200,91018,1712,91019,1485,121019,1342,171019,982,121019,1376,121019,183,31019,150,311019,236,141020,108,281020,138,31020,228,91020,320,31020,581,31021,180,91021,1180,181022,709,181022,910,201022,891,131022,877,121022,630,141022,201,271022,180,41022,261,201022,441,131022,151,271026,153,31026,124,271026,690,31026,849,31028,110,151030,180,181032,1766,31032,1361,31032,1211,31032,977,31032,707,31032,680,31032,507,31032,463,31032,836,31032,384,31032,296,31032,268,261032,423,191032,251,51032,171,431032,160,31032,128,301033,1321,61033,1079,61033,959,61033,668,51033,892,61033,447,51033,947,31034,456,271034,808,31034,562,251034,508,31034,402,141034,1331,261034,354,211034,217,741034,169,351034,154,31034,133,191034,385,151035,795,171035,529,151035,507,161035,464,191035,196,171035,286,401035,136,161035,59,511035,328,51037,1196,161037,766,181037,162,161037,557,141038,106,181038,194,191038,1730,31038,1737,31038,2392,241039,120,241039,146,31039,299,151039,1657,31039,1710,241040,394,31040,1152,31040,246,31040,1209,161040,76,281040,53,161040,151,61043,1581,21043,1518,21043,1391,21043,1366,21043,1358,31043,1226,21043,1210,71043,1182,111043,1095,31043,1062,21043,866,21043,851,31043,696,21043,571,171043,535,21043,431,21043,423,31043,271,21043,264,31043,128,21043,108,181043,99,31043,85,121044,1697,51044,1612,251044,1353,51044,252,51044,195,51044,149,441044,1122,51045,1004,281046,1292,31047,183,411047,356,81047,415,81047,425,21047,1005,81048,750,241048,817,61048,269,291048,167,211048,458,291049,496,51050,493,141050,933,141050,1194,261050,1532,351050,1689,261052,1327,31052,1348,31052,1012,181052,1355,31052,1391,31052,829,31052,415,271052,720,251052,262,231052,491,271052,747,31052,566,31052,520,31053,191,311053,294,311054,162,291054,982,161055,2255,141055,2019,231055,1961,221055,1945,141055,211,221055,732,261055,191,151055,1810,161055,166,131056,1055,31056,1224,31056,1107,31056,431,31056,141,371056,117,31056,100,151056,194,31057,72,531057,214,211057,274,321057,492,161057,999,171057,1168,121058,1698,41058,1609,31058,1172,41058,675,121058,689,31058,206,381058,339,121058,259,391058,196,21058,178,161059,879,21059,1185,21059,1064,21059,635,21059,1138,21059,304,21059,257,21059,233,221059,532,21060,1632,71060,1554,31060,1508,151060,1472,111060,1452,181060,1405,181060,1111,31060,1425,251060,457,211060,392,71060,326,111060,288,181060,258,281060,134,271060,480,31061,1548,21061,217,81061,488,21061,515,21061,628,21061,657,21061,692,21061,1607,21062,425,271062,63,311063,1178,281063,1430,201063,1157,171063,978,201063,916,171063,785,201063,709,211063,1099,151063,638,201063,461,201063,346,171063,298,171063,277,171063,180,31063,671,171064,2952,21064,1957,191064,1978,31064,1899,141064,142,21064,517,21064,122,181064,3140,21064,1642,51065,707,41065,703,31065,1208,141065,596,311065,214,171065,269,41065,242,251065,233,31065,538,201066,151,221066,536,211066,559,21066,768,21066,1044,21066,1457,21066,1677,21067,1652,31067,1576,31067,905,31067,673,31067,589,31067,125,151067,216,391067,165,381067,142,31067,328,31068,669,31068,335,191068,595,191068,83,271068,112,31069,78,411069,126,491069,382,171069,515,121069,960,171070,454,221070,1069,301071,869,481071,715,171071,589,241071,563,171071,243,171071,181,211071,160,161071,77,171071,397,471073,2165,121073,2070,101073,2044,121073,1973,121073,1822,61073,1799,61073,1742,141073,1221,61073,1310,51073,332,31073,434,51073,304,261073,247,281073,159,591073,1715,211074,131,131075,1558,91075,1485,141075,1338,211075,1309,241075,1215,101075,1203,61075,1501,141075,1094,141075,1026,161075,770,171075,186,371075,124,451075,1137,181076,1696,151076,1736,171076,1792,31076,1909,31076,1933,291076,1978,231076,2136,171076,2170,111076,2745,171076,2361,171076,2485,31076,2509,111076,2595,111076,2626,171076,2714,111076,1442,31076,2256,231076,1241,241076,2429,171076,1138,151076,166,241076,201,171076,1184,311076,364,311076,411,121076,435,171076,459,261076,498,231076,319,311076,588,461076,742,161076,778,51076,796,111076,895,31076,907,111076,938,171076,543,31077,241,301077,217,31077,193,221078,81,181078,101,21078,142,181078,265,141078,284,191079,1275,21079,1004,21079,495,21079,686,21079,360,281079,254,21079,233,191079,426,21080,1903,61080,1616,61080,1005,41080,601,61080,447,41080,434,111080,201,201080,586,21081,1307,181081,1266,361081,1206,211081,1097,311081,793,311081,578,31081,417,131081,238,291081,194,251081,184,31081,154,281081,508,31082,266,121083,250,281083,280,31083,365,31083,385,141083,1157,151083,1686,141084,736,171084,801,151084,633,151084,297,151084,399,201084,69,531084,441,351085,66,311085,99,51085,720,271086,283,201086,308,201086,707,161087,190,261087,160,231087,348,101087,121,291087,152,31088,224,151089,81,221089,131,131089,987,131090,428,51090,1091,31090,667,51090,654,31090,502,31090,384,31090,515,51090,294,171090,235,221090,179,161090,144,51090,110,311090,349,271091,1778,481091,1737,31091,1621,31091,1523,31091,1499,191091,1333,31091,1183,31091,1054,271091,609,31091,469,31091,418,31091,282,351091,276,31091,249,251091,985,171092,719,31092,1025,111092,889,111092,835,111092,817,31092,572,31092,1003,31092,525,31092,153,31092,179,111092,551,111092,363,31092,348,111092,469,111093,620,211093,939,61093,1097,131093,1416,51093,1454,221093,1631,191094,507,81094,426,201095,123,301095,279,81095,324,101095,1270,301095,155,31096,53,261096,81,31096,113,191096,211,391096,489,31096,1136,31098,1164,211098,1485,81098,1457,211098,1377,211098,1090,211098,827,191098,759,391098,286,191098,870,111099,282,241099,309,31099,925,221099,951,31100,129,51100,183,301100,218,671100,347,31101,1793,231101,1705,91101,1386,151101,423,231101,213,21101,180,311101,1115,91103,1341,361103,2129,291103,2112,111103,2082,241103,2064,161103,1975,351103,1876,261103,1571,221103,1417,231103,1442,31103,1310,291103,670,191103,1101,241103,1042,381103,977,141103,796,361103,765,291103,736,31103,1130,241103,656,31103,590,171103,269,221104,1885,71104,1907,211104,1932,171104,1957,121104,1983,151104,2043,121104,2582,191104,2387,161104,2405,401104,2451,231104,1839,81104,2692,81104,2790,191104,2353,81104,1795,81104,1488,201104,1621,111104,176,191104,302,211104,497,191104,548,61104,628,191104,1711,71104,907,71104,1026,81104,1156,81104,1226,61104,1433,161104,1550,141104,989,61105,1032,31105,1724,31105,1526,31105,924,31105,1614,31105,633,31105,208,221105,779,31105,232,31106,1643,331106,130,251106,179,241106,562,241106,824,221106,942,191106,1059,341106,1279,221106,1305,31107,171,121107,114,521107,64,221108,392,141109,162,101109,489,121109,744,121109,826,121109,906,121109,1231,121110,514,31110,1581,31110,1605,91110,281,31110,124,31110,94,281110,380,31111,1384,31111,714,31111,699,31111,561,31111,1388,31111,526,31111,497,271111,133,41111,534,251112,1464,141112,140,131112,189,131112,248,131112,330,131112,588,131112,767,151112,1042,131112,1094,231112,1246,131112,1349,141113,508,151113,258,141114,196,51114,325,61114,340,181116,177,141116,1573,221117,1805,251117,1719,61117,1659,171117,1410,151117,1392,21117,1274,21117,1051,71117,947,61117,813,21117,805,21117,708,61117,585,211117,531,251117,349,21117,327,201117,315,71117,193,71117,130,251119,1945,31119,1634,31119,1260,31119,1090,31119,579,251119,166,271119,277,31119,458,31119,124,31119,94,281120,101,171121,207,271121,236,21121,324,21121,716,21122,165,361123,314,131123,143,541124,142,271124,171,31124,191,141124,238,211124,586,31124,1828,271125,360,201125,864,211125,818,241125,389,211125,336,201125,313,211125,265,231125,249,121125,217,201125,97,121126,1192,241126,1183,41126,1178,31126,1149,271126,1132,151126,1097,141126,1000,191126,803,291126,658,261126,594,41126,562,301126,522,271126,487,171126,375,171126,44,141126,787,91127,1078,161127,945,191127,1100,121127,908,151127,258,361127,925,181128,590,311129,314,161130,1540,151130,1211,151130,1102,151130,1027,151130,1317,151130,351,131130,301,251130,277,151130,136,191130,398,131131,1411,181131,1181,161131,1038,181131,857,91131,708,181131,466,181131,122,181131,584,181132,980,31132,1400,121132,1082,31132,1066,121132,912,121132,195,221132,443,301132,655,31132,336,201132,280,171132,219,31134,1616,221134,1570,221134,1517,241134,1331,221134,1017,81134,995,161134,1258,321134,940,151134,759,161134,746,81134,570,221134,416,221134,197,221134,966,81135,787,41135,178,41135,214,171135,145,311136,79,221136,108,281136,367,121136,1302,121137,1391,201137,327,161137,253,41137,225,261137,278,181138,626,71138,641,231139,279,211139,361,211139,400,251139,903,211139,1203,321139,1239,61140,218,181140,1852,61140,1829,211140,1084,21140,353,21140,145,171140,173,271140,164,21140,238,211141,1082,121141,1050,271141,1190,271141,1035,131141,1459,101141,1724,211141,1474,31141,1450,31141,869,171141,296,291141,707,161141,689,131141,674,131141,654,181141,374,31141,343,291141,327,31141,179,391141,825,271142,208,141142,128,491143,84,51143,303,51143,484,61143,595,251143,737,61143,1009,221143,1039,71144,1675,31144,1603,31144,1503,31144,1199,71144,545,31144,356,31144,131,141144,867,31145,318,141145,335,151145,357,201146,1909,101146,1847,31146,1364,31146,1131,141146,255,201146,368,31146,337,291146,277,51146,912,31147,1320,31147,1428,31147,1390,31147,595,101147,572,31147,424,141147,231,481147,152,581147,141,31147,120,191148,99,191148,293,191148,478,191149,1549,31149,1229,21149,184,281149,214,31149,710,31150,1555,191150,1737,181150,1593,271150,1529,241150,252,41150,751,191150,333,411150,223,271150,1210,191151,878,121151,1804,251151,1715,121151,1621,111151,1471,31151,1242,111151,1209,281151,1189,151151,841,111151,501,121151,648,111151,641,31151,423,31151,376,31151,321,481151,274,31151,243,291151,159,311151,767,31152,945,61152,777,131152,707,211152,687,151152,627,261152,210,171152,344,181152,192,131152,169,141152,118,251152,537,61153,956,121153,1486,121153,1396,81153,1366,121153,1159,121153,866,121153,290,101153,618,81153,517,81153,475,81153,788,81153,200,121154,961,31154,735,131154,365,31154,348,131154,213,31154,182,291154,147,221154,228,501155,507,31155,1200,31155,1035,31155,682,31155,623,31155,326,31155,1124,31155,258,61155,203,171155,145,281155,134,31155,111,211155,266,191157,146,261157,174,21157,183,151157,351,361157,389,231157,779,21158,930,131158,845,131158,765,71158,516,131158,448,51158,260,171158,568,171159,667,141159,1247,81159,1206,131159,1185,91159,1092,211159,1078,121159,1049,131159,862,471159,687,281159,649,151159,73,251159,462,171159,631,111159,37,21159,100,31159,22,131159,265,451159,218,271159,398,21159,453,21160,129,191160,215,181160,261,231161,105,251161,163,181161,227,281161,540,91161,623,181161,743,41161,1227,271162,340,241162,1370,251162,127,281162,157,31164,2078,181164,2045,251164,1960,31164,1811,201164,1703,91164,1686,101164,1665,181164,1283,31164,1091,31164,1076,121164,1003,131164,283,281164,156,581164,1455,31165,1101,41165,1497,41165,494,31165,1421,61165,319,181165,280,151165,214,441165,409,151166,535,351166,530,31166,231,281166,215,151166,389,291167,169,81167,198,251167,391,301167,862,251167,904,201167,929,61167,1104,81170,1552,261170,1389,61170,1343,131170,1055,61170,964,71170,1112,61170,600,121170,425,251170,345,61170,120,251170,744,141171,1473,41171,1403,141171,1379,41171,1188,41171,1180,41171,1027,181171,981,41171,861,41171,543,41171,516,131171,508,41171,482,121171,425,41171,384,391171,126,291171,157,41171,755,41172,1244,271172,1203,211172,1186,151172,1273,211172,1312,151172,1167,171172,1296,141172,594,151172,1226,161172,551,211172,522,271172,504,161172,574,141172,481,211172,464,151172,449,131172,131,271174,1439,61174,746,241174,615,111174,366,31174,506,61174,134,191174,124,31174,100,221174,180,541175,113,531175,193,211175,291,131175,322,121175,366,241175,523,121175,890,131176,916,151176,974,521176,804,151176,528,201176,45,151176,691,151177,166,31177,284,171177,302,211177,326,31177,408,31177,563,31178,371,141178,1234,31178,426,31178,358,31178,86,31178,57,271180,705,31180,773,51180,1310,271180,1375,161180,1501,161181,136,251181,1374,51181,1408,161183,876,201184,1582,181184,2885,181184,1181,31184,1160,191185,88,251185,147,61185,222,51185,367,251185,497,251188,178,391188,219,41188,1583,41189,301,21190,960,31190,289,251190,261,31190,162,241191,1707,21191,1676,161191,1582,31191,1324,31191,1261,161191,1204,21191,1118,31191,1043,161191,977,261191,861,161191,931,251191,696,31191,764,21191,146,161191,215,261191,246,251191,169,281191,377,161191,536,161191,596,251191,623,21191,359,141192,700,221192,1172,131192,1095,281192,1018,201192,899,131192,603,171192,327,31192,550,271192,479,131192,182,31192,111,41192,78,311192,579,221193,792,211193,1061,151193,1018,31193,1209,121193,824,781193,716,221193,191,171193,406,21193,131,191193,152,31193,529,781193,274,211193,255,131194,1449,31194,1309,31194,1287,201194,1279,21194,1261,161194,786,131194,669,131194,553,111194,535,141194,1486,131194,471,131194,505,211194,113,111194,126,81194,136,271194,1429,21194,203,131194,281,221194,169,181194,345,111194,395,81195,779,141195,1409,51196,437,21196,720,21196,176,181196,197,21197,1844,31197,1632,31197,1497,201197,1146,31197,1382,31197,681,151197,270,31197,116,201197,813,31197,454,31198,1027,61198,1988,141198,1876,241198,1828,171198,1269,61198,1126,61198,953,61198,539,61198,508,51198,433,251198,281,141198,146,51198,118,251198,915,61199,1497,61199,1459,51199,1329,31199,1321,31199,1248,31199,1207,31199,1123,31199,1062,31199,672,261199,1574,51199,583,31199,575,31199,438,221199,415,61199,389,221199,357,51199,278,51199,224,111199,623,301200,1819,151200,1791,241200,1677,181200,1383,101200,1360,101200,1303,241200,1143,101200,1046,251200,973,171200,769,171200,195,101200,103,241200,89,71200,1117,211202,806,61202,355,171202,282,141203,1788,201203,358,21203,298,131203,473,21203,523,201203,716,21203,980,21204,417,191204,401,91204,529,161204,506,191204,236,31204,619,201204,163,51204,104,191204,94,31204,64,281204,173,91206,1666,31206,1147,31206,1137,31206,886,101206,827,71206,740,61206,213,171206,479,31206,292,31206,270,201206,232,31206,501,31208,207,241208,937,291208,1031,241210,1669,21210,1496,21210,1350,21210,1151,221210,659,21210,947,21210,415,21210,378,221210,176,21210,154,201210,473,21211,283,141211,299,91211,500,151211,1118,311211,1237,311211,1306,151213,1591,21213,1937,21213,1748,221213,1741,21213,1528,171213,304,251213,639,21213,579,21213,331,21213,278,161213,1428,21215,1286,31215,1092,161215,1043,81215,1032,81215,986,31215,787,231215,623,131215,527,31215,460,231215,244,31215,220,211215,747,31216,190,1021217,688,131217,1233,131217,1085,131217,876,131217,536,131217,393,131217,422,51217,451,131217,304,131217,160,131218,171,271218,1166,31218,1149,151219,1519,51219,179,41219,191,281219,132,451219,334,151220,924,201220,1289,171220,1200,271220,1065,181220,1052,81220,609,451220,1543,171220,369,31220,236,341220,192,391220,162,161220,129,31220,481,31221,1795,61221,1689,211221,1668,161221,1595,61221,1206,91221,1650,161221,746,131221,722,81221,584,171221,509,161221,147,61221,783,211222,213,131222,671,141222,149,141224,136,141224,338,221224,362,31224,436,31224,567,31224,694,181224,716,31225,1677,131225,679,131225,557,201225,304,201226,94,81226,136,361226,261,151226,336,191226,681,381226,974,211227,680,31227,640,121227,615,201227,588,251227,564,221227,285,101227,260,201227,180,31227,148,301227,553,91228,629,21228,1161,151228,618,91228,728,21228,524,131228,745,21228,290,131228,263,211228,254,71228,122,171228,97,181228,494,181229,1290,31229,1065,31229,1044,191229,797,31229,719,31229,995,31229,384,31229,357,131229,159,461229,148,31229,131,151229,421,31231,71,21231,520,21231,55,141232,186,131232,460,131232,1636,131233,1397,31233,1898,31233,1656,151233,1630,211233,1476,201233,1046,51233,826,31233,941,31233,332,31233,258,31233,138,31233,115,211233,1037,31235,417,151235,623,301235,898,91235,945,31237,904,171237,1034,261237,1118,171237,1187,31237,1231,191237,1558,31237,1324,141237,1452,31237,1477,241237,1593,131237,1633,341237,1268,141237,627,31237,840,31237,548,361237,593,61237,180,261237,231,171237,276,301237,308,31237,208,31237,373,31237,393,171237,416,31237,480,261237,518,191237,340,191238,101,381238,170,271238,199,31238,264,141240,230,261240,1128,151241,112,191241,81,181242,1145,161242,1054,211242,1239,321242,1015,181242,193,641242,363,201242,548,31242,299,211243,1782,151243,1730,181243,1697,161243,1372,141243,1185,431243,896,201243,883,111243,626,331243,786,371243,508,391243,490,161243,473,151243,453,181243,391,181243,296,291243,176,291243,76,311243,605,191244,681,61244,920,71244,878,171244,638,171244,802,171244,429,51244,254,121244,560,171244,285,171246,77,181246,97,21246,357,21246,704,21246,898,21247,769,21247,1930,21247,1754,301247,1723,21247,1644,21247,1548,21247,619,221247,1216,21247,643,21247,1995,21247,368,191247,1448,21248,974,21248,130,161248,1831,311248,608,21248,1636,181248,523,211248,121,21248,228,31248,204,221248,162,121248,493,251248,103,161249,67,201249,94,451249,262,181249,455,201249,836,151249,873,201249,1023,201250,1091,21250,899,21250,805,21250,643,21250,1409,21250,477,31250,203,21250,104,21250,85,171250,568,21251,1548,21251,1492,31251,1460,31251,1088,41251,1025,31251,982,41251,892,41251,861,41251,777,21251,743,241251,694,121251,514,31251,489,21251,475,111251,444,211251,422,31251,400,191251,381,31251,358,201251,1598,21251,726,31252,644,211252,354,81252,105,211252,299,121253,221,221253,245,21253,571,21253,654,21253,790,221253,859,21253,1332,21254,495,21254,1014,21254,214,231254,473,201255,139,231255,412,271255,500,121255,514,391255,753,131255,768,191255,1170,241256,751,251256,1463,21256,838,131256,816,21256,711,351256,226,21256,667,21256,206,181256,121,491256,78,181256,695,21257,1353,241257,1278,191257,1098,211257,1010,171257,676,201257,698,31257,559,111257,195,151257,165,131257,929,211259,438,31259,100,231259,125,31259,157,241259,186,221259,225,141260,451,241260,1294,241260,218,631260,283,41261,319,181262,35,211262,103,191262,127,221262,350,341263,1782,231263,1718,221263,1576,261263,1551,201263,1527,181263,1506,151263,1224,261263,1122,161263,787,111263,847,141263,481,131263,347,291263,268,201263,244,181263,185,361263,153,151263,994,181264,1321,151264,1219,171264,836,171264,379,151264,66,151264,525,151265,45,21265,49,181265,74,371265,145,181265,336,21265,411,201265,480,21266,852,231266,284,211266,120,301266,250,131267,183,261267,211,31267,274,171267,293,31267,698,31267,986,31267,1755,31268,465,51268,972,171268,705,171268,507,31268,545,171268,421,421268,475,301268,169,61268,115,171269,660,21269,596,141269,340,131269,317,61269,274,21269,96,281269,226,61269,77,171269,63,21269,40,211270,216,131270,311,211270,335,31270,1270,31271,1592,301271,1482,171271,1470,101271,1453,151271,1432,101271,375,171271,452,261271,337,101271,160,131271,128,261271,1336,151272,185,271272,742,31273,1187,31273,1283,31273,1179,31273,1276,31273,872,31273,137,331273,128,31273,906,31274,1747,171274,1527,171274,1447,171274,1407,91274,1391,81274,1283,61274,936,61274,788,61274,829,61274,590,61274,422,171274,363,21274,315,171274,234,311274,131,231274,112,21274,92,181274,700,61275,201,161276,1138,21276,1464,21276,1091,21276,1541,21276,1749,21276,1494,21276,975,21276,247,221276,739,21276,624,21276,543,21276,473,21276,271,21276,857,21277,1387,191277,1345,231277,920,31277,860,81277,623,141277,548,491277,391,61277,319,31277,291,261277,231,121277,222,31277,190,301277,496,31278,1136,31278,1712,31278,1728,281278,805,31278,209,31278,559,31278,172,351278,620,211280,1194,81280,1120,21280,917,21280,821,21280,857,21280,323,21280,307,141280,173,81280,437,81281,302,141281,393,301282,705,271282,1688,31282,1471,31282,612,31282,1149,31282,135,161282,452,31282,102,211282,767,31282,125,31283,96,321283,303,191283,342,301283,374,31283,415,81283,480,91283,808,191283,867,31284,233,151284,1213,91284,1238,131284,1261,241285,1373,31285,924,31285,1702,211285,919,31285,1643,31285,637,301285,705,301285,388,271285,380,31285,357,211285,253,61285,235,131285,557,31286,135,401286,911,181287,216,201287,1027,181287,911,51287,552,41287,536,121287,127,51287,253,51287,1149,51287,96,291287,467,51287,1313,51288,1030,131288,1090,31288,848,131288,1278,291288,1205,31288,707,141288,587,21288,564,211288,546,131288,517,181288,475,31288,316,71288,220,261288,215,31290,145,141290,161,31291,677,211291,589,301291,453,31291,162,281291,128,211291,342,71292,198,291292,314,31292,414,201292,549,41292,697,191292,726,201292,1068,391292,830,201293,518,101293,1103,221293,850,151293,246,131293,141,221293,86,401293,360,141295,1856,131295,1774,241295,1393,151295,1634,131295,233,531295,214,131295,129,631295,809,141296,523,71296,1084,71296,979,51296,712,191296,583,171296,450,61296,1096,171296,246,171296,234,71296,188,301296,111,411296,73,181296,437,81297,906,191297,1755,181297,611,181297,473,41297,240,271297,335,171297,209,181297,174,341298,95,131298,119,281298,259,241298,459,131298,1148,131298,1412,131299,1221,31299,994,201299,926,51299,810,201299,453,201299,229,201300,128,181300,148,31300,175,281300,561,31301,1380,21301,939,21301,851,21301,617,371301,322,21301,435,21301,168,21301,148,181301,565,21302,1567,31302,1457,181302,1408,321302,1214,31302,1133,101302,641,31302,492,31302,384,141302,196,161302,164,31302,135,271302,431,151304,739,191304,1076,201304,772,141304,995,41304,716,41304,300,81304,250,231304,146,41304,105,391304,386,41305,1537,21305,1502,131305,1407,151305,1341,131305,1135,21305,1119,21305,838,21305,777,181305,746,21305,1520,131305,605,21305,549,21305,435,211305,389,181305,372,21305,341,291305,161,181305,134,191305,722,21306,1104,261306,916,131306,860,31306,607,361306,562,131306,327,31306,234,21306,210,221306,167,281306,152,31306,124,261306,260,131307,63,181307,115,281307,181,281307,584,181307,700,221307,822,181307,916,181308,1129,231308,1175,161308,1242,221308,1197,91308,1439,31308,1492,201308,1516,201308,1074,61308,1349,201308,1031,201308,1158,81308,948,61308,141,251308,168,31308,179,361308,221,131308,500,181308,977,31308,565,31308,522,31308,659,181308,716,61309,134,61309,145,171310,744,31310,1322,121310,234,141310,718,241311,163,201311,185,21311,1109,211311,1302,261311,1420,361311,1531,201312,1233,31312,1181,31312,695,31312,457,31312,643,31312,199,141312,248,31312,78,311312,135,31312,111,221313,1357,31313,140,381313,356,31313,412,51313,468,31313,959,31313,1206,41313,1589,31314,1031,71314,940,171314,705,171314,96,131314,396,231314,162,261314,559,181315,1021,61315,1132,31315,1112,61315,1628,61315,687,61315,652,121315,278,161315,666,41315,422,31315,296,31316,328,71316,361,321316,839,171316,1133,21316,1460,31316,1717,71316,1729,151316,2062,401319,2241,51319,2169,41319,851,31320,153,261320,1322,151320,1438,31321,1517,181321,1463,201321,1440,191321,1290,201321,1274,81321,968,201321,749,181321,603,181321,565,81321,423,201321,400,151321,375,231321,227,231321,202,201321,171,181321,161,81321,581,201322,1983,81322,1851,81322,1569,161322,1543,161322,1526,111322,1240,211322,1226,61322,930,281322,544,81322,801,51322,659,241322,415,241322,367,181322,220,241322,100,241322,2051,241322,865,111323,471,221323,461,31323,668,31323,432,271323,846,31323,340,241323,218,241323,112,141323,244,141323,132,161324,101,141324,181,181324,493,311324,526,31324,1647,31325,58,161325,93,311325,427,251325,735,161325,1071,161326,1437,171326,1367,271326,1344,121326,616,301326,1171,201326,136,141326,1224,211327,902,171327,137,251327,190,151327,352,121327,768,141327,786,71327,923,251328,332,251329,668,251329,698,161329,888,31329,1264,241329,1631,111330,143,151330,319,191331,1118,181331,1030,181331,1565,181331,648,81331,100,181331,696,271332,158,271332,206,181332,275,271332,350,141332,367,21332,507,231332,627,361332,751,131332,791,21332,1169,21333,1933,101333,620,501333,113,111333,527,581335,114,181335,135,21335,161,171335,269,21336,1274,21336,729,21336,531,21336,1281,181336,311,21336,165,21336,144,181336,376,21336,931,21337,107,221337,183,221337,332,91337,475,91337,657,91337,688,201337,1152,201338,1485,131338,1088,231338,350,211338,1846,211338,1045,201338,1686,231338,1004,201338,158,251338,776,21338,651,151338,610,131338,572,231338,252,151338,784,201339,173,331339,209,41339,1164,151339,1183,41339,1478,41339,1712,41340,1312,301340,1608,191340,1221,31340,1661,121340,1810,31340,1641,141340,839,31340,1574,31340,309,111340,140,211340,163,31340,173,241340,431,31340,288,71340,212,191341,153,521341,223,201341,245,261341,276,211341,326,401341,583,131341,645,131342,1749,21342,1601,91342,263,281342,316,251343,242,291343,273,41343,280,621343,1192,61343,1446,401344,768,21344,835,181344,406,31344,938,151344,997,241344,859,211344,588,301344,323,191344,383,211344,136,41344,147,331344,94,401344,620,21345,918,151345,901,151345,855,131345,604,241345,836,141345,392,161345,156,461345,92,571345,551,161346,252,291346,283,21346,536,181346,556,31347,1180,211348,481,71348,599,71348,1925,71348,276,141348,201,71348,173,251348,217,461349,1026,21349,921,111349,835,21349,769,21349,578,21349,1179,111349,436,31349,328,261349,282,201349,99,31349,69,281349,481,21350,179,161351,772,211351,706,181351,578,31351,474,271351,316,31351,285,281351,504,31352,1185,181352,1624,51352,1528,51352,1390,181352,1660,81352,1105,161352,425,281352,567,261352,176,31352,207,51352,148,261352,1052,181353,105,281353,140,281353,196,361353,237,91353,926,31354,1611,81354,674,81354,227,271355,208,141356,1470,161356,1726,161356,1708,71356,1687,161356,1660,21356,1636,21356,1546,101356,1519,71356,1273,141356,1389,161356,147,281356,1085,161356,1002,191356,628,61356,539,61356,477,211356,439,21356,201,121356,1215,241357,143,251357,608,251358,517,141358,690,51358,1188,21358,984,51358,499,131358,697,31358,489,81358,144,31358,310,111358,181,301358,176,21358,163,111358,103,391358,406,31358,1340,111359,270,371361,174,181361,195,21361,242,21361,325,21361,396,21361,653,21361,896,21361,954,21361,1179,21362,1320,151362,1169,131362,1135,301362,1010,141362,912,141362,714,141362,404,361362,505,321362,462,71362,1726,141362,341,141362,132,251362,642,281363,576,31363,326,31363,1116,31363,249,31363,129,211363,178,231363,100,281363,204,271363,153,31364,1463,201364,1275,31364,1166,201364,405,21364,1359,201364,383,201364,337,201364,86,31364,476,201364,1254,191365,514,31365,1437,31365,1408,271365,1256,31365,1200,31365,1006,251365,975,31365,566,241365,547,31365,371,31365,291,111365,269,111365,255,121365,235,181365,193,181365,135,201365,107,31365,84,211365,310,71366,474,61366,430,141366,189,141366,327,141367,71,131367,86,31367,96,261367,202,501367,299,31367,459,31367,667,31367,1413,31368,1079,241368,318,151368,1134,151368,101,151368,132,161369,1621,31369,1574,31369,756,31369,1539,31369,1498,31369,1337,31369,433,31369,153,31369,290,191369,258,31369,230,261369,164,381369,121,301369,354,151370,346,31370,959,21370,1252,21371,826,41371,806,181371,128,161371,91,181372,137,191372,158,31372,166,161372,292,671372,391,391372,479,251372,533,331373,593,81373,507,281373,190,81373,103,301373,43,21374,1701,71374,1604,51374,1425,61374,1401,51374,1360,51374,1346,31374,1290,61374,1224,101374,1129,121374,1104,101374,1013,61374,868,61374,783,101374,763,151374,638,121374,602,51374,471,51376,721,121376,442,31376,368,201376,681,31376,303,31376,279,221376,328,121377,1418,171377,1078,191377,569,181377,366,81377,591,81377,263,81377,187,191377,177,81377,64,171377,287,81378,158,131378,278,131378,385,321378,438,131378,464,211378,521,221378,634,131379,223,271379,167,281379,291,131379,73,251379,100,31380,91,141380,107,31380,118,291380,393,31381,1327,131381,801,31381,268,31381,234,321381,93,31382,2022,61382,2046,101382,1983,61382,1940,31382,1433,31382,1300,31382,407,61382,817,31382,594,31382,461,31382,433,261382,308,181382,2225,31382,1175,361383,604,61383,572,61383,881,81383,777,31383,312,31383,213,251383,240,31383,380,61383,184,31383,161,211384,73,211385,1849,171385,1752,201385,1701,51385,1648,171385,1383,61385,1358,231385,1093,251385,1143,51385,731,171385,612,171385,574,41385,518,171385,443,171385,158,191385,1075,91386,775,151386,1861,51386,1539,221386,1163,221386,1151,31386,1082,211386,806,31386,702,211386,116,751386,661,31386,638,211386,585,121386,394,351386,93,51386,69,221386,667,171387,118,181387,171,141388,857,351388,1537,31388,1522,31388,1179,61388,1092,91388,1042,31388,820,201388,1450,31388,522,31388,107,301388,139,31388,185,171388,697,221388,409,31388,437,31388,232,31389,828,61389,814,91389,349,191389,262,361389,532,351389,194,221389,137,281389,114,161389,218,61390,64,301390,374,31390,387,171390,603,101390,739,191390,763,151390,844,101392,609,231392,1306,31392,1186,31392,1005,31392,1390,31392,417,31392,244,41392,216,261392,142,571392,387,281393,191,241393,934,141393,1000,141394,715,21394,348,91394,303,401394,380,21394,112,221394,102,21394,74,261394,282,91396,1984,71396,1956,191396,1868,271396,1829,171396,1011,191396,691,191396,465,171396,154,171396,896,191397,1718,121397,1555,121397,1575,31398,250,231398,171,261399,72,31399,77,281399,110,361399,194,181399,664,221400,1080,21400,1029,21400,971,21400,768,21400,579,21400,482,21400,630,21400,343,21400,216,171400,174,171400,136,21400,115,181400,388,21401,89,291401,120,41401,659,201401,707,71401,716,131402,1281,151402,1548,151402,1437,291402,1258,131402,363,151402,813,141402,644,151402,260,181402,932,151403,116,351403,191,821403,153,41403,440,191403,550,191403,880,271404,662,31404,326,31404,322,31404,189,271404,226,251404,218,31404,253,31405,1141,21405,1602,21405,1256,21405,672,21405,736,171405,553,21405,125,21405,135,541405,99,241406,158,201406,228,61406,538,161406,717,71406,774,141406,850,61407,1675,171407,1520,21407,430,21407,1662,21407,138,21407,118,181407,261,21408,425,161408,447,261408,1113,91408,1555,161408,1930,191409,961,31409,1080,31409,1244,121409,1261,191409,209,141409,1426,31409,1483,121409,842,121409,1318,31409,778,51409,104,31409,632,31409,558,161409,513,121409,459,31409,341,161409,173,311409,80,221409,745,31410,104,251410,140,361410,285,251410,1181,151411,1336,161411,898,161411,1550,151411,197,241411,170,161411,223,31411,1692,151412,949,131412,366,211413,656,31413,846,31413,945,31413,1249,31415,1175,151415,1527,361415,237,151415,257,211416,165,201416,238,201417,768,31417,287,211417,311,31417,380,31417,555,31418,966,101418,1257,121418,336,121419,888,101419,88,511419,618,151419,727,151419,993,161419,1011,51419,1169,151420,825,71420,95,151421,152,51421,259,181421,512,151421,693,191421,833,191422,836,31422,888,31422,992,31422,1118,81422,1368,221422,1479,41422,1526,191422,1631,81422,1765,31422,807,41422,1471,31422,793,31422,924,41422,676,21422,179,301422,211,31422,228,301422,260,41422,278,191422,696,21422,571,191422,618,171422,442,31422,643,81422,653,21423,1051,81423,1027,191423,182,81423,158,191424,510,41425,284,141425,769,141426,612,491427,144,281427,174,31428,1207,41428,1114,41428,1013,41428,943,351428,868,41428,415,41428,378,41428,305,181428,173,41428,134,371428,683,41429,242,171429,560,61429,621,61429,1495,151430,1544,31430,559,31430,435,31430,252,31430,127,31430,97,281430,138,281431,655,211431,76,291431,112,271431,219,211431,290,171431,509,211431,547,241431,605,291432,1247,111432,1602,111432,96,161432,118,181434,1229,171434,1189,221434,954,251434,861,161434,703,291434,389,141434,135,161434,783,251435,175,81435,297,81435,756,81435,991,81435,1079,281435,1230,81436,933,71436,608,121436,577,111436,102,51438,214,211439,113,211439,248,211439,283,201440,1621,211440,1611,71440,1567,181440,1060,151440,1504,141440,838,71440,718,71440,463,211440,407,151440,1019,71441,1356,71441,1165,101441,673,351441,498,161441,738,71441,436,181441,216,101441,164,81441,149,101441,464,271442,1112,81442,1084,231442,1055,31442,975,31442,929,211442,893,221442,835,141442,820,61442,696,161442,600,231442,645,31442,505,31442,433,31442,368,31442,281,231442,228,391442,207,191442,161,281442,150,31442,132,161442,580,31443,1318,61443,942,61443,918,41443,299,261443,619,301443,457,181443,651,41444,1392,61444,1111,291444,1026,381444,968,21444,579,21444,780,21444,225,181444,193,21444,173,181444,78,21444,50,261444,378,151445,2116,41445,1438,321445,1407,181445,1581,211445,1100,231445,211,41445,155,541445,1231,191448,284,61449,187,291449,554,151449,583,271449,982,151450,179,151450,196,21450,276,21450,376,21451,627,51451,916,61451,462,341451,1542,61451,229,121451,199,61451,143,541451,400,61452,1346,151452,170,81453,166,201453,188,21453,209,261453,559,21453,921,21453,1267,21453,1375,21453,1807,21454,1250,281454,1224,151454,1158,281454,1061,121454,993,151454,947,151454,540,121454,505,71454,445,71454,295,71454,88,231454,930,121455,283,61456,932,131456,1739,31456,1575,31456,1409,41456,923,31456,877,181456,392,41456,272,41456,245,251456,897,241457,1163,31457,183,241457,209,31457,787,31457,873,191457,1000,121457,1336,31458,1017,31458,511,31458,281,231458,62,171458,81,31458,1300,171458,113,261459,1223,201459,1072,201459,1421,171459,1346,71459,969,151459,1058,61459,797,171459,652,91459,496,171459,349,261459,209,171460,167,191460,193,281460,264,61460,466,61460,932,181461,405,151461,340,71461,329,61461,178,401461,100,391461,79,151462,80,241462,106,31462,125,161462,143,61462,155,161462,574,31463,1259,21463,1116,21463,567,21463,378,21463,134,21463,118,141463,144,281464,506,41464,908,251464,874,181464,855,131464,729,211464,635,41464,332,411464,307,191464,261,121464,241,181464,227,121464,162,271464,137,41464,109,261465,1075,181466,150,161466,403,31466,168,31467,1097,231467,1082,31467,1188,31467,1163,31467,971,31467,742,31467,563,241467,513,251467,494,31467,248,281467,278,31467,673,141469,281,191469,161,151469,302,21469,413,261469,451,21469,1507,261469,1738,261470,766,151470,694,181470,652,31470,1125,141470,373,171470,125,371470,415,101470,392,31470,167,101470,95,141470,455,221471,1448,71471,873,31471,1193,31471,1050,31471,658,31471,76,261471,523,31471,493,181471,104,31471,570,211472,624,181472,644,31473,543,251473,997,141473,1061,141473,1180,241473,1275,141474,1162,231474,1038,81474,1443,151474,856,81474,1608,151474,1670,141474,1519,31474,696,31474,729,81474,632,31474,520,31474,460,41474,452,31474,380,31474,362,81474,310,441474,129,31474,659,171475,876,51475,800,51475,565,131475,492,131475,385,121475,295,71475,202,51475,445,151476,1406,201476,1285,41476,1160,31476,1033,31476,904,31476,845,41476,700,231476,837,31476,364,181476,281,41476,241,381476,193,31476,164,271476,111,161476,74,321476,483,381477,1387,21477,1052,21477,767,31477,640,21477,796,21477,383,181477,209,31477,185,221477,403,21479,162,131479,177,51479,184,211479,207,341479,1024,131482,1384,31482,166,221482,190,31482,296,141482,347,31482,406,31482,1720,31483,1133,351483,444,151483,249,491483,124,251484,116,221484,142,291484,173,31484,195,271484,434,31484,597,31484,671,31484,730,31484,818,141484,851,31484,955,31489,1448,21489,992,21489,844,21489,833,91489,941,131490,595,361490,633,41490,726,141490,765,141490,1032,31491,1022,131491,1089,131491,919,131491,809,131491,388,211491,361,61491,71,131491,702,131492,80,191494,990,31494,261,221494,285,31494,440,221494,463,31494,602,31494,689,31494,1043,31495,891,51495,850,391496,73,351496,488,201497,259,181497,280,21497,387,21497,1295,21497,1435,21497,1780,21497,1887,21498,1196,141498,1308,151498,1080,131498,126,141498,241,141498,461,141499,1453,201499,719,101499,483,131499,469,121499,535,181499,423,181499,154,211499,108,201499,443,241500,649,121500,739,121501,1070,151501,1805,231501,1742,361501,1670,131501,918,201501,128,351501,454,131501,171,351501,108,131501,816,131502,1089,251502,1534,131502,1475,141502,1330,141502,1289,231502,1056,141502,196,131502,916,251502,808,141502,786,131502,714,131502,507,131502,1024,131503,1389,31503,523,21503,444,21503,420,31503,292,31503,185,21503,167,161503,265,251504,290,161504,696,251504,688,31504,257,61504,420,31504,163,171504,239,161504,152,31504,126,241505,74,81505,500,31505,492,61505,649,31506,452,241506,326,201506,291,31506,272,171507,1622,21507,1605,131507,1332,31507,1218,131507,1144,131507,1010,131507,927,31507,1737,31507,703,31507,455,521507,325,171507,262,171507,138,441507,127,31507,103,221507,823,31508,386,221509,117,141510,346,111510,475,41512,2014,101512,799,31512,581,31512,1137,271512,419,171512,234,251512,438,31513,147,141513,173,491513,409,61513,533,151513,761,261513,1779,141514,1229,151514,876,201515,158,531515,356,191515,504,221515,759,241515,1040,181516,1503,81516,1386,81516,479,81516,136,81516,178,91516,292,211517,143,191517,164,21517,187,181517,387,21517,502,21517,643,21517,784,21517,1725,21518,642,31518,199,31518,84,31518,59,231518,95,361519,139,191519,163,111519,909,81519,922,111521,506,31521,469,351522,399,51522,151,261522,205,261522,97,291522,128,31523,68,161523,230,161523,480,301523,688,161523,837,161523,912,161525,1837,241525,1768,41525,1635,21525,1552,41525,1488,91525,1088,41525,609,261525,582,101525,198,301525,248,141525,230,41525,774,141526,187,321526,970,31526,1134,91528,1499,41528,1638,51528,1331,41528,1272,41528,409,61528,191,51528,160,291529,101,441529,147,41529,157,271529,372,41529,405,81529,894,41529,1006,81530,82,131531,54,291531,85,51531,111,201531,250,51531,799,51532,1745,31532,1720,211532,1225,31532,800,531532,743,21532,1443,31532,524,31532,246,31532,186,31532,163,201532,722,181534,1413,231534,1386,91534,1284,31534,1233,31534,1104,31534,873,21534,855,141534,801,21534,793,31534,743,271534,455,61534,440,21534,325,21534,289,31534,213,21534,194,171534,185,31534,151,321534,141,61534,693,21535,881,71535,716,81535,853,81535,229,81535,187,81535,498,81536,254,71536,694,71536,802,301536,834,111536,959,151536,979,131536,1064,51537,198,111538,1633,131538,1549,191538,1379,131538,1294,251538,1098,251538,987,141538,386,361538,621,131538,316,141538,277,261538,849,131538,357,141539,1438,51539,745,181539,1181,251539,993,51539,275,51539,41,151539,16,141539,32,21541,441,131541,584,61541,734,131541,995,61542,113,181543,198,271544,1345,171544,888,171544,1221,61544,342,191544,570,171545,161,171545,181,31545,222,201545,457,51545,758,201545,930,31545,1193,71545,1374,61547,564,181547,871,21547,195,21547,684,21547,102,21547,172,21547,81,181549,1714,51549,1612,51549,1486,31549,1218,31549,566,51549,216,481549,153,331549,141,51549,110,291549,515,51551,514,31551,1061,31551,716,21551,627,31551,500,21551,966,21551,297,21551,224,31551,201,211551,194,21551,321,31551,175,171552,1008,31552,812,31552,770,31552,1788,31552,254,31552,147,31552,128,171552,467,31553,719,31553,573,31553,452,101553,563,31553,223,31553,137,31553,299,31554,1292,61554,1646,121554,1730,21554,1480,281554,1412,121554,1306,161554,1252,211554,1632,101554,815,21554,766,21554,591,151554,563,21554,134,451554,126,21554,109,151554,944,21555,1328,51555,1311,31555,1293,31555,1222,51555,1151,151555,1120,31555,1026,31555,836,331555,704,31555,580,61555,348,31555,309,171555,293,31555,260,301555,217,191555,123,311555,737,231556,590,71556,1022,71556,763,31556,729,261556,703,31556,1057,31556,552,31556,354,121556,522,31556,386,71556,182,181556,147,31556,119,261556,102,51556,530,71557,174,271557,260,371557,361,321557,712,21557,857,221557,958,171558,496,351558,447,151558,473,211559,253,301559,631,201559,670,181559,693,291559,724,31559,1237,301559,1473,301560,1051,171560,1227,51560,1157,121560,968,51560,334,31560,537,181560,418,61560,921,201560,315,171561,107,511562,170,141562,1350,151562,1366,221562,1541,141563,1358,301563,853,131563,766,61563,727,131563,592,131563,335,211563,402,141563,198,211563,182,141563,497,211565,1378,121565,104,141565,124,261565,286,141567,175,121567,241,21568,114,561568,198,411568,270,501568,431,251568,752,351568,1193,261569,1011,81569,308,81569,1735,81569,173,381569,267,81570,155,251570,304,251570,796,251571,1011,381571,1724,31571,1648,151571,1904,31571,1452,291571,1250,151571,1064,31571,659,31571,1551,31571,447,221571,388,241571,313,231571,265,181571,251,31571,196,31571,164,301571,566,31572,1623,221572,417,651572,77,191572,102,211574,451,71574,589,131574,567,161574,717,21574,548,171574,409,181574,112,291574,247,191574,211,131574,179,171574,103,21574,83,181574,329,261576,1845,341576,1658,181576,1631,211576,1611,181576,1591,181576,1548,221576,1517,231576,1487,201576,1416,191576,1572,171577,204,131579,1569,171579,1520,31579,1423,101579,1257,131579,1129,31579,707,31579,562,281579,533,31579,485,31579,461,221579,423,141579,339,381579,256,61579,244,81579,507,171580,614,231581,1371,51581,1296,21581,1009,31581,921,21581,611,21581,314,211581,337,31581,182,21581,150,301581,603,31582,1008,151582,627,151582,357,171582,313,161582,287,201582,491,151582,232,141582,200,141582,185,131582,125,381582,105,151582,248,111584,748,101584,809,121584,823,141584,839,111584,852,141584,925,151584,885,161584,1028,91584,1463,221584,738,81584,868,151584,682,151584,453,41584,69,271584,337,111584,319,41584,256,221584,232,181584,202,161584,183,81584,169,91584,98,41584,51,131584,613,91585,1417,371585,989,251585,575,201585,186,321585,383,251585,129,251585,504,251587,942,151587,1382,31587,812,181587,1106,221587,1224,211587,987,281587,788,221587,466,131587,520,231587,22,291587,53,31587,692,211587,481,331587,436,261590,1353,91590,1217,91590,1333,151590,271,91590,251,151590,962,221591,968,151591,1448,31591,1395,111591,1337,31591,1213,111591,1201,31591,1185,31591,1133,31591,1005,361591,1410,31591,857,111591,770,151591,698,111591,677,31591,619,151591,473,31591,360,151591,197,131591,170,31591,140,281591,836,31592,375,221592,285,111592,277,31592,255,201593,119,511593,172,61594,1754,31594,1715,21594,1637,151594,1627,21594,1198,131594,1091,51594,745,31594,1026,31594,650,181594,385,21594,309,21594,221,21594,96,271594,87,21594,68,171594,670,31600,1586,131600,1290,241600,772,251600,1418,131600,382,131600,158,141600,140,131600,434,131602,1005,261602,638,171602,823,261602,339,251602,178,261602,510,81603,188,341603,940,231605,620,31605,1115,31605,1100,31605,728,31605,458,31605,824,31605,208,121605,166,51605,156,31605,128,211605,222,31606,410,181606,579,151607,383,191607,434,71607,979,421607,1023,301607,1055,31608,1841,151608,1281,151608,590,151608,1791,181608,186,281608,103,151608,256,151609,122,221609,150,161609,479,121609,642,121609,770,121609,1027,121610,227,391610,155,231612,1431,141612,1407,201612,1371,121612,1300,31612,1205,361612,736,301612,684,31612,630,121612,560,31612,119,631612,448,131612,405,131612,337,211612,318,171612,295,181612,199,361612,184,31612,525,131613,1457,31613,558,31613,194,211613,456,31613,130,31613,103,251613,145,381614,104,151615,126,141615,160,331615,221,101615,277,31615,1143,221615,1334,31616,1577,31616,1459,31616,649,31616,542,31616,190,371616,276,141616,179,31616,155,221616,397,31617,92,141617,108,21617,163,141617,234,521617,446,21617,1643,21618,1544,181618,1521,181619,1763,21619,1570,21619,1399,21619,1279,21619,1176,21619,1125,21619,1072,21619,1028,21619,913,191619,745,261619,125,21619,499,21619,484,21619,455,21619,432,211619,315,261619,223,21619,135,281619,463,191619,108,151620,1749,221620,1710,141620,1630,221620,1573,141620,1330,141620,1272,141620,1204,141620,1372,141620,937,141620,855,221620,750,141620,534,141620,478,221620,431,141620,1117,141621,452,141621,439,81621,385,291621,208,141621,51,81621,312,81622,802,31622,1793,31622,1460,31622,1413,31622,1389,181622,1331,31622,1140,31622,929,161622,790,31622,1529,281622,729,31622,101,381622,141,31622,149,31622,172,171622,758,171622,490,181622,681,31622,725,31622,277,31623,931,201623,913,31623,836,281623,798,201623,752,31623,714,201623,560,31623,417,281623,232,201623,181,31623,146,331623,108,201623,538,201623,323,131624,860,231624,1227,241624,1191,151624,1064,131624,979,521624,798,101624,658,151624,480,161624,405,151624,105,261624,845,131626,760,191626,1470,291627,1062,191627,797,131627,737,191627,1480,291627,153,341627,125,141627,216,41628,744,201628,414,131628,353,211628,376,181628,163,171628,152,91628,113,201628,205,111629,1290,31629,1725,141629,1686,231629,1627,61629,1503,31629,1067,221629,724,31629,784,31629,1091,31629,696,261629,267,31629,248,171630,220,201630,242,31630,694,161631,1069,201631,1053,121631,938,31631,960,121631,297,121631,208,121631,423,31632,951,251632,1600,31632,1366,31632,1322,31632,1268,61632,1212,231632,1183,71632,1092,51632,982,31632,1728,181632,915,31632,573,31632,689,31632,865,61632,99,91632,175,31632,225,201632,131,311632,316,31632,380,31632,292,41632,661,61632,265,251633,234,201633,270,111633,589,291634,809,81634,204,81634,705,201634,1416,81635,1473,41635,1368,141635,556,201635,1184,41635,159,351635,131,181635,397,41635,153,41636,1166,241636,1283,81636,1238,121636,1109,181636,975,161636,965,81636,1161,31636,263,281636,241,31636,170,221636,58,31636,29,271636,846,161637,260,181638,292,151638,346,141638,485,31638,645,151638,1866,151639,125,161639,376,291640,612,61640,823,61640,503,141640,88,261640,559,231641,53,611641,230,191641,416,141641,581,141642,281,251642,648,321642,1361,71642,1471,261645,1362,141645,1333,51645,993,51645,889,81645,809,31645,340,261645,490,31645,462,211645,189,31645,152,351645,639,31647,1415,151647,918,151647,900,131647,347,261647,479,151647,284,191647,129,381647,69,21647,52,151647,305,131649,1376,181649,1036,171649,1022,61649,1073,141649,524,61649,512,71649,994,131650,1068,31650,1753,31650,1684,251650,1508,31650,1451,381650,1365,251650,1868,51650,1149,141650,1077,151650,968,71650,1204,241650,769,261650,615,161650,579,31650,534,221650,503,31650,251,81650,226,61650,177,31650,108,311650,893,31652,1076,51652,969,101652,856,61652,872,81652,367,361652,340,51652,291,51652,648,51653,1212,51653,1114,181653,1033,181653,994,51653,917,71653,484,181653,386,181653,344,51653,251,51653,164,181653,609,171655,427,221655,162,571655,74,221655,103,461657,201,111658,102,201658,389,221658,413,291658,631,201658,784,401658,947,131658,969,201660,1202,181660,1130,221660,1008,181660,924,391660,623,351660,392,181660,551,151660,198,181660,140,361660,581,181662,219,221662,80,221662,113,271662,183,211663,855,301663,741,231663,276,301664,325,61664,448,31664,611,31664,788,131664,1506,31665,1919,151665,1862,241665,1682,141665,1590,141665,1564,171665,1540,181665,1424,51665,1409,61665,1282,211665,1177,141665,1109,211665,127,311665,1502,331668,420,201668,241,251668,310,71668,329,131668,631,201669,967,171669,379,81669,316,151669,208,81670,1566,21670,1535,271670,1449,21670,1352,21670,1247,271670,1278,21670,397,291670,349,21670,272,21670,210,21670,192,161670,1014,21672,482,191672,805,31672,667,31672,578,31672,403,221672,180,81672,161,141672,106,271672,95,31672,66,271672,237,31673,798,31673,1578,161673,1540,31673,1497,31673,1450,201673,1423,211673,1400,121673,1093,391673,861,161673,839,31673,738,121673,480,31673,682,161673,658,211673,578,31673,554,211673,333,31673,233,211673,187,431673,155,151673,144,31673,112,291673,731,31674,1803,201674,455,201674,195,301675,800,61675,2246,241675,1754,151675,1656,231675,1385,41675,1351,321675,974,161675,950,141675,874,231675,839,211675,769,171675,379,121675,542,171675,513,121675,503,31675,468,161675,435,111675,412,61675,393,171675,583,31675,337,31675,66,431676,1476,21676,560,21676,182,191676,203,21677,129,301677,161,61677,175,271677,240,151677,620,61677,955,61678,1467,221678,1494,231678,1047,131678,983,131678,203,221678,361,61678,230,231678,138,131678,821,141679,128,161679,398,141679,520,161679,657,151680,1717,181680,1192,31680,996,251680,838,161680,965,241680,856,61680,991,31681,1161,121681,1233,121681,936,221681,830,121681,633,221681,560,191681,124,221682,166,261682,194,31682,681,201683,1744,131683,1711,171683,1400,101683,1375,131683,1144,81683,233,31683,160,31683,130,31683,111,171683,76,31683,898,131685,1023,161685,1346,31685,1216,171685,659,171685,244,181685,540,161685,363,91685,611,131686,1148,61686,1851,231686,1739,521686,1701,361686,1644,401686,1591,251686,1548,251686,1328,341686,1228,51686,959,241686,229,371686,846,241686,754,491686,540,31686,411,231686,332,91686,279,251686,166,241686,132,251686,910,241687,1878,21687,939,191687,268,201687,117,141687,167,21687,133,21687,236,21689,1519,61689,1834,31689,1727,61689,507,181689,128,291689,245,141689,215,271689,262,61690,778,31690,726,91690,580,51690,492,31690,821,31690,341,31690,288,31690,256,71690,69,291690,397,51691,404,251692,239,111692,170,301692,109,181692,134,271693,28,311693,189,131693,222,421693,310,341693,843,131694,1004,341694,1042,131694,491,131694,205,131694,268,171694,246,201694,291,341695,57,241695,83,31695,365,211695,397,251696,2411,101696,631,121696,1936,101696,287,171696,313,121697,457,61697,1797,201697,1939,61698,1846,141698,2084,141698,2072,71698,1982,141698,1954,141698,1742,131698,1700,141698,1605,31698,1383,141698,842,141698,815,141698,644,141698,461,141699,1264,141699,1162,141699,817,161699,806,61699,727,151699,405,141699,162,141699,760,181701,380,31701,1732,31701,1625,251701,1451,281701,787,301701,782,31701,612,31701,313,31701,573,31701,258,31701,235,71701,197,111701,170,211701,150,181701,135,131701,110,31701,87,211701,286,251702,816,31702,539,31702,325,31702,262,31702,139,261702,127,31702,99,251702,185,401703,112,131703,319,141703,428,221703,644,231703,723,131703,1045,61703,1096,231704,355,281704,712,281705,1334,61705,1293,31705,901,61705,720,61705,217,261706,126,351706,179,281706,245,221706,723,351706,1113,71707,1750,321707,1713,261707,1629,191707,1343,281707,1254,251707,1235,141707,1204,191707,1031,31707,369,31707,454,91707,353,141707,316,191707,249,331707,137,621707,123,31707,88,331707,691,31708,118,131709,748,31709,768,31709,484,31709,196,31709,339,31709,367,31710,218,241710,324,41710,291,311710,587,161710,652,261710,680,31710,917,151710,1478,31710,1688,31711,615,31714,179,311714,275,231714,466,201714,986,201714,1604,241715,1765,161715,1624,161715,1574,311715,1527,171715,1347,161715,1134,191715,1031,171715,1003,131715,975,201715,960,131715,933,171715,882,221715,659,191715,140,221716,866,231716,1140,151716,457,151716,855,91717,53,241717,94,151717,178,251717,1139,271718,577,31718,898,191718,872,31718,774,31718,753,191718,525,31718,919,31718,433,31718,369,301718,335,31718,156,31718,514,31718,133,211719,768,31719,327,31719,232,31719,83,291719,145,281719,176,31720,1708,31720,900,31720,618,31720,565,31720,418,231720,886,31720,269,211720,200,171720,109,211720,301,231720,293,31722,97,311722,130,41722,172,201722,270,141722,1476,41722,1784,41723,945,31723,840,51723,115,51723,1457,51723,815,31723,1248,51723,616,51723,84,291723,246,51723,183,211723,206,31723,536,51723,238,51723,214,221724,208,391724,249,41724,1144,31724,1123,191725,1099,411725,1000,61725,953,151725,396,151725,310,221725,173,431725,218,71725,133,261725,1153,271725,290,181725,70,151727,996,171727,151,81727,503,181728,559,251728,94,181728,115,21728,126,271728,301,21728,335,141728,420,171728,482,21728,882,21729,532,111729,444,111729,389,31729,88,31729,162,31729,905,111729,309,111730,1020,131730,1596,121730,260,291730,1645,201731,785,111731,149,241732,409,171732,319,191733,359,181733,559,201735,164,211735,188,31736,1444,111736,1419,31736,1324,31736,1298,111736,1184,111736,714,111736,659,31736,510,101736,456,231736,449,51736,411,161736,381,201736,326,161736,276,91736,200,431736,162,311736,485,211737,250,21737,230,181737,1025,21737,89,181737,151,451738,324,21738,297,251738,651,31738,627,221738,839,31738,1395,21738,1480,31739,1515,211739,357,151739,1653,161739,219,151739,293,171740,145,161740,163,31740,433,31740,552,151740,609,301740,941,31740,994,171740,1193,31740,1242,221740,1282,31741,564,181741,647,191741,532,261741,527,31741,253,31741,226,251741,175,181742,94,191742,115,31742,142,711742,430,131742,935,31742,1137,111742,1164,71742,1265,111744,1277,31744,1149,61744,890,31744,773,31744,428,31744,370,51744,152,261744,180,31744,502,281745,179,211745,202,31745,215,351745,420,31746,544,141746,1342,31746,1279,251746,620,31746,560,31746,471,251746,203,221746,341,251746,293,31746,242,301746,227,31746,141,251746,406,31747,1343,121747,1312,31747,1145,121747,656,121747,541,151747,324,31747,292,301747,214,201747,121,501747,472,121748,629,261748,1543,81748,770,81748,603,81748,460,281748,495,251748,443,141748,432,81749,117,301749,149,31749,221,261749,249,31749,360,31749,480,31749,842,31750,1063,181750,972,21750,296,21750,148,21750,127,181750,217,21751,898,21751,271,21751,330,181751,351,21751,664,21751,683,161751,1187,21752,209,41752,189,171753,120,251753,147,41753,332,501754,288,141754,1400,171754,1460,321754,1689,61755,132,251756,277,101756,1211,31756,944,31756,543,101756,519,191756,168,31756,195,231756,138,281756,303,31757,410,51757,61,51757,267,151759,793,81759,762,281759,817,161759,804,81759,736,21759,632,271759,107,21759,86,181759,664,271760,1171,31760,162,31760,135,251760,418,91760,429,181760,453,101760,781,31760,958,31760,994,351761,705,21761,631,151761,308,21761,191,21761,1280,21761,80,411761,71,21761,54,151761,171,181762,260,151762,199,261762,104,181762,175,151764,1364,51764,1287,61764,1180,61764,1165,61764,1103,61764,1044,81764,1019,101764,980,301764,939,81764,621,51764,575,171764,516,161764,465,161764,308,101764,199,211764,894,81764,1409,61767,638,321767,672,41767,518,141767,1089,251767,773,31767,434,451767,701,251767,360,31767,188,301767,116,181767,386,241767,101,31767,72,271768,113,21768,149,281768,97,141769,392,121769,840,631769,629,221769,589,31769,468,211769,443,121769,406,111769,370,151769,734,31769,247,181769,225,161769,210,131769,198,31769,166,81769,111,381769,96,31769,299,221769,70,241770,364,131770,331,161770,251,211770,111,221770,163,221770,135,31770,236,31771,768,181771,1195,181771,1037,241771,718,151771,156,181771,325,71771,257,181771,664,71772,183,281772,694,241773,350,281773,1091,281774,798,241774,2004,321774,1737,241774,1280,321774,745,231774,834,221774,543,321774,302,241774,330,181774,579,191774,366,241775,1031,241775,1562,171775,1439,311775,1381,301775,1289,201775,1192,201775,1108,301775,998,151775,948,301775,889,101775,757,301775,549,241775,467,241775,291,341775,225,201775,156,211775,70,31776,376,201776,345,131776,331,121776,261,181776,237,191776,645,181776,204,121776,195,71776,176,171776,162,121776,122,121776,83,331776,218,171778,210,191778,294,71778,512,181778,598,401778,654,61778,985,61779,1426,191779,1262,161779,146,51779,86,211779,556,191780,157,171780,187,61780,296,51780,603,171780,674,51780,772,51780,835,171781,1129,151781,1621,191781,448,161781,197,261781,281,151781,423,151782,115,401782,205,251782,437,211782,711,401782,753,51782,861,141782,1115,211783,1123,161783,1059,431783,975,301783,777,161783,629,301783,418,391783,325,381783,217,271783,713,431784,87,311784,120,41784,1205,61784,1340,61785,488,171786,672,161786,1661,181786,608,171786,134,461786,491,171786,520,161787,613,31787,585,261787,381,31787,349,301787,729,151787,153,121787,143,41787,117,241787,329,151788,1172,231788,1580,121788,1495,151788,1488,31788,1457,31788,1425,31788,1396,271788,1317,201788,1754,121788,1141,201788,1243,171788,1070,81788,1057,111788,1042,131788,867,121788,547,121788,395,241788,365,121788,244,201788,143,121788,1112,181789,1751,31789,805,31789,723,131789,245,251789,337,31789,272,31789,484,241790,564,161790,686,181790,731,131790,638,161790,530,251790,656,151790,461,201790,420,131790,147,171790,135,41790,99,341790,506,181791,338,311791,371,41792,1375,361792,810,311792,644,151792,1195,31792,248,31792,219,271792,158,301792,440,241793,1873,251793,1777,311793,935,131793,1620,241793,584,61793,196,131793,759,61794,905,101794,1538,21794,1430,21794,1384,191794,1185,31794,1003,41794,813,21794,1897,21794,685,21794,521,21794,261,31794,279,251794,231,281794,226,21794,204,201794,746,21796,351,61797,1018,171797,886,171797,669,151797,759,171797,596,171797,493,171797,118,171797,623,281798,100,281798,130,31798,156,211798,988,31798,1583,31800,1361,201800,1847,71800,1733,251800,1582,121800,1548,71800,1478,201800,1262,161800,431,71800,1151,71800,1073,151800,952,121800,775,141800,261,71800,105,251800,1216,151801,77,251801,590,361802,602,281802,376,281802,425,281803,142,191803,200,31803,219,271803,571,31803,711,31804,925,281804,1017,161804,463,61804,577,51804,165,21804,131,321804,102,151804,214,171807,1398,31807,1356,31807,981,31807,896,61807,599,41807,564,331807,389,121807,486,91807,274,221807,248,31807,209,31807,158,251807,143,31807,117,241807,355,31808,220,291809,1939,61809,1674,151809,1580,151809,1482,171809,973,151809,1432,71809,747,151809,676,151809,574,151809,373,151809,133,151809,828,151810,142,181810,162,31810,452,31810,567,191810,610,31812,126,561813,1195,31813,738,31813,502,31813,257,151813,188,31813,159,271813,277,141814,1514,111815,933,31815,1581,31815,1529,31815,1429,31815,1335,311815,1291,31815,1224,31815,1132,31815,896,201815,1436,21815,497,31815,442,321815,265,31815,233,301815,177,191815,147,241815,722,31815,118,21815,102,141816,1778,221816,1485,171816,1458,211816,1384,171816,1246,171816,1107,171816,231,151816,278,201816,248,181816,204,181816,159,271816,136,171816,746,221817,1336,31817,1854,181817,1389,31817,1142,71817,1755,31817,526,341817,264,21817,248,141817,186,71817,928,31818,311,101818,406,131818,460,131818,795,131818,888,131818,942,131818,1644,131819,913,171819,793,171819,638,171819,605,151819,497,61819,362,251819,341,131819,318,141819,178,251819,404,71820,233,31820,1401,131820,84,161820,209,221821,211,141821,264,141821,366,141821,419,251821,937,141821,1269,141821,1570,141822,391,331822,365,211822,112,211822,188,331822,300,211823,80,91823,153,91823,291,181823,345,151823,639,151823,968,171823,1054,91823,1151,91823,1336,91823,1467,91823,1588,171824,513,341824,434,61824,415,141824,343,121824,292,161824,185,221824,361,291825,1052,331825,1738,221825,1667,131825,1662,31825,1641,161825,1570,131825,1374,241825,1134,31825,1087,301825,1767,161825,970,201825,688,131825,882,81825,910,241825,77,241825,114,281825,175,301825,103,31825,463,31825,514,211825,635,81825,711,101825,231,261826,1682,181826,1601,41826,1624,121826,1641,201826,1666,111826,1719,51826,2417,31826,2003,41826,2040,51826,2083,61826,2657,41826,2616,31826,1561,61826,1792,61826,1077,31826,326,271826,941,41826,1021,41826,273,241826,299,31826,386,361826,433,111826,446,101826,458,191826,518,361826,556,31826,634,181826,708,41826,756,31826,870,51826,483,171827,210,261827,299,81827,170,281827,429,31827,120,141827,69,81827,142,201828,111,261828,139,31828,173,61828,207,31828,348,81828,455,81828,485,361828,537,171830,576,131830,552,21830,405,131830,135,21830,231,131830,109,241830,258,301831,1303,251831,1891,151831,1648,61831,1584,131831,1386,31831,1357,271831,1034,261831,1750,151831,652,31831,633,171831,611,131831,552,151831,289,151831,209,151831,953,151832,1257,421832,1315,181832,1387,31832,1427,241832,1757,31832,1653,181832,1689,31832,1228,131832,1939,31832,1498,31832,1207,191832,1806,181832,1164,131832,148,181832,216,31832,420,31832,705,181832,193,211832,922,201832,1026,181832,1140,181832,783,181832,1179,261833,150,331833,187,151834,743,31834,912,51834,893,31834,787,31834,499,91834,418,351834,148,31834,305,491834,129,171834,455,31835,1398,61835,946,171835,904,271835,882,181835,589,81835,475,181835,412,61835,166,181835,83,181835,71,61835,551,201836,1555,171836,1330,281836,977,481836,788,61836,549,271836,453,281836,252,211836,642,291838,163,111838,204,191839,91,81839,105,181839,215,81839,249,641839,569,81840,1340,31840,1289,31840,929,31840,837,31840,755,31840,666,71840,318,61840,363,31840,336,251840,305,71840,177,61840,623,31841,419,131841,462,501841,1025,131841,322,201841,97,371841,271,201841,73,131841,293,231843,426,141843,482,131843,497,91843,515,201843,549,261844,718,201844,682,101844,591,101844,507,221844,332,181844,157,221844,122,101844,469,101845,1123,191845,997,21845,966,191845,906,21845,362,181845,600,21845,68,171845,148,191845,96,271845,87,21845,264,21846,114,151847,353,301847,760,141847,138,271847,199,261848,811,241848,837,31848,1019,211848,1078,31848,1583,161848,1603,221848,2152,511849,729,111849,948,31849,443,31849,969,111849,204,311849,182,31849,160,201849,296,61850,175,201850,197,31850,205,111853,940,31853,746,31853,684,31853,548,31853,771,211853,147,541853,136,31853,108,261853,421,31854,198,201854,556,201856,1348,141856,1364,291856,1562,71856,1687,141856,2001,141856,1859,151856,2193,141856,2068,61856,1211,71856,1769,141856,1178,141856,141,241856,993,71856,119,141856,179,61856,265,141856,353,221856,1063,71856,543,71856,741,81856,815,331856,872,71856,451,221857,236,241858,525,31858,363,221858,612,31858,225,31858,167,191858,269,281861,83,281861,118,711861,226,321861,293,281861,323,31861,658,31861,792,31862,721,61862,1221,221862,1204,151862,741,151862,649,251862,467,151862,337,151862,223,161862,196,181862,131,511863,1241,41863,737,41863,599,41863,559,371863,444,31863,476,31863,370,261863,284,181863,221,171863,181,31863,148,301863,399,31864,1044,41864,193,191864,214,41864,223,251864,631,261864,962,41865,653,21865,304,21865,115,21865,87,261866,673,151866,721,261866,927,151867,182,111867,349,111867,894,111869,658,31869,301,141869,317,861869,106,31869,84,201869,116,191871,321,171871,377,71871,661,71871,682,231871,786,301872,550,31872,1770,31872,1555,31872,1312,31872,691,31872,670,171872,430,31872,318,131872,272,31872,214,31872,132,271872,121,31872,95,241874,901,51874,819,191874,929,381874,633,461874,446,241874,699,401875,265,151875,323,171876,1081,61876,64,271876,116,61876,225,121876,380,241876,530,51876,675,151876,719,151876,1158,241878,2060,191878,1740,141878,643,141878,181,181878,142,141878,231,271879,1298,61880,118,131880,133,121880,150,271880,244,201881,181,31881,144,351881,306,591881,723,311881,788,191881,832,31882,972,151882,1184,151882,619,151882,1057,151882,520,151882,1143,151882,246,131882,175,231882,96,131882,76,151882,380,151884,101,131884,191,131884,241,201884,823,131884,1012,191884,1415,131884,1465,191884,1567,51885,1172,61885,274,61885,1011,61886,166,411886,354,411886,417,241886,508,171886,746,171886,812,81886,855,171887,309,31887,239,111887,266,111887,291,161887,1272,41888,1279,31888,485,31888,316,261888,223,201893,157,251893,183,291893,684,131893,801,31894,349,361894,860,151894,991,141895,1641,31895,1513,31895,910,21895,416,31895,356,31895,265,31895,235,281896,99,171896,118,31896,745,31896,962,31897,941,321897,1359,131898,1430,21898,326,21898,456,21898,179,151898,196,21899,185,141899,1161,381899,1959,321900,1513,41900,1650,41900,1135,41900,705,41900,859,41900,439,41900,168,41900,149,171900,558,41901,1185,171901,88,171901,107,41901,134,161901,192,41901,522,41901,963,201901,1444,41902,692,41902,182,101902,661,291903,598,281903,697,271903,531,621903,291,221903,448,221903,266,231903,205,181904,574,221905,2218,231905,2199,171905,1545,171905,1229,31905,1234,31905,754,171905,1524,161905,557,301905,537,181905,288,31905,266,201905,238,171905,723,291906,1336,161906,117,161906,149,381906,205,161906,226,211906,525,161906,553,161906,621,181906,755,161908,1164,211908,1111,241908,1137,221909,66,211909,126,431909,294,151909,353,211910,1897,61910,2248,61910,2055,61910,1520,131910,1351,131910,1114,31910,441,61910,367,61910,214,61910,1184,41911,1661,131911,78,131911,116,61911,93,21911,709,21911,809,231913,569,181913,530,31913,752,31913,450,131913,147,31913,157,201913,118,271913,188,551915,182,621915,246,31915,668,31915,899,221915,930,31915,1117,51915,1138,211916,625,31916,540,31916,397,131916,378,31916,333,51916,350,261917,219,141917,235,21917,298,251917,482,341917,721,61917,914,41918,1276,21918,1091,21918,1033,201918,115,141918,135,131918,102,111918,903,101920,235,281920,266,31920,508,31920,926,231920,1286,31922,1852,31922,1606,251922,1593,31922,332,31922,260,31922,243,151922,1463,31923,249,351923,662,41923,731,41925,1542,21925,1725,21925,819,21925,224,21925,199,221925,251,21926,1536,251926,168,251926,195,31926,282,511926,356,31926,413,361926,660,91926,1144,31926,1510,161926,1614,31926,1897,111927,1147,461927,1068,181927,1020,141927,182,141927,468,241927,381,351927,547,301928,529,31928,690,31928,327,131928,1181,31928,462,271928,970,31928,391,141928,134,211928,364,121928,342,201928,307,181928,167,161928,157,31928,378,111929,337,271929,469,71929,478,111929,491,231929,1446,71929,1468,231930,1048,171930,721,51930,1257,171930,430,171930,148,171930,595,171931,1034,31931,927,201931,770,101931,598,71931,486,61931,121,31931,95,241931,529,61932,116,251932,143,31932,1377,211932,1450,61934,1697,241934,989,61934,1406,61934,511,241934,324,131934,707,131936,331,211936,355,31936,397,31936,684,31936,754,31937,84,371938,472,401939,444,131939,1049,191939,563,31939,206,191939,73,151939,166,161939,122,261939,90,121939,184,161940,70,111940,93,391940,146,691940,433,111940,519,111940,636,511940,833,111940,1104,271940,1176,111941,1029,381941,1185,211941,704,61941,1717,141941,296,51941,202,201941,100,431941,451,61942,1309,251942,672,31942,1698,31942,617,31942,1400,31942,271,31942,179,331942,168,31942,450,31942,141,241943,317,21943,167,21943,147,181945,1373,121945,1347,141945,1266,161945,1240,141945,823,81945,803,141945,737,81945,1066,141945,222,141945,202,131945,191,81945,98,261945,92,31945,65,241945,263,31946,644,31946,109,291946,140,31946,165,211946,191,221946,294,31947,555,131947,1528,181947,546,51948,1196,21948,66,181948,86,21948,123,71948,204,21948,565,21948,870,171948,898,401948,947,81948,960,361949,583,251949,454,31949,436,141949,307,31949,97,31949,191,271949,70,251949,232,131950,344,321950,241,211950,400,171950,141,21950,120,191950,230,61951,849,411951,438,221951,181,301952,1307,141952,510,131952,485,121952,424,201952,397,31952,1437,261952,334,271952,274,31952,169,261952,104,31952,67,351952,363,31953,420,391953,581,291954,349,201955,353,131955,368,21955,439,21955,569,21955,776,21955,1001,21956,1013,31956,989,161956,916,31956,656,31956,254,31956,105,31956,74,281957,126,121957,192,361957,244,311957,290,131957,230,41957,565,271958,1599,41958,1534,41958,1283,241958,1157,401958,1088,201958,1072,111958,278,41958,431,41958,124,281958,112,41958,93,171958,834,41959,908,231959,1367,311959,1036,311959,625,31959,117,351959,169,281959,265,261959,154,31960,160,511960,278,191960,329,141960,463,31960,662,151960,767,241960,824,151961,1482,421961,1290,51961,1187,51961,1067,51961,967,51961,932,161961,904,51961,1373,41961,741,191961,640,101961,340,41961,218,51961,142,41961,115,251961,762,241962,1524,141962,1339,141962,1104,51962,1040,71962,1015,141962,842,51962,751,141962,506,201962,471,161962,374,121962,262,241962,162,191962,121,301962,633,301963,79,211963,102,31963,503,211963,622,131963,1576,111964,354,211965,727,241965,770,31965,834,31965,479,31965,365,261965,393,31965,688,121966,374,161966,435,231966,463,261967,862,51967,712,21967,596,141967,612,21967,492,51967,277,51967,172,41967,139,311968,1215,121968,1396,61968,1105,211968,908,51968,641,51968,1193,61968,488,51968,411,61968,209,61968,150,61968,533,221969,74,231969,126,181969,270,581969,372,141969,416,311970,1474,311970,1447,151970,1356,201970,1268,171970,1194,211970,1035,291970,1112,201970,557,171970,206,171970,1138,191971,1105,31971,1715,31971,1597,31971,1363,31971,1148,71971,984,31971,1458,31971,673,31971,640,31971,602,361971,347,31971,756,31971,253,251971,280,31972,1810,201972,1545,31972,1419,141972,169,201973,175,511973,287,141973,615,151973,694,151973,1738,171973,2082,141977,1515,31977,1059,31977,863,31977,1230,31977,361,31977,186,31977,162,221977,617,31978,673,181978,540,41978,532,31978,409,41978,313,41978,191,221978,161,281978,115,281978,100,31978,74,241978,693,31978,275,361979,290,131979,77,511979,166,141980,422,221980,567,291980,809,181980,1381,341981,738,131981,1106,151981,1088,171981,1063,211981,1052,61981,1016,101981,976,101981,956,61981,711,151981,1428,151981,563,61981,535,61981,477,131981,450,151981,346,131981,120,251981,887,141981,1562,61982,911,31982,1046,211982,691,111982,313,251982,411,31982,438,261983,97,51983,165,71983,197,51983,275,61983,517,61983,745,131983,1420,71984,1514,31984,1481,31984,1420,151984,1388,31984,1245,31984,968,31984,869,31984,302,31984,277,221984,1041,31985,102,221985,332,131986,1256,171986,1653,121986,1579,121986,1494,61986,1680,171986,1405,131986,1331,181986,707,61986,317,171986,541,121986,466,121986,341,121986,263,121986,133,151986,98,121986,662,181987,2266,151987,825,91987,247,151987,784,91988,1258,271988,1711,21988,1611,21988,1400,31988,1382,21988,1325,401988,1303,201988,1045,61988,692,211988,973,41988,982,211988,37,21988,105,171988,24,111988,592,61988,944,201988,327,21989,186,441989,233,41989,246,261989,283,591989,471,41989,1197,41990,251,201991,225,391991,1144,261991,953,221991,886,311991,812,241991,158,271991,189,251991,99,451991,476,221992,569,121992,587,451992,279,31992,754,281992,669,31992,515,121992,193,301992,179,121992,164,131992,121,131992,255,221993,296,281993,326,141993,388,351994,436,161994,562,41994,305,111994,248,111995,82,351995,119,31995,178,161995,253,31995,312,281995,398,31996,991,311996,1274,161996,732,161996,166,181997,1668,31997,1707,171997,1610,91997,1586,191997,1530,31997,1419,361997,1322,31997,1149,31997,1089,91997,1060,191997,981,81997,942,171997,122,301997,691,81997,724,31997,254,191997,278,91997,338,31997,362,31997,155,31997,478,31997,500,261997,554,191997,645,31997,369,371999,775,201999,1336,261999,1282,291999,1235,271999,1050,211999,939,381999,909,141999,861,171999,800,131999,759,141999,356,291999,625,291999,542,381999,501,271999,387,241999,336,181999,185,231999,158,161999,124,211999,89,331999,656,932001,539,32001,501,302001,400,212001,200,232001,156,252001,225,32002,1183,502004,996,32004,795,32004,780,32004,1155,32004,357,32004,207,32004,184,212004,605,32005,88,262005,116,32005,124,292005,300,92005,508,92005,730,32008,51,262008,79,42009,594,22009,993,22009,819,282009,459,22009,1156,22009,194,182009,159,22009,139,182009,329,22010,167,142010,483,142010,1054,142011,1350,32011,1275,222011,1008,32011,947,32011,1685,32011,691,172011,679,32011,656,212011,855,32014,1527,172014,1487,352014,1418,142014,975,162014,1043,142014,580,132014,177,402014,142,142014,896,342015,665,82015,978,182015,930,212015,840,222015,800,72015,689,82015,547,222015,67,262015,359,82015,267,32015,204,302015,167,32015,139,262015,462,32017,160,352017,451,352017,1213,232017,1255,172018,738,22020,78,62020,478,62020,558,182020,581,142020,668,62020,725,62023,289,172023,309,32023,1504,32023,1956,162025,304,262026,25,332026,102,702026,197,142026,291,142026,420,172026,805,232027,595,62027,437,62027,395,32027,343,32027,72,182027,63,32027,40,212027,124,162028,123,202028,206,232028,231,212028,538,352028,599,252028,822,332030,1075,52030,642,292030,789,52030,209,52030,151,562030,368,52031,102,72031,125,162031,159,182031,179,172031,198,132031,216,82031,260,172031,290,132031,500,162033,405,502033,367,342033,177,182034,115,142034,147,62034,173,142034,191,122034,435,62036,223,172037,750,82037,681,32037,430,242037,319,462037,639,82037,382,182037,295,32037,214,32037,153,592037,309,82039,107,252039,134,32039,183,152039,718,32039,811,32039,815,32042,1123,22042,978,222042,967,62042,683,122042,490,112042,192,22042,168,222042,655,202043,846,392043,1128,222044,1751,62044,1670,182044,1144,82044,1033,202044,1008,82044,998,82044,893,1002044,851,152044,712,62044,101,182044,134,62044,783,182044,625,182044,193,182045,770,172045,612,52045,524,52045,925,32045,276,172045,199,62045,80,302045,451,62045,693,32047,221,122048,224,62048,432,62048,447,62049,819,82049,1067,82049,1036,42049,969,82049,952,122049,878,82049,842,122049,699,122049,689,82049,570,122049,543,172049,163,252049,112,82049,94,122049,717,122050,91,372050,637,182051,367,142055,1082,32055,983,32055,811,32055,626,32055,508,372055,265,272055,803,32056,544,272056,727,272056,756,32056,847,332057,278,372057,826,372057,657,372057,196,292057,153,192057,174,162057,109,282057,55,372063,44,272063,73,32063,90,262063,411,32063,752,32063,1077,32064,139,182064,735,302064,107,202064,129,32065,150,202065,172,22065,334,22065,920,22065,1112,22066,85,202066,116,222066,178,162066,572,202066,809,212066,898,192067,668,142067,449,62067,354,142067,274,182067,172,62069,234,202069,256,32069,342,52069,807,132071,83,142071,190,292071,248,142071,343,262072,206,152072,223,32073,661,222073,545,32074,108,92075,73,182075,93,22075,113,402075,201,22075,310,22075,385,22075,549,22075,591,22076,240,162077,913,22077,973,22077,729,22077,99,202077,121,22077,463,22078,207,22078,452,102078,543,22078,752,62078,801,22078,884,82078,977,22078,984,22078,1425,22078,1432,22080,1519,62080,1503,42080,1202,62080,917,212080,940,42080,1070,162081,172,242087,159,212087,365,202087,389,262088,770,182090,1009,22090,605,22090,108,152090,125,22090,508,22095,72,242095,257,322095,305,162095,556,122095,584,142095,660,342097,753,132097,698,62097,504,62097,149,132097,362,62098,165,312098,318,142098,334,182098,358,142098,468,172098,496,102098,640,172098,908,282101,1081,142101,193,122101,151,252102,212,142102,282,302102,314,32102,566,32102,620,32103,421,192103,402,132103,370,82103,342,182103,334,62103,94,292103,362,62105,479,262105,1163,262105,595,332105,316,282105,1144,52105,191,272105,233,262105,91,262105,839,262105,124,272106,837,152106,810,152106,541,162106,513,162106,787,132106,361,142106,207,142106,137,382106,389,132107,92,122107,525,142108,703,22109,142,152109,251,252109,534,252109,805,152109,908,622112,1019,92112,1257,52112,941,142112,844,142112,1070,92112,558,152112,501,142112,274,52112,600,162112,125,142114,239,102114,251,32114,291,262114,613,302114,725,172114,800,192114,844,32114,898,202114,1044,32121,144,352122,32,202122,245,162122,281,202122,414,172122,500,202124,201,212124,224,32124,460,32126,142,32126,2243,142127,949,52127,732,52127,609,122127,590,142127,121,62127,313,122127,238,172127,195,52127,535,62128,1294,42128,715,42128,684,292128,637,202128,529,192128,565,92128,449,122128,426,212128,102,32128,76,242128,472,182130,868,682130,104,62130,333,252130,391,92130,701,512130,823,382130,947,332131,817,152131,625,322131,1010,152131,225,282131,255,152132,359,342132,530,172132,156,242135,158,222135,182,112135,195,142135,211,212135,234,162135,267,82135,1118,372135,1246,202135,1394,162135,1595,252136,485,142136,333,142136,414,72137,110,182137,130,22137,397,132137,414,182137,437,162137,776,22138,1301,62138,1562,122138,1373,152138,1114,172138,1005,142138,954,122138,777,92138,362,92138,146,152140,550,172140,579,162140,597,32140,1543,32141,190,382141,587,182142,197,392143,811,22143,329,22144,72,272144,112,212144,174,212144,197,162144,228,262144,622,272144,667,272144,763,162144,783,182144,984,162146,337,132146,197,132146,150,132147,307,222147,1082,122149,1355,152149,1256,152149,952,262149,805,152149,991,152149,689,152149,262,262149,113,372149,775,152152,320,162152,368,512154,549,32154,912,32154,796,32154,663,32154,416,32154,961,32154,233,32154,159,252154,130,32154,104,242154,313,32155,103,242155,372,62155,408,212158,148,222158,172,22158,408,22158,557,22158,624,22159,1483,62159,892,72159,275,232161,1405,222161,65,222161,237,242161,263,32161,320,32161,298,202161,652,32161,796,332161,923,142161,997,222161,1232,142161,1286,222161,1341,142162,1987,122162,1757,172162,177,222162,1310,122164,82,582164,181,132164,221,202164,360,242164,784,122164,1075,92164,1176,232164,2053,122164,2171,202164,2335,122166,704,212166,602,72166,277,302166,482,512167,221,142167,296,162167,380,122167,426,322167,494,302167,593,372168,708,102168,833,142168,629,212168,332,162168,286,142168,245,242168,183,102170,147,442170,273,252170,364,442172,97,142172,128,262172,173,462172,458,142172,740,392173,911,122173,671,122173,495,122173,229,172173,337,122173,60,262173,372,122174,238,152175,428,212175,1241,182176,202,282177,557,22177,642,22177,672,322177,393,22177,83,22177,268,172177,63,182177,326,22179,1701,162179,1581,122179,1423,332179,1369,162179,1198,202179,1087,172179,1040,262179,954,162179,895,282179,536,122179,395,162179,362,192179,315,82179,248,182179,168,172179,117,282179,77,332179,443,122180,1230,232180,1099,182180,880,322180,268,232180,153,342180,791,232182,169,142182,225,322182,279,122182,293,82182,369,142182,564,142183,1080,272185,179,272186,96,202187,48,142187,69,202187,179,162187,236,182188,44,332188,130,512188,207,82188,241,262188,450,172189,799,242189,917,32189,712,222189,865,102189,674,222189,851,102189,450,152189,320,142189,523,242189,78,32189,54,222190,135,222190,171,192190,675,242190,1070,272191,278,142191,879,142192,987,22192,1108,272192,265,152192,282,22194,263,202194,310,212194,345,172194,376,202194,947,212194,991,132195,123,82195,176,502195,521,182195,597,82196,646,162196,739,592196,708,192196,339,172196,303,282196,235,562196,214,192196,145,232197,163,252197,236,142197,340,332197,375,42197,505,152197,1132,192197,1542,262199,231,162199,860,172199,164,172199,183,422200,83,262200,203,312200,323,302200,606,262201,129,122201,552,122202,712,152208,1836,132208,2200,22208,2102,22208,2029,22208,1942,372208,1882,92208,1627,132208,1464,452208,1581,92208,1594,222208,1251,272208,711,302208,459,22208,435,222209,40,222209,64,32209,205,32209,340,222210,1022,182210,843,182210,592,182210,686,182210,339,182210,530,342210,230,182210,188,172211,590,22211,807,232211,852,492211,963,222213,118,212214,1796,32214,1748,32214,2036,32214,1719,32214,1592,32214,1441,32214,1277,72214,1220,32214,300,182214,1123,172214,1115,32214,740,32214,575,32214,320,32214,255,32214,231,222214,1185,32217,823,32217,159,32217,122,352218,1570,22218,1922,372218,1904,132218,1965,22218,1836,132218,1709,132218,1654,22218,1598,222218,1456,222218,356,22218,1306,212218,197,92218,332,222218,458,92218,1348,92218,503,92218,641,262218,496,22218,542,22219,637,22220,209,282220,239,32220,271,252220,414,212221,900,52221,224,62221,117,142222,113,212222,136,32222,207,252222,234,312222,416,32225,932,452225,899,152225,791,552225,1215,222225,558,62225,338,192225,188,452225,147,222225,385,192226,75,562226,137,392226,364,112226,523,222227,882,292227,807,312227,627,212227,701,392228,171,132228,254,212228,1175,132228,1239,132228,1376,132232,153,182232,388,262232,1129,262246,1099,202253,974,242253,133,242255,140,172255,929,172255,847,142255,793,172255,759,142255,611,172255,868,82255,311,262255,274,132255,210,142255,177,142255,564,142257,75,142257,237,142258,169,202258,233,752274,1178,142274,79,512274,768,142274,958,172274,1108,222274,1242,152278,350,222278,152,222279,97,142279,168,14]

In [ ]:
BannerAnnotate.annotate()

In [ ]:
class BannerAnnotate:
    def annotate(self,):
        return (0,0,22,\
0,54,38,\
0,159,18,\
1,976,10,\
1,1232,36,\
1,1359,18,\
2,17,30,\
2,77,26,\
3,4,21,\
3,34,28,\
3,68,21,\
3,91,3,\
3,343,3,\
3,392,13,\
3,470,16,\
3,510,9,\
3,913,6,\
3,953,14,\
3,1554,3,\
5,86,15,\
5,103,6,\
6,56,16,\
6,152,18,\
6,522,15,\
6,539,27,\
6,568,15,\
6,585,25,\
6,612,23,\
6,641,20,\
7,19,9,\
9,53,12,\
9,74,6,\
9,104,6,\
9,156,6,\
9,222,32,\
9,564,5,\
9,920,5,\
9,1160,7,\
9,1357,5,\
9,1593,5,\
9,1614,13,\
10,54,16,\
10,167,26,\
10,278,17,\
10,355,17,\
11,131,31,\
11,164,3,\
11,170,3,\
11,215,40,\
11,870,3,\
12,48,24,\
12,87,21,\
13,39,21,\
13,62,3,\
13,68,21,\
13,91,3,\
13,102,17,\
13,137,18,\
13,157,19,\
13,181,30,\
13,213,3,\
13,348,3,\
13,400,3,\
13,1430,3,\
14,53,19,\
15,74,15,\
15,370,15,\
15,708,14,\
15,1021,15,\
15,1518,5,\
15,1737,5,\
15,1770,12,\
15,1819,12,\
15,2083,6,\
15,2108,15,\
15,2711,15,\
17,7,11,\
17,33,19,\
18,30,12,\
18,47,8,\
18,72,9,\
18,145,27,\
18,276,9,\
18,306,30,\
18,345,27,\
18,374,3,\
18,383,12,\
18,584,8,\
18,665,12,\
18,844,8,\
18,944,4,\
18,995,3,\
18,1134,3,\
18,1242,15,\
18,1498,18,\
18,1526,15,\
19,12,14,\
19,162,51,\
19,525,14,\
19,1017,13,\
19,1293,15,\
19,1547,14,\
20,1377,43,\
21,50,15,\
21,90,39,\
21,166,13,\
21,200,3,\
21,310,13,\
21,369,9,\
21,380,16,\
21,398,15,\
21,418,37,\
21,1154,12,\
22,26,14,\
22,88,31,\
22,180,14,\
22,262,31,\
22,1424,31,\
23,11,16,\
23,31,30,\
23,79,19,\
23,100,56,\
23,197,30,\
23,229,3,\
23,244,16,\
23,300,26,\
23,331,18,\
23,414,3,\
23,444,3,\
23,466,16,\
23,507,17,\
23,547,19,\
23,640,3,\
23,717,16,\
23,763,19,\
25,0,28,\
25,41,26,\
25,119,28,\
25,149,3,\
25,250,19,\
25,301,27,\
25,358,26,\
25,432,3,\
25,580,3,\
25,971,9,\
25,985,18,\
25,1169,12,\
25,1222,3,\
25,1290,12,\
25,1691,12,\
25,1708,3,\
25,1765,11,\
25,1819,3,\
26,233,18,\
26,412,23,\
27,13,30,\
27,82,12,\
28,24,15,\
28,61,15,\
28,78,2,\
28,269,19,\
28,298,24,\
28,324,2,\
28,491,15,\
28,759,15,\
28,840,15,\
28,981,2,\
29,79,23,\
29,184,14,\
29,2174,14,\
30,1315,19,\
31,74,27,\
31,155,27,\
31,184,3,\
31,190,3,\
31,349,28,\
31,963,3,\
31,1088,3,\
32,0,15,\
32,20,21,\
32,137,27,\
32,167,3,\
32,240,21,\
32,264,3,\
32,304,18,\
32,326,3,\
32,553,3,\
32,893,3,\
32,990,3,\
32,1238,15,\
32,1552,32,\
32,1587,18,\
32,1769,3,\
32,1887,3,\
34,21,18,\
34,90,23,\
34,231,27,\
34,268,20,\
34,666,3,\
35,51,7,\
35,142,7,\
35,719,6,\
35,806,6,\
35,1864,6,\
35,1894,7,\
37,42,20,\
38,0,3,\
38,49,29,\
38,158,29,\
38,189,3,\
38,226,3,\
38,261,3,\
38,396,3,\
38,405,20,\
38,499,21,\
38,927,3,\
39,13,32,\
39,69,26,\
40,51,7,\
40,78,21,\
41,28,18,\
41,48,18,\
41,68,3,\
41,103,18,\
41,126,26,\
41,747,29,\
41,781,16,\
41,1051,22,\
41,1247,40,\
41,1289,4,\
41,1296,17,\
41,1325,23,\
41,1350,17,\
41,1369,14,\
41,1389,32,\
42,0,6,\
42,31,9,\
42,190,23,\
42,245,18,\
42,307,16,\
42,547,9,\
42,847,6,\
42,1062,11,\
42,1166,6,\
42,1221,20,\
42,1274,11,\
42,1300,20,\
42,1419,16,\
43,72,18,\
43,215,29,\
43,339,17,\
43,361,9,\
43,457,19,\
43,497,14,\
43,625,18,\
43,681,22,\
43,1068,22,\
43,1159,10,\
43,1349,22,\
43,1410,17,\
44,150,23,\
44,243,18,\
44,263,2,\
44,358,21,\
44,676,10,\
44,724,24,\
44,979,32,\
44,1139,31,\
45,28,23,\
46,217,10,\
46,313,27,\
46,1567,18,\
47,73,25,\
47,139,16,\
47,157,14,\
47,177,18,\
47,197,13,\
47,329,13,\
47,349,28,\
47,508,16,\
47,992,18,\
47,1042,25,\
47,1252,54,\
48,537,20,\
48,721,5,\
48,1472,6,\
48,1573,6,\
48,2247,6,\
49,239,16,\
49,260,9,\
50,0,21,\
50,649,21,\
50,1034,9,\
51,29,3,\
51,66,30,\
51,153,26,\
51,181,3,\
51,211,30,\
51,243,3,\
51,369,3,\
51,476,3,\
51,614,14,\
51,997,3,\
52,150,14,\
52,257,22,\
52,297,20,\
52,378,14,\
52,413,22,\
52,825,14,\
52,935,14,\
52,1914,14,\
53,324,6,\
53,521,11,\
53,728,41,\
53,979,3,\
54,105,21,\
54,128,2,\
54,521,2,\
54,847,25,\
54,1026,6,\
54,1184,6,\
54,1299,6,\
54,1324,2,\
55,12,17,\
56,75,27,\
56,104,18,\
56,127,27,\
56,536,27,\
58,18,20,\
58,215,18,\
58,245,20,\
58,339,10,\
58,376,60,\
58,571,28,\
58,634,20,\
58,665,24,\
58,700,27,\
58,791,14,\
58,817,30,\
58,851,20,\
58,940,20,\
58,1010,42,\
58,1154,20,\
58,1205,27,\
58,1313,20,\
60,67,35,\
60,104,35,\
60,141,3,\
60,151,20,\
60,257,42,\
60,450,3,\
60,877,3,\
60,1194,3,\
60,1380,3,\
60,1430,15,\
60,1466,3,\
60,1594,19,\
60,1709,3,\
61,261,29,\
61,293,50,\
61,467,50,\
61,1026,21,\
62,9,18,\
62,31,43,\
62,131,18,\
62,274,53,\
62,333,17,\
62,599,18,\
62,645,18,\
62,743,17,\
62,835,18,\
62,988,14,\
62,1463,18,\
62,2134,18,\
62,2208,18,\
62,2567,13,\
62,2758,27,\
62,2930,18,\
63,85,17,\
63,107,8,\
63,117,41,\
63,160,4,\
63,181,15,\
63,217,28,\
63,275,21,\
63,298,17,\
63,320,22,\
63,1283,13,\
64,0,15,\
64,126,15,\
64,299,3,\
64,1193,3,\
65,110,28,\
65,140,28,\
65,170,3,\
65,180,14,\
65,292,3,\
65,519,3,\
65,799,26,\
65,849,3,\
66,1,31,\
66,99,31,\
66,132,3,\
66,162,3,\
66,172,28,\
66,274,3,\
67,15,52,\
67,158,58,\
67,218,2,\
67,376,3,\
67,384,2,\
67,443,16,\
68,0,31,\
68,57,31,\
68,90,4,\
68,120,15,\
68,148,40,\
68,467,7,\
68,671,10,\
68,861,31,\
69,52,18,\
69,124,44,\
69,189,22,\
70,683,12,\
71,0,24,\
71,174,15,\
71,191,3,\
71,312,29,\
71,347,3,\
71,379,3,\
71,528,3,\
71,687,3,\
71,755,3,\
71,829,3,\
71,1117,3,\
71,1297,3,\
71,1347,21,\
71,1567,3,\
71,1623,13,\
72,58,5,\
72,90,6,\
72,187,10,\
72,243,6,\
72,330,5,\
72,655,14,\
72,842,5,\
72,945,6,\
72,1017,14,\
72,1127,6,\
72,1168,6,\
72,1264,12,\
72,1366,6,\
72,1456,6,\
73,55,13,\
73,79,13,\
73,94,3,\
73,103,37,\
73,160,23,\
73,220,3,\
73,374,3,\
73,438,3,\
73,581,3,\
73,988,3,\
73,1193,3,\
73,1644,3,\
75,38,20,\
75,95,31,\
75,446,39,\
75,493,21,\
75,525,15,\
77,52,27,\
77,171,20,\
77,193,2,\
77,239,36,\
77,277,5,\
77,303,2,\
77,559,2,\
77,631,2,\
77,1037,2,\
77,1375,20,\
77,1440,2,\
78,232,20,\
78,254,2,\
78,263,26,\
78,407,2,\
78,1138,13,\
78,1607,2,\
79,8,16,\
79,43,12,\
80,0,20,\
80,45,18,\
80,86,18,\
80,106,2,\
80,123,21,\
80,179,32,\
80,578,24,\
80,688,2,\
80,747,21,\
80,770,3,\
80,844,4,\
80,809,33,\
80,882,2,\
80,905,3,\
80,998,27,\
80,1042,4,\
80,1149,2,\
80,1161,2,\
80,1258,2,\
80,1459,2,\
80,1523,4,\
80,1582,2,\
80,1624,2,\
80,1792,2,\
80,1901,2,\
80,2038,24,\
81,0,23,\
81,121,23,\
81,262,23,\
81,329,23,\
81,408,14,\
81,578,23,\
81,633,14,\
81,749,14,\
81,858,23,\
81,989,23,\
81,1052,14,\
81,1155,24,\
81,1275,15,\
81,1322,23,\
83,15,16,\
83,33,22,\
83,90,16,\
83,163,39,\
83,326,37,\
83,365,6,\
83,414,16,\
83,496,3,\
83,646,23,\
83,691,11,\
83,876,16,\
83,894,22,\
84,45,27,\
84,110,26,\
84,172,30,\
84,328,20,\
84,369,27,\
84,519,20,\
86,0,54,\
86,131,54,\
86,188,5,\
86,202,17,\
86,459,18,\
86,1092,42,\
86,1213,17,\
87,19,27,\
87,119,26,\
87,147,66,\
87,367,16,\
87,434,27,\
87,464,12,\
87,479,14,\
87,500,9,\
87,542,16,\
87,935,23,\
87,1077,16,\
87,1165,14,\
88,65,17,\
88,193,5,\
88,228,13,\
88,348,17,\
88,418,5,\
88,491,17,\
88,690,17,\
88,758,5,\
88,803,17,\
88,905,5,\
88,926,17,\
88,997,6,\
88,1078,17,\
88,1140,17,\
88,1348,5,\
88,1465,13,\
89,55,21,\
89,261,21,\
89,284,3,\
89,337,28,\
89,413,33,\
89,659,3,\
89,973,3,\
89,1044,3,\
90,21,9,\
90,83,14,\
90,230,14,\
90,246,2,\
90,401,23,\
90,483,39,\
90,768,2,\
90,857,10,\
90,888,21,\
91,100,24,\
91,180,24,\
91,206,3,\
91,248,16,\
91,266,6,\
91,277,17,\
91,310,3,\
91,338,25,\
91,677,3,\
91,745,3,\
91,781,24,\
91,1090,3,\
91,1141,3,\
91,1502,3,\
91,1622,3,\
92,0,18,\
92,95,36,\
92,133,18,\
92,153,2,\
92,249,22,\
92,320,2,\
92,386,2,\
92,475,21,\
92,641,16,\
92,1455,21,\
92,1515,2,\
93,86,5,\
93,162,23,\
93,420,13,\
93,445,23,\
93,573,5,\
93,660,5,\
93,1860,5,\
94,0,12,\
94,14,14,\
94,30,19,\
94,51,13,\
94,66,13,\
94,81,9,\
94,99,23,\
94,128,22,\
94,173,14,\
94,281,9,\
94,292,16,\
94,310,12,\
94,324,14,\
94,340,19,\
94,361,13,\
94,376,6,\
94,384,14,\
94,400,13,\
94,422,18,\
94,446,22,\
95,30,25,\
95,99,31,\
95,141,5,\
95,342,3,\
95,703,3,\
95,995,3,\
95,1244,22,\
96,0,16,\
96,50,16,\
96,68,16,\
96,97,16,\
96,320,13,\
96,673,16,\
97,14,11,\
98,159,13,\
98,183,6,\
98,220,37,\
98,733,5,\
98,1183,13,\
98,1206,7,\
98,1248,37,\
99,156,28,\
99,250,28,\
99,280,3,\
99,288,18,\
99,510,3,\
99,1356,3,\
99,1876,15,\
100,64,18,\
100,135,6,\
100,245,18,\
100,378,5,\
100,414,17,\
100,467,27,\
100,496,3,\
100,683,18,\
100,718,6,\
100,915,6,\
100,1035,6,\
100,1438,6,\
100,1677,18,\
100,1744,5,\
100,1884,17,\
101,57,29,\
101,88,29,\
101,119,5,\
101,143,26,\
101,394,5,\
101,587,5,\
101,598,31,\
101,788,5,\
101,847,5,\
101,867,39,\
101,1301,5,\
101,1721,12,\
101,1853,5,\
101,1869,36,\
103,82,42,\
103,126,19,\
103,194,46,\
103,147,4,\
103,620,35,\
103,1244,8,\
104,56,20,\
104,144,31,\
104,237,24,\
104,530,29,\
105,122,10,\
105,143,10,\
105,159,15,\
105,208,13,\
105,422,10,\
105,850,10,\
106,66,23,\
106,285,23,\
106,356,23,\
106,408,14,\
106,1061,5,\
106,1084,5,\
106,1115,5,\
106,1180,16,\
106,1207,5,\
106,1908,38,\
106,2238,14,\
107,28,26,\
107,240,6,\
107,604,5,\
107,847,6,\
107,897,21,\
107,1009,6,\
107,1239,19,\
107,1332,5,\
107,1582,6,\
107,1792,5,\
107,2040,22,\
107,2078,39,\
107,2142,5,\
107,2394,6,\
109,0,31,\
109,36,24,\
109,181,24,\
109,208,3,\
109,217,8,\
109,230,18,\
109,341,24,\
109,368,3,\
109,392,10,\
109,417,18,\
109,679,3,\
109,735,12,\
109,751,3,\
109,808,22,\
109,921,12,\
109,984,41,\
109,1229,14,\
109,1297,3,\
109,1392,3,\
110,14,26,\
110,98,26,\
110,126,2,\
110,297,24,\
110,399,13,\
110,414,23,\
110,701,2,\
110,819,2,\
112,38,24,\
112,67,26,\
112,288,17,\
112,311,6,\
112,337,10,\
112,364,26,\
112,392,3,\
112,402,12,\
112,441,21,\
112,745,3,\
112,1075,3,\
112,1083,3,\
112,1414,3,\
112,1540,24,\
113,86,17,\
113,131,8,\
113,172,19,\
113,795,17,\
114,623,19,\
115,60,19,\
116,558,37,\
119,81,29,\
119,580,16,\
119,790,33,\
119,2413,34,\
120,146,21,\
120,177,15,\
121,86,20,\
121,234,9,\
121,267,32,\
122,506,45,\
122,553,3,\
123,0,35,\
123,137,10,\
123,187,7,\
123,202,42,\
123,772,18,\
124,345,28,\
124,775,62,\
124,1119,12,\
124,1133,3,\
124,1160,9,\
124,1190,12,\
124,1204,3,\
124,1221,9,\
124,1278,17,\
124,1511,3,\
124,1519,22,\
124,1702,20,\
124,1990,12,\
125,44,17,\
126,142,27,\
126,171,3,\
126,329,3,\
126,423,19,\
128,4,23,\
128,91,34,\
128,140,23,\
128,182,3,\
128,231,31,\
128,265,3,\
128,276,3,\
128,314,25,\
128,475,43,\
128,566,3,\
128,688,3,\
128,748,16,\
128,769,3,\
128,960,3,\
128,982,22,\
128,1132,3,\
128,1266,3,\
128,1394,3,\
128,1698,3,\
128,1856,25,\
128,1886,3,\
129,151,6,\
129,168,51,\
129,249,7,\
129,424,17,\
129,517,14,\
129,632,15,\
129,760,5,\
129,928,3,\
129,1032,14,\
129,1200,14,\
129,1287,14,\
129,1733,21,\
129,1837,14,\
129,1975,14,\
129,2115,17,\
130,121,26,\
130,149,2,\
130,165,47,\
130,231,2,\
130,1262,2,\
131,73,21,\
131,96,32,\
131,144,37,\
131,587,52,\
131,641,4,\
131,646,3,\
131,895,16,\
132,101,14,\
132,1007,8,\
134,30,23,\
134,57,19,\
134,78,16,\
134,109,13,\
134,143,19,\
134,181,18,\
134,229,14,\
134,335,14,\
134,374,23,\
134,1297,23,\
134,1406,23,\
134,1608,22,\
135,11,10,\
135,68,28,\
135,98,3,\
135,123,48,\
135,344,3,\
135,439,10,\
136,63,26,\
136,188,12,\
136,270,32,\
136,341,18,\
136,588,26,\
136,1566,22,\
137,0,24,\
137,99,24,\
138,56,21,\
138,126,16,\
138,143,21,\
138,167,3,\
138,597,3,\
138,988,3,\
139,82,36,\
139,565,36,\
139,811,36,\
139,852,24,\
140,181,16,\
141,305,19,\
141,757,52,\
141,930,19,\
141,1448,3,\
141,1657,33,\
142,23,6,\
142,159,6,\
142,216,16,\
142,234,3,\
142,301,6,\
142,525,6,\
142,603,6,\
142,736,6,\
142,937,25,\
142,1125,6,\
142,1677,21,\
142,1845,6,\
142,2052,17,\
142,2260,6,\
142,2302,23,\
142,2383,6,\
143,96,22,\
143,346,24,\
143,1774,15,\
143,1849,15,\
144,39,25,\
144,110,25,\
146,56,16,\
146,247,29,\
146,281,22,\
146,494,9,\
146,540,14,\
146,568,5,\
146,665,35,\
146,746,14,\
146,984,12,\
147,0,69,\
147,101,34,\
147,202,24,\
147,304,40,\
149,119,5,\
149,206,20,\
149,492,8,\
149,504,24,\
149,780,44,\
149,1115,6,\
149,1143,15,\
149,1275,14,\
149,1435,27,\
150,44,26,\
150,72,15,\
150,289,15,\
150,376,48,\
150,1437,26,\
150,1667,34,\
151,85,18,\
151,105,18,\
151,261,28,\
152,32,6,\
153,332,6,\
155,0,15,\
155,331,6,\
155,444,8,\
155,454,27,\
155,538,6,\
155,987,6,\
155,1147,34,\
155,1531,34,\
156,35,20,\
156,67,14,\
157,0,3,\
157,23,23,\
157,50,30,\
157,112,3,\
157,199,21,\
157,278,3,\
157,358,3,\
157,640,14,\
157,715,23,\
157,807,3,\
157,863,19,\
157,1429,8,\
157,1566,8,\
157,1672,21,\
158,201,2,\
158,181,18,\
158,958,8,\
159,31,6,\
159,109,15,\
159,135,27,\
159,211,28,\
159,257,15,\
159,353,6,\
159,397,3,\
159,508,3,\
159,749,3,\
159,774,2,\
160,73,24,\
160,141,24,\
160,171,24,\
160,197,3,\
160,215,33,\
160,276,24,\
160,368,3,\
160,1078,24,\
160,1164,8,\
161,430,14,\
161,806,17,\
161,1332,15,\
163,25,15,\
163,195,26,\
163,420,15,\
164,98,20,\
165,1340,18,\
166,28,8,\
166,188,3,\
166,208,3,\
166,263,8,\
166,177,9,\
166,654,3,\
166,1217,3,\
167,1205,21,\
168,28,25,\
168,143,25,\
168,238,25,\
168,340,17,\
168,380,26,\
168,554,6,\
168,638,25,\
168,1659,17,\
168,1730,19,\
168,1861,17,\
169,37,6,\
170,0,38,\
170,120,21,\
170,143,40,\
170,331,3,\
170,381,20,\
170,440,18,\
170,665,21,\
172,775,3,\
172,734,39,\
173,5,24,\
173,144,17,\
173,231,16,\
174,14,26,\
174,156,15,\
174,257,15,\
174,2045,26,\
176,84,26,\
176,175,16,\
176,1258,34,\
177,37,18,\
177,161,20,\
177,254,13,\
177,348,18,\
177,453,13,\
177,468,2,\
177,1108,18,\
177,1314,2,\
178,36,21,\
178,151,28,\
178,180,21,\
178,204,3,\
178,288,3,\
178,1045,3,\
178,1104,3,\
178,1559,3,\
179,97,7,\
179,239,6,\
179,261,23,\
179,324,5,\
179,435,15,\
179,614,27,\
179,667,34,\
179,782,9,\
179,818,21,\
179,848,17,\
179,989,6,\
179,1138,27,\
180,39,26,\
180,186,26,\
180,227,3,\
180,1001,3,\
180,1034,3,\
180,1412,3,\
181,46,25,\
181,80,42,\
182,0,29,\
183,553,12,\
184,158,17,\
184,1212,20,\
185,440,25,\
186,0,3,\
186,26,17,\
186,116,3,\
186,159,18,\
186,323,5,\
186,384,30,\
186,416,3,\
186,505,10,\
186,635,3,\
186,694,17,\
186,730,7,\
186,767,3,\
186,823,3,\
186,892,17,\
186,951,3,\
186,996,18,\
186,1053,6,\
186,1094,3,\
186,1272,3,\
186,1288,17,\
186,1320,3,\
186,1325,3,\
186,1432,28,\
186,1681,3,\
187,93,28,\
187,123,3,\
187,134,27,\
187,189,20,\
187,372,3,\
187,473,3,\
187,563,18,\
187,756,3,\
187,786,3,\
187,1213,3,\
188,0,18,\
188,126,18,\
188,146,2,\
188,787,2,\
188,930,2,\
189,81,13,\
189,239,5,\
189,680,13,\
189,1200,6,\
190,111,14,\
190,127,40,\
190,196,14,\
190,212,2,\
190,422,2,\
190,429,14,\
190,478,2,\
190,485,14,\
190,1575,35,\
190,1678,2,\
191,159,5,\
191,223,43,\
192,78,23,\
192,103,14,\
192,119,2,\
192,142,23,\
192,258,26,\
192,286,3,\
192,370,39,\
192,534,3,\
192,777,6,\
193,57,12,\
193,115,9,\
193,126,26,\
193,154,3,\
193,189,24,\
193,247,26,\
193,348,20,\
193,526,9,\
193,614,24,\
193,800,41,\
193,861,12,\
193,1073,26,\
193,1150,12,\
193,1334,20,\
194,78,13,\
194,106,13,\
194,179,23,\
194,300,13,\
194,333,13,\
194,427,24,\
194,703,21,\
194,847,20,\
194,1009,20,\
194,1664,13,\
194,1797,14,\
194,1938,13,\
195,54,28,\
195,137,13,\
195,406,13,\
195,649,6,\
195,920,20,\
195,1194,25,\
195,1237,18,\
195,1349,20,\
196,516,36,\
197,111,13,\
197,1259,13,\
197,1410,28,\
198,45,13,\
198,175,43,\
198,244,26,\
198,308,13,\
198,333,21,\
198,220,4,\
198,420,13,\
198,1703,4,\
198,1766,4,\
200,69,6,\
200,269,6,\
200,834,6,\
201,67,3,\
201,72,21,\
201,124,21,\
201,742,28,\
202,45,22,\
202,111,16,\
203,15,32,\
203,64,26,\
203,92,3,\
203,149,18,\
203,171,12,\
203,208,35,\
203,245,3,\
203,263,3,\
203,295,19,\
203,319,10,\
203,370,12,\
203,413,5,\
203,442,3,\
203,485,12,\
203,547,5,\
203,670,35,\
203,798,3,\
203,1097,18,\
203,1145,3,\
203,1163,3,\
203,1336,17,\
203,1428,7,\
204,42,24,\
204,95,24,\
204,134,32,\
204,290,32,\
204,343,19,\
204,388,24,\
204,437,17,\
204,643,31,\
204,694,21,\
204,768,32,\
205,4,33,\
205,693,33,\
205,729,4,\
205,879,4,\
206,80,18,\
206,100,3,\
206,450,28,\
206,565,24,\
206,754,14,\
206,932,23,\
206,957,3,\
206,969,9,\
206,1067,9,\
206,1095,12,\
206,1399,9,\
206,1595,48,\
207,771,6,\
207,890,6,\
207,964,6,\
207,1067,18,\
207,1090,19,\
209,85,15,\
209,102,15,\
209,145,43,\
209,250,18,\
209,604,15,\
209,714,15,\
209,1374,15,\
209,1503,15,\
209,1582,24,\
209,1934,33,\
210,73,19,\
210,191,21,\
210,379,21,\
210,402,2,\
210,409,20,\
210,898,3,\
210,1101,19,\
211,44,28,\
211,295,28,\
211,325,3,\
211,893,3,\
211,1062,3,\
211,1308,19,\
211,1395,17,\
211,1481,10,\
211,1598,22,\
212,0,14,\
212,199,13,\
212,503,14,\
212,869,5,\
212,929,5,\
213,239,12,\
214,81,32,\
214,119,26,\
214,147,4,\
214,171,12,\
214,191,36,\
214,267,27,\
214,296,37,\
214,338,18,\
215,33,28,\
215,123,13,\
215,184,13,\
215,326,28,\
215,394,13,\
215,900,24,\
215,1429,13,\
215,1520,28,\
215,1556,13,\
216,173,7,\
216,286,25,\
216,339,7,\
216,658,7,\
216,1043,2,\
216,1309,17,\
217,31,6,\
217,112,6,\
217,151,26,\
217,1450,15,\
217,1581,24,\
217,1698,15,\
218,86,28,\
218,116,28,\
218,146,3,\
218,179,17,\
218,244,3,\
218,290,5,\
218,372,17,\
218,402,2,\
218,406,11,\
218,390,10,\
218,513,11,\
218,530,3,\
218,757,3,\
218,803,3,\
219,0,27,\
219,29,3,\
219,597,3,\
219,698,27,\
219,727,3,\
219,1285,26,\
220,31,14,\
220,91,14,\
220,107,3,\
220,116,28,\
220,220,36,\
221,32,85,\
221,136,91,\
221,245,28,\
221,335,10,\
221,347,23,\
221,1216,13,\
221,1300,13,\
221,1393,13,\
221,1680,12,\
221,1710,19,\
222,0,20,\
222,203,20,\
222,225,2,\
222,570,2,\
223,380,16,\
224,212,16,\
224,230,3,\
224,236,12,\
224,357,3,\
224,455,3,\
224,572,3,\
224,1195,5,\
225,2035,5,\
226,72,25,\
226,99,26,\
226,265,13,\
226,439,6,\
226,509,25,\
226,620,24,\
226,681,13,\
226,789,14,\
226,1011,18,\
226,1049,20,\
226,1092,14,\
226,1195,13,\
226,1281,14,\
226,1364,25,\
226,1456,13,\
226,1537,25,\
226,1625,13,\
226,1687,24,\
226,1778,14,\
226,1924,25,\
227,188,37,\
227,229,36,\
227,267,4,\
227,281,4,\
228,0,20,\
228,39,29,\
228,73,21,\
228,118,20,\
228,177,17,\
228,140,2,\
228,278,18,\
228,319,53,\
229,84,15,\
229,162,15,\
229,213,19,\
229,383,20,\
230,86,17,\
230,142,17,\
230,161,3,\
230,183,18,\
230,216,27,\
230,357,3,\
230,886,3,\
230,1000,3,\
231,16,18,\
231,97,18,\
231,142,28,\
231,1196,18,\
231,1317,19,\
232,74,14,\
233,0,9,\
233,169,18,\
233,1315,18,\
233,1573,9,\
235,129,20,\
235,151,2,\
235,174,20,\
235,257,33,\
235,300,20,\
236,9,17,\
236,148,17,\
236,238,18,\
236,286,18,\
236,306,2,\
236,311,2,\
236,338,19,\
236,600,5,\
236,1362,5,\
236,1470,22,\
236,1611,2,\
236,1681,2,\
236,1701,5,\
237,26,3,\
237,96,3,\
237,155,17,\
237,227,17,\
237,300,30,\
237,333,3,\
237,384,26,\
237,413,3,\
237,484,3,\
237,625,3,\
237,650,17,\
237,683,3,\
237,722,3,\
237,741,17,\
237,955,3,\
237,1085,17,\
237,1237,3,\
237,1636,3,\
237,1661,3,\
237,1677,17,\
238,37,20,\
238,66,20,\
238,111,27,\
238,259,20,\
238,878,19,\
239,32,14,\
239,175,14,\
239,191,3,\
239,974,31,\
239,1582,3,\
239,1703,3,\
240,0,21,\
240,156,31,\
240,189,3,\
240,224,14,\
240,240,4,\
240,349,9,\
240,461,21,\
240,621,3,\
240,952,3,\
240,1083,3,\
240,1149,3,\
240,1176,12,\
240,1221,3,\
240,1404,3,\
240,1461,3,\
240,1582,3,\
241,47,25,\
241,138,25,\
241,165,6,\
241,561,3,\
241,689,3,\
242,112,51,\
242,196,19,\
242,241,23,\
242,696,14,\
242,1209,14,\
243,67,21,\
243,154,21,\
243,177,3,\
243,186,17,\
243,205,2,\
243,280,3,\
243,354,2,\
243,441,20,\
243,466,18,\
243,838,3,\
244,50,34,\
244,242,34,\
244,278,4,\
244,519,4,\
244,1663,4,\
245,46,37,\
245,152,39,\
245,193,6,\
245,363,4,\
245,573,26,\
245,616,8,\
245,1306,15,\
245,1604,8,\
245,1707,8,\
248,43,21,\
248,130,16,\
248,147,21,\
248,170,3,\
248,442,3,\
248,573,3,\
248,851,3,\
248,1185,3,\
249,24,48,\
249,188,41,\
249,1213,16,\
250,49,14,\
250,964,31,\
250,1000,23,\
250,1276,19,\
251,536,16,\
252,78,18,\
252,123,39,\
252,175,19,\
252,424,32,\
252,1059,18,\
253,0,14,\
253,202,13,\
253,235,14,\
253,613,6,\
253,722,7,\
253,742,13,\
253,852,36,\
253,999,14,\
253,1302,25,\
253,1354,13,\
253,1535,6,\
255,81,18,\
255,128,18,\
255,149,2,\
255,270,2,\
255,447,2,\
255,764,2,\
255,1261,18,\
256,134,13,\
256,385,5,\
256,502,3,\
256,581,7,\
257,844,5,\
258,114,19,\
258,135,2,\
258,145,27,\
258,278,26,\
258,332,2,\
258,738,2,\
258,793,2,\
258,856,32,\
259,15,26,\
259,43,3,\
259,75,9,\
259,126,38,\
259,248,3,\
259,331,28,\
259,361,4,\
259,395,3,\
259,631,3,\
259,652,6,\
259,700,3,\
259,776,3,\
259,853,3,\
260,63,41,\
260,209,8,\
260,260,39,\
260,470,12,\
260,1115,12,\
260,1231,12,\
262,53,42,\
262,214,42,\
262,258,5,\
262,580,28,\
262,620,30,\
262,694,5,\
262,715,18,\
262,772,21,\
262,947,20,\
262,1360,5,\
262,2015,5,\
263,1402,11,\
263,1589,11,\
263,1944,11,\
264,92,28,\
264,215,53,\
264,271,3,\
264,307,25,\
264,387,2,\
264,561,2,\
264,591,3,\
264,1324,3,\
264,1478,3,\
264,1615,2,\
265,17,31,\
265,82,3,\
265,105,24,\
265,354,3,\
265,381,24,\
265,490,31,\
265,523,3,\
266,117,35,\
266,733,6,\
267,0,35,\
267,185,35,\
267,222,3,\
267,286,26,\
267,314,3,\
267,336,3,\
267,891,3,\
267,1191,6,\
267,1226,3,\
268,0,24,\
268,259,31,\
268,292,3,\
268,314,24,\
268,340,3,\
268,404,8,\
269,43,35,\
269,108,35,\
269,145,3,\
269,255,42,\
269,494,44,\
269,752,3,\
269,1384,18,\
269,1406,3,\
270,66,25,\
270,113,25,\
270,141,2,\
270,362,24,\
270,458,19,\
270,1461,3,\
270,2019,16,\
271,43,15,\
271,67,19,\
271,88,64,\
271,168,13,\
271,210,27,\
271,264,20,\
271,556,43,\
271,606,13,\
271,641,19,\
271,1499,15,\
272,0,22,\
272,182,22,\
272,206,3,\
272,452,14,\
272,663,3,\
272,870,3,\
272,1266,3,\
272,1373,3,\
272,1404,3,\
272,1570,3,\
272,1621,3,\
273,31,16,\
273,219,27,\
273,248,2,\
273,375,24,\
273,401,3,\
273,470,2,\
273,577,28,\
273,607,4,\
273,824,4,\
273,957,2,\
273,1110,3,\
274,47,32,\
274,116,32,\
274,150,3,\
274,825,32,\
274,973,3,\
275,59,18,\
275,94,18,\
275,128,27,\
275,613,9,\
275,795,48,\
275,845,2,\
275,1134,15,\
275,1164,43,\
275,1240,52,\
275,1296,2,\
276,94,20,\
276,198,20,\
276,283,7,\
276,417,18,\
276,554,7,\
276,729,10,\
276,965,6,\
276,1497,21,\
276,1609,3,\
277,71,8,\
277,111,28,\
277,141,4,\
277,477,3,\
277,1129,51,\
277,1231,3,\
278,133,12,\
278,199,21,\
278,234,27,\
280,353,16,\
280,381,14,\
280,400,25,\
280,1159,3,\
281,833,24,\
281,970,21,\
282,107,26,\
282,286,26,\
282,726,15,\
282,743,11,\
283,44,26,\
283,142,25,\
283,175,19,\
283,254,29,\
283,343,30,\
283,479,26,\
283,569,14,\
283,636,10,\
283,661,18,\
283,1155,26,\
283,1323,17,\
283,1400,17,\
283,1486,15,\
283,1577,19,\
284,120,13,\
284,445,13,\
284,611,4,\
284,812,4,\
284,2104,4,\
284,2412,4,\
284,2612,5,\
285,116,16,\
285,221,23,\
285,1861,16,\
286,134,20,\
286,156,2,\
286,413,20,\
286,578,2,\
286,587,15,\
287,154,17,\
287,421,33,\
287,579,39,\
287,687,15,\
287,1392,15,\
288,170,15,\
288,255,18,\
289,30,3,\
289,71,40,\
289,237,26,\
289,265,3,\
289,433,3,\
289,516,13,\
289,533,15,\
289,566,26,\
289,696,3,\
289,732,40,\
289,936,15,\
289,1017,6,\
289,1090,3,\
289,1135,40,\
289,1288,3,\
289,1324,20,\
289,1392,39,\
290,67,21,\
290,218,21,\
290,324,32,\
290,974,23,\
290,1070,7,\
290,1201,13,\
290,1458,13,\
290,1480,22,\
290,1599,13,\
290,1748,13,\
291,116,25,\
291,202,12,\
291,345,37,\
291,1050,25,\
292,91,15,\
292,108,3,\
292,117,28,\
292,158,47,\
292,507,15,\
293,1,24,\
293,26,26,\
294,22,19,\
294,86,19,\
294,107,4,\
294,119,46,\
294,279,19,\
294,394,19,\
294,453,4,\
294,603,19,\
294,842,4,\
294,889,19,\
295,78,14,\
295,162,3,\
295,202,16,\
295,123,37,\
295,504,3,\
295,876,3,\
295,995,3,\
295,1180,3,\
295,1425,12,\
295,1735,3,\
295,2185,3,\
296,112,39,\
296,153,4,\
297,18,18,\
297,39,38,\
297,189,16,\
297,289,22,\
297,320,14,\
297,437,15,\
297,526,27,\
297,749,16,\
297,817,14,\
297,919,14,\
297,973,22,\
297,997,15,\
298,163,25,\
298,241,5,\
298,1422,5,\
299,1,28,\
299,62,3,\
299,71,19,\
299,184,11,\
299,233,14,\
299,299,3,\
299,471,11,\
301,92,36,\
301,726,13,\
301,808,6,\
302,25,49,\
302,200,57,\
302,266,25,\
302,333,12,\
302,350,12,\
302,527,16,\
302,574,49,\
302,853,12,\
303,27,32,\
303,107,32,\
303,272,2,\
303,257,13,\
303,141,3,\
303,594,8,\
303,788,18,\
303,883,29,\
303,975,17,\
304,57,17,\
304,164,18,\
304,303,10,\
304,624,18,\
304,764,6,\
304,1182,6,\
304,1266,17,\
305,67,25,\
305,172,36,\
305,319,21,\
305,978,6,\
307,15,19,\
307,80,19,\
307,104,10,\
307,178,22,\
307,226,23,\
307,377,9,\
307,509,10,\
307,559,19,\
307,611,19,\
307,649,9,\
307,774,12,\
307,885,12,\
307,962,19,\
308,0,28,\
308,30,28,\
308,60,3,\
308,86,31,\
308,169,5,\
308,179,9,\
308,318,3,\
308,612,3,\
308,643,21,\
308,814,11,\
308,1052,17,\
309,31,21,\
309,202,3,\
309,286,38,\
309,175,25,\
309,505,3,\
309,932,56,\
309,1344,18,\
309,1400,38,\
310,15,52,\
310,154,27,\
310,277,23,\
310,302,3,\
310,722,3,\
310,780,30,\
310,812,3,\
310,818,3,\
310,827,3,\
310,842,23,\
310,984,3,\
310,1056,6,\
310,1144,6,\
310,1394,27,\
310,1423,2,\
310,1606,5,\
310,1690,2,\
310,1763,5,\
311,35,8,\
311,130,14,\
311,156,16,\
311,880,3,\
311,862,16,\
311,1069,3,\
311,1095,3,\
311,1210,3,\
311,1348,3,\
311,1606,14,\
312,35,26,\
312,63,3,\
312,119,18,\
312,171,26,\
312,199,3,\
312,570,15,\
312,767,3,\
312,805,15,\
312,1636,27,\
313,73,25,\
313,135,25,\
313,239,36,\
313,810,6,\
313,925,25,\
314,23,34,\
314,257,25,\
314,419,16,\
314,1583,24,\
314,1652,45,\
315,370,25,\
315,405,25,\
315,397,2,\
315,1168,2,\
315,1282,2,\
315,1154,12,\
315,1652,2,\
315,1971,6,\
315,1982,2,\
316,239,31,\
316,1052,10,\
316,1735,21,\
317,91,23,\
317,207,23,\
317,453,27,\
317,657,23,\
317,685,30,\
317,928,14,\
317,955,18,\
317,984,29,\
317,1048,18,\
318,111,52,\
318,165,6,\
318,739,52,\
318,793,7,\
319,105,5,\
319,353,5,\
319,716,21,\
319,739,2,\
319,776,2,\
320,24,20,\
320,136,24,\
320,211,22,\
320,323,31,\
320,1143,24,\
320,1347,24,\
320,1743,12,\
321,158,26,\
321,191,46,\
321,253,35,\
321,305,13,\
321,1121,16,\
321,1327,25,\
321,1529,51,\
322,92,5,\
322,204,16,\
322,893,9,\
322,1288,15,\
322,1501,13,\
322,1807,5,\
323,43,20,\
323,98,33,\
323,138,27,\
323,213,42,\
323,263,18,\
323,287,14,\
323,382,3,\
323,752,15,\
323,771,19,\
323,1176,17,\
323,1219,20,\
323,1266,22,\
324,26,23,\
324,69,23,\
324,153,13,\
324,272,13,\
324,317,23,\
324,371,13,\
324,493,7,\
324,533,23,\
324,649,21,\
324,816,22,\
324,976,13,\
324,1146,13,\
324,1483,6,\
324,1527,13,\
324,1679,23,\
326,84,24,\
326,156,23,\
326,208,13,\
326,311,13,\
326,398,24,\
326,906,12,\
327,90,18,\
327,111,2,\
327,229,2,\
327,748,2,\
328,14,22,\
328,215,31,\
328,1593,43,\
329,72,17,\
329,91,17,\
329,112,26,\
329,286,18,\
329,313,18,\
329,558,17,\
329,1013,17,\
330,53,14,\
330,72,24,\
330,102,9,\
330,155,6,\
330,436,43,\
330,514,7,\
330,530,22,\
330,555,38,\
330,665,9,\
330,686,14,\
330,703,2,\
330,752,30,\
330,825,2,\
330,847,3,\
330,955,2,\
330,1249,2,\
330,1338,2,\
330,1625,3,\
330,1854,2,\
330,1874,3,\
330,1956,3,\
330,2024,2,\
330,2168,2,\
330,2185,14,\
330,2295,18,\
330,2472,24,\
330,2902,2,\
332,0,18,\
332,52,25,\
332,86,25,\
332,113,2,\
332,162,4,\
332,126,34,\
332,428,9,\
334,16,13,\
334,33,18,\
334,84,23,\
334,126,18,\
334,146,18,\
334,171,27,\
334,545,18,\
334,905,18,\
334,971,18,\
334,1045,13,\
334,1072,13,\
334,1134,18,\
335,17,21,\
335,232,18,\
335,335,23,\
335,396,21,\
335,419,3,\
335,438,29,\
335,360,3,\
335,1587,3,\
335,2497,31,\
336,165,12,\
337,1266,22,\
337,1414,8,\
338,69,36,\
338,162,15,\
338,321,26,\
338,349,3,\
338,441,3,\
338,562,3,\
338,585,24,\
338,1035,3,\
338,1336,15,\
338,1484,15,\
338,1939,15,\
338,2093,15,\
338,2203,15,\
338,2359,15,\
339,94,17,\
339,275,16,\
339,316,7,\
339,757,16,\
339,958,21,\
339,1108,16,\
340,52,24,\
340,171,30,\
340,321,24,\
340,491,35,\
340,1021,16,\
340,1701,24,\
341,84,24,\
341,114,19,\
341,135,24,\
341,165,22,\
341,308,24,\
341,349,8,\
341,384,24,\
341,425,8,\
341,482,8,\
341,560,24,\
341,648,8,\
341,714,15,\
341,790,8,\
341,1014,8,\
341,1302,24,\
341,1343,8,\
341,1453,19,\
342,1,39,\
342,41,4,\
344,43,18,\
344,79,18,\
345,416,7,\
345,1062,6,\
345,1421,21,\
345,1455,9,\
346,588,22,\
347,61,24,\
347,91,24,\
347,117,3,\
347,128,25,\
347,342,3,\
348,72,13,\
348,133,13,\
348,197,35,\
348,399,36,\
348,507,26,\
348,544,13,\
348,852,13,\
348,891,14,\
348,1500,25,\
348,1632,13,\
348,1770,14,\
348,1809,16,\
349,27,18,\
349,109,18,\
349,129,2,\
349,226,2,\
349,848,46,\
349,896,13,\
349,1084,2,\
350,105,26,\
350,159,26,\
350,188,4,\
350,254,5,\
350,269,9,\
350,415,4,\
350,553,4,\
350,661,4,\
350,705,17,\
350,773,20,\
350,946,4,\
350,1520,4,\
351,161,3,\
351,202,23,\
351,137,22,\
352,53,15,\
352,77,26,\
352,202,15,\
352,647,27,\
353,89,6,\
353,100,15,\
353,426,15,\
353,469,15,\
353,1369,15,\
354,205,5,\
354,246,6,\
354,435,6,\
354,640,6,\
354,1786,6,\
354,2148,5,\
355,0,69,\
355,86,16,\
355,107,21,\
355,186,16,\
355,204,49,\
355,256,6,\
355,271,26,\
355,420,15,\
355,457,10,\
355,471,21,\
355,584,15,\
355,624,13,\
355,701,3,\
355,879,25,\
355,1230,28,\
355,1718,16,\
355,1807,5,\
355,1819,17,\
355,1866,16,\
355,2060,5,\
356,37,15,\
356,268,15,\
356,285,2,\
356,915,2,\
356,1681,2,\
357,70,22,\
357,250,24,\
357,549,39,\
357,971,24,\
357,1287,24,\
358,71,23,\
358,240,23,\
358,270,35,\
358,514,41,\
358,557,2,\
358,611,4,\
358,1009,20,\
358,1142,2,\
358,1216,20,\
359,334,29,\
359,365,2,\
359,418,30,\
359,855,2,\
359,908,2,\
359,1012,2,\
360,26,16,\
360,44,16,\
360,62,3,\
360,108,29,\
360,172,12,\
360,205,3,\
360,692,24,\
360,721,23,\
360,853,18,\
360,911,18,\
362,11,25,\
362,41,20,\
363,213,25,\
363,255,6,\
363,310,17,\
363,467,8,\
363,905,6,\
363,954,17,\
364,1318,41,\
365,0,44,\
365,159,23,\
365,184,35,\
365,221,3,\
365,582,23,\
365,925,3,\
365,1016,3,\
365,1058,15,\
365,1075,2,\
365,1165,3,\
365,1193,15,\
365,1210,2,\
365,1752,3,\
365,1768,21,\
365,1793,3,\
365,1885,3,\
365,1996,27,\
365,2067,3,\
365,2086,15,\
365,2221,23,\
365,2250,3,\
366,26,18,\
366,46,18,\
366,67,2,\
366,78,26,\
366,219,29,\
366,251,3,\
366,283,2,\
366,414,2,\
366,486,2,\
366,524,2,\
366,939,2,\
366,1004,2,\
366,1094,2,\
367,335,3,\
367,312,21,\
367,828,15,\
367,1454,3,\
367,1516,3,\
367,2197,3,\
368,0,37,\
368,103,10,\
368,117,26,\
368,182,15,\
368,228,26,\
368,305,35,\
368,363,10,\
368,433,26,\
368,475,17,\
368,519,26,\
368,558,10,\
368,591,26,\
368,641,17,\
368,724,17,\
368,1016,10,\
368,1041,26,\
368,1132,26,\
368,1180,10,\
368,1208,26,\
368,1236,2,\
368,1292,10,\
368,1317,26,\
368,1357,10,\
368,1385,26,\
368,1491,26,\
368,1578,35,\
368,1834,26,\
370,64,12,\
370,81,24,\
370,172,40,\
370,767,12,\
371,55,16,\
371,82,41,\
371,152,15,\
371,207,27,\
371,489,18,\
372,106,28,\
372,149,28,\
372,179,3,\
372,207,18,\
372,227,2,\
372,667,22,\
372,1837,32,\
373,533,5,\
373,849,13,\
373,988,36,\
373,1080,26,\
373,1245,4,\
373,1639,5,\
373,1947,11,\
373,1964,5,\
374,48,6,\
374,65,19,\
374,86,4,\
374,132,33,\
374,167,3,\
374,179,46,\
374,389,3,\
374,438,4,\
374,767,15,\
375,242,33,\
375,1744,36,\
376,44,15,\
376,81,15,\
376,98,3,\
376,129,36,\
376,176,3,\
376,188,18,\
376,532,3,\
376,621,3,\
378,92,22,\
378,116,22,\
378,140,3,\
378,155,25,\
378,199,61,\
378,418,18,\
378,438,3,\
379,20,29,\
379,63,18,\
379,92,17,\
379,271,14,\
379,287,9,\
379,298,15,\
379,457,29,\
379,526,9,\
379,537,23,\
379,562,15,\
380,0,21,\
380,40,9,\
380,54,8,\
380,64,21,\
380,87,2,\
380,114,28,\
380,160,21,\
380,228,6,\
380,257,2,\
380,280,6,\
380,319,21,\
380,358,2,\
380,411,18,\
380,445,8,\
380,476,9,\
380,1298,2,\
380,1349,13,\
380,1435,2,\
380,1467,15,\
380,1557,19,\
380,1594,2,\
380,1631,8,\
380,1763,28,\
380,1880,19,\
380,1990,40,\
380,2047,29,\
380,2226,30,\
380,2501,21,\
380,2570,6,\
380,2745,2,\
381,66,16,\
381,86,12,\
381,299,15,\
381,413,12,\
381,461,15,\
381,709,12,\
381,766,16,\
381,858,16,\
381,1061,16,\
381,1582,16,\
381,1880,16,\
381,1900,12,\
382,358,17,\
382,707,19,\
382,833,18,\
382,877,11,\
382,1304,8,\
382,1512,21,\
382,1728,12,\
382,1934,12,\
382,2207,10,\
383,88,15,\
383,130,16,\
383,422,15,\
383,680,5,\
383,731,15,\
383,748,13,\
383,1229,24,\
383,1366,18,\
383,1394,15,\
384,35,15,\
384,337,15,\
384,483,15,\
384,604,26,\
384,632,3,\
384,735,15,\
384,1246,3,\
384,1324,3,\
384,1423,24,\
384,1957,15,\
385,25,3,\
385,87,22,\
385,136,22,\
385,160,3,\
385,171,45,\
385,347,22,\
385,416,24,\
385,804,3,\
385,955,3,\
385,1424,9,\
385,1696,3,\
387,51,12,\
387,137,12,\
387,152,5,\
387,1792,12,\
388,81,42,\
388,442,53,\
388,899,8,\
390,0,24,\
390,39,21,\
390,164,32,\
390,320,13,\
390,448,20,\
390,552,24,\
390,633,21,\
390,949,9,\
391,70,19,\
391,122,53,\
391,631,18,\
391,1174,21,\
391,1858,21,\
391,1939,20,\
392,215,75,\
393,23,12,\
393,208,21,\
393,231,3,\
393,247,17,\
393,266,2,\
393,337,36,\
393,397,14,\
393,375,5,\
393,488,3,\
393,495,2,\
393,516,21,\
393,854,3,\
393,861,2,\
393,966,3,\
393,970,2,\
393,1084,3,\
393,1338,3,\
393,1526,14,\
393,1799,3,\
393,1803,2,\
394,18,5,\
394,131,15,\
394,293,15,\
394,599,15,\
394,656,18,\
394,733,15,\
394,1031,5,\
394,1040,4,\
394,1521,15,\
394,1606,6,\
394,1750,15,\
395,81,15,\
395,145,15,\
395,162,2,\
395,170,28,\
395,538,38,\
395,818,2,\
395,1143,8,\
395,1232,8,\
395,1454,8,\
395,1878,15,\
395,2372,8,\
396,76,14,\
396,227,37,\
396,690,14,\
397,114,5,\
397,151,7,\
397,215,5,\
397,302,6,\
397,529,3,\
397,717,27,\
397,925,6,\
397,968,6,\
398,9,29,\
398,181,29,\
398,212,3,\
398,376,3,\
398,419,3,\
398,595,10,\
398,878,31,\
398,1138,3,\
398,1288,3,\
399,18,26,\
399,81,26,\
399,135,28,\
399,1507,14,\
400,494,28,\
400,523,21,\
400,546,2,\
400,940,3,\
400,1419,2,\
401,54,19,\
401,256,18,\
401,310,7,\
401,785,7,\
402,64,8,\
402,77,25,\
402,133,8,\
402,147,17,\
402,175,29,\
402,394,8,\
402,470,17,\
402,575,3,\
402,683,8,\
402,721,3,\
402,829,8,\
402,842,18,\
402,1168,8,\
403,0,28,\
403,72,3,\
403,312,3,\
403,477,3,\
403,1522,3,\
404,618,53,\
405,21,23,\
405,112,15,\
405,129,2,\
405,160,26,\
405,201,45,\
405,330,19,\
405,545,10,\
405,782,2,\
405,853,10,\
405,901,2,\
405,1011,2,\
405,1165,10,\
406,78,21,\
406,141,16,\
406,159,5,\
406,807,5,\
407,283,31,\
408,0,19,\
408,147,17,\
408,275,17,\
408,302,19,\
408,612,19,\
408,789,18,\
408,905,19,\
408,1010,19,\
408,1591,18,\
408,1643,19,\
409,917,5,\
409,1698,4,\
410,82,34,\
410,121,21,\
410,156,15,\
410,173,2,\
410,280,13,\
410,295,22,\
410,326,20,\
410,478,2,\
410,708,39,\
410,784,17,\
410,803,2,\
410,914,2,\
410,1708,18,\
412,43,14,\
412,194,14,\
412,305,20,\
413,11,18,\
414,288,5,\
414,480,23,\
414,591,14,\
414,1021,15,\
414,1302,22,\
415,1271,8,\
415,1315,11,\
416,0,32,\
416,50,15,\
416,124,15,\
416,141,3,\
416,253,3,\
416,349,24,\
416,442,3,\
416,523,24,\
416,613,3,\
416,663,24,\
417,255,21,\
418,62,22,\
418,442,22,\
418,466,3,\
418,774,3,\
419,0,29,\
419,121,29,\
419,367,31,\
419,737,18,\
419,841,18,\
421,25,22,\
421,49,22,\
421,73,3,\
421,90,39,\
421,206,3,\
421,260,28,\
421,350,11,\
421,515,3,\
421,739,15,\
421,874,3,\
421,922,3,\
421,941,16,\
421,1081,3,\
422,310,27,\
422,368,19,\
422,1983,13,\
423,167,14,\
423,286,2,\
423,266,18,\
423,576,18,\
423,777,2,\
424,32,20,\
424,85,20,\
424,156,20,\
424,234,20,\
424,772,27,\
425,7,22,\
425,216,29,\
425,290,14,\
425,1714,29,\
425,1956,43,\
426,1,20,\
427,54,24,\
427,175,19,\
427,247,13,\
427,315,31,\
427,371,28,\
427,822,22,\
427,853,34,\
427,985,24,\
427,1716,13,\
428,274,22,\
428,766,20,\
428,790,8,\
429,187,11,\
429,453,3,\
430,65,8,\
430,565,8,\
430,714,3,\
430,680,32,\
431,309,13,\
431,447,13,\
431,625,13,\
432,17,26,\
432,102,16,\
432,287,37,\
432,368,23,\
432,733,18,\
432,780,18,\
432,810,10,\
433,46,16,\
433,106,16,\
433,128,17,\
433,409,16,\
433,610,16,\
433,949,15,\
433,1565,25,\
433,1707,17,\
433,1937,17,\
433,2045,16,\
434,86,27,\
434,115,3,\
434,152,19,\
434,269,3,\
434,387,20,\
435,70,17,\
435,152,36,\
435,242,16,\
435,300,16,\
435,455,26,\
435,484,15,\
435,1014,19,\
435,1127,40,\
435,1189,10,\
435,1292,17,\
435,1411,23,\
436,127,13,\
436,170,12,\
436,184,2,\
436,1263,17,\
436,1360,17,\
436,1385,26,\
437,598,23,\
437,692,26,\
438,26,20,\
438,51,38,\
438,124,28,\
438,202,24,\
438,528,18,\
439,70,27,\
439,99,3,\
439,226,24,\
439,269,24,\
439,315,19,\
439,339,31,\
439,551,21,\
440,0,67,\
440,254,6,\
440,262,2,\
440,408,14,\
440,661,14,\
440,862,6,\
440,1036,37,\
440,1191,2,\
441,0,33,\
441,356,33,\
441,736,28,\
441,766,3,\
441,826,4,\
441,964,3,\
441,1074,3,\
441,1079,33,\
441,1118,21,\
441,1256,19,\
441,1280,16,\
441,1326,3,\
441,1352,18,\
441,1416,54,\
441,1472,4,\
441,1538,18,\
441,1810,18,\
441,1830,2,\
441,1960,2,\
441,2033,2,\
441,2053,11,\
441,2147,19,\
441,2420,14,\
441,2516,14,\
441,2573,19,\
441,2839,14,\
442,0,28,\
442,30,3,\
442,46,11,\
442,122,28,\
442,152,3,\
442,163,28,\
442,233,5,\
442,243,9,\
442,254,17,\
442,382,3,\
442,400,11,\
442,522,11,\
442,559,3,\
442,675,11,\
442,737,18,\
442,774,19,\
442,990,18,\
442,1037,11,\
442,1163,13,\
442,1522,11,\
442,1570,3,\
442,1627,13,\
442,1676,11,\
442,1702,18,\
442,1778,18,\
442,1815,11,\
442,1987,3,\
442,2008,3,\
442,2023,11,\
443,0,41,\
443,45,17,\
443,206,12,\
443,248,22,\
443,272,2,\
443,277,9,\
443,288,6,\
443,296,26,\
443,324,3,\
443,703,2,\
443,744,2,\
443,778,11,\
443,836,2,\
443,886,2,\
443,988,2,\
443,997,18,\
443,1135,6,\
443,1301,14,\
443,1367,15,\
443,1434,18,\
443,1506,9,\
443,1517,3,\
443,1522,6,\
443,1533,2,\
443,1578,2,\
443,1600,3,\
443,1605,9,\
443,1619,6,\
443,1669,3,\
443,1737,2,\
443,1798,2,\
443,1815,6,\
443,1938,28,\
443,2058,2,\
443,2166,21,\
443,2203,29,\
443,2362,9,\
443,2376,6,\
443,2418,2,\
443,2473,2,\
443,2493,9,\
443,2507,3,\
443,2575,2,\
444,164,21,\
444,256,6,\
444,318,5,\
444,422,15,\
447,0,12,\
447,22,34,\
447,71,34,\
447,118,16,\
447,107,4,\
447,559,29,\
447,1109,3,\
447,1116,20,\
447,1219,4,\
447,1352,10,\
447,1445,5,\
447,1645,4,\
448,54,36,\
448,274,24,\
448,369,36,\
448,407,4,\
449,159,15,\
449,695,5,\
449,1079,13,\
450,30,3,\
450,64,24,\
450,90,24,\
450,116,3,\
450,134,25,\
450,177,19,\
450,198,6,\
450,210,21,\
450,233,3,\
450,313,3,\
450,367,3,\
450,422,15,\
450,439,6,\
450,451,27,\
450,561,3,\
451,44,24,\
451,395,13,\
451,476,28,\
451,1177,25,\
451,1340,13,\
451,1486,13,\
452,13,14,\
453,78,24,\
453,104,16,\
453,458,16,\
453,477,3,\
453,748,3,\
453,1186,3,\
454,39,15,\
454,86,24,\
454,143,27,\
454,196,24,\
454,222,8,\
454,236,21,\
454,308,15,\
454,388,24,\
454,476,24,\
454,890,15,\
454,937,24,\
454,1129,24,\
454,1339,24,\
454,1444,24,\
455,68,24,\
455,129,24,\
455,155,3,\
455,164,27,\
455,221,35,\
455,307,6,\
455,319,16,\
455,536,3,\
455,1264,25,\
455,1462,3,\
455,1732,3,\
456,152,27,\
456,269,36,\
456,307,4,\
457,20,14,\
457,36,2,\
457,69,2,\
457,159,14,\
457,176,2,\
457,234,2,\
457,279,2,\
457,593,2,\
457,700,27,\
457,995,2,\
457,1114,40,\
458,40,21,\
458,81,16,\
458,99,21,\
458,123,3,\
458,241,6,\
458,250,30,\
458,312,3,\
458,359,23,\
458,398,6,\
458,428,40,\
458,488,17,\
458,536,16,\
458,561,3,\
458,677,3,\
458,728,9,\
458,1090,3,\
458,1183,6,\
458,1235,9,\
458,1303,6,\
458,1544,7,\
458,1621,14,\
458,1668,6,\
459,0,37,\
459,92,23,\
459,218,16,\
459,281,17,\
459,343,27,\
459,412,17,\
459,477,25,\
459,534,17,\
459,587,25,\
459,700,17,\
459,728,5,\
459,778,17,\
459,810,36,\
459,914,23,\
459,976,16,\
459,1098,23,\
460,317,34,\
460,374,2,\
460,678,2,\
461,137,17,\
461,167,13,\
461,298,17,\
461,612,13,\
462,156,18,\
462,737,9,\
462,892,9,\
462,1221,18,\
462,1458,3,\
463,37,21,\
463,113,17,\
463,244,26,\
463,329,26,\
463,451,17,\
463,565,10,\
463,747,10,\
463,777,17,\
463,1102,23,\
464,165,21,\
464,211,21,\
464,234,3,\
464,257,58,\
464,519,3,\
464,593,3,\
464,785,3,\
464,884,3,\
464,941,2,\
464,925,14,\
464,1148,3,\
464,1324,22,\
464,1476,2,\
464,1586,3,\
464,1643,3,\
464,1759,3,\
464,1825,7,\
464,1836,3,\
464,1845,26,\
465,96,25,\
465,123,25,\
465,151,7,\
465,167,46,\
465,226,14,\
465,497,17,\
465,786,7,\
466,114,5,\
466,308,21,\
467,23,15,\
467,66,15,\
467,127,18,\
467,210,16,\
467,320,15,\
467,599,15,\
467,704,18,\
467,834,15,\
467,965,15,\
467,1268,18,\
467,1534,18,\
468,82,26,\
468,115,44,\
469,8,21,\
469,48,21,\
469,108,21,\
469,131,3,\
469,229,32,\
469,336,2,\
469,362,3,\
469,529,3,\
469,687,21,\
469,733,3,\
469,795,3,\
469,803,19,\
470,0,44,\
470,90,51,\
470,148,30,\
470,347,14,\
470,373,14,\
470,415,14,\
470,491,24,\
470,517,3,\
470,540,15,\
470,566,3,\
470,730,9,\
470,949,15,\
470,1122,15,\
470,1183,16,\
471,41,3,\
471,53,29,\
471,172,25,\
471,199,3,\
471,323,21,\
471,364,3,\
471,346,4,\
471,534,3,\
471,657,3,\
471,780,3,\
471,850,3,\
471,961,12,\
471,1022,3,\
471,1232,3,\
471,1275,5,\
471,1322,5,\
471,1416,3,\
471,1605,9,\
474,59,42,\
474,124,51,\
474,178,5,\
474,330,17,\
474,621,17,\
476,53,13,\
476,145,21,\
476,168,3,\
476,199,13,\
476,229,3,\
476,324,13,\
476,584,13,\
476,1039,13,\
476,1567,13,\
476,1670,13,\
477,1,49,\
478,68,17,\
478,90,13,\
478,105,16,\
478,124,16,\
478,143,3,\
478,169,46,\
478,250,35,\
478,302,13,\
478,464,3,\
478,653,3,\
479,95,26,\
479,124,2,\
479,825,2,\
479,1077,6,\
480,98,47,\
480,911,48,\
481,63,17,\
481,138,8,\
481,150,23,\
481,191,15,\
481,243,21,\
481,360,17,\
481,480,8,\
483,53,31,\
483,141,17,\
483,188,35,\
483,236,18,\
483,225,4,\
483,930,40,\
483,972,4,\
483,983,39,\
483,1024,5,\
483,1135,13,\
483,1175,5,\
483,1593,5,\
483,1637,4,\
483,1683,4,\
483,1645,36,\
483,1988,4,\
483,2278,4,\
483,2440,4,\
484,16,3,\
484,75,35,\
484,147,3,\
484,178,30,\
484,214,43,\
484,301,3,\
484,386,2,\
484,458,35,\
484,516,33,\
484,568,35,\
484,668,2,\
484,738,21,\
484,779,28,\
484,812,21,\
484,958,3,\
484,983,7,\
484,1070,3,\
484,1129,34,\
485,12,30,\
485,47,11,\
485,62,18,\
485,145,16,\
485,179,23,\
485,207,11,\
485,236,37,\
485,310,19,\
485,333,18,\
485,388,19,\
485,601,18,\
485,1306,3,\
485,1650,16,\
485,1670,18,\
486,0,20,\
486,48,15,\
486,71,20,\
486,127,29,\
486,204,6,\
486,369,3,\
486,494,14,\
486,534,7,\
487,0,22,\
487,24,22,\
487,48,3,\
487,59,40,\
487,142,53,\
487,201,20,\
487,252,3,\
488,0,16,\
488,21,22,\
488,64,14,\
488,80,14,\
488,101,40,\
488,143,8,\
488,157,18,\
488,388,12,\
488,421,8,\
488,471,16,\
489,56,28,\
489,86,28,\
489,116,3,\
489,131,23,\
489,433,26,\
489,463,3,\
489,485,3,\
489,1348,3,\
490,398,25,\
491,190,21,\
491,635,19,\
491,1088,16,\
492,350,17,\
492,369,31,\
493,0,39,\
493,63,24,\
493,92,21,\
493,163,39,\
493,204,4,\
493,284,15,\
493,301,12,\
493,329,20,\
493,387,23,\
493,682,21,\
493,1031,24,\
493,1071,10,\
493,1125,4,\
493,1324,4,\
494,56,37,\
494,172,50,\
495,65,24,\
495,95,32,\
495,145,31,\
495,318,16,\
495,626,24,\
495,762,16,\
495,1092,19,\
495,1366,16,\
496,52,16,\
496,372,2,\
496,348,22,\
496,588,16,\
496,606,4,\
496,836,13,\
496,893,21,\
497,17,15,\
497,34,25,\
497,62,3,\
497,74,28,\
497,115,47,\
497,222,3,\
497,1035,3,\
497,1150,18,\
498,8,15,\
499,69,24,\
499,99,24,\
499,125,3,\
499,136,35,\
499,212,3,\
499,264,6,\
499,272,16,\
499,300,16,\
499,520,3,\
500,36,18,\
500,98,14,\
500,118,15,\
500,175,13,\
500,189,21,\
500,573,21,\
500,650,15,\
500,730,15,\
500,903,14,\
500,1306,16,\
500,1557,5,\
501,4,11,\
501,127,11,\
501,786,16,\
502,1641,20,\
503,42,18,\
503,105,18,\
503,200,33,\
503,314,30,\
503,435,23,\
503,584,13,\
503,720,13,\
503,793,6,\
503,819,14,\
503,901,18,\
503,945,23,\
504,220,34,\
504,603,5,\
504,967,6,\
504,1115,3,\
504,1064,49,\
505,41,24,\
505,67,3,\
505,161,17,\
505,199,24,\
505,225,3,\
505,258,3,\
505,431,3,\
505,544,3,\
505,969,3,\
505,1569,3,\
505,1677,3,\
506,0,8,\
506,22,13,\
506,70,11,\
506,83,8,\
506,117,13,\
506,132,12,\
506,146,8,\
506,156,23,\
506,185,18,\
506,286,13,\
506,325,11,\
506,654,12,\
506,716,13,\
506,749,11,\
506,959,6,\
506,995,8,\
506,1034,14,\
507,0,27,\
507,89,17,\
507,691,26,\
507,974,27,\
507,1270,25,\
507,1399,19,\
507,1445,17,\
507,1549,16,\
507,1639,17,\
507,2238,14,\
509,52,25,\
509,152,45,\
510,70,15,\
510,206,15,\
510,251,15,\
511,4,23,\
511,94,25,\
511,126,3,\
511,168,13,\
511,215,3,\
511,242,11,\
511,316,3,\
511,365,12,\
511,428,3,\
511,613,3,\
511,800,3,\
511,872,23,\
511,1064,3,\
511,1069,3,\
511,1099,6,\
511,1297,3,\
511,1386,24,\
512,26,16,\
512,44,16,\
512,62,3,\
512,82,18,\
512,858,12,\
512,971,5,\
512,1017,5,\
513,271,26,\
513,299,2,\
513,382,2,\
513,1274,15,\
513,1319,14,\
514,48,17,\
515,19,17,\
515,80,17,\
515,258,40,\
515,382,17,\
515,410,28,\
515,473,23,\
515,498,26,\
515,526,7,\
515,535,39,\
515,580,19,\
515,647,13,\
515,666,15,\
515,688,16,\
515,706,28,\
515,765,23,\
515,821,13,\
515,941,13,\
516,0,12,\
516,148,11,\
516,556,21,\
516,581,20,\
517,0,19,\
517,97,16,\
517,145,22,\
517,288,6,\
517,491,19,\
517,662,19,\
517,913,28,\
517,1048,19,\
517,1103,15,\
517,1422,25,\
517,1513,35,\
518,104,21,\
518,128,2,\
518,350,2,\
518,1093,13,\
518,1154,2,\
520,78,13,\
520,367,20,\
520,468,5,\
520,495,13,\
520,604,5,\
520,630,13,\
520,729,13,\
520,809,23,\
520,894,13,\
520,1050,14,\
520,1164,13,\
520,1522,14,\
520,1584,13,\
520,1641,43,\
521,40,28,\
521,200,28,\
521,230,3,\
521,252,13,\
521,277,9,\
521,300,11,\
521,703,3,\
521,910,29,\
521,962,3,\
523,93,29,\
523,147,29,\
523,178,5,\
523,224,5,\
523,286,5,\
523,304,32,\
523,357,21,\
523,380,3,\
523,392,3,\
523,438,3,\
523,443,26,\
523,471,4,\
523,496,3,\
523,540,3,\
523,643,17,\
523,691,3,\
523,662,27,\
523,926,5,\
523,1043,3,\
523,1085,3,\
523,1104,5,\
523,1113,4,\
523,1214,5,\
523,1237,17,\
523,1380,5,\
523,1586,10,\
524,24,47,\
524,461,22,\
524,485,3,\
524,544,12,\
524,590,20,\
524,616,8,\
524,667,19,\
524,688,28,\
524,718,3,\
524,804,26,\
524,1027,3,\
524,1128,3,\
524,1184,3,\
524,1251,32,\
525,37,13,\
525,694,13,\
525,977,16,\
525,1074,13,\
526,1994,11,\
527,341,44,\
527,425,20,\
527,500,23,\
527,559,13,\
527,666,13,\
527,1007,13,\
528,29,6,\
528,221,6,\
528,279,26,\
528,307,3,\
528,365,6,\
528,1386,6,\
529,314,3,\
529,296,16,\
529,420,37,\
529,1468,3,\
529,1538,2,\
530,81,30,\
531,75,54,\
531,131,54,\
531,200,17,\
531,240,46,\
531,187,5,\
533,105,48,\
533,257,48,\
533,354,27,\
533,456,13,\
533,516,13,\
533,609,13,\
533,743,13,\
533,1344,26,\
533,1473,13,\
533,1515,13,\
533,1693,13,\
534,54,26,\
534,90,34,\
534,132,13,\
534,235,17,\
534,336,14,\
534,491,14,\
534,620,13,\
534,711,13,\
534,770,19,\
534,917,16,\
534,1253,14,\
534,1300,7,\
534,1419,7,\
535,75,18,\
536,106,21,\
536,129,3,\
536,141,27,\
536,227,27,\
536,271,3,\
536,452,3,\
536,740,3,\
536,1010,16,\
536,1209,3,\
536,1370,17,\
537,59,19,\
537,193,27,\
537,222,4,\
537,339,10,\
537,748,21,\
537,796,18,\
537,816,10,\
537,835,15,\
537,859,18,\
537,884,18,\
537,946,6,\
537,956,10,\
537,1045,22,\
537,1353,2,\
537,1379,19,\
538,196,46,\
538,301,34,\
538,491,12,\
538,505,23,\
538,530,12,\
538,544,12,\
538,596,14,\
538,796,53,\
538,903,21,\
538,1273,5,\
538,1288,20,\
538,1374,19,\
538,1454,22,\
538,1482,8,\
538,1494,23,\
538,1519,12,\
538,1533,6,\
538,1541,21,\
538,1571,12,\
538,1611,14,\
539,82,21,\
539,153,25,\
539,180,7,\
539,218,32,\
539,381,7,\
539,562,3,\
539,657,3,\
539,910,3,\
539,1393,7,\
540,112,21,\
540,211,28,\
540,458,13,\
540,651,21,\
540,794,28,\
540,922,9,\
540,1324,13,\
540,1576,24,\
540,1691,13,\
541,12,27,\
541,106,17,\
541,165,17,\
541,233,31,\
541,184,3,\
541,285,3,\
541,322,9,\
541,409,3,\
541,611,6,\
541,754,6,\
541,1057,3,\
541,1145,22,\
541,1304,3,\
541,1390,3,\
541,1581,3,\
541,1703,3,\
542,10,15,\
542,104,21,\
542,127,21,\
542,150,3,\
542,165,39,\
542,306,14,\
542,385,15,\
542,521,29,\
543,153,23,\
543,178,3,\
543,227,3,\
543,306,3,\
543,673,3,\
543,681,27,\
543,710,32,\
543,777,41,\
543,824,18,\
543,1064,20,\
543,2377,23,\
543,2495,54,\
543,3044,23,\
543,3156,15,\
543,3441,20,\
543,3651,3,\
543,3729,24,\
544,37,20,\
545,22,28,\
546,22,28,\
546,193,33,\
546,239,22,\
546,299,16,\
546,417,3,\
546,422,33,\
546,457,3,\
546,511,16,\
547,0,29,\
547,49,18,\
547,91,3,\
547,211,22,\
547,235,3,\
547,435,3,\
547,556,22,\
547,593,35,\
547,643,39,\
547,630,4,\
547,684,5,\
547,794,3,\
547,824,3,\
547,872,3,\
547,914,3,\
547,957,3,\
547,1008,3,\
547,1099,12,\
547,1219,3,\
547,1502,12,\
547,1845,3,\
547,2080,3,\
548,78,28,\
548,109,28,\
548,139,3,\
548,150,25,\
548,186,29,\
548,345,3,\
548,396,3,\
548,439,3,\
548,462,9,\
548,1025,3,\
549,31,18,\
551,36,13,\
551,175,13,\
551,275,13,\
551,333,13,\
551,589,13,\
551,718,14,\
551,1066,13,\
551,1226,13,\
551,1440,13,\
551,1612,13,\
551,1975,13,\
551,2054,13,\
552,31,12,\
552,108,12,\
552,122,2,\
553,117,10,\
553,652,14,\
553,751,18,\
553,1052,14,\
554,65,15,\
554,116,63,\
554,274,22,\
554,544,15,\
554,1209,14,\
555,100,25,\
555,142,52,\
555,216,25,\
555,243,4,\
555,285,15,\
555,1732,15,\
555,1787,15,\
556,0,26,\
556,468,12,\
556,482,26,\
556,514,26,\
557,0,26,\
557,45,66,\
557,134,64,\
557,219,13,\
557,293,14,\
557,369,13,\
557,388,24,\
557,433,26,\
557,535,20,\
557,574,15,\
557,715,50,\
557,881,21,\
557,1004,15,\
557,1029,11,\
557,1078,15,\
558,453,33,\
558,655,17,\
558,1226,5,\
559,636,27,\
559,717,23,\
559,1259,27,\
559,1304,31,\
559,1353,23,\
559,1397,34,\
560,0,49,\
560,70,33,\
560,242,32,\
560,483,12,\
560,675,21,\
560,791,21,\
560,904,21,\
561,48,21,\
561,107,21,\
561,131,3,\
561,266,3,\
561,485,3,\
561,836,17,\
561,1019,3,\
561,1111,3,\
562,4,21,\
562,157,21,\
562,510,16,\
562,528,21,\
562,719,16,\
562,937,16,\
562,1238,21,\
562,1366,21,\
562,1403,12,\
563,19,7,\
563,60,7,\
563,71,18,\
563,214,18,\
563,339,26,\
563,421,7,\
563,841,16,\
563,959,7,\
565,156,15,\
565,621,15,\
565,2199,15,\
566,46,22,\
566,73,11,\
566,111,22,\
566,135,3,\
566,146,38,\
566,238,5,\
566,340,3,\
566,507,11,\
566,557,3,\
566,601,3,\
566,687,5,\
566,636,49,\
566,1096,3,\
566,1269,9,\
566,1781,3,\
566,1851,9,\
566,1897,4,\
566,1965,4,\
566,2020,26,\
567,178,25,\
567,208,21,\
567,644,15,\
568,161,26,\
568,188,21,\
568,214,20,\
568,515,16,\
568,860,6,\
568,1529,26,\
569,59,21,\
569,85,12,\
569,450,13,\
569,480,6,\
569,490,9,\
569,502,14,\
569,745,17,\
569,767,9,\
569,784,24,\
569,810,3,\
569,1235,13,\
569,1311,3,\
569,1388,3,\
569,1449,9,\
569,1588,9,\
569,1598,9,\
569,1802,3,\
569,1970,13,\
569,1988,3,\
571,45,6,\
571,123,42,\
571,167,5,\
571,178,40,\
571,708,3,\
571,716,6,\
571,1235,6,\
571,1372,6,\
571,1592,16,\
572,120,14,\
572,277,14,\
572,293,2,\
572,329,2,\
572,523,2,\
572,698,2,\
573,18,35,\
573,87,14,\
573,155,34,\
573,241,20,\
573,342,19,\
573,450,30,\
573,556,12,\
573,862,21,\
574,11,28,\
574,167,28,\
574,208,3,\
574,246,3,\
574,360,3,\
574,639,28,\
574,678,19,\
574,701,17,\
574,720,18,\
574,901,43,\
574,946,18,\
574,970,14,\
574,990,14,\
574,1338,28,\
575,117,6,\
575,185,22,\
575,588,17,\
576,266,25,\
576,509,13,\
576,580,19,\
576,601,3,\
576,619,30,\
576,651,4,\
576,719,13,\
576,745,13,\
576,769,12,\
576,787,17,\
576,814,14,\
576,830,13,\
576,936,2,\
576,970,16,\
576,1019,21,\
576,1048,2,\
576,1430,3,\
576,1547,11,\
576,1578,13,\
576,1705,2,\
577,22,12,\
578,24,29,\
578,55,3,\
578,101,29,\
578,132,3,\
578,142,27,\
578,254,3,\
578,358,3,\
578,1201,20,\
578,1304,20,\
578,1398,5,\
579,11,18,\
579,69,18,\
579,89,2,\
579,185,18,\
579,339,2,\
579,506,21,\
579,531,2,\
579,1373,2,\
580,46,29,\
580,77,29,\
580,112,26,\
580,233,21,\
580,260,15,\
580,804,29,\
580,1328,29,\
581,978,2,\
581,1022,23,\
584,18,32,\
584,99,33,\
584,254,3,\
584,306,50,\
584,359,20,\
584,384,17,\
584,446,3,\
584,470,29,\
584,504,17,\
584,622,3,\
584,891,3,\
584,1049,3,\
584,1128,3,\
584,1220,3,\
584,1431,3,\
584,1546,3,\
584,1646,3,\
584,1725,3,\
584,1826,3,\
585,14,31,\
585,153,21,\
585,379,20,\
585,401,21,\
585,1283,20,\
586,110,15,\
586,127,15,\
586,147,24,\
586,601,15,\
586,1489,17,\
586,1802,15,\
587,0,32,\
587,90,32,\
587,124,4,\
587,217,4,\
588,24,17,\
588,113,4,\
588,124,17,\
588,225,4,\
589,358,15,\
589,1508,15,\
589,1822,15,\
590,0,33,\
590,72,33,\
590,107,4,\
590,118,36,\
590,182,14,\
590,201,25,\
590,377,20,\
590,430,10,\
590,830,4,\
590,1115,15,\
591,10,30,\
591,42,3,\
591,61,31,\
591,110,3,\
591,141,6,\
591,194,26,\
591,222,3,\
591,255,30,\
591,287,3,\
591,362,3,\
591,389,3,\
591,508,3,\
591,549,3,\
591,615,3,\
591,780,3,\
591,860,22,\
591,925,3,\
591,972,8,\
591,1041,29,\
591,1074,15,\
591,1152,51,\
591,1238,35,\
591,1275,4,\
591,1327,31,\
591,1497,18,\
591,1546,3,\
591,1600,3,\
591,1887,6,\
592,44,46,\
592,161,37,\
592,220,33,\
592,255,4,\
592,347,32,\
592,516,33,\
593,0,15,\
593,245,3,\
593,695,3,\
594,12,48,\
594,124,31,\
594,157,2,\
594,269,5,\
594,821,6,\
594,1078,5,\
594,1191,25,\
594,2155,24,\
594,2181,12,\
594,2201,24,\
594,2287,2,\
595,79,39,\
596,78,21,\
596,119,14,\
596,271,22,\
596,389,22,\
596,482,25,\
596,524,24,\
596,646,48,\
596,700,18,\
596,722,30,\
596,769,26,\
596,840,37,\
596,879,23,\
596,912,21,\
597,261,25,\
597,348,7,\
597,425,5,\
597,689,16,\
597,1694,7,\
598,8,19,\
598,45,20,\
598,128,20,\
598,150,2,\
598,180,19,\
598,227,20,\
598,281,19,\
598,384,19,\
598,433,2,\
598,500,19,\
598,646,17,\
598,936,6,\
598,1032,19,\
598,1243,2,\
598,1268,19,\
598,1567,2,\
598,1635,19,\
598,1756,19,\
599,89,8,\
599,209,8,\
599,1039,8,\
599,1387,15,\
599,1625,8,\
600,38,18,\
600,92,18,\
600,116,20,\
600,139,2,\
600,152,39,\
600,359,2,\
600,478,38,\
600,1019,2,\
600,1486,2,\
601,29,41,\
601,87,5,\
601,158,5,\
601,648,35,\
601,1044,5,\
601,1063,5,\
601,1161,13,\
601,1232,19,\
601,1536,6,\
601,1742,5,\
602,12,15,\
603,767,15,\
603,888,27,\
604,763,17,\
604,811,2,\
605,16,17,\
605,53,17,\
605,72,3,\
605,83,46,\
605,131,3,\
606,24,24,\
606,53,6,\
607,47,35,\
607,87,35,\
607,124,3,\
607,378,20,\
607,829,18,\
607,916,3,\
607,1103,14,\
607,1167,19,\
608,13,43,\
608,210,49,\
608,1167,15,\
609,26,14,\
609,57,25,\
609,217,25,\
609,244,3,\
609,316,33,\
609,409,15,\
609,448,18,\
609,483,3,\
609,578,15,\
609,772,3,\
609,839,3,\
609,894,15,\
609,980,15,\
609,1019,11,\
609,1071,28,\
609,1111,24,\
609,1175,18,\
609,1236,14,\
609,1313,19,\
609,1512,18,\
609,1931,19,\
609,2045,13,\
609,2148,14,\
609,2195,3,\
610,277,26,\
610,476,14,\
610,557,3,\
610,624,19,\
610,2086,22,\
610,2149,13,\
612,86,25,\
612,119,7,\
612,221,18,\
612,356,12,\
612,373,13,\
612,465,6,\
612,596,6,\
612,678,6,\
612,778,25,\
612,1114,25,\
613,52,30,\
613,201,22,\
613,225,3,\
613,294,3,\
613,622,29,\
613,779,3,\
613,859,3,\
613,971,6,\
613,989,4,\
613,1047,9,\
613,1061,13,\
613,1079,65,\
613,1149,4,\
613,1159,20,\
613,1184,4,\
613,1237,4,\
613,1254,4,\
613,1691,3,\
613,1793,12,\
614,22,20,\
614,97,20,\
614,119,3,\
614,146,67,\
614,274,15,\
614,618,16,\
614,1698,3,\
615,76,5,\
615,101,17,\
615,217,5,\
615,240,20,\
615,363,17,\
615,490,5,\
615,725,40,\
615,802,17,\
615,896,3,\
615,1135,9,\
615,1218,17,\
615,1290,5,\
617,266,14,\
618,154,5,\
618,198,6,\
619,48,24,\
620,0,63,\
620,65,63,\
620,143,13,\
620,179,47,\
620,130,6,\
621,34,21,\
621,133,21,\
621,157,2,\
622,0,15,\
622,51,8,\
622,71,15,\
622,130,8,\
622,409,44,\
622,467,22,\
622,501,14,\
622,539,32,\
622,663,15,\
622,765,11,\
623,65,50,\
623,242,50,\
623,294,6,\
623,396,6,\
623,1195,6,\
623,1361,6,\
624,0,24,\
624,145,15,\
624,162,3,\
624,404,3,\
624,798,24,\
624,1740,3,\
624,1851,3,\
624,1967,3,\
624,2034,3,\
625,42,33,\
625,77,28,\
625,107,3,\
625,116,33,\
625,151,4,\
625,177,39,\
625,248,3,\
625,258,17,\
625,305,4,\
625,316,29,\
625,419,21,\
625,452,15,\
625,645,3,\
626,69,22,\
626,264,22,\
626,288,2,\
626,654,2,\
626,847,22,\
626,910,22,\
626,1032,2,\
626,1182,2,\
626,1899,2,\
626,2199,2,\
628,43,17,\
628,153,17,\
628,172,4,\
628,195,16,\
628,693,4,\
628,1183,4,\
629,95,20,\
629,329,21,\
629,361,39,\
629,402,4,\
629,524,4,\
630,33,8,\
630,108,37,\
630,152,18,\
631,42,21,\
631,152,21,\
631,176,3,\
631,241,28,\
631,461,3,\
631,476,6,\
631,628,3,\
631,655,3,\
631,852,3,\
631,1105,3,\
631,1214,29,\
631,1316,5,\
632,81,28,\
632,283,28,\
632,313,3,\
632,657,3,\
632,1693,3,\
633,89,18,\
633,331,18,\
633,394,27,\
633,423,3,\
633,429,25,\
633,456,3,\
633,466,29,\
633,497,3,\
633,917,18,\
633,1127,10,\
633,1306,3,\
633,1314,3,\
633,1560,3,\
633,1568,3,\
633,1585,17,\
633,1896,15,\
634,150,20,\
634,1058,18,\
635,29,22,\
635,104,22,\
635,128,3,\
635,139,39,\
635,346,3,\
635,857,3,\
636,449,3,\
636,491,3,\
636,634,13,\
636,842,3,\
637,40,16,\
637,58,31,\
637,91,17,\
637,114,19,\
637,293,36,\
637,334,65,\
637,515,17,\
637,575,20,\
637,597,16,\
637,679,18,\
637,699,17,\
637,718,13,\
637,737,8,\
637,747,16,\
637,1240,16,\
637,1261,6,\
638,0,26,\
638,96,15,\
638,116,17,\
638,255,26,\
638,283,3,\
638,351,23,\
638,422,6,\
638,437,21,\
638,460,2,\
638,637,31,\
638,714,10,\
638,773,6,\
638,903,14,\
638,1020,17,\
638,1051,20,\
638,1206,18,\
638,1357,14,\
638,1413,32,\
638,1647,3,\
638,1800,14,\
638,1849,14,\
638,1879,10,\
638,1932,3,\
638,2034,20,\
638,2151,6,\
638,2251,5,\
638,2361,18,\
638,2417,3,\
638,2566,6,\
638,2590,3,\
638,2700,9,\
638,2716,12,\
639,0,17,\
639,65,18,\
639,152,26,\
639,180,3,\
639,216,30,\
639,274,27,\
639,315,3,\
639,708,3,\
639,830,3,\
639,948,18,\
639,1116,7,\
639,1314,37,\
639,1517,21,\
640,23,11,\
640,59,11,\
640,95,11,\
640,108,4,\
640,293,9,\
640,427,9,\
640,505,4,\
640,542,16,\
640,697,16,\
640,778,28,\
640,808,4,\
640,840,18,\
640,891,24,\
640,931,20,\
640,977,4,\
640,987,9,\
640,1138,35,\
640,1202,4,\
640,1404,9,\
640,1415,22,\
640,1622,24,\
640,1732,9,\
640,1743,24,\
640,1888,11,\
640,2223,4,\
641,793,56,\
642,24,34,\
642,128,17,\
642,223,16,\
642,559,3,\
642,975,42,\
642,1107,33,\
642,2011,14,\
643,379,20,\
643,449,3,\
643,491,39,\
643,532,4,\
644,0,9,\
646,122,23,\
646,148,12,\
646,163,2,\
646,313,13,\
646,387,20,\
646,410,17,\
646,430,15,\
646,466,2,\
646,560,30,\
646,629,36,\
646,688,22,\
647,29,28,\
647,59,14,\
647,87,48,\
647,137,3,\
647,258,6,\
647,75,2,\
647,266,14,\
647,706,17,\
647,863,3,\
647,973,12,\
647,1216,17,\
647,1450,7,\
648,21,17,\
648,123,18,\
648,316,17,\
648,541,8,\
649,31,25,\
649,150,30,\
649,185,6,\
649,204,17,\
649,335,6,\
649,645,6,\
649,697,13,\
649,933,7,\
649,1114,17,\
650,161,30,\
650,193,3,\
651,1,30,\
651,184,14,\
651,215,30,\
651,635,21,\
652,39,12,\
652,171,12,\
652,185,2,\
652,222,20,\
652,261,8,\
652,278,27,\
652,412,5,\
652,584,2,\
652,743,5,\
653,312,17,\
653,335,3,\
653,340,30,\
653,491,17,\
653,569,18,\
653,597,3,\
653,779,6,\
653,864,17,\
653,1069,15,\
653,1130,6,\
653,1215,3,\
653,1220,26,\
653,1260,3,\
654,275,20,\
654,498,25,\
655,78,7,\
655,147,16,\
655,480,5,\
655,576,14,\
655,636,24,\
655,672,13,\
655,693,5,\
655,761,12,\
655,775,17,\
655,798,49,\
655,1054,5,\
655,1101,25,\
655,1152,12,\
655,1218,13,\
656,164,27,\
656,193,3,\
656,197,3,\
656,573,3,\
656,577,3,\
657,33,28,\
657,674,11,\
657,770,20,\
657,837,18,\
658,1128,15,\
660,107,16,\
660,350,28,\
660,825,15,\
660,992,16,\
662,53,27,\
662,103,27,\
662,132,3,\
663,0,38,\
663,86,27,\
663,115,3,\
663,124,25,\
663,151,3,\
663,257,3,\
663,261,3,\
663,586,3,\
663,611,3,\
663,699,3,\
663,708,21,\
663,828,10,\
663,985,16,\
663,1003,19,\
663,1036,58,\
663,1198,3,\
663,1202,3,\
665,57,27,\
665,110,27,\
665,139,3,\
665,176,28,\
665,403,16,\
665,459,3,\
665,690,3,\
665,741,3,\
666,12,27,\
668,42,35,\
668,131,13,\
668,333,35,\
668,371,3,\
668,399,3,\
668,561,12,\
668,652,3,\
668,663,26,\
668,970,3,\
668,1276,3,\
669,11,26,\
672,24,3,\
672,111,34,\
673,34,18,\
673,233,18,\
673,913,18,\
674,0,18,\
674,79,18,\
674,432,18,\
674,703,18,\
674,1045,17,\
676,0,20,\
676,118,33,\
676,153,4,\
676,228,17,\
676,250,27,\
676,279,3,\
677,149,14,\
678,16,44,\
679,58,22,\
679,250,20,\
680,40,8,\
680,129,8,\
680,398,8,\
680,577,8,\
680,752,8,\
680,827,8,\
680,886,24,\
681,74,15,\
681,188,15,\
681,575,13,\
681,676,15,\
681,917,15,\
682,31,25,\
682,118,25,\
682,145,4,\
682,174,22,\
682,214,22,\
682,241,25,\
682,273,4,\
682,544,4,\
682,1073,4,\
683,34,3,\
683,52,17,\
683,75,3,\
683,80,26,\
683,132,30,\
683,210,40,\
683,295,3,\
683,308,14,\
683,417,21,\
683,440,25,\
683,471,32,\
683,524,6,\
683,790,18,\
683,858,3,\
683,945,3,\
684,254,3,\
685,22,27,\
685,97,17,\
685,119,27,\
685,148,3,\
685,235,3,\
685,529,19,\
686,0,39,\
688,0,17,\
688,22,23,\
688,64,15,\
690,22,17,\
690,81,19,\
691,0,29,\
691,102,15,\
691,142,15,\
691,159,3,\
691,242,3,\
691,439,13,\
691,470,20,\
691,492,18,\
691,622,31,\
691,744,17,\
691,847,3,\
691,922,10,\
691,960,28,\
691,1109,18,\
691,1199,3,\
691,1242,3,\
691,1386,3,\
692,42,25,\
692,155,25,\
693,502,13,\
695,68,24,\
695,156,17,\
695,234,23,\
695,445,6,\
695,548,6,\
695,630,6,\
695,691,6,\
695,808,13,\
695,1114,23,\
695,1219,17,\
695,1277,12,\
695,1566,6,\
695,2012,12,\
695,2034,12,\
695,2254,24,\
696,258,4,\
696,290,21,\
696,231,25,\
696,486,22,\
696,765,4,\
696,933,4,\
697,25,24,\
697,131,24,\
697,199,16,\
697,220,14,\
697,860,18,\
697,881,13,\
697,939,6,\
697,948,13,\
698,143,33,\
698,347,14,\
698,379,18,\
698,429,18,\
698,555,14,\
698,1524,33,\
699,128,45,\
699,334,26,\
699,694,17,\
700,90,19,\
701,35,18,\
701,296,28,\
701,592,18,\
701,1004,28,\
702,91,18,\
702,134,18,\
702,155,2,\
702,571,2,\
702,1257,2,\
703,82,26,\
703,110,3,\
703,120,5,\
703,202,3,\
703,616,17,\
703,793,3,\
704,38,23,\
704,140,13,\
704,219,3,\
704,187,30,\
704,496,2,\
704,515,8,\
704,560,31,\
704,956,28,\
704,1084,22,\
704,1110,13,\
704,1171,33,\
704,1226,19,\
705,4,18,\
705,501,18,\
705,734,18,\
705,1187,9,\
705,1369,9,\
707,75,20,\
707,301,20,\
707,661,21,\
708,0,28,\
708,112,28,\
708,142,3,\
708,153,34,\
708,217,35,\
708,1666,3,\
709,15,52,\
709,136,17,\
709,155,25,\
709,187,28,\
709,228,55,\
709,1156,24,\
710,251,30,\
710,283,3,\
710,549,3,\
710,643,3,\
710,683,17,\
710,759,25,\
710,856,3,\
710,1051,3,\
711,193,5,\
711,300,16,\
712,20,26,\
712,57,26,\
712,85,3,\
712,243,3,\
712,296,3,\
712,653,12,\
712,863,3,\
712,937,3,\
713,6,24,\
713,89,24,\
713,115,3,\
713,124,25,\
713,151,3,\
713,174,29,\
713,221,16,\
713,270,3,\
713,1370,3,\
713,1374,3,\
713,1684,16,\
713,1702,16,\
713,1720,17,\
713,1754,18,\
714,21,13,\
714,62,21,\
714,166,21,\
714,367,13,\
714,452,25,\
714,823,6,\
714,1225,33,\
714,1370,6,\
715,23,14,\
715,182,14,\
715,513,14,\
716,48,49,\
716,193,24,\
716,268,48,\
716,351,9,\
716,368,15,\
716,410,37,\
716,733,57,\
716,802,16,\
717,48,32,\
717,350,15,\
717,367,3,\
717,433,3,\
717,564,3,\
717,574,3,\
717,583,26,\
717,611,3,\
717,882,3,\
717,906,3,\
717,929,3,\
717,1083,3,\
717,1269,3,\
717,1310,3,\
717,1867,3,\
717,1970,3,\
718,193,52,\
718,249,26,\
718,1672,25,\
719,44,18,\
719,152,22,\
719,176,3,\
719,291,22,\
719,457,16,\
719,507,18,\
719,527,3,\
719,546,18,\
719,574,26,\
719,856,27,\
719,1507,23,\
720,41,18,\
720,124,15,\
720,171,11,\
720,324,19,\
720,414,10,\
720,504,19,\
720,998,5,\
720,1042,24,\
720,1161,23,\
720,1569,23,\
720,1609,5,\
720,1636,18,\
720,1748,24,\
721,102,14,\
721,130,16,\
721,163,10,\
721,310,10,\
721,367,14,\
721,1288,27,\
722,48,16,\
722,81,17,\
722,184,23,\
722,218,16,\
722,239,27,\
722,293,16,\
722,386,58,\
722,791,23,\
723,110,5,\
723,325,19,\
724,25,25,\
724,59,16,\
724,135,20,\
724,157,3,\
724,309,37,\
724,348,4,\
724,479,17,\
724,498,3,\
724,581,12,\
724,698,16,\
724,806,11,\
724,997,3,\
724,1072,7,\
724,1090,3,\
724,1199,10,\
724,1353,9,\
724,1391,27,\
724,1430,3,\
724,1490,10,\
724,1552,9,\
724,1618,3,\
725,66,38,\
725,115,38,\
725,155,3,\
725,163,3,\
725,282,3,\
725,1212,3,\
725,1316,3,\
725,1320,3,\
725,1347,3,\
725,1426,3,\
725,1466,3,\
725,1582,3,\
725,1653,3,\
726,45,20,\
726,67,76,\
726,169,20,\
726,401,20,\
726,599,15,\
726,945,15,\
726,963,24,\
726,1220,20,\
727,41,21,\
727,108,21,\
727,131,3,\
727,197,7,\
727,231,19,\
727,328,25,\
727,401,3,\
727,549,12,\
727,578,24,\
727,824,21,\
727,878,3,\
727,987,19,\
727,1035,3,\
727,1178,3,\
727,1269,3,\
728,39,13,\
728,129,6,\
728,140,18,\
728,160,38,\
728,206,12,\
728,746,42,\
728,833,28,\
728,874,52,\
729,131,28,\
729,161,28,\
729,200,42,\
729,731,28,\
729,1017,24,\
729,1531,28,\
729,1745,28,\
730,62,5,\
731,106,18,\
731,154,18,\
731,174,2,\
731,314,2,\
731,705,2,\
731,857,2,\
732,21,22,\
732,108,15,\
732,237,15,\
732,1004,18,\
732,1059,18,\
732,1297,31,\
732,1489,15,\
733,129,18,\
733,173,17,\
733,301,28,\
734,59,17,\
734,98,17,\
734,130,27,\
734,187,17,\
734,283,40,\
734,325,6,\
734,403,17,\
734,629,6,\
734,778,19,\
734,1463,26,\
734,1573,13,\
734,1491,3,\
734,1801,20,\
734,1963,14,\
735,18,30,\
735,69,12,\
735,114,30,\
735,1177,23,\
735,1324,12,\
735,1475,12,\
735,1699,12,\
736,33,26,\
736,65,11,\
736,144,26,\
736,326,32,\
736,396,10,\
736,407,9,\
736,473,10,\
736,633,23,\
736,721,30,\
736,830,15,\
736,860,29,\
736,910,18,\
736,930,29,\
736,964,14,\
737,90,18,\
737,174,18,\
737,216,26,\
737,767,21,\
738,53,20,\
738,98,20,\
738,120,3,\
738,135,27,\
738,382,3,\
738,537,3,\
738,790,20,\
738,1113,3,\
739,1,21,\
739,59,21,\
739,82,3,\
739,112,16,\
739,298,25,\
739,364,3,\
739,457,3,\
739,601,3,\
739,701,3,\
739,831,3,\
739,1208,3,\
739,1330,27,\
739,1455,3,\
739,1601,3,\
739,1825,3,\
739,1881,3,\
739,1969,3,\
740,73,16,\
740,212,29,\
740,243,3,\
740,330,3,\
740,496,3,\
740,603,18,\
740,849,3,\
740,857,23,\
740,910,26,\
740,940,8,\
740,882,4,\
740,1179,21,\
740,1242,21,\
740,1347,3,\
740,1398,18,\
740,1520,16,\
740,1578,3,\
741,0,43,\
741,138,43,\
741,229,17,\
741,318,13,\
742,171,2,\
742,1655,16,\
742,1688,19,\
742,1709,16,\
743,82,18,\
743,153,18,\
743,185,5,\
743,370,6,\
743,449,18,\
743,599,17,\
743,770,18,\
743,928,17,\
743,1083,5,\
743,1343,18,\
744,136,23,\
744,234,13,\
744,322,22,\
744,357,33,\
744,459,23,\
744,988,13,\
744,1056,18,\
744,1259,13,\
744,1767,24,\
745,15,14,\
745,62,12,\
745,124,15,\
745,232,15,\
745,372,2,\
745,350,20,\
745,480,5,\
745,512,2,\
745,533,5,\
745,569,14,\
745,780,15,\
745,1084,5,\
745,1251,12,\
745,1350,6,\
745,1422,12,\
745,1493,16,\
745,1590,5,\
745,1676,11,\
745,1809,5,\
746,0,13,\
746,82,21,\
746,151,13,\
746,230,21,\
746,253,20,\
746,709,12,\
746,993,18,\
746,1071,13,\
746,1126,13,\
746,1168,11,\
747,0,25,\
748,1378,27,\
749,91,17,\
749,110,17,\
749,129,4,\
749,152,17,\
749,243,4,\
749,672,4,\
749,877,4,\
749,1094,4,\
749,1387,4,\
751,0,23,\
751,27,14,\
751,156,22,\
751,288,14,\
754,22,12,\
755,27,26,\
755,67,13,\
755,97,13,\
755,213,26,\
755,244,29,\
755,291,22,\
755,338,13,\
755,430,13,\
755,576,13,\
755,603,13,\
755,744,3,\
755,903,3,\
755,1214,13,\
755,1312,13,\
755,1626,13,\
755,1732,3,\
755,1787,13,\
755,1940,13,\
755,2055,3,\
756,77,35,\
756,259,17,\
757,455,21,\
757,1600,12,\
758,259,25,\
758,1225,15,\
759,4,18,\
759,203,28,\
759,271,9,\
759,646,28,\
760,56,12,\
760,70,51,\
760,134,31,\
761,108,30,\
761,342,20,\
762,15,28,\
762,138,28,\
762,168,3,\
762,178,29,\
762,346,17,\
762,553,11,\
762,568,3,\
762,887,3,\
762,905,11,\
762,924,3,\
762,945,11,\
762,1076,11,\
762,1210,11,\
762,1283,3,\
762,1301,11,\
762,1441,11,\
762,1564,11,\
762,1627,11,\
763,0,24,\
763,40,22,\
763,64,22,\
763,88,3,\
763,99,27,\
763,141,23,\
763,169,8,\
763,188,31,\
763,273,3,\
763,302,32,\
763,380,3,\
763,483,3,\
763,509,3,\
763,654,3,\
763,729,3,\
763,745,33,\
763,780,24,\
763,878,3,\
763,987,3,\
763,1191,28,\
763,1270,3,\
764,52,37,\
764,101,38,\
764,141,3,\
764,149,26,\
764,183,26,\
765,0,26,\
765,135,12,\
765,1257,12,\
765,1387,13,\
765,1402,3,\
765,1421,19,\
765,1474,12,\
765,1651,3,\
766,0,24,\
766,28,25,\
766,63,31,\
766,100,38,\
766,170,30,\
766,255,11,\
766,349,17,\
766,429,11,\
766,489,3,\
766,701,11,\
766,946,21,\
766,1024,11,\
766,1102,10,\
766,1189,11,\
766,1202,12,\
766,1272,14,\
766,1288,14,\
766,1353,9,\
766,1398,22,\
766,1517,10,\
766,1541,17,\
766,1576,11,\
766,1626,11,\
766,1895,12,\
766,1941,28,\
766,2004,11,\
766,2048,9,\
766,2108,11,\
766,2145,24,\
766,2201,11,\
766,2276,11,\
766,2318,25,\
766,2359,11,\
766,2465,18,\
766,2521,10,\
766,2584,3,\
766,2637,3,\
767,362,20,\
767,778,23,\
767,1098,21,\
769,272,53,\
769,1324,14,\
770,31,10,\
770,46,12,\
770,87,12,\
770,110,29,\
770,381,22,\
770,421,18,\
770,478,14,\
770,521,40,\
770,609,14,\
770,664,22,\
770,844,12,\
770,1466,22,\
770,1514,18,\
771,169,20,\
771,259,21,\
771,191,4,\
772,92,18,\
772,140,18,\
773,108,26,\
773,171,3,\
773,205,19,\
773,143,26,\
773,416,3,\
773,484,3,\
773,544,12,\
773,558,3,\
773,670,3,\
773,855,3,\
773,978,10,\
773,1382,15,\
773,1420,5,\
774,48,13,\
774,67,34,\
774,103,4,\
774,138,17,\
774,179,50,\
774,256,4,\
775,16,6,\
775,24,6,\
775,293,12,\
775,307,6,\
775,557,6,\
775,635,23,\
775,866,32,\
775,900,3,\
775,949,16,\
775,1055,13,\
775,1185,3,\
775,1199,17,\
775,1218,17,\
775,1237,23,\
775,1266,3,\
775,1398,6,\
775,1557,6,\
775,1723,15,\
775,1883,6,\
775,1951,6,\
775,2049,18,\
775,2080,18,\
775,2208,25,\
775,2264,18,\
775,2292,18,\
775,2420,6,\
775,2432,18,\
775,2484,3,\
775,2657,18,\
775,3024,6,\
775,3213,8,\
775,3264,16,\
775,3364,2,\
775,3448,6,\
775,4010,6,\
775,4223,25,\
775,4531,6,\
775,4654,6,\
775,4752,6,\
775,4818,25,\
775,4902,6,\
775,5275,6,\
775,5639,20,\
775,5661,18,\
775,5704,23,\
775,6007,6,\
775,6162,31,\
775,6474,6,\
775,6641,6,\
776,33,3,\
776,137,17,\
776,192,30,\
776,224,3,\
776,233,41,\
776,276,5,\
776,156,3,\
776,461,3,\
776,580,3,\
776,639,30,\
776,778,3,\
776,953,17,\
776,1035,3,\
776,1447,17,\
776,1528,3,\
776,1753,17,\
777,30,22,\
777,100,22,\
777,124,3,\
777,245,45,\
777,429,3,\
777,491,18,\
777,515,13,\
777,837,3,\
777,919,3,\
777,1001,3,\
778,29,18,\
778,104,18,\
778,124,2,\
778,312,2,\
778,500,2,\
779,92,13,\
779,120,6,\
779,229,22,\
779,318,16,\
779,609,13,\
779,738,17,\
779,924,7,\
779,1135,18,\
779,1301,7,\
779,1360,23,\
779,1455,6,\
779,1538,3,\
779,1627,13,\
780,48,39,\
780,129,21,\
780,452,39,\
780,494,4,\
780,598,4,\
780,798,12,\
780,815,15,\
780,863,10,\
781,43,25,\
781,215,25,\
781,242,3,\
782,59,15,\
782,134,15,\
782,170,16,\
782,428,15,\
782,829,15,\
782,1695,15,\
783,4,5,\
783,194,5,\
783,227,25,\
783,347,24,\
783,1732,14,\
784,10,17,\
784,74,20,\
784,304,31,\
784,708,17,\
784,740,16,\
784,1010,20,\
785,50,18,\
785,80,22,\
785,169,18,\
785,281,18,\
785,312,4,\
785,441,18,\
785,1063,4,\
786,17,20,\
786,55,20,\
786,77,3,\
786,92,48,\
786,299,24,\
786,393,24,\
786,506,18,\
787,743,19,\
788,56,15,\
788,198,15,\
788,215,14,\
788,256,22,\
788,468,14,\
788,528,5,\
788,570,14,\
788,745,15,\
788,943,15,\
788,1006,14,\
788,1094,5,\
788,1281,14,\
788,1453,15,\
788,1569,6,\
788,1637,15,\
788,1774,15,\
788,1863,14,\
788,2136,14,\
788,2300,14,\
789,28,36,\
789,87,17,\
789,123,16,\
789,191,23,\
789,258,17,\
789,380,46,\
789,489,36,\
789,746,4,\
790,204,8,\
790,275,19,\
790,349,17,\
790,369,3,\
790,753,3,\
790,992,3,\
790,1136,3,\
791,85,36,\
791,135,18,\
791,155,2,\
791,189,11,\
791,202,15,\
791,219,5,\
791,230,14,\
791,264,18,\
791,402,23,\
791,427,4,\
791,493,12,\
791,598,12,\
791,612,2,\
791,620,32,\
791,869,36,\
791,907,4,\
791,1432,2,\
791,1595,24,\
791,1820,21,\
791,1940,12,\
792,50,14,\
792,370,14,\
792,647,14,\
792,750,14,\
792,890,14,\
792,990,14,\
792,1295,14,\
793,0,22,\
793,48,22,\
793,73,3,\
793,89,35,\
793,139,58,\
793,360,3,\
793,568,3,\
794,55,3,\
794,176,3,\
794,199,9,\
794,307,3,\
794,414,15,\
794,553,9,\
794,609,3,\
794,647,3,\
794,722,3,\
794,758,9,\
794,953,30,\
794,985,3,\
794,1010,8,\
794,1045,3,\
794,1240,3,\
794,1255,3,\
794,1312,9,\
794,1362,3,\
794,1412,3,\
794,1594,3,\
794,1626,3,\
794,1686,20,\
795,75,15,\
795,145,16,\
795,708,27,\
796,38,15,\
796,79,15,\
796,383,15,\
796,452,15,\
796,520,6,\
796,814,15,\
796,1335,15,\
797,6,30,\
797,53,25,\
797,206,19,\
797,288,36,\
797,327,3,\
797,712,2,\
797,738,25,\
797,805,23,\
797,894,2,\
797,906,19,\
797,938,21,\
797,976,3,\
797,1020,2,\
797,1032,2,\
797,1068,14,\
799,63,18,\
799,172,15,\
799,219,21,\
799,377,15,\
799,560,10,\
799,781,20,\
799,899,18,\
799,962,18,\
799,1062,15,\
799,1564,15,\
799,2115,18,\
799,2355,15,\
799,2485,15,\
800,0,25,\
800,103,25,\
800,198,25,\
800,262,28,\
800,598,13,\
800,646,13,\
800,664,21,\
800,707,15,\
800,780,18,\
800,800,12,\
800,814,28,\
800,844,31,\
800,973,25,\
800,1004,13,\
801,0,26,\
801,93,3,\
801,135,26,\
801,163,3,\
801,942,10,\
801,985,13,\
801,1255,3,\
801,1451,12,\
801,1480,13,\
802,0,58,\
802,111,35,\
802,151,35,\
802,188,3,\
802,505,16,\
802,975,3,\
802,1522,3,\
803,203,19,\
803,224,10,\
803,236,12,\
803,250,49,\
803,1288,16,\
803,1401,18,\
803,1424,19,\
803,1468,88,\
804,121,20,\
804,234,14,\
804,528,28,\
804,1513,15,\
804,1600,14,\
804,1670,15,\
805,15,21,\
805,103,21,\
805,141,12,\
805,286,3,\
805,596,3,\
805,635,12,\
807,66,30,\
807,149,22,\
807,173,3,\
807,388,3,\
807,644,3,\
807,819,3,\
807,1008,3,\
807,1114,3,\
807,1298,3,\
807,1509,3,\
807,1649,3,\
808,486,23,\
809,29,17,\
809,61,17,\
809,85,48,\
809,192,4,\
809,616,17,\
809,907,17,\
810,93,11,\
810,532,13,\
810,572,27,\
810,1071,17,\
811,25,24,\
811,107,13,\
811,261,24,\
811,406,24,\
811,489,25,\
811,591,35,\
811,694,13,\
811,793,24,\
811,985,17,\
811,1437,24,\
811,1597,24,\
812,31,39,\
812,154,28,\
812,191,39,\
812,474,13,\
813,184,17,\
813,219,26,\
813,296,19,\
813,320,6,\
813,654,21,\
813,677,3,\
814,19,6,\
814,31,22,\
815,274,36,\
815,1265,3,\
815,1514,27,\
816,1585,4,\
818,63,36,\
818,151,19,\
818,218,35,\
818,362,19,\
818,400,5,\
818,777,36,\
818,841,26,\
818,877,6,\
818,1073,36,\
818,1122,36,\
818,1443,28,\
818,1556,36,\
818,1793,28,\
818,1925,5,\
818,2104,28,\
818,2248,5,\
818,2285,35,\
819,63,6,\
819,129,6,\
819,658,6,\
819,707,6,\
819,890,24,\
819,1069,6,\
819,1236,6,\
819,1287,6,\
819,1354,6,\
820,180,17,\
820,1299,55,\
821,25,27,\
821,57,23,\
821,121,9,\
821,184,20,\
821,362,27,\
821,391,2,\
821,399,23,\
821,424,3,\
821,476,2,\
821,517,2,\
821,979,2,\
821,1082,14,\
821,1138,2,\
821,1293,10,\
821,1390,26,\
821,1461,10,\
821,1531,5,\
821,1853,2,\
821,1876,3,\
821,2124,10,\
821,2182,2,\
821,2625,14,\
821,2669,27,\
822,302,35,\
822,339,4,\
822,704,2,\
823,40,13,\
823,182,13,\
823,312,6,\
823,491,13,\
823,687,13,\
823,746,33,\
823,992,13,\
823,1066,24,\
823,1197,27,\
823,1636,13,\
823,1788,13,\
824,0,26,\
824,147,3,\
824,339,3,\
824,420,2,\
824,882,3,\
825,0,24,\
825,45,24,\
825,100,24,\
825,284,24,\
825,569,24,\
825,674,24,\
825,855,24,\
825,987,24,\
825,1079,24,\
825,1108,25,\
825,1148,16,\
825,1169,11,\
825,1182,24,\
825,1326,18,\
825,1518,24,\
825,1617,24,\
826,274,12,\
826,808,24,\
827,142,27,\
827,171,2,\
827,180,16,\
827,409,23,\
827,834,2,\
827,2130,20,\
828,173,17,\
828,246,27,\
829,160,20,\
829,182,3,\
829,284,3,\
829,502,3,\
829,632,3,\
829,1089,3,\
829,1224,3,\
829,1318,3,\
830,105,17,\
830,514,18,\
831,30,18,\
831,65,13,\
831,103,20,\
831,125,13,\
831,185,16,\
831,213,8,\
831,262,13,\
831,279,18,\
831,459,18,\
831,494,13,\
831,509,20,\
831,531,3,\
831,664,11,\
831,680,18,\
831,717,3,\
831,1045,13,\
831,1108,18,\
831,1505,18,\
831,1783,18,\
831,1945,13,\
831,2261,18,\
832,256,2,\
832,239,15,\
833,98,29,\
833,305,29,\
833,336,2,\
833,364,2,\
833,403,2,\
833,429,2,\
833,656,2,\
833,791,2,\
833,1083,2,\
833,1305,2,\
833,1454,2,\
834,33,8,\
834,46,6,\
834,66,16,\
834,131,16,\
834,149,2,\
834,181,18,\
834,236,32,\
834,309,26,\
834,366,14,\
834,440,8,\
834,468,28,\
834,505,6,\
834,573,2,\
834,668,2,\
834,840,12,\
834,901,20,\
834,932,23,\
834,1093,23,\
834,1310,13,\
834,1430,2,\
835,0,69,\
835,120,23,\
835,161,21,\
835,275,22,\
835,374,21,\
835,413,21,\
835,485,22,\
835,658,8,\
835,692,32,\
835,748,48,\
835,840,23,\
835,1308,20,\
836,828,6,\
837,20,18,\
837,111,7,\
837,288,41,\
837,332,5,\
837,365,7,\
837,599,26,\
837,650,22,\
837,874,21,\
837,906,21,\
837,1628,7,\
839,102,18,\
840,128,26,\
840,176,29,\
840,250,16,\
840,676,15,\
840,717,23,\
840,742,17,\
840,764,17,\
840,820,28,\
840,880,15,\
841,72,34,\
841,914,10,\
841,948,14,\
842,23,19,\
842,140,16,\
842,174,23,\
842,208,42,\
842,308,19,\
842,853,18,\
843,118,19,\
843,138,21,\
843,161,3,\
843,185,34,\
843,334,21,\
843,430,7,\
843,589,28,\
843,1125,17,\
843,1198,3,\
844,87,16,\
844,118,16,\
844,141,27,\
844,206,26,\
844,237,32,\
844,300,16,\
844,409,45,\
844,817,8,\
844,855,35,\
844,895,16,\
844,944,3,\
844,1062,15,\
845,0,22,\
845,26,29,\
846,32,36,\
846,177,15,\
846,195,3,\
846,210,21,\
846,234,3,\
846,244,17,\
846,420,14,\
846,1465,14,\
846,1667,21,\
847,18,17,\
847,95,20,\
847,123,3,\
847,165,17,\
847,249,20,\
847,437,20,\
847,2180,20,\
848,745,24,\
849,30,18,\
849,162,20,\
849,184,2,\
849,573,2,\
849,579,2,\
849,901,2,\
849,1056,2,\
849,1366,2,\
850,55,8,\
850,121,8,\
850,279,8,\
851,60,82,\
851,144,51,\
851,215,11,\
851,231,16,\
851,319,20,\
851,379,16,\
851,403,15,\
851,443,17,\
851,552,16,\
851,681,16,\
851,703,15,\
851,862,21,\
851,942,8,\
851,1047,9,\
851,1098,2,\
851,1128,15,\
852,0,32,\
852,160,37,\
852,328,16,\
852,349,10,\
852,199,6,\
852,980,20,\
852,1002,3,\
852,1051,10,\
852,1118,4,\
852,1187,10,\
852,1222,10,\
852,1394,22,\
852,1418,3,\
852,1476,12,\
852,1589,20,\
853,36,16,\
853,57,13,\
853,100,15,\
853,179,6,\
853,249,6,\
853,264,6,\
853,325,16,\
853,460,34,\
853,541,35,\
853,671,16,\
853,721,22,\
853,801,13,\
853,893,6,\
853,986,13,\
853,1010,7,\
853,1224,14,\
853,1294,6,\
853,1361,7,\
853,1379,16,\
853,1453,6,\
853,1517,17,\
854,51,18,\
854,176,18,\
854,226,17,\
854,363,18,\
854,448,20,\
854,701,18,\
854,764,20,\
854,808,9,\
854,969,18,\
854,1129,18,\
855,100,24,\
855,271,24,\
855,503,24,\
855,630,3,\
855,1463,3,\
855,1538,14,\
855,1740,3,\
856,81,25,\
856,108,3,\
856,246,25,\
856,727,25,\
856,754,3,\
856,1444,24,\
856,1545,16,\
856,1594,10,\
856,1676,15,\
856,1720,16,\
856,1765,17,\
856,1815,16,\
856,1898,14,\
856,1941,15,\
856,1985,16,\
856,2033,15,\
856,2388,25,\
856,2475,5,\
856,2650,5,\
856,2697,17,\
857,61,17,\
857,80,17,\
857,412,19,\
857,570,28,\
857,616,17,\
857,714,34,\
858,74,27,\
858,103,27,\
858,132,3,\
858,143,50,\
858,484,3,\
858,779,3,\
859,4,16,\
859,32,26,\
859,112,16,\
859,137,19,\
859,196,17,\
859,215,18,\
859,235,13,\
859,250,14,\
859,425,18,\
859,719,16,\
860,0,28,\
861,0,28,\
861,114,28,\
861,144,3,\
861,155,27,\
861,210,41,\
861,612,3,\
861,1228,11,\
861,1552,11,\
861,1658,11,\
861,1673,3,\
861,1876,11,\
862,46,14,\
862,124,24,\
862,254,20,\
862,276,3,\
862,539,3,\
862,633,24,\
862,1090,5,\
862,1242,24,\
862,1436,6,\
862,2084,3,\
862,2224,14,\
863,0,29,\
863,152,29,\
863,193,17,\
863,615,2,\
863,2195,29,\
864,114,18,\
864,230,18,\
864,634,8,\
864,659,26,\
864,779,14,\
864,1259,14,\
865,41,18,\
865,94,18,\
865,115,2,\
865,234,2,\
865,432,2,\
866,112,18,\
866,158,26,\
866,197,32,\
866,346,24,\
866,716,14,\
866,1689,15,\
866,1896,15,\
867,46,17,\
867,84,17,\
867,280,10,\
867,310,21,\
867,462,17,\
867,558,17,\
867,653,6,\
867,690,21,\
867,825,5,\
867,1049,17,\
867,1114,19,\
867,1221,23,\
867,1291,23,\
867,1741,17,\
868,96,15,\
868,118,37,\
868,236,15,\
868,486,15,\
869,1,28,\
869,93,17,\
869,112,3,\
869,124,14,\
869,192,42,\
869,236,5,\
869,264,19,\
869,294,30,\
869,326,3,\
869,332,21,\
869,355,3,\
869,365,18,\
869,385,2,\
869,412,6,\
869,744,3,\
869,761,3,\
869,769,27,\
869,859,16,\
869,1251,15,\
869,1272,33,\
869,1332,21,\
869,1372,3,\
869,1377,5,\
869,1489,9,\
869,1535,10,\
869,1615,3,\
869,1683,6,\
869,1699,63,\
869,1873,15,\
869,1972,2,\
871,42,6,\
871,185,6,\
871,248,51,\
871,458,15,\
872,19,5,\
872,79,11,\
872,242,11,\
872,285,5,\
872,317,13,\
872,400,5,\
872,432,62,\
872,521,29,\
872,658,24,\
872,779,22,\
872,1168,22,\
872,1358,11,\
873,17,3,\
873,69,10,\
874,13,40,\
874,192,40,\
874,234,3,\
874,238,3,\
874,879,3,\
874,883,3,\
874,1822,3,\
874,1826,3,\
874,2032,3,\
874,2036,3,\
875,121,28,\
875,151,3,\
875,161,22,\
875,423,26,\
875,492,3,\
875,1087,3,\
875,1232,3,\
876,24,19,\
876,49,17,\
876,92,17,\
876,114,57,\
876,261,17,\
876,697,17,\
876,1325,17,\
876,1861,19,\
876,1997,17,\
876,2035,25,\
877,86,39,\
877,127,29,\
877,158,5,\
877,190,35,\
877,383,15,\
877,415,5,\
877,437,31,\
877,497,27,\
877,545,5,\
877,886,5,\
877,1156,15,\
877,1318,5,\
877,1458,56,\
877,1568,5,\
878,23,18,\
878,44,18,\
878,63,2,\
878,104,18,\
878,268,2,\
878,625,2,\
878,740,2,\
879,20,17,\
879,94,17,\
879,113,3,\
879,134,46,\
879,253,3,\
879,579,3,\
879,1125,3,\
879,1286,3,\
879,1375,3,\
879,1393,3,\
880,104,28,\
880,137,32,\
880,278,8,\
880,483,33,\
880,521,28,\
880,607,25,\
880,663,12,\
881,85,21,\
882,42,28,\
882,329,33,\
882,376,30,\
882,408,28,\
882,438,3,\
882,447,18,\
882,620,3,\
882,656,3,\
882,747,33,\
882,782,4,\
882,911,3,\
882,970,4,\
882,1453,4,\
882,1639,4,\
883,0,35,\
883,53,43,\
883,146,17,\
883,319,31,\
883,490,19,\
883,521,25,\
883,558,27,\
883,975,14,\
883,998,27,\
884,67,27,\
884,96,27,\
884,134,40,\
884,197,14,\
884,255,16,\
884,313,27,\
884,1403,27,\
886,0,14,\
886,25,28,\
887,18,13,\
887,1634,13,\
890,4,14,\
890,27,13,\
890,134,14,\
890,187,13,\
890,318,13,\
890,354,15,\
890,581,6,\
890,657,6,\
890,782,13,\
890,1103,14,\
891,42,12,\
891,227,12,\
891,1338,12,\
892,0,22,\
892,168,22,\
892,201,21,\
892,1079,22,\
892,1285,22,\
892,1389,22,\
893,0,20,\
893,21,14,\
893,40,18,\
893,76,33,\
893,114,30,\
893,154,33,\
893,189,4,\
893,199,30,\
893,268,20,\
893,301,14,\
893,317,36,\
893,358,43,\
893,475,4,\
893,562,4,\
893,757,23,\
893,840,34,\
893,988,2,\
893,1030,24,\
893,1124,35,\
893,1174,18,\
893,1280,20,\
893,1304,4,\
893,1338,11,\
893,1351,18,\
893,1371,19,\
893,1396,17,\
893,1423,12,\
893,1520,16,\
893,1542,18,\
894,89,20,\
894,111,2,\
894,166,27,\
894,256,21,\
894,284,2,\
894,347,16,\
894,454,2,\
894,481,2,\
894,627,2,\
894,783,2,\
894,857,2,\
894,926,21,\
894,1071,2,\
894,1177,2,\
894,1190,39,\
895,30,17,\
895,58,17,\
895,78,2,\
895,151,27,\
895,181,3,\
895,411,2,\
895,1001,2,\
895,1046,2,\
895,1089,2,\
897,112,16,\
897,342,28,\
897,485,38,\
897,665,20,\
897,701,15,\
897,855,18,\
897,940,15,\
897,1150,18,\
898,23,24,\
898,49,24,\
898,122,20,\
898,156,10,\
898,177,22,\
898,1117,3,\
898,1161,37,\
899,18,34,\
899,87,8,\
899,105,34,\
899,184,19,\
899,141,4,\
899,280,8,\
899,437,4,\
899,474,8,\
899,484,37,\
899,523,25,\
899,581,8,\
899,631,8,\
899,644,22,\
899,752,8,\
899,843,8,\
899,922,19,\
899,943,8,\
899,1084,4,\
899,1157,26,\
899,1404,13,\
899,1586,23,\
899,1765,22,\
899,1801,8,\
899,2006,27,\
899,2074,4,\
899,2123,24,\
899,2225,8,\
899,2306,9,\
899,2353,26,\
900,117,21,\
900,152,21,\
900,188,3,\
900,366,27,\
900,599,3,\
900,656,3,\
900,719,3,\
900,1089,18,\
900,1113,20,\
900,1135,12,\
900,1149,13,\
900,1175,7,\
900,1365,3,\
900,2031,3,\
901,57,15,\
901,789,15,\
902,102,21,\
902,1761,13,\
903,54,5,\
903,115,18,\
903,135,25,\
903,250,19,\
903,654,5,\
903,1099,3,\
903,1418,20,\
904,28,34,\
904,111,15,\
904,128,41,\
904,471,9,\
904,977,19,\
904,1061,12,\
904,1157,17,\
904,1425,16,\
905,0,32,\
905,101,32,\
905,148,29,\
905,199,10,\
905,214,22,\
905,135,3,\
905,439,20,\
905,465,37,\
905,806,21,\
906,231,31,\
906,301,28,\
906,331,3,\
906,369,25,\
906,408,16,\
906,396,5,\
906,426,3,\
906,516,3,\
906,637,32,\
906,1072,3,\
907,30,22,\
907,95,9,\
907,265,22,\
907,289,2,\
907,369,2,\
907,525,2,\
907,663,2,\
907,754,2,\
907,984,2,\
907,1153,2,\
907,1296,2,\
907,1437,2,\
907,1538,2,\
907,1761,2,\
908,49,29,\
908,92,29,\
908,123,5,\
908,149,20,\
908,199,73,\
908,418,5,\
908,540,21,\
908,934,5,\
908,1004,3,\
908,1159,3,\
908,2228,5,\
908,2337,5,\
909,710,16,\
909,1032,2,\
909,1393,11,\
909,1440,11,\
909,1463,17,\
909,1529,33,\
910,0,23,\
910,69,16,\
910,124,12,\
910,147,42,\
910,414,10,\
910,430,11,\
910,494,28,\
910,757,20,\
910,794,11,\
910,896,15,\
910,958,15,\
910,1165,23,\
910,1519,12,\
910,1652,23,\
911,155,17,\
911,396,33,\
912,56,20,\
912,80,27,\
912,205,27,\
912,235,3,\
912,328,3,\
912,484,3,\
912,1048,20,\
912,1180,3,\
912,1275,3,\
913,28,15,\
913,180,15,\
913,979,2,\
913,1097,2,\
913,1357,2,\
914,103,28,\
914,725,38,\
914,1248,5,\
914,1366,5,\
915,56,20,\
915,85,14,\
915,105,29,\
915,170,13,\
916,14,35,\
916,92,21,\
916,184,27,\
916,212,21,\
916,235,3,\
916,805,18,\
916,1024,16,\
916,1398,29,\
916,1636,3,\
917,0,24,\
918,171,6,\
918,343,17,\
918,596,15,\
918,717,5,\
918,918,16,\
918,942,6,\
918,1090,15,\
918,1406,15,\
919,57,15,\
919,140,15,\
919,195,21,\
919,713,2,\
919,752,15,\
920,57,13,\
920,223,13,\
920,238,3,\
920,276,42,\
920,395,3,\
920,651,3,\
920,746,3,\
920,923,13,\
921,47,33,\
921,100,36,\
921,285,17,\
921,451,5,\
921,527,17,\
921,607,17,\
921,990,20,\
921,1018,36,\
921,1133,17,\
921,1238,18,\
921,1454,17,\
921,1476,18,\
921,1621,6,\
921,1695,35,\
921,1734,14,\
921,1936,17,\
921,1958,18,\
922,214,26,\
922,249,39,\
922,348,10,\
922,1310,15,\
922,1506,15,\
922,1546,15,\
922,1627,6,\
923,27,24,\
923,58,24,\
923,84,3,\
923,95,33,\
923,174,3,\
923,577,3,\
923,794,3,\
923,993,3,\
924,0,29,\
924,113,26,\
924,169,30,\
924,267,14,\
924,1223,17,\
924,2036,15,\
924,2055,4,\
924,2150,20,\
925,0,18,\
925,202,17,\
925,1444,24,\
925,1568,15,\
925,1790,19,\
925,1854,17,\
925,1895,31,\
925,2069,18,\
925,2273,17,\
926,26,5,\
926,102,21,\
926,187,13,\
926,221,23,\
926,311,11,\
926,519,12,\
926,581,12,\
926,675,12,\
926,714,3,\
926,840,12,\
926,888,5,\
926,1189,12,\
926,1343,12,\
926,1481,5,\
927,59,3,\
927,85,41,\
927,130,18,\
927,184,18,\
927,315,9,\
927,366,19,\
927,514,38,\
927,812,9,\
927,843,24,\
927,871,22,\
927,963,19,\
927,1057,23,\
927,1126,9,\
928,70,15,\
928,214,15,\
928,234,6,\
928,328,5,\
928,471,15,\
928,544,15,\
928,1010,5,\
928,1063,6,\
928,1288,5,\
929,22,66,\
929,237,20,\
930,87,21,\
930,110,3,\
930,121,27,\
930,220,21,\
930,252,3,\
930,314,9,\
930,328,9,\
930,424,29,\
930,476,29,\
930,1227,28,\
931,65,18,\
931,118,20,\
931,143,24,\
931,1971,18,\
932,233,19,\
932,254,2,\
932,688,4,\
932,822,2,\
932,935,2,\
932,1097,2,\
932,1371,2,\
933,31,3,\
933,78,30,\
933,110,3,\
933,178,26,\
933,206,3,\
933,308,5,\
933,417,8,\
933,429,3,\
933,270,36,\
933,705,10,\
933,717,8,\
933,846,7,\
933,871,9,\
933,996,3,\
933,1088,3,\
933,1120,3,\
933,1373,3,\
933,1395,3,\
933,1436,3,\
934,81,8,\
934,213,8,\
934,223,2,\
934,480,13,\
934,919,17,\
935,86,3,\
935,117,3,\
935,169,29,\
935,200,3,\
935,211,9,\
935,315,3,\
935,356,14,\
935,407,3,\
935,513,3,\
935,784,17,\
935,842,3,\
935,901,27,\
935,951,3,\
935,993,22,\
935,1057,18,\
935,1077,13,\
935,1117,3,\
935,1258,3,\
935,1304,39,\
935,1374,3,\
935,1391,17,\
935,1490,30,\
935,1612,3,\
935,1735,13,\
935,1762,3,\
936,33,17,\
936,136,26,\
936,272,17,\
936,375,5,\
936,413,20,\
936,588,25,\
936,707,5,\
936,730,5,\
936,889,31,\
936,940,16,\
936,963,6,\
936,1060,19,\
936,1154,17,\
936,1339,17,\
936,1371,21,\
936,1484,11,\
938,43,14,\
938,95,51,\
938,166,22,\
938,447,14,\
938,546,14,\
938,782,14,\
939,21,26,\
939,70,35,\
939,107,3,\
939,250,3,\
939,440,18,\
939,463,24,\
939,578,12,\
939,641,3,\
939,750,15,\
939,773,35,\
939,912,15,\
939,942,12,\
939,1393,3,\
940,20,25,\
940,47,25,\
940,79,27,\
940,1220,9,\
940,1265,3,\
940,1674,3,\
940,1714,25,\
941,35,37,\
941,101,5,\
941,118,3,\
941,140,37,\
941,180,4,\
941,202,30,\
941,235,3,\
941,289,16,\
941,327,17,\
941,437,4,\
941,528,3,\
941,582,4,\
941,775,3,\
941,924,19,\
941,1160,8,\
941,1290,4,\
941,1336,19,\
941,1360,20,\
941,1399,14,\
941,1472,4,\
941,1508,3,\
941,1667,3,\
942,0,28,\
942,53,14,\
942,107,28,\
942,137,3,\
942,148,27,\
942,267,3,\
942,296,10,\
942,353,3,\
942,562,14,\
942,581,3,\
942,661,14,\
942,1001,32,\
942,1087,3,\
942,1113,14,\
943,14,26,\
944,0,20,\
944,37,12,\
944,63,20,\
944,120,17,\
944,85,2,\
944,162,12,\
944,188,2,\
944,341,2,\
944,364,2,\
944,573,28,\
944,618,23,\
945,0,22,\
945,85,30,\
945,187,22,\
945,372,14,\
945,388,21,\
945,411,21,\
945,540,10,\
945,581,8,\
945,637,14,\
945,656,12,\
945,670,20,\
945,769,30,\
946,25,6,\
946,96,3,\
946,132,30,\
946,165,3,\
946,192,27,\
946,321,17,\
946,341,3,\
946,349,3,\
946,358,6,\
946,387,12,\
946,410,7,\
946,808,6,\
946,902,37,\
947,105,16,\
947,245,16,\
947,263,2,\
947,315,2,\
947,495,2,\
947,1083,2,\
947,1340,2,\
947,1371,2,\
947,1479,2,\
947,1589,2,\
947,1737,2,\
948,85,12,\
948,108,21,\
948,131,3,\
948,141,23,\
948,194,13,\
948,209,25,\
948,276,11,\
948,292,7,\
948,301,38,\
948,371,3,\
948,489,2,\
948,516,3,\
948,585,13,\
948,1258,3,\
948,1668,3,\
948,1782,9,\
948,1843,3,\
949,71,28,\
949,101,3,\
949,323,28,\
949,370,25,\
949,450,19,\
949,803,19,\
949,1025,2,\
950,131,13,\
950,149,14,\
950,203,16,\
950,361,21,\
951,803,2,\
951,792,9,\
951,939,2,\
951,1078,2,\
951,1570,5,\
952,46,14,\
952,169,14,\
952,297,14,\
953,35,5,\
953,78,15,\
953,200,27,\
953,243,7,\
953,263,34,\
953,324,32,\
953,538,15,\
953,677,15,\
953,709,26,\
953,809,15,\
953,1104,9,\
953,1547,16,\
954,128,2,\
954,158,16,\
954,110,16,\
954,360,16,\
954,383,5,\
954,395,26,\
954,462,2,\
954,493,2,\
954,502,41,\
954,611,5,\
954,627,5,\
954,1455,21,\
955,8,30,\
955,44,24,\
955,80,30,\
955,112,3,\
955,123,41,\
955,305,3,\
955,317,9,\
955,356,18,\
955,415,16,\
955,520,19,\
955,596,16,\
955,656,18,\
955,742,24,\
955,1258,26,\
956,115,26,\
956,185,26,\
956,213,3,\
956,236,2,\
956,244,18,\
956,264,2,\
956,219,15,\
956,1060,2,\
956,1283,2,\
956,1357,2,\
956,1496,2,\
956,1524,2,\
956,1658,2,\
957,61,15,\
957,173,2,\
957,163,8,\
957,344,25,\
957,371,3,\
957,408,18,\
957,443,20,\
957,478,2,\
957,554,2,\
957,592,3,\
957,652,2,\
957,776,3,\
957,1375,2,\
957,1439,17,\
958,18,22,\
958,95,24,\
958,150,39,\
958,121,4,\
958,330,31,\
958,363,3,\
958,444,3,\
958,482,3,\
958,734,3,\
958,847,3,\
960,4,21,\
960,152,21,\
960,693,21,\
961,157,24,\
961,299,24,\
961,325,3,\
961,639,3,\
961,840,24,\
961,1359,24,\
961,1910,14,\
961,1964,3,\
962,87,3,\
962,53,32,\
962,276,3,\
962,585,3,\
962,884,3,\
963,115,6,\
963,181,25,\
963,366,3,\
963,531,17,\
963,600,3,\
963,570,28,\
963,781,3,\
963,804,13,\
963,873,3,\
963,1013,3,\
963,1090,3,\
963,1352,3,\
964,271,30,\
964,304,3,\
965,4,15,\
965,185,26,\
965,2019,26,\
966,0,27,\
966,32,42,\
966,81,5,\
967,76,30,\
967,205,17,\
967,572,17,\
967,710,12,\
967,1122,8,\
967,1282,23,\
967,1349,9,\
967,1456,12,\
967,1566,22,\
968,605,24,\
969,0,29,\
969,97,30,\
969,129,7,\
969,163,10,\
969,180,32,\
969,500,14,\
969,609,7,\
969,678,7,\
969,1443,15,\
969,1626,15,\
970,507,15,\
971,23,18,\
971,93,2,\
971,232,2,\
971,317,2,\
971,387,2,\
971,456,2,\
971,489,2,\
971,619,2,\
971,648,2,\
971,878,2,\
971,1176,2,\
971,1292,2,\
971,1486,2,\
973,322,18,\
974,55,28,\
974,148,17,\
974,238,21,\
974,369,28,\
974,399,3,\
974,735,3,\
974,1042,3,\
974,1318,3,\
974,1841,3,\
975,11,15,\
975,108,26,\
975,145,28,\
975,310,25,\
975,828,15,\
975,1126,15,\
976,31,14,\
976,146,14,\
976,414,22,\
976,1220,6,\
977,27,24,\
977,332,54,\
977,430,24,\
977,829,16,\
977,1363,16,\
977,1655,16,\
977,1860,16,\
977,2039,37,\
978,28,13,\
978,293,13,\
978,395,13,\
978,453,25,\
978,576,13,\
978,718,13,\
978,866,23,\
978,969,13,\
978,1153,6,\
978,1279,22,\
978,1453,13,\
978,1694,12,\
978,1731,6,\
978,1767,6,\
978,1851,13,\
978,2048,13,\
978,2125,22,\
979,32,6,\
979,147,25,\
979,195,28,\
979,174,4,\
979,495,6,\
979,1693,6,\
979,1973,8,\
979,2084,6,\
980,67,14,\
980,103,14,\
980,130,16,\
980,163,29,\
980,277,14,\
980,464,14,\
980,494,33,\
980,776,8,\
980,813,21,\
980,1025,18,\
981,0,30,\
981,61,3,\
981,65,3,\
981,626,3,\
981,630,3,\
981,1276,3,\
982,37,30,\
982,69,30,\
982,101,3,\
982,116,28,\
982,472,3,\
982,528,3,\
982,581,20,\
982,616,14,\
983,56,19,\
983,141,28,\
983,171,4,\
983,755,10,\
983,862,4,\
983,1001,20,\
983,1258,27,\
984,22,32,\
984,56,15,\
984,231,32,\
984,326,15,\
984,543,6,\
984,578,6,\
986,42,32,\
986,97,6,\
986,187,32,\
986,319,32,\
986,653,3,\
986,874,22,\
987,24,14,\
987,61,16,\
987,99,30,\
987,143,30,\
987,175,3,\
987,202,26,\
987,230,3,\
987,286,18,\
987,322,17,\
987,456,14,\
987,657,14,\
987,676,16,\
987,708,3,\
987,870,17,\
987,892,6,\
987,917,3,\
987,1014,14,\
987,1032,3,\
987,1101,3,\
987,1167,14,\
987,1268,17,\
987,1318,17,\
987,1376,14,\
987,1407,3,\
987,1501,11,\
987,1586,3,\
988,503,26,\
988,531,3,\
988,753,3,\
988,842,3,\
988,1172,3,\
988,1325,3,\
989,156,5,\
989,351,21,\
989,637,14,\
991,43,13,\
991,130,5,\
991,203,12,\
991,217,2,\
991,500,5,\
991,606,6,\
991,680,6,\
991,761,5,\
991,912,18,\
991,1053,34,\
992,129,3,\
992,224,13,\
992,239,40,\
992,104,23,\
992,285,21,\
992,419,15,\
992,506,12,\
992,662,12,\
992,730,12,\
992,999,12,\
992,1167,12,\
992,1379,34,\
992,1467,12,\
992,1504,16,\
992,1687,12,\
992,1731,16,\
992,1963,12,\
993,4,22,\
993,54,43,\
993,111,14,\
993,265,70,\
993,380,29,\
993,412,3,\
993,502,11,\
993,897,18,\
993,1171,4,\
994,81,41,\
994,124,3,\
994,416,5,\
994,448,26,\
994,1232,13,\
994,1335,12,\
995,22,38,\
995,137,40,\
995,189,39,\
995,230,4,\
995,249,38,\
995,289,4,\
995,666,19,\
995,764,23,\
995,901,38,\
995,944,4,\
996,112,45,\
996,246,28,\
996,533,25,\
996,598,6,\
996,750,22,\
996,846,14,\
996,909,6,\
996,1566,13,\
996,1635,14,\
996,1705,13,\
997,44,23,\
997,926,19,\
998,91,29,\
998,122,29,\
998,153,5,\
998,172,26,\
998,371,5,\
998,416,5,\
998,1598,5,\
999,509,18,\
1000,566,7,\
1000,651,7,\
1000,770,3,\
1000,811,15,\
1000,987,7,\
1001,0,25,\
1001,70,27,\
1001,136,17,\
1001,158,22,\
1001,234,31,\
1001,269,26,\
1001,343,21,\
1001,422,9,\
1001,620,16,\
1001,762,3,\
1001,905,5,\
1001,966,3,\
1001,1110,11,\
1001,1132,25,\
1001,1172,20,\
1001,1313,3,\
1001,1370,3,\
1001,1387,16,\
1001,1633,19,\
1001,1727,3,\
1001,1764,20,\
1002,16,22,\
1002,105,15,\
1002,160,17,\
1002,179,3,\
1002,222,15,\
1002,709,3,\
1002,798,3,\
1002,953,3,\
1002,1275,3,\
1005,107,23,\
1006,35,22,\
1006,128,13,\
1006,396,14,\
1006,459,7,\
1006,520,14,\
1006,542,14,\
1006,606,14,\
1006,658,18,\
1006,881,13,\
1006,1024,13,\
1006,1248,18,\
1006,1407,17,\
1006,1664,18,\
1007,81,17,\
1007,386,17,\
1007,497,35,\
1007,538,23,\
1007,566,18,\
1007,589,5,\
1007,624,18,\
1007,845,6,\
1007,1280,6,\
1007,1492,17,\
1007,1579,18,\
1007,1702,20,\
1008,260,10,\
1008,284,10,\
1008,482,46,\
1008,603,20,\
1008,985,20,\
1008,1672,26,\
1008,1828,25,\
1009,62,13,\
1009,107,27,\
1009,254,13,\
1009,269,2,\
1009,313,27,\
1009,342,3,\
1009,707,2,\
1009,1115,11,\
1009,1144,19,\
1009,1223,2,\
1010,300,6,\
1010,686,5,\
1010,1308,3,\
1011,169,28,\
1011,211,28,\
1011,250,35,\
1011,739,23,\
1012,404,21,\
1013,273,19,\
1014,94,6,\
1014,181,13,\
1014,389,13,\
1015,705,22,\
1015,774,21,\
1015,797,22,\
1015,1183,2,\
1015,1673,22,\
1015,1905,16,\
1016,17,26,\
1016,52,23,\
1016,98,17,\
1016,196,34,\
1016,232,3,\
1016,241,18,\
1016,261,2,\
1016,294,3,\
1016,380,6,\
1016,390,26,\
1016,440,26,\
1016,468,3,\
1016,523,3,\
1016,531,2,\
1016,645,20,\
1016,752,17,\
1016,791,15,\
1016,874,3,\
1016,1322,5,\
1017,27,29,\
1017,91,29,\
1017,122,3,\
1017,171,19,\
1017,214,3,\
1017,456,3,\
1017,557,3,\
1017,1202,14,\
1017,1440,3,\
1017,1509,3,\
1018,90,9,\
1018,937,15,\
1018,1200,9,\
1018,1712,9,\
1019,24,31,\
1019,150,31,\
1019,183,3,\
1019,236,14,\
1019,982,12,\
1019,1342,17,\
1019,1376,12,\
1019,1485,12,\
1020,72,28,\
1020,102,3,\
1020,108,28,\
1020,138,3,\
1020,228,9,\
1020,320,3,\
1020,581,3,\
1021,70,9,\
1021,180,9,\
1021,1180,18,\
1022,151,27,\
1022,201,27,\
1022,261,20,\
1022,180,4,\
1022,441,13,\
1022,630,14,\
1022,709,18,\
1022,877,12,\
1022,891,13,\
1022,910,20,\
1024,19,28,\
1026,59,31,\
1026,153,3,\
1026,124,27,\
1026,690,3,\
1026,849,3,\
1028,21,44,\
1028,110,15,\
1030,34,9,\
1030,180,18,\
1032,13,3,\
1032,96,30,\
1032,128,30,\
1032,160,3,\
1032,171,43,\
1032,251,5,\
1032,268,26,\
1032,296,3,\
1032,384,3,\
1032,423,19,\
1032,463,3,\
1032,507,3,\
1032,680,3,\
1032,707,3,\
1032,836,3,\
1032,977,3,\
1032,1211,3,\
1032,1361,3,\
1032,1766,3,\
1033,40,6,\
1033,447,5,\
1033,668,5,\
1033,892,6,\
1033,947,3,\
1033,959,6,\
1033,1079,6,\
1033,1321,6,\
1034,0,19,\
1034,133,19,\
1034,154,3,\
1034,169,35,\
1034,217,74,\
1034,354,21,\
1034,385,15,\
1034,402,14,\
1034,456,27,\
1034,508,3,\
1034,562,25,\
1034,808,3,\
1034,1331,26,\
1035,41,16,\
1035,59,51,\
1035,136,16,\
1035,196,17,\
1035,286,40,\
1035,328,5,\
1035,464,19,\
1035,507,16,\
1035,529,15,\
1035,795,17,\
1037,26,17,\
1037,162,16,\
1037,557,14,\
1037,766,18,\
1037,1196,16,\
1038,106,18,\
1038,194,19,\
1038,1730,3,\
1038,1737,3,\
1038,2392,24,\
1039,82,24,\
1039,120,24,\
1039,146,3,\
1039,299,15,\
1039,1657,3,\
1039,1710,24,\
1040,35,16,\
1040,53,16,\
1040,76,28,\
1040,151,6,\
1040,246,3,\
1040,394,3,\
1040,1152,3,\
1040,1209,16,\
1043,47,21,\
1043,85,12,\
1043,99,3,\
1043,108,18,\
1043,128,2,\
1043,264,3,\
1043,271,2,\
1043,423,3,\
1043,431,2,\
1043,535,2,\
1043,571,17,\
1043,696,2,\
1043,851,3,\
1043,866,2,\
1043,1062,2,\
1043,1095,3,\
1043,1182,11,\
1043,1210,7,\
1043,1226,2,\
1043,1358,3,\
1043,1366,2,\
1043,1391,2,\
1043,1518,2,\
1043,1581,2,\
1044,30,44,\
1044,149,44,\
1044,195,5,\
1044,252,5,\
1044,1122,5,\
1044,1353,5,\
1044,1612,25,\
1044,1697,5,\
1045,95,28,\
1045,1004,28,\
1046,1292,3,\
1047,183,41,\
1047,356,8,\
1047,415,8,\
1047,425,2,\
1047,1005,8,\
1048,95,36,\
1048,167,21,\
1048,269,29,\
1048,458,29,\
1048,750,24,\
1048,817,6,\
1049,496,5,\
1050,493,14,\
1050,933,14,\
1050,1194,26,\
1050,1532,35,\
1050,1689,26,\
1052,71,22,\
1052,262,23,\
1052,415,27,\
1052,491,27,\
1052,520,3,\
1052,566,3,\
1052,720,25,\
1052,747,3,\
1052,829,3,\
1052,1012,18,\
1052,1327,3,\
1052,1348,3,\
1052,1355,3,\
1052,1391,3,\
1053,191,31,\
1053,294,31,\
1054,162,29,\
1054,982,16,\
1055,0,25,\
1055,166,13,\
1055,191,15,\
1055,211,22,\
1055,732,26,\
1055,1810,16,\
1055,1945,14,\
1055,1961,22,\
1055,2019,23,\
1055,2255,14,\
1056,58,15,\
1056,77,6,\
1056,100,15,\
1056,117,3,\
1056,141,37,\
1056,194,3,\
1056,431,3,\
1056,1055,3,\
1056,1107,3,\
1056,1224,3,\
1057,23,13,\
1057,72,53,\
1057,214,21,\
1057,274,32,\
1057,492,16,\
1057,999,17,\
1057,1168,12,\
1058,196,2,\
1058,206,38,\
1058,259,39,\
1058,339,12,\
1058,178,16,\
1058,689,3,\
1058,675,12,\
1058,1172,4,\
1058,1609,3,\
1058,1698,4,\
1059,120,22,\
1059,233,22,\
1059,257,2,\
1059,304,2,\
1059,532,2,\
1059,635,2,\
1059,879,2,\
1059,1064,2,\
1059,1138,2,\
1059,1185,2,\
1060,31,30,\
1060,134,27,\
1060,258,28,\
1060,288,18,\
1060,326,11,\
1060,392,7,\
1060,457,21,\
1060,480,3,\
1060,1111,3,\
1060,1405,18,\
1060,1425,25,\
1060,1452,18,\
1060,1472,11,\
1060,1508,15,\
1060,1554,3,\
1060,1632,7,\
1061,217,8,\
1061,488,2,\
1061,515,2,\
1061,628,2,\
1061,657,2,\
1061,692,2,\
1061,1548,2,\
1061,1607,2,\
1062,40,4,\
1062,4,34,\
1062,63,31,\
1062,425,27,\
1063,54,3,\
1063,124,20,\
1063,180,3,\
1063,277,17,\
1063,298,17,\
1063,346,17,\
1063,461,20,\
1063,638,20,\
1063,671,17,\
1063,709,21,\
1063,785,20,\
1063,916,17,\
1063,978,20,\
1063,1099,15,\
1063,1157,17,\
1063,1178,28,\
1063,1430,20,\
1064,42,18,\
1064,122,18,\
1064,142,2,\
1064,517,2,\
1064,1642,5,\
1064,1899,14,\
1064,1978,3,\
1064,1957,19,\
1064,2952,2,\
1064,3140,2,\
1065,33,34,\
1065,214,17,\
1065,233,3,\
1065,242,25,\
1065,269,4,\
1065,538,20,\
1065,596,31,\
1065,703,3,\
1065,707,4,\
1065,1208,14,\
1066,57,36,\
1066,151,22,\
1066,536,21,\
1066,559,2,\
1066,768,2,\
1066,1044,2,\
1066,1457,2,\
1066,1677,2,\
1067,108,15,\
1067,125,15,\
1067,142,3,\
1067,165,38,\
1067,216,39,\
1067,328,3,\
1067,589,3,\
1067,673,3,\
1067,905,3,\
1067,1576,3,\
1067,1652,3,\
1068,31,27,\
1068,83,27,\
1068,112,3,\
1068,335,19,\
1068,595,19,\
1068,669,3,\
1069,51,17,\
1069,78,41,\
1069,126,49,\
1069,382,17,\
1069,515,12,\
1069,960,17,\
1070,75,26,\
1070,454,22,\
1070,1069,30,\
1071,6,17,\
1071,32,16,\
1071,77,17,\
1071,160,16,\
1071,181,21,\
1071,243,17,\
1071,397,47,\
1071,563,17,\
1071,589,24,\
1071,715,17,\
1071,869,48,\
1073,0,26,\
1073,104,42,\
1073,159,59,\
1073,247,28,\
1073,304,26,\
1073,434,5,\
1073,332,3,\
1073,1221,6,\
1073,1310,5,\
1073,1715,21,\
1073,1742,14,\
1073,1799,6,\
1073,1822,6,\
1073,1973,12,\
1073,2044,12,\
1073,2070,10,\
1073,2165,12,\
1074,0,24,\
1074,40,17,\
1074,131,13,\
1075,38,21,\
1075,77,45,\
1075,124,45,\
1075,186,37,\
1075,770,17,\
1075,1026,16,\
1075,1094,14,\
1075,1137,18,\
1075,1203,6,\
1075,1215,10,\
1075,1309,24,\
1075,1338,21,\
1075,1485,14,\
1075,1501,14,\
1075,1558,9,\
1076,71,17,\
1076,105,25,\
1076,166,24,\
1076,201,17,\
1076,319,31,\
1076,364,31,\
1076,411,12,\
1076,435,17,\
1076,459,26,\
1076,498,23,\
1076,543,3,\
1076,588,46,\
1076,742,16,\
1076,778,5,\
1076,796,11,\
1076,895,3,\
1076,907,11,\
1076,938,17,\
1076,1138,15,\
1076,1184,31,\
1076,1241,24,\
1076,1442,3,\
1076,1696,15,\
1076,1736,17,\
1076,1792,3,\
1076,1909,3,\
1076,1933,29,\
1076,1978,23,\
1076,2136,17,\
1076,2170,11,\
1076,2256,23,\
1076,2361,17,\
1076,2429,17,\
1076,2485,3,\
1076,2509,11,\
1076,2595,11,\
1076,2626,17,\
1076,2714,11,\
1076,2745,17,\
1077,73,19,\
1077,193,22,\
1077,217,3,\
1077,241,30,\
1078,61,18,\
1078,81,18,\
1078,101,2,\
1078,142,18,\
1078,265,14,\
1078,284,19,\
1079,25,28,\
1079,233,19,\
1079,254,2,\
1079,360,28,\
1079,426,2,\
1079,495,2,\
1079,686,2,\
1079,1004,2,\
1079,1275,2,\
1080,68,20,\
1080,201,20,\
1080,434,11,\
1080,447,4,\
1080,586,2,\
1080,601,6,\
1080,1005,4,\
1080,1616,6,\
1080,1903,6,\
1081,58,28,\
1081,154,28,\
1081,184,3,\
1081,194,25,\
1081,238,29,\
1081,417,13,\
1081,508,3,\
1081,578,3,\
1081,793,31,\
1081,1097,31,\
1081,1206,21,\
1081,1266,36,\
1081,1307,18,\
1082,266,12,\
1083,91,28,\
1083,250,28,\
1083,280,3,\
1083,365,3,\
1083,385,14,\
1083,1157,15,\
1083,1686,14,\
1084,21,46,\
1084,69,53,\
1084,297,15,\
1084,399,20,\
1084,441,35,\
1084,633,15,\
1084,736,17,\
1084,801,15,\
1085,66,31,\
1085,99,5,\
1085,720,27,\
1086,57,15,\
1086,283,20,\
1086,308,20,\
1086,707,16,\
1087,23,29,\
1087,121,29,\
1087,152,3,\
1087,160,23,\
1087,190,26,\
1087,348,10,\
1088,49,15,\
1088,224,15,\
1089,66,13,\
1089,81,22,\
1089,131,13,\
1089,987,13,\
1090,57,3,\
1090,77,31,\
1090,110,31,\
1090,144,5,\
1090,179,16,\
1090,235,22,\
1090,294,17,\
1090,349,27,\
1090,384,3,\
1090,428,5,\
1090,502,3,\
1090,515,5,\
1090,654,3,\
1090,667,5,\
1090,1091,3,\
1091,94,25,\
1091,249,25,\
1091,276,3,\
1091,282,35,\
1091,418,3,\
1091,469,3,\
1091,609,3,\
1091,985,17,\
1091,1054,27,\
1091,1183,3,\
1091,1333,3,\
1091,1499,19,\
1091,1523,3,\
1091,1621,3,\
1091,1737,3,\
1091,1778,48,\
1092,63,21,\
1092,88,28,\
1092,153,3,\
1092,179,11,\
1092,348,11,\
1092,363,3,\
1092,469,11,\
1092,525,3,\
1092,551,11,\
1092,572,3,\
1092,719,3,\
1092,817,3,\
1092,835,11,\
1092,889,11,\
1092,1003,3,\
1092,1025,11,\
1093,28,31,\
1093,61,5,\
1093,620,21,\
1093,939,6,\
1093,1097,13,\
1093,1416,5,\
1093,1454,22,\
1093,1631,19,\
1094,426,20,\
1094,507,8,\
1095,10,30,\
1095,123,30,\
1095,155,3,\
1095,279,8,\
1095,324,10,\
1095,1270,30,\
1096,0,26,\
1096,53,26,\
1096,81,3,\
1096,113,19,\
1096,211,39,\
1096,489,3,\
1096,1136,3,\
1098,0,21,\
1098,286,19,\
1098,759,39,\
1098,827,19,\
1098,870,11,\
1098,1090,21,\
1098,1164,21,\
1098,1377,21,\
1098,1457,21,\
1098,1485,8,\
1099,0,24,\
1099,282,24,\
1099,309,3,\
1099,925,22,\
1099,951,3,\
1100,31,3,\
1100,129,5,\
1100,183,30,\
1100,218,67,\
1100,347,3,\
1101,40,23,\
1101,180,31,\
1101,213,2,\
1101,423,23,\
1101,1115,9,\
1101,1386,15,\
1101,1705,9,\
1101,1793,23,\
1103,0,35,\
1103,46,11,\
1103,62,29,\
1103,269,22,\
1103,590,17,\
1103,656,3,\
1103,670,19,\
1103,736,3,\
1103,765,29,\
1103,796,36,\
1103,977,14,\
1103,1042,38,\
1103,1101,24,\
1103,1130,24,\
1103,1310,29,\
1103,1341,36,\
1103,1442,3,\
1103,1417,23,\
1103,1571,22,\
1103,1876,26,\
1103,1975,35,\
1103,2064,16,\
1103,2082,24,\
1103,2112,11,\
1103,2129,29,\
1104,7,6,\
1104,78,19,\
1104,176,19,\
1104,302,21,\
1104,497,19,\
1104,548,6,\
1104,628,19,\
1104,907,7,\
1104,989,6,\
1104,1026,8,\
1104,1156,8,\
1104,1226,6,\
1104,1433,16,\
1104,1488,20,\
1104,1550,14,\
1104,1621,11,\
1104,1711,7,\
1104,1795,8,\
1104,1839,8,\
1104,1885,7,\
1104,1907,21,\
1104,1932,17,\
1104,1957,12,\
1104,1983,15,\
1104,2043,12,\
1104,2353,8,\
1104,2387,16,\
1104,2405,40,\
1104,2451,23,\
1104,2582,19,\
1104,2692,8,\
1104,2790,19,\
1105,107,22,\
1105,208,22,\
1105,232,3,\
1105,633,3,\
1105,779,3,\
1105,924,3,\
1105,1032,3,\
1105,1526,3,\
1105,1614,3,\
1105,1724,3,\
1106,0,28,\
1106,49,25,\
1106,130,25,\
1106,179,24,\
1106,562,24,\
1106,824,22,\
1106,942,19,\
1106,1059,34,\
1106,1279,22,\
1106,1305,3,\
1106,1643,33,\
1107,0,22,\
1107,64,22,\
1107,114,52,\
1107,171,12,\
1108,86,14,\
1108,392,14,\
1109,105,12,\
1109,162,10,\
1109,489,12,\
1109,744,12,\
1109,826,12,\
1109,906,12,\
1109,1231,12,\
1110,29,28,\
1110,63,4,\
1110,94,28,\
1110,124,3,\
1110,281,3,\
1110,380,3,\
1110,514,3,\
1110,1581,3,\
1110,1605,9,\
1111,133,4,\
1111,497,27,\
1111,526,3,\
1111,534,25,\
1111,561,3,\
1111,699,3,\
1111,714,3,\
1111,1384,3,\
1111,1388,3,\
1112,10,13,\
1112,140,13,\
1112,189,13,\
1112,248,13,\
1112,330,13,\
1112,588,13,\
1112,767,15,\
1112,1042,13,\
1112,1094,23,\
1112,1246,13,\
1112,1349,14,\
1112,1464,14,\
1113,258,14,\
1113,508,15,\
1114,196,5,\
1114,325,6,\
1114,340,18,\
1116,22,14,\
1116,80,14,\
1116,177,14,\
1116,1573,22,\
1117,0,43,\
1117,47,14,\
1117,130,25,\
1117,193,7,\
1117,315,7,\
1117,327,20,\
1117,349,2,\
1117,531,25,\
1117,585,21,\
1117,708,6,\
1117,805,2,\
1117,813,2,\
1117,947,6,\
1117,1051,7,\
1117,1274,2,\
1117,1392,2,\
1117,1410,15,\
1117,1659,17,\
1117,1719,6,\
1117,1805,25,\
1118,154,37,\
1119,0,28,\
1119,94,28,\
1119,124,3,\
1119,166,27,\
1119,277,3,\
1119,458,3,\
1119,579,25,\
1119,1090,3,\
1119,1260,3,\
1119,1634,3,\
1119,1945,3,\
1120,55,17,\
1120,101,17,\
1121,48,27,\
1121,207,27,\
1121,236,2,\
1121,324,2,\
1121,716,2,\
1122,85,25,\
1122,165,36,\
1123,21,46,\
1123,143,54,\
1123,314,13,\
1124,113,27,\
1124,142,27,\
1124,171,3,\
1124,191,14,\
1124,238,21,\
1124,586,3,\
1124,1828,27,\
1125,0,12,\
1125,97,12,\
1125,217,20,\
1125,249,12,\
1125,265,23,\
1125,313,21,\
1125,336,20,\
1125,360,20,\
1125,389,21,\
1125,818,24,\
1125,864,21,\
1126,0,14,\
1126,44,14,\
1126,375,17,\
1126,487,17,\
1126,522,27,\
1126,562,30,\
1126,594,4,\
1126,658,26,\
1126,787,9,\
1126,803,29,\
1126,1000,19,\
1126,1097,14,\
1126,1132,15,\
1126,1149,27,\
1126,1178,3,\
1126,1183,4,\
1126,1192,24,\
1127,258,36,\
1127,908,15,\
1127,925,18,\
1127,945,19,\
1127,1078,16,\
1127,1100,12,\
1128,590,31,\
1129,314,16,\
1130,84,50,\
1130,136,19,\
1130,277,15,\
1130,301,25,\
1130,351,13,\
1130,398,13,\
1130,1027,15,\
1130,1102,15,\
1130,1211,15,\
1130,1317,15,\
1130,1540,15,\
1131,0,18,\
1131,20,9,\
1131,122,18,\
1131,466,18,\
1131,584,18,\
1131,708,18,\
1131,857,9,\
1131,1038,18,\
1131,1181,16,\
1131,1411,18,\
1132,51,22,\
1132,75,3,\
1132,126,12,\
1132,142,3,\
1132,219,3,\
1132,280,17,\
1132,336,20,\
1132,195,22,\
1132,443,30,\
1132,655,3,\
1132,912,12,\
1132,980,3,\
1132,1066,12,\
1132,1082,3,\
1132,1400,12,\
1134,33,22,\
1134,197,22,\
1134,416,22,\
1134,570,22,\
1134,746,8,\
1134,759,16,\
1134,940,15,\
1134,966,8,\
1134,995,16,\
1134,1017,8,\
1134,1258,32,\
1134,1331,22,\
1134,1517,24,\
1134,1570,22,\
1134,1616,22,\
1135,112,31,\
1135,145,31,\
1135,214,17,\
1135,178,4,\
1135,787,4,\
1136,79,22,\
1136,108,28,\
1136,367,12,\
1136,1302,12,\
1137,73,26,\
1137,225,26,\
1137,253,4,\
1137,278,18,\
1137,327,16,\
1137,1391,20,\
1138,626,7,\
1138,641,23,\
1139,48,43,\
1139,279,21,\
1139,361,21,\
1139,400,25,\
1139,903,21,\
1139,1203,32,\
1139,1239,6,\
1140,49,17,\
1140,145,17,\
1140,164,2,\
1140,173,27,\
1140,218,18,\
1140,238,21,\
1140,353,2,\
1140,1084,2,\
1140,1829,21,\
1140,1852,6,\
1141,68,26,\
1141,96,32,\
1141,133,29,\
1141,179,39,\
1141,296,29,\
1141,327,3,\
1141,343,29,\
1141,374,3,\
1141,654,18,\
1141,674,13,\
1141,689,13,\
1141,707,16,\
1141,825,27,\
1141,869,17,\
1141,1035,13,\
1141,1050,27,\
1141,1082,12,\
1141,1190,27,\
1141,1450,3,\
1141,1459,10,\
1141,1474,3,\
1141,1724,21,\
1142,128,49,\
1142,208,14,\
1143,84,5,\
1143,303,5,\
1143,484,6,\
1143,595,25,\
1143,737,6,\
1143,1009,22,\
1143,1039,7,\
1144,10,29,\
1144,78,10,\
1144,93,3,\
1144,131,14,\
1144,356,3,\
1144,545,3,\
1144,867,3,\
1144,1199,7,\
1144,1503,3,\
1144,1603,3,\
1144,1675,3,\
1145,318,14,\
1145,335,15,\
1145,357,20,\
1146,110,3,\
1146,277,5,\
1146,337,29,\
1146,368,3,\
1146,255,20,\
1146,912,3,\
1146,1131,14,\
1146,1364,3,\
1146,1847,3,\
1146,1909,10,\
1147,35,28,\
1147,99,19,\
1147,65,3,\
1147,120,19,\
1147,141,3,\
1147,152,58,\
1147,231,48,\
1147,424,14,\
1147,572,3,\
1147,595,10,\
1147,1320,3,\
1147,1390,3,\
1147,1428,3,\
1148,31,19,\
1148,99,19,\
1148,293,19,\
1148,478,19,\
1149,102,28,\
1149,184,28,\
1149,214,3,\
1149,710,3,\
1149,1229,2,\
1149,1549,3,\
1150,223,27,\
1150,333,41,\
1150,252,4,\
1150,751,19,\
1150,1210,19,\
1150,1529,24,\
1150,1555,19,\
1150,1593,27,\
1150,1737,18,\
1151,90,29,\
1151,159,31,\
1151,243,29,\
1151,274,3,\
1151,321,48,\
1151,376,3,\
1151,423,3,\
1151,501,12,\
1151,641,3,\
1151,648,11,\
1151,767,3,\
1151,841,11,\
1151,878,12,\
1151,1189,15,\
1151,1209,28,\
1151,1242,11,\
1151,1471,3,\
1151,1621,11,\
1151,1715,12,\
1151,1804,25,\
1152,118,25,\
1152,169,14,\
1152,192,13,\
1152,210,17,\
1152,344,18,\
1152,537,6,\
1152,627,26,\
1152,687,15,\
1152,707,21,\
1152,777,13,\
1152,945,6,\
1153,15,12,\
1153,45,8,\
1153,200,12,\
1153,290,10,\
1153,475,8,\
1153,517,8,\
1153,618,8,\
1153,788,8,\
1153,866,12,\
1153,956,12,\
1153,1159,12,\
1153,1366,12,\
1153,1396,8,\
1153,1486,12,\
1154,147,22,\
1154,182,29,\
1154,213,3,\
1154,228,50,\
1154,348,13,\
1154,365,3,\
1154,735,13,\
1154,961,3,\
1155,0,21,\
1155,111,21,\
1155,134,3,\
1155,145,28,\
1155,203,17,\
1155,258,6,\
1155,266,19,\
1155,326,3,\
1155,507,3,\
1155,623,3,\
1155,682,3,\
1155,1035,3,\
1155,1124,3,\
1155,1200,3,\
1157,14,26,\
1157,146,26,\
1157,174,2,\
1157,183,15,\
1157,351,36,\
1157,389,23,\
1157,779,2,\
1158,4,12,\
1158,98,13,\
1158,260,17,\
1158,448,5,\
1158,516,13,\
1158,568,17,\
1158,765,7,\
1158,845,13,\
1158,930,13,\
1159,0,20,\
1159,22,13,\
1159,37,2,\
1159,73,25,\
1159,100,3,\
1159,218,27,\
1159,265,45,\
1159,398,2,\
1159,453,2,\
1159,462,17,\
1159,631,11,\
1159,649,15,\
1159,667,14,\
1159,687,28,\
1159,862,47,\
1159,1049,13,\
1159,1078,12,\
1159,1092,21,\
1159,1185,9,\
1159,1206,13,\
1159,1247,8,\
1160,61,18,\
1160,129,19,\
1160,215,18,\
1160,261,23,\
1161,105,25,\
1161,163,18,\
1161,227,28,\
1161,540,9,\
1161,623,18,\
1161,743,4,\
1161,1227,27,\
1162,97,28,\
1162,127,28,\
1162,157,3,\
1162,340,24,\
1162,1370,25,\
1163,15,44,\
1163,76,24,\
1164,0,58,\
1164,156,58,\
1164,283,28,\
1164,1003,13,\
1164,1076,12,\
1164,1091,3,\
1164,1283,3,\
1164,1455,3,\
1164,1665,18,\
1164,1686,10,\
1164,1703,9,\
1164,1811,20,\
1164,1960,3,\
1164,2045,25,\
1164,2078,18,\
1165,73,26,\
1165,214,44,\
1165,280,15,\
1165,319,18,\
1165,409,15,\
1165,494,3,\
1165,1101,4,\
1165,1421,6,\
1165,1497,4,\
1166,123,30,\
1166,215,15,\
1166,231,28,\
1166,389,29,\
1166,530,3,\
1166,535,35,\
1167,76,17,\
1167,169,8,\
1167,198,25,\
1167,391,30,\
1167,862,25,\
1167,904,20,\
1167,929,6,\
1167,1104,8,\
1168,0,53,\
1169,57,46,\
1170,120,25,\
1170,345,6,\
1170,425,25,\
1170,600,12,\
1170,744,14,\
1170,964,7,\
1170,1055,6,\
1170,1112,6,\
1170,1343,13,\
1170,1389,6,\
1170,1552,26,\
1171,54,14,\
1171,157,4,\
1171,126,29,\
1171,384,39,\
1171,425,4,\
1171,482,12,\
1171,508,4,\
1171,516,13,\
1171,543,4,\
1171,755,4,\
1171,861,4,\
1171,981,4,\
1171,1027,18,\
1171,1180,4,\
1171,1188,4,\
1171,1379,4,\
1171,1403,14,\
1171,1473,4,\
1172,131,27,\
1172,449,13,\
1172,464,15,\
1172,481,21,\
1172,504,16,\
1172,522,27,\
1172,551,21,\
1172,574,14,\
1172,594,15,\
1172,1167,17,\
1172,1186,15,\
1172,1203,21,\
1172,1226,16,\
1172,1244,27,\
1172,1273,21,\
1172,1296,14,\
1172,1312,15,\
1174,68,30,\
1174,100,22,\
1174,124,3,\
1174,134,19,\
1174,180,54,\
1174,366,3,\
1174,506,6,\
1174,615,11,\
1174,746,24,\
1174,1439,6,\
1175,78,33,\
1175,113,53,\
1175,193,21,\
1175,291,13,\
1175,322,12,\
1175,366,24,\
1175,523,12,\
1175,890,13,\
1176,13,15,\
1176,45,15,\
1176,528,20,\
1176,691,15,\
1176,804,15,\
1176,916,15,\
1176,974,52,\
1177,47,21,\
1177,166,3,\
1177,284,17,\
1177,302,21,\
1177,326,3,\
1177,408,3,\
1177,563,3,\
1178,57,27,\
1178,86,3,\
1178,358,3,\
1178,371,14,\
1178,426,3,\
1178,1234,3,\
1180,705,3,\
1180,773,5,\
1180,1310,27,\
1180,1375,16,\
1180,1501,16,\
1181,43,25,\
1181,136,25,\
1181,1374,5,\
1181,1408,16,\
1183,876,20,\
1184,1181,3,\
1184,1160,19,\
1184,1582,18,\
1184,2885,18,\
1185,43,25,\
1185,88,25,\
1185,147,6,\
1185,222,5,\
1185,367,25,\
1185,497,25,\
1186,0,18,\
1188,19,39,\
1188,178,39,\
1188,219,4,\
1188,1583,4,\
1189,301,2,\
1190,0,24,\
1190,139,4,\
1190,162,24,\
1190,261,3,\
1190,289,25,\
1190,960,3,\
1191,0,16,\
1191,37,25,\
1191,123,3,\
1191,146,16,\
1191,169,28,\
1191,215,26,\
1191,246,25,\
1191,359,14,\
1191,377,16,\
1191,536,16,\
1191,596,25,\
1191,623,2,\
1191,696,3,\
1191,764,2,\
1191,861,16,\
1191,931,25,\
1191,977,26,\
1191,1043,16,\
1191,1118,3,\
1191,1204,2,\
1191,1261,16,\
1191,1324,3,\
1191,1582,3,\
1191,1676,16,\
1191,1707,2,\
1192,30,22,\
1192,78,31,\
1192,111,4,\
1192,182,3,\
1192,327,3,\
1192,479,13,\
1192,550,27,\
1192,579,22,\
1192,603,17,\
1192,700,22,\
1192,899,13,\
1192,1018,20,\
1192,1095,28,\
1192,1172,13,\
1193,131,19,\
1193,152,3,\
1193,191,17,\
1193,255,13,\
1193,274,21,\
1193,406,2,\
1193,529,78,\
1193,716,22,\
1193,792,21,\
1193,824,78,\
1193,1018,3,\
1193,1061,15,\
1193,1209,12,\
1194,27,13,\
1194,113,11,\
1194,126,8,\
1194,136,27,\
1194,169,18,\
1194,203,13,\
1194,281,22,\
1194,345,11,\
1194,395,8,\
1194,471,13,\
1194,505,21,\
1194,535,14,\
1194,553,11,\
1194,669,13,\
1194,786,13,\
1194,1261,16,\
1194,1279,2,\
1194,1287,20,\
1194,1309,3,\
1194,1429,2,\
1194,1449,3,\
1194,1486,13,\
1195,779,14,\
1195,1409,5,\
1196,176,18,\
1196,197,2,\
1196,437,2,\
1196,720,2,\
1197,60,20,\
1197,116,20,\
1197,270,3,\
1197,454,3,\
1197,681,15,\
1197,813,3,\
1197,1146,3,\
1197,1382,3,\
1197,1497,20,\
1197,1632,3,\
1197,1844,3,\
1198,60,25,\
1198,118,25,\
1198,146,5,\
1198,281,14,\
1198,433,25,\
1198,508,5,\
1198,539,6,\
1198,915,6,\
1198,953,6,\
1198,1027,6,\
1198,1126,6,\
1198,1269,6,\
1198,1828,17,\
1198,1876,24,\
1198,1988,14,\
1199,13,3,\
1199,21,3,\
1199,224,11,\
1199,278,5,\
1199,357,5,\
1199,389,22,\
1199,415,6,\
1199,438,22,\
1199,575,3,\
1199,583,3,\
1199,623,30,\
1199,672,26,\
1199,1062,3,\
1199,1123,3,\
1199,1207,3,\
1199,1248,3,\
1199,1321,3,\
1199,1329,3,\
1199,1459,5,\
1199,1497,6,\
1199,1574,5,\
1200,32,24,\
1200,60,11,\
1200,89,7,\
1200,103,24,\
1200,195,10,\
1200,769,17,\
1200,973,17,\
1200,1046,25,\
1200,1117,21,\
1200,1143,10,\
1200,1303,24,\
1200,1360,10,\
1200,1383,10,\
1200,1677,18,\
1200,1791,24,\
1200,1819,15,\
1202,45,14,\
1202,105,5,\
1202,282,14,\
1202,355,17,\
1202,806,6,\
1203,358,2,\
1203,298,13,\
1203,473,2,\
1203,523,20,\
1203,716,2,\
1203,980,2,\
1203,1788,20,\
1204,0,15,\
1204,34,28,\
1204,64,28,\
1204,94,3,\
1204,104,19,\
1204,163,5,\
1204,173,9,\
1204,236,3,\
1204,401,9,\
1204,417,19,\
1204,506,19,\
1204,529,16,\
1204,619,20,\
1205,0,22,\
1206,56,3,\
1206,82,20,\
1206,213,17,\
1206,232,3,\
1206,270,20,\
1206,292,3,\
1206,479,3,\
1206,501,3,\
1206,740,6,\
1206,827,7,\
1206,886,10,\
1206,1137,3,\
1206,1147,3,\
1206,1666,3,\
1208,25,24,\
1208,207,24,\
1208,937,29,\
1208,1031,24,\
1210,46,20,\
1210,154,20,\
1210,176,2,\
1210,378,22,\
1210,415,2,\
1210,473,2,\
1210,659,2,\
1210,947,2,\
1210,1151,22,\
1210,1350,2,\
1210,1496,2,\
1210,1669,2,\
1211,70,15,\
1211,283,14,\
1211,299,9,\
1211,500,15,\
1211,1118,31,\
1211,1237,31,\
1211,1306,15,\
1212,28,11,\
1213,97,25,\
1213,278,16,\
1213,304,25,\
1213,331,2,\
1213,579,2,\
1213,639,2,\
1213,1428,2,\
1213,1528,17,\
1213,1591,2,\
1213,1741,2,\
1213,1748,22,\
1213,1937,2,\
1214,14,17,\
1215,32,21,\
1215,126,8,\
1215,136,8,\
1215,150,13,\
1215,220,21,\
1215,244,3,\
1215,460,23,\
1215,527,3,\
1215,623,13,\
1215,747,3,\
1215,787,23,\
1215,986,3,\
1215,1032,8,\
1215,1043,8,\
1215,1092,16,\
1215,1286,3,\
1216,89,20,\
1216,190,102,\
1217,60,13,\
1217,160,13,\
1217,304,13,\
1217,393,13,\
1217,422,5,\
1217,451,13,\
1217,536,13,\
1217,688,13,\
1217,876,13,\
1217,1085,13,\
1217,1233,13,\
1218,171,27,\
1218,1166,3,\
1218,1149,15,\
1219,0,45,\
1219,132,45,\
1219,191,28,\
1219,334,15,\
1219,179,4,\
1219,1519,5,\
1220,0,30,\
1220,35,6,\
1220,129,3,\
1220,162,16,\
1220,192,39,\
1220,236,34,\
1220,369,3,\
1220,481,3,\
1220,609,45,\
1220,924,20,\
1220,1052,8,\
1220,1065,18,\
1220,1200,27,\
1220,1289,17,\
1220,1543,17,\
1221,78,6,\
1221,147,6,\
1221,509,16,\
1221,584,17,\
1221,722,8,\
1221,746,13,\
1221,783,21,\
1221,1206,9,\
1221,1595,6,\
1221,1650,16,\
1221,1668,16,\
1221,1689,21,\
1221,1795,6,\
1222,31,14,\
1222,149,14,\
1222,213,13,\
1222,671,14,\
1224,32,14,\
1224,91,22,\
1224,136,14,\
1224,338,22,\
1224,362,3,\
1224,436,3,\
1224,567,3,\
1224,694,18,\
1224,716,3,\
1225,304,20,\
1225,557,20,\
1225,679,13,\
1225,1677,13,\
1226,75,17,\
1226,94,8,\
1226,136,36,\
1226,261,15,\
1226,336,19,\
1226,681,38,\
1226,974,21,\
1227,0,30,\
1227,36,19,\
1227,148,30,\
1227,180,3,\
1227,260,20,\
1227,285,10,\
1227,553,9,\
1227,564,22,\
1227,588,25,\
1227,615,20,\
1227,640,12,\
1227,680,3,\
1228,76,9,\
1228,97,18,\
1228,122,17,\
1228,254,7,\
1228,263,21,\
1228,290,13,\
1228,494,18,\
1228,524,13,\
1228,629,2,\
1228,728,2,\
1228,745,2,\
1228,618,9,\
1228,1161,15,\
1229,24,3,\
1229,131,15,\
1229,148,3,\
1229,159,46,\
1229,357,13,\
1229,384,3,\
1229,421,3,\
1229,719,3,\
1229,797,3,\
1229,995,3,\
1229,1044,19,\
1229,1065,3,\
1229,1290,3,\
1230,14,19,\
1230,37,6,\
1231,19,14,\
1231,55,14,\
1231,71,2,\
1231,520,2,\
1232,34,5,\
1232,125,13,\
1232,186,13,\
1232,460,13,\
1232,1636,13,\
1233,32,21,\
1233,115,21,\
1233,138,3,\
1233,258,3,\
1233,332,3,\
1233,826,3,\
1233,941,3,\
1233,1037,3,\
1233,1046,5,\
1233,1397,3,\
1233,1476,20,\
1233,1630,21,\
1233,1656,15,\
1233,1898,3,\
1235,417,15,\
1235,623,30,\
1235,898,9,\
1235,945,3,\
1236,0,19,\
1236,24,38,\
1236,67,18,\
1237,33,3,\
1237,67,19,\
1237,90,3,\
1237,180,26,\
1237,208,3,\
1237,231,17,\
1237,276,30,\
1237,308,3,\
1237,340,19,\
1237,373,3,\
1237,393,17,\
1237,416,3,\
1237,480,26,\
1237,518,19,\
1237,548,36,\
1237,593,6,\
1237,627,3,\
1237,840,3,\
1237,904,17,\
1237,1034,26,\
1237,1118,17,\
1237,1187,3,\
1237,1231,19,\
1237,1268,14,\
1237,1324,14,\
1237,1452,3,\
1237,1477,24,\
1237,1558,3,\
1237,1593,13,\
1237,1633,34,\
1238,13,15,\
1238,101,38,\
1238,170,27,\
1238,199,3,\
1238,264,14,\
1239,28,9,\
1239,61,24,\
1240,230,26,\
1240,1128,15,\
1241,54,18,\
1241,81,18,\
1241,112,19,\
1242,0,51,\
1242,60,21,\
1242,193,64,\
1242,299,21,\
1242,363,20,\
1242,548,3,\
1242,1015,18,\
1242,1054,21,\
1242,1145,16,\
1242,1239,32,\
1243,44,18,\
1243,76,31,\
1243,176,29,\
1243,296,29,\
1243,391,18,\
1243,453,18,\
1243,473,15,\
1243,490,16,\
1243,508,39,\
1243,605,19,\
1243,626,33,\
1243,786,37,\
1243,883,11,\
1243,896,20,\
1243,1185,43,\
1243,1372,14,\
1243,1697,16,\
1243,1730,18,\
1243,1782,15,\
1244,53,17,\
1244,254,12,\
1244,285,17,\
1244,429,5,\
1244,560,17,\
1244,638,17,\
1244,681,6,\
1244,802,17,\
1244,878,17,\
1244,920,7,\
1246,31,18,\
1246,77,18,\
1246,97,2,\
1246,357,2,\
1246,704,2,\
1246,898,2,\
1247,62,22,\
1247,368,19,\
1247,619,22,\
1247,643,2,\
1247,769,2,\
1247,1216,2,\
1247,1448,2,\
1247,1548,2,\
1247,1644,2,\
1247,1723,2,\
1247,1754,30,\
1247,1930,2,\
1247,1995,2,\
1248,45,16,\
1248,67,22,\
1248,103,16,\
1248,121,2,\
1248,130,16,\
1248,162,12,\
1248,204,22,\
1248,228,3,\
1248,493,25,\
1248,523,21,\
1248,608,2,\
1248,974,2,\
1248,1636,18,\
1248,1831,31,\
1249,29,36,\
1249,67,20,\
1249,94,45,\
1249,262,18,\
1249,455,20,\
1249,836,15,\
1249,873,20,\
1249,1023,20,\
1250,66,17,\
1250,85,17,\
1250,104,2,\
1250,203,2,\
1250,477,3,\
1250,568,2,\
1250,643,2,\
1250,805,2,\
1250,899,2,\
1250,1091,2,\
1250,1409,2,\
1251,82,19,\
1251,358,20,\
1251,381,3,\
1251,400,19,\
1251,422,3,\
1251,444,21,\
1251,475,11,\
1251,489,2,\
1251,514,3,\
1251,694,12,\
1251,726,3,\
1251,743,24,\
1251,777,2,\
1251,861,4,\
1251,892,4,\
1251,982,4,\
1251,1025,3,\
1251,1088,4,\
1251,1460,3,\
1251,1492,3,\
1251,1548,2,\
1251,1598,2,\
1252,0,19,\
1252,23,21,\
1252,105,21,\
1252,299,12,\
1252,354,8,\
1252,644,21,\
1253,59,22,\
1253,221,22,\
1253,245,2,\
1253,571,2,\
1253,654,2,\
1253,790,22,\
1253,859,2,\
1253,1332,2,\
1254,0,18,\
1254,214,23,\
1254,473,20,\
1254,495,2,\
1254,1014,2,\
1255,55,19,\
1255,139,23,\
1255,412,27,\
1255,500,12,\
1255,514,39,\
1255,753,13,\
1255,768,19,\
1255,1170,24,\
1256,0,25,\
1256,78,18,\
1256,121,49,\
1256,206,18,\
1256,226,2,\
1256,667,2,\
1256,695,2,\
1256,711,35,\
1256,751,25,\
1256,816,2,\
1256,838,13,\
1256,1463,2,\
1257,67,15,\
1257,165,13,\
1257,195,15,\
1257,559,11,\
1257,698,3,\
1257,929,21,\
1257,676,20,\
1257,1010,17,\
1257,1098,21,\
1257,1278,19,\
1257,1353,24,\
1258,24,28,\
1258,57,18,\
1259,71,23,\
1259,100,23,\
1259,125,3,\
1259,157,24,\
1259,186,22,\
1259,225,14,\
1259,438,3,\
1260,45,63,\
1260,218,63,\
1260,283,4,\
1260,451,24,\
1260,1294,24,\
1261,35,18,\
1261,319,18,\
1262,0,33,\
1262,35,21,\
1262,103,19,\
1262,127,22,\
1262,350,34,\
1263,0,30,\
1263,35,38,\
1263,79,20,\
1263,153,15,\
1263,185,36,\
1263,244,18,\
1263,268,20,\
1263,347,29,\
1263,481,13,\
1263,787,11,\
1263,847,14,\
1263,994,18,\
1263,1122,16,\
1263,1224,26,\
1263,1506,15,\
1263,1527,18,\
1263,1551,20,\
1263,1576,26,\
1263,1718,22,\
1263,1782,23,\
1264,49,15,\
1264,66,15,\
1264,379,15,\
1264,525,15,\
1264,836,17,\
1264,1219,17,\
1264,1321,15,\
1265,25,18,\
1265,45,2,\
1265,49,18,\
1265,74,37,\
1265,145,18,\
1265,336,2,\
1265,411,20,\
1265,480,2,\
1266,48,28,\
1266,120,30,\
1266,250,13,\
1266,284,21,\
1266,852,23,\
1267,72,17,\
1267,98,3,\
1267,183,26,\
1267,211,3,\
1267,274,17,\
1267,293,3,\
1267,698,3,\
1267,986,3,\
1267,1755,3,\
1268,29,17,\
1268,115,17,\
1268,169,6,\
1268,421,42,\
1268,475,30,\
1268,507,3,\
1268,465,5,\
1268,545,17,\
1268,705,17,\
1268,972,17,\
1269,17,21,\
1269,40,21,\
1269,63,2,\
1269,77,17,\
1269,96,28,\
1269,226,6,\
1269,274,2,\
1269,317,6,\
1269,340,13,\
1269,596,14,\
1269,660,2,\
1270,0,21,\
1270,216,13,\
1270,311,21,\
1270,335,3,\
1270,1270,3,\
1271,0,15,\
1271,128,26,\
1271,160,13,\
1271,337,10,\
1271,375,17,\
1271,452,26,\
1271,1336,15,\
1271,1432,10,\
1271,1453,15,\
1271,1470,10,\
1271,1482,17,\
1271,1592,30,\
1272,185,27,\
1272,742,3,\
1273,128,3,\
1273,137,33,\
1273,872,3,\
1273,906,3,\
1273,1179,3,\
1273,1187,3,\
1273,1276,3,\
1273,1283,3,\
1274,23,18,\
1274,92,18,\
1274,112,2,\
1274,131,23,\
1274,234,31,\
1274,315,17,\
1274,363,2,\
1274,422,17,\
1274,590,6,\
1274,700,6,\
1274,788,6,\
1274,829,6,\
1274,936,6,\
1274,1283,6,\
1274,1391,8,\
1274,1407,9,\
1274,1447,17,\
1274,1527,17,\
1274,1747,17,\
1275,201,16,\
1276,0,22,\
1276,247,22,\
1276,271,2,\
1276,473,2,\
1276,543,2,\
1276,624,2,\
1276,739,2,\
1276,857,2,\
1276,975,2,\
1276,1091,2,\
1276,1138,2,\
1276,1464,2,\
1276,1494,2,\
1276,1541,2,\
1276,1749,2,\
1277,0,3,\
1277,65,17,\
1277,190,30,\
1277,222,3,\
1277,231,12,\
1277,291,26,\
1277,319,3,\
1277,391,6,\
1277,496,3,\
1277,548,49,\
1277,623,14,\
1277,860,8,\
1277,920,3,\
1277,1345,23,\
1277,1387,19,\
1278,123,35,\
1278,172,35,\
1278,209,3,\
1278,559,3,\
1278,620,21,\
1278,805,3,\
1278,1136,3,\
1278,1712,3,\
1278,1728,28,\
1279,0,15,\
1280,71,8,\
1280,173,8,\
1280,307,14,\
1280,323,2,\
1280,437,8,\
1280,821,2,\
1280,857,2,\
1280,917,2,\
1280,1120,2,\
1280,1194,8,\
1281,38,30,\
1281,90,5,\
1281,302,14,\
1281,393,30,\
1282,71,21,\
1282,102,21,\
1282,125,3,\
1282,135,16,\
1282,452,3,\
1282,612,3,\
1282,705,27,\
1282,767,3,\
1282,1149,3,\
1282,1471,3,\
1282,1688,3,\
1283,26,19,\
1283,96,32,\
1283,303,19,\
1283,342,30,\
1283,374,3,\
1283,415,8,\
1283,480,9,\
1283,808,19,\
1283,867,3,\
1284,233,15,\
1284,1213,9,\
1284,1238,13,\
1284,1261,24,\
1285,16,23,\
1285,89,21,\
1285,235,13,\
1285,253,6,\
1285,357,21,\
1285,380,3,\
1285,388,27,\
1285,557,3,\
1285,637,30,\
1285,705,30,\
1285,919,3,\
1285,924,3,\
1285,1373,3,\
1285,1643,3,\
1285,1702,21,\
1286,135,40,\
1286,911,18,\
1287,65,29,\
1287,96,29,\
1287,127,5,\
1287,216,20,\
1287,253,5,\
1287,467,5,\
1287,536,12,\
1287,552,4,\
1287,911,5,\
1287,1027,18,\
1287,1149,5,\
1287,1313,5,\
1288,37,26,\
1288,81,13,\
1288,99,21,\
1288,215,3,\
1288,220,26,\
1288,316,7,\
1288,475,3,\
1288,517,18,\
1288,546,13,\
1288,564,21,\
1288,587,2,\
1288,707,14,\
1288,848,13,\
1288,1030,13,\
1288,1090,3,\
1288,1205,3,\
1288,1278,29,\
1289,17,15,\
1290,35,14,\
1290,145,14,\
1290,161,3,\
1291,1,17,\
1291,23,20,\
1291,128,21,\
1291,162,28,\
1291,342,7,\
1291,453,3,\
1291,589,30,\
1291,677,21,\
1292,55,20,\
1292,80,39,\
1292,198,29,\
1292,314,3,\
1292,414,20,\
1292,549,4,\
1292,697,19,\
1292,726,20,\
1292,830,20,\
1292,1068,39,\
1293,24,22,\
1293,59,10,\
1293,74,10,\
1293,86,40,\
1293,141,22,\
1293,246,13,\
1293,360,14,\
1293,518,10,\
1293,850,15,\
1293,1103,22,\
1295,97,13,\
1295,129,63,\
1295,214,13,\
1295,233,53,\
1295,809,14,\
1295,1393,15,\
1295,1634,13,\
1295,1774,24,\
1295,1856,13,\
1296,0,18,\
1296,42,17,\
1296,64,7,\
1296,73,18,\
1296,111,41,\
1296,188,30,\
1296,234,7,\
1296,246,17,\
1296,437,8,\
1296,450,6,\
1296,523,7,\
1296,583,17,\
1296,712,19,\
1296,979,5,\
1296,1084,7,\
1296,1096,17,\
1297,130,18,\
1297,174,34,\
1297,209,18,\
1297,240,27,\
1297,335,17,\
1297,473,4,\
1297,611,18,\
1297,906,19,\
1297,1755,18,\
1298,0,13,\
1298,95,13,\
1298,119,28,\
1298,259,24,\
1298,459,13,\
1298,1148,13,\
1298,1412,13,\
1299,79,20,\
1299,229,20,\
1299,453,20,\
1299,810,20,\
1299,926,5,\
1299,994,20,\
1299,1221,3,\
1300,128,18,\
1300,148,3,\
1300,175,28,\
1300,561,3,\
1301,60,18,\
1301,148,18,\
1301,168,2,\
1301,322,2,\
1301,435,2,\
1301,565,2,\
1301,617,37,\
1301,851,2,\
1301,939,2,\
1301,1380,2,\
1302,88,21,\
1302,135,27,\
1302,164,3,\
1302,196,16,\
1302,384,14,\
1302,431,15,\
1302,492,3,\
1302,641,3,\
1302,1133,10,\
1302,1214,3,\
1302,1408,32,\
1302,1457,18,\
1302,1567,3,\
1304,4,39,\
1304,45,15,\
1304,105,39,\
1304,146,4,\
1304,250,23,\
1304,300,8,\
1304,386,4,\
1304,716,4,\
1304,739,19,\
1304,772,14,\
1304,995,4,\
1304,1076,20,\
1305,134,19,\
1305,161,18,\
1305,341,29,\
1305,372,2,\
1305,389,18,\
1305,435,21,\
1305,549,2,\
1305,605,2,\
1305,722,2,\
1305,746,2,\
1305,777,18,\
1305,838,2,\
1305,1119,2,\
1305,1135,2,\
1305,1341,13,\
1305,1407,15,\
1305,1502,13,\
1305,1520,13,\
1305,1537,2,\
1306,54,26,\
1306,124,26,\
1306,152,3,\
1306,167,28,\
1306,210,22,\
1306,234,2,\
1306,260,13,\
1306,327,3,\
1306,562,13,\
1306,607,36,\
1306,860,3,\
1306,916,13,\
1306,1104,26,\
1307,25,28,\
1307,63,18,\
1307,115,28,\
1307,181,28,\
1307,584,18,\
1307,700,22,\
1307,822,18,\
1307,916,18,\
1308,34,25,\
1308,94,20,\
1308,119,20,\
1308,141,25,\
1308,168,3,\
1308,179,36,\
1308,221,13,\
1308,500,18,\
1308,522,3,\
1308,565,3,\
1308,659,18,\
1308,716,6,\
1308,948,6,\
1308,977,3,\
1308,1031,20,\
1308,1074,6,\
1308,1129,23,\
1308,1158,8,\
1308,1175,16,\
1308,1197,9,\
1308,1242,22,\
1308,1349,20,\
1308,1439,3,\
1308,1492,20,\
1308,1516,20,\
1309,44,15,\
1309,134,6,\
1309,145,17,\
1310,39,14,\
1310,122,24,\
1310,234,14,\
1310,718,24,\
1310,744,3,\
1310,1322,12,\
1311,82,39,\
1311,163,20,\
1311,185,2,\
1311,1109,21,\
1311,1302,26,\
1311,1420,36,\
1311,1531,20,\
1312,21,22,\
1312,78,31,\
1312,111,22,\
1312,135,3,\
1312,199,14,\
1312,248,3,\
1312,457,3,\
1312,643,3,\
1312,695,3,\
1312,1181,3,\
1312,1233,3,\
1313,53,32,\
1313,140,38,\
1313,356,3,\
1313,412,5,\
1313,468,3,\
1313,959,3,\
1313,1206,4,\
1313,1357,3,\
1313,1589,3,\
1314,51,26,\
1314,96,13,\
1314,162,26,\
1314,396,23,\
1314,559,18,\
1314,705,17,\
1314,940,17,\
1314,1031,7,\
1315,61,16,\
1315,138,6,\
1315,278,16,\
1315,296,3,\
1315,422,3,\
1315,652,12,\
1315,666,4,\
1315,687,6,\
1315,1021,6,\
1315,1112,6,\
1315,1132,3,\
1315,1628,6,\
1316,140,7,\
1316,152,32,\
1316,328,7,\
1316,361,32,\
1316,839,17,\
1316,1133,2,\
1316,1460,3,\
1316,1717,7,\
1316,1729,15,\
1316,2062,40,\
1317,31,19,\
1319,851,3,\
1319,2169,4,\
1319,2241,5,\
1320,153,26,\
1320,1322,15,\
1320,1438,3,\
1321,0,18,\
1321,161,8,\
1321,171,18,\
1321,202,20,\
1321,227,23,\
1321,375,23,\
1321,400,15,\
1321,423,20,\
1321,565,8,\
1321,581,20,\
1321,603,18,\
1321,749,18,\
1321,968,20,\
1321,1274,8,\
1321,1290,20,\
1321,1440,19,\
1321,1463,20,\
1321,1517,18,\
1322,51,47,\
1322,100,24,\
1322,220,24,\
1322,367,18,\
1322,415,24,\
1322,544,8,\
1322,659,24,\
1322,801,5,\
1322,865,11,\
1322,930,28,\
1322,1226,6,\
1322,1240,21,\
1322,1526,11,\
1322,1543,16,\
1322,1569,16,\
1322,1851,8,\
1322,1983,8,\
1322,2051,24,\
1323,112,14,\
1323,132,16,\
1323,218,24,\
1323,244,14,\
1323,340,24,\
1323,461,3,\
1323,471,22,\
1323,432,27,\
1323,668,3,\
1323,846,3,\
1324,59,22,\
1324,101,14,\
1324,181,18,\
1324,493,31,\
1324,526,3,\
1324,1647,3,\
1325,40,16,\
1325,58,16,\
1325,93,31,\
1325,427,25,\
1325,735,16,\
1325,1071,16,\
1326,33,14,\
1326,136,14,\
1326,616,30,\
1326,1171,20,\
1326,1224,21,\
1326,1344,12,\
1326,1367,27,\
1326,1437,17,\
1327,85,14,\
1327,103,17,\
1327,137,25,\
1327,190,15,\
1327,352,12,\
1327,768,14,\
1327,786,7,\
1327,902,17,\
1327,923,25,\
1328,124,18,\
1328,332,25,\
1329,668,25,\
1329,698,16,\
1329,888,3,\
1329,1264,24,\
1329,1631,11,\
1330,44,15,\
1330,143,15,\
1330,319,19,\
1331,70,18,\
1331,100,18,\
1331,648,8,\
1331,696,27,\
1331,1030,18,\
1331,1118,18,\
1331,1565,18,\
1332,23,16,\
1332,158,27,\
1332,206,18,\
1332,275,27,\
1332,350,14,\
1332,367,2,\
1332,507,23,\
1332,627,36,\
1332,751,13,\
1332,791,2,\
1332,1169,2,\
1333,25,6,\
1333,113,11,\
1333,527,58,\
1333,620,50,\
1333,1933,10,\
1334,12,38,\
1335,0,18,\
1335,114,18,\
1335,135,2,\
1335,161,17,\
1335,269,2,\
1336,57,18,\
1336,144,18,\
1336,165,2,\
1336,311,2,\
1336,376,2,\
1336,531,2,\
1336,729,2,\
1336,931,2,\
1336,1274,2,\
1336,1281,18,\
1337,48,21,\
1337,107,22,\
1337,183,22,\
1337,332,9,\
1337,475,9,\
1337,657,9,\
1337,688,20,\
1337,1152,20,\
1338,42,22,\
1338,93,25,\
1338,158,25,\
1338,252,15,\
1338,350,21,\
1338,572,23,\
1338,610,13,\
1338,651,15,\
1338,776,2,\
1338,784,20,\
1338,1004,20,\
1338,1045,20,\
1338,1088,23,\
1338,1485,13,\
1338,1686,23,\
1338,1846,21,\
1339,68,33,\
1339,173,33,\
1339,209,4,\
1339,1164,15,\
1339,1183,4,\
1339,1478,4,\
1339,1712,4,\
1340,117,21,\
1340,140,21,\
1340,163,3,\
1340,173,24,\
1340,212,19,\
1340,288,7,\
1340,309,11,\
1340,431,3,\
1340,839,3,\
1340,1221,3,\
1340,1312,30,\
1340,1574,3,\
1340,1608,19,\
1340,1641,14,\
1340,1661,12,\
1340,1810,3,\
1341,34,21,\
1341,153,52,\
1341,223,20,\
1341,245,26,\
1341,276,21,\
1341,326,40,\
1341,583,13,\
1341,645,13,\
1342,116,28,\
1342,263,28,\
1342,316,25,\
1342,1601,9,\
1342,1749,2,\
1343,24,17,\
1343,242,29,\
1343,273,4,\
1343,280,62,\
1343,1192,6,\
1343,1446,40,\
1344,74,18,\
1344,94,40,\
1344,147,33,\
1344,136,4,\
1344,323,19,\
1344,383,21,\
1344,406,3,\
1344,620,2,\
1344,588,30,\
1344,768,2,\
1344,835,18,\
1344,859,21,\
1344,938,15,\
1344,997,24,\
1345,25,16,\
1345,92,57,\
1345,156,46,\
1345,392,16,\
1345,551,16,\
1345,604,24,\
1345,836,14,\
1345,855,13,\
1345,901,15,\
1345,918,15,\
1346,252,29,\
1346,283,2,\
1346,536,18,\
1346,556,3,\
1347,1180,21,\
1348,27,25,\
1348,173,25,\
1348,201,7,\
1348,217,46,\
1348,276,14,\
1348,481,7,\
1348,599,7,\
1348,1925,7,\
1349,27,20,\
1349,69,28,\
1349,99,3,\
1349,282,20,\
1349,328,26,\
1349,436,3,\
1349,481,2,\
1349,578,2,\
1349,769,2,\
1349,835,2,\
1349,921,11,\
1349,1026,2,\
1349,1179,11,\
1350,179,16,\
1351,102,28,\
1351,285,28,\
1351,316,3,\
1351,474,27,\
1351,504,3,\
1351,578,3,\
1351,706,18,\
1351,772,21,\
1352,79,26,\
1352,148,26,\
1352,207,5,\
1352,176,3,\
1352,425,28,\
1352,567,26,\
1352,1052,18,\
1352,1105,16,\
1352,1185,18,\
1352,1390,18,\
1352,1528,5,\
1352,1624,5,\
1352,1660,8,\
1353,0,28,\
1353,105,28,\
1353,140,28,\
1353,196,36,\
1353,237,9,\
1353,926,3,\
1354,94,8,\
1354,227,27,\
1354,674,8,\
1354,1611,8,\
1355,73,14,\
1355,208,14,\
1356,48,15,\
1356,68,20,\
1356,147,28,\
1356,201,12,\
1356,439,2,\
1356,477,21,\
1356,539,6,\
1356,628,6,\
1356,1002,19,\
1356,1085,16,\
1356,1215,24,\
1356,1273,14,\
1356,1389,16,\
1356,1470,16,\
1356,1519,7,\
1356,1546,10,\
1356,1636,2,\
1356,1660,2,\
1356,1687,16,\
1356,1708,7,\
1356,1726,16,\
1357,143,25,\
1357,608,25,\
1358,0,11,\
1358,36,39,\
1358,103,39,\
1358,144,3,\
1358,163,11,\
1358,176,2,\
1358,181,30,\
1358,310,11,\
1358,406,3,\
1358,489,8,\
1358,499,13,\
1358,517,14,\
1358,690,5,\
1358,697,3,\
1358,984,5,\
1358,1188,2,\
1358,1340,11,\
1359,64,27,\
1359,270,37,\
1361,0,18,\
1361,119,18,\
1361,174,18,\
1361,195,2,\
1361,242,2,\
1361,325,2,\
1361,396,2,\
1361,653,2,\
1361,896,2,\
1361,954,2,\
1361,1179,2,\
1362,34,14,\
1362,132,25,\
1362,341,14,\
1362,404,36,\
1362,462,7,\
1362,505,32,\
1362,642,28,\
1362,714,14,\
1362,912,14,\
1362,1010,14,\
1362,1135,30,\
1362,1169,13,\
1362,1320,15,\
1362,1726,14,\
1363,0,21,\
1363,100,28,\
1363,129,21,\
1363,153,3,\
1363,178,23,\
1363,204,27,\
1363,249,3,\
1363,326,3,\
1363,576,3,\
1363,1116,3,\
1364,32,20,\
1364,86,3,\
1364,337,20,\
1364,383,20,\
1364,476,20,\
1364,405,2,\
1364,1166,20,\
1364,1275,3,\
1364,1254,19,\
1364,1359,20,\
1364,1463,20,\
1365,61,21,\
1365,84,21,\
1365,107,3,\
1365,135,20,\
1365,193,18,\
1365,235,18,\
1365,255,12,\
1365,269,11,\
1365,291,11,\
1365,310,7,\
1365,371,3,\
1365,514,3,\
1365,547,3,\
1365,566,24,\
1365,975,3,\
1365,1006,25,\
1365,1200,3,\
1365,1256,3,\
1365,1408,27,\
1365,1437,3,\
1366,73,14,\
1366,189,14,\
1366,327,14,\
1366,430,14,\
1366,474,6,\
1367,71,13,\
1367,86,3,\
1367,96,26,\
1367,202,50,\
1367,299,3,\
1367,459,3,\
1367,667,3,\
1367,1413,3,\
1368,0,27,\
1368,101,15,\
1368,132,16,\
1368,318,15,\
1368,1079,24,\
1368,1134,15,\
1369,0,3,\
1369,35,30,\
1369,121,30,\
1369,153,3,\
1369,164,38,\
1369,230,26,\
1369,258,3,\
1369,290,19,\
1369,354,15,\
1369,433,3,\
1369,756,3,\
1369,1337,3,\
1369,1498,3,\
1369,1539,3,\
1369,1574,3,\
1369,1621,3,\
1370,346,3,\
1370,959,2,\
1370,1252,2,\
1371,91,18,\
1371,128,16,\
1371,806,18,\
1371,826,4,\
1372,116,19,\
1372,137,19,\
1372,158,3,\
1372,166,16,\
1372,292,67,\
1372,391,39,\
1372,479,25,\
1372,533,33,\
1373,0,18,\
1373,43,2,\
1373,103,30,\
1373,190,8,\
1373,507,28,\
1373,593,8,\
1374,36,5,\
1374,99,12,\
1374,128,7,\
1374,471,5,\
1374,602,5,\
1374,638,12,\
1374,763,15,\
1374,783,10,\
1374,868,6,\
1374,1013,6,\
1374,1104,10,\
1374,1129,12,\
1374,1224,10,\
1374,1290,6,\
1374,1346,3,\
1374,1360,5,\
1374,1401,5,\
1374,1425,6,\
1374,1604,5,\
1374,1701,7,\
1375,0,26,\
1375,31,18,\
1376,0,49,\
1376,53,56,\
1376,122,12,\
1376,279,22,\
1376,303,3,\
1376,328,12,\
1376,368,20,\
1376,442,3,\
1376,681,3,\
1376,721,12,\
1377,20,8,\
1377,64,17,\
1377,177,8,\
1377,187,19,\
1377,263,8,\
1377,287,8,\
1377,366,8,\
1377,569,18,\
1377,591,8,\
1377,1078,19,\
1377,1418,17,\
1378,24,13,\
1378,158,13,\
1378,278,13,\
1378,385,32,\
1378,438,13,\
1378,464,21,\
1378,521,22,\
1378,634,13,\
1379,0,25,\
1379,73,25,\
1379,100,3,\
1379,167,28,\
1379,223,27,\
1379,291,13,\
1380,75,14,\
1380,91,14,\
1380,107,3,\
1380,118,29,\
1380,393,3,\
1381,59,32,\
1381,93,3,\
1381,234,32,\
1381,268,3,\
1381,801,3,\
1381,1327,13,\
1382,83,48,\
1382,308,18,\
1382,407,6,\
1382,433,26,\
1382,461,3,\
1382,594,3,\
1382,817,3,\
1382,1175,36,\
1382,1300,3,\
1382,1433,3,\
1382,1940,3,\
1382,1983,6,\
1382,2022,6,\
1382,2046,10,\
1382,2225,3,\
1383,16,21,\
1383,54,25,\
1383,161,21,\
1383,184,3,\
1383,213,25,\
1383,240,3,\
1383,312,3,\
1383,380,6,\
1383,572,6,\
1383,604,6,\
1383,777,3,\
1383,881,8,\
1384,73,21,\
1385,123,17,\
1385,158,19,\
1385,443,17,\
1385,518,17,\
1385,574,4,\
1385,612,17,\
1385,731,17,\
1385,1075,9,\
1385,1093,25,\
1385,1143,5,\
1385,1358,23,\
1385,1383,6,\
1385,1648,17,\
1385,1701,5,\
1385,1752,20,\
1385,1849,17,\
1386,38,29,\
1386,69,22,\
1386,93,5,\
1386,116,75,\
1386,394,35,\
1386,585,12,\
1386,638,21,\
1386,661,3,\
1386,667,17,\
1386,702,21,\
1386,775,15,\
1386,806,3,\
1386,1082,21,\
1386,1151,3,\
1386,1163,22,\
1386,1539,22,\
1386,1861,5,\
1387,87,18,\
1387,118,18,\
1387,171,14,\
1388,64,30,\
1388,107,30,\
1388,139,3,\
1388,185,17,\
1388,232,3,\
1388,409,3,\
1388,437,3,\
1388,522,3,\
1388,697,22,\
1388,820,20,\
1388,857,35,\
1388,1042,3,\
1388,1092,9,\
1388,1179,6,\
1388,1450,3,\
1388,1522,3,\
1388,1537,3,\
1389,86,26,\
1389,114,16,\
1389,137,28,\
1389,194,22,\
1389,218,6,\
1389,262,36,\
1389,349,19,\
1389,532,35,\
1389,814,9,\
1389,828,6,\
1390,19,23,\
1390,47,10,\
1390,64,30,\
1390,374,3,\
1390,387,17,\
1390,603,10,\
1390,739,19,\
1390,763,15,\
1390,844,10,\
1391,13,25,\
1392,0,23,\
1392,142,57,\
1392,216,26,\
1392,244,4,\
1392,387,28,\
1392,417,3,\
1392,609,23,\
1392,1005,3,\
1392,1186,3,\
1392,1306,3,\
1392,1390,3,\
1393,191,24,\
1393,934,14,\
1393,1000,14,\
1394,74,26,\
1394,102,2,\
1394,112,22,\
1394,282,9,\
1394,303,40,\
1394,348,9,\
1394,380,2,\
1394,715,2,\
1395,31,14,\
1396,46,17,\
1396,154,17,\
1396,465,17,\
1396,691,19,\
1396,896,19,\
1396,1011,19,\
1396,1829,17,\
1396,1868,27,\
1396,1956,19,\
1396,1984,7,\
1397,69,12,\
1397,1555,12,\
1397,1575,3,\
1397,1718,12,\
1398,4,5,\
1398,171,26,\
1398,250,23,\
1399,53,17,\
1399,72,3,\
1399,77,28,\
1399,110,36,\
1399,194,18,\
1399,664,22,\
1400,36,18,\
1400,115,18,\
1400,136,2,\
1400,174,17,\
1400,216,17,\
1400,343,2,\
1400,388,2,\
1400,482,2,\
1400,579,2,\
1400,630,2,\
1400,768,2,\
1400,971,2,\
1400,1029,2,\
1400,1080,2,\
1401,58,29,\
1401,89,29,\
1401,120,4,\
1401,659,20,\
1401,707,7,\
1401,716,13,\
1402,96,15,\
1402,260,18,\
1402,363,15,\
1402,644,15,\
1402,813,14,\
1402,932,15,\
1402,1258,13,\
1402,1281,15,\
1402,1437,29,\
1402,1548,15,\
1403,95,6,\
1403,116,35,\
1403,191,82,\
1403,153,4,\
1403,440,19,\
1403,550,19,\
1403,880,27,\
1404,59,40,\
1404,189,27,\
1404,218,3,\
1404,226,25,\
1404,253,3,\
1404,322,3,\
1404,326,3,\
1404,662,3,\
1405,61,24,\
1405,99,24,\
1405,135,54,\
1405,125,2,\
1405,553,2,\
1405,672,2,\
1405,736,17,\
1405,1141,2,\
1405,1256,2,\
1405,1602,2,\
1406,93,20,\
1406,158,20,\
1406,228,6,\
1406,538,16,\
1406,717,7,\
1406,774,14,\
1406,850,6,\
1407,68,18,\
1407,118,18,\
1407,138,2,\
1407,261,2,\
1407,430,2,\
1407,1520,2,\
1407,1662,2,\
1407,1675,17,\
1408,23,27,\
1408,425,16,\
1408,447,26,\
1408,1113,9,\
1408,1555,16,\
1408,1930,19,\
1409,26,12,\
1409,56,22,\
1409,80,22,\
1409,104,3,\
1409,173,31,\
1409,209,14,\
1409,341,16,\
1409,459,3,\
1409,513,12,\
1409,558,16,\
1409,632,3,\
1409,745,3,\
1409,778,5,\
1409,842,12,\
1409,961,3,\
1409,1080,3,\
1409,1244,12,\
1409,1261,19,\
1409,1318,3,\
1409,1426,3,\
1409,1483,12,\
1410,34,15,\
1410,104,25,\
1410,140,36,\
1410,285,25,\
1410,1181,15,\
1411,170,16,\
1411,197,24,\
1411,223,3,\
1411,898,16,\
1411,1336,16,\
1411,1550,15,\
1411,1692,15,\
1412,30,10,\
1412,366,21,\
1412,949,13,\
1413,120,21,\
1413,656,3,\
1413,846,3,\
1413,945,3,\
1413,1249,3,\
1415,237,15,\
1415,257,21,\
1415,1175,15,\
1415,1527,36,\
1416,0,20,\
1416,165,20,\
1416,238,20,\
1417,18,21,\
1417,287,21,\
1417,311,3,\
1417,380,3,\
1417,555,3,\
1417,768,3,\
1418,0,12,\
1418,336,12,\
1418,966,10,\
1418,1257,12,\
1419,49,37,\
1419,88,51,\
1419,618,15,\
1419,727,15,\
1419,888,10,\
1419,993,16,\
1419,1011,5,\
1419,1169,15,\
1420,54,15,\
1420,95,15,\
1420,825,7,\
1421,4,5,\
1421,152,5,\
1421,259,18,\
1421,512,15,\
1421,693,19,\
1421,833,19,\
1422,0,3,\
1422,95,3,\
1422,103,4,\
1422,149,8,\
1422,179,30,\
1422,211,3,\
1422,228,30,\
1422,260,4,\
1422,278,19,\
1422,442,3,\
1422,571,19,\
1422,618,17,\
1422,643,8,\
1422,653,2,\
1422,676,2,\
1422,696,2,\
1422,793,3,\
1422,807,4,\
1422,836,3,\
1422,888,3,\
1422,924,4,\
1422,992,3,\
1422,1118,8,\
1422,1368,22,\
1422,1471,3,\
1422,1479,4,\
1422,1526,19,\
1422,1631,8,\
1422,1765,3,\
1423,53,19,\
1423,77,8,\
1423,158,19,\
1423,182,8,\
1423,1027,19,\
1423,1051,8,\
1424,510,4,\
1425,111,14,\
1425,284,14,\
1425,769,14,\
1426,88,19,\
1426,612,49,\
1427,33,19,\
1427,144,28,\
1427,174,3,\
1428,134,37,\
1428,173,4,\
1428,305,18,\
1428,378,4,\
1428,415,4,\
1428,683,4,\
1428,868,4,\
1428,943,35,\
1428,1013,4,\
1428,1114,4,\
1428,1207,4,\
1429,53,17,\
1429,242,17,\
1429,560,6,\
1429,621,6,\
1429,1495,15,\
1430,37,28,\
1430,97,28,\
1430,127,3,\
1430,138,28,\
1430,252,3,\
1430,435,3,\
1430,559,3,\
1430,1544,3,\
1431,16,7,\
1431,42,17,\
1431,76,29,\
1431,112,27,\
1431,219,21,\
1431,290,17,\
1431,509,21,\
1431,547,24,\
1431,605,29,\
1431,655,21,\
1432,15,15,\
1432,96,16,\
1432,118,18,\
1432,1247,11,\
1432,1602,11,\
1434,135,16,\
1434,389,14,\
1434,703,29,\
1434,783,25,\
1434,861,16,\
1434,954,25,\
1434,1189,22,\
1434,1229,17,\
1435,103,28,\
1435,175,8,\
1435,297,8,\
1435,756,8,\
1435,991,8,\
1435,1079,28,\
1435,1230,8,\
1436,56,22,\
1436,102,5,\
1436,577,11,\
1436,608,12,\
1436,933,7,\
1437,119,31,\
1438,105,21,\
1438,214,21,\
1439,113,21,\
1439,248,21,\
1439,283,20,\
1440,19,7,\
1440,28,21,\
1440,407,15,\
1440,463,21,\
1440,718,7,\
1440,838,7,\
1440,1019,7,\
1440,1060,15,\
1440,1504,14,\
1440,1567,18,\
1440,1611,7,\
1440,1621,21,\
1441,57,18,\
1441,149,10,\
1441,164,8,\
1441,216,10,\
1441,436,18,\
1441,464,27,\
1441,498,16,\
1441,673,35,\
1441,738,7,\
1441,1165,10,\
1441,1356,7,\
1442,11,23,\
1442,36,8,\
1442,96,34,\
1442,132,16,\
1442,150,3,\
1442,161,28,\
1442,207,19,\
1442,228,39,\
1442,281,23,\
1442,368,3,\
1442,433,3,\
1442,505,3,\
1442,580,3,\
1442,600,23,\
1442,645,3,\
1442,696,16,\
1442,820,6,\
1442,835,14,\
1442,893,22,\
1442,929,21,\
1442,975,3,\
1442,1055,3,\
1442,1084,23,\
1442,1112,8,\
1443,55,43,\
1443,103,31,\
1443,299,26,\
1443,457,18,\
1443,619,30,\
1443,651,4,\
1443,918,4,\
1443,942,6,\
1443,1318,6,\
1444,22,26,\
1444,50,26,\
1444,78,2,\
1444,173,18,\
1444,193,2,\
1444,225,18,\
1444,378,15,\
1444,579,2,\
1444,780,2,\
1444,968,2,\
1444,1026,38,\
1444,1111,29,\
1444,1392,6,\
1445,65,54,\
1445,155,54,\
1445,211,4,\
1445,1100,23,\
1445,1231,19,\
1445,1407,18,\
1445,1438,32,\
1445,1581,21,\
1445,2116,4,\
1446,84,20,\
1448,284,6,\
1449,61,15,\
1449,187,29,\
1449,554,15,\
1449,583,27,\
1449,982,15,\
1450,56,15,\
1450,179,15,\
1450,196,2,\
1450,276,2,\
1450,376,2,\
1451,87,54,\
1451,143,54,\
1451,199,6,\
1451,229,12,\
1451,400,6,\
1451,462,34,\
1451,627,5,\
1451,916,6,\
1451,1542,6,\
1452,141,8,\
1452,170,8,\
1452,1346,15,\
1453,166,20,\
1453,188,2,\
1453,209,26,\
1453,559,2,\
1453,921,2,\
1453,1267,2,\
1453,1375,2,\
1453,1807,2,\
1454,26,15,\
1454,88,23,\
1454,295,7,\
1454,445,7,\
1454,505,7,\
1454,540,12,\
1454,930,12,\
1454,947,15,\
1454,993,15,\
1454,1061,12,\
1454,1158,28,\
1454,1224,15,\
1454,1250,28,\
1455,283,6,\
1456,31,39,\
1456,245,25,\
1456,272,4,\
1456,392,4,\
1456,877,18,\
1456,897,24,\
1456,923,3,\
1456,932,13,\
1456,1409,4,\
1456,1575,3,\
1456,1739,3,\
1457,81,22,\
1457,183,24,\
1457,209,3,\
1457,787,3,\
1457,873,19,\
1457,1000,12,\
1457,1163,3,\
1457,1336,3,\
1458,42,9,\
1458,62,17,\
1458,81,3,\
1458,113,26,\
1458,281,23,\
1458,511,3,\
1458,1017,3,\
1458,1300,17,\
1459,98,17,\
1459,209,17,\
1459,349,26,\
1459,496,17,\
1459,652,9,\
1459,797,17,\
1459,969,15,\
1459,1058,6,\
1459,1072,20,\
1459,1223,20,\
1459,1346,7,\
1459,1421,17,\
1460,33,12,\
1460,64,49,\
1460,167,19,\
1460,193,28,\
1460,264,6,\
1460,466,6,\
1460,932,18,\
1461,0,13,\
1461,17,15,\
1461,53,24,\
1461,79,15,\
1461,100,39,\
1461,178,40,\
1461,329,6,\
1461,340,7,\
1461,405,15,\
1462,25,24,\
1462,80,24,\
1462,106,3,\
1462,125,16,\
1462,143,6,\
1462,155,16,\
1462,574,3,\
1463,102,14,\
1463,118,14,\
1463,134,2,\
1463,144,28,\
1463,378,2,\
1463,567,2,\
1463,1116,2,\
1463,1259,2,\
1464,54,26,\
1464,109,26,\
1464,137,4,\
1464,162,27,\
1464,227,12,\
1464,241,18,\
1464,261,12,\
1464,307,19,\
1464,332,41,\
1464,506,4,\
1464,635,4,\
1464,729,21,\
1464,855,13,\
1464,874,18,\
1464,908,25,\
1465,1075,18,\
1466,168,3,\
1466,150,16,\
1466,403,3,\
1467,0,3,\
1467,69,23,\
1467,278,3,\
1467,248,28,\
1467,494,3,\
1467,513,25,\
1467,563,24,\
1467,673,14,\
1467,742,3,\
1467,971,3,\
1467,1082,3,\
1467,1097,23,\
1467,1163,3,\
1467,1188,3,\
1468,0,12,\
1468,24,44,\
1469,161,15,\
1469,302,2,\
1469,281,19,\
1469,413,26,\
1469,451,2,\
1469,1507,26,\
1469,1738,26,\
1470,33,18,\
1470,95,14,\
1470,125,37,\
1470,167,10,\
1470,392,3,\
1470,415,10,\
1470,455,22,\
1470,373,17,\
1470,652,3,\
1470,694,18,\
1470,766,15,\
1470,1125,14,\
1471,48,26,\
1471,76,26,\
1471,104,3,\
1471,493,18,\
1471,523,3,\
1471,570,21,\
1471,658,3,\
1471,873,3,\
1471,1050,3,\
1471,1193,3,\
1471,1448,7,\
1472,624,18,\
1472,644,3,\
1473,35,14,\
1473,543,25,\
1473,997,14,\
1473,1061,14,\
1473,1180,24,\
1473,1275,14,\
1474,80,12,\
1474,129,3,\
1474,310,44,\
1474,362,8,\
1474,380,3,\
1474,452,3,\
1474,460,4,\
1474,520,3,\
1474,632,3,\
1474,659,17,\
1474,696,3,\
1474,729,8,\
1474,856,8,\
1474,1038,8,\
1474,1162,23,\
1474,1443,15,\
1474,1519,3,\
1474,1608,15,\
1474,1670,14,\
1475,87,15,\
1475,202,5,\
1475,295,7,\
1475,385,12,\
1475,445,15,\
1475,492,13,\
1475,565,13,\
1475,800,5,\
1475,876,5,\
1476,74,32,\
1476,111,16,\
1476,164,27,\
1476,193,3,\
1476,241,38,\
1476,281,4,\
1476,364,18,\
1476,483,38,\
1476,700,23,\
1476,837,3,\
1476,845,4,\
1476,904,3,\
1476,1033,3,\
1476,1160,3,\
1476,1285,4,\
1476,1406,20,\
1477,0,76,\
1477,185,22,\
1477,209,3,\
1477,383,18,\
1477,403,2,\
1477,640,2,\
1477,767,3,\
1477,796,2,\
1477,1052,2,\
1477,1387,2,\
1478,38,13,\
1479,69,13,\
1479,162,13,\
1479,177,5,\
1479,184,21,\
1479,207,34,\
1479,1024,13,\
1480,24,17,\
1481,0,12,\
1482,7,22,\
1482,45,14,\
1482,166,22,\
1482,190,3,\
1482,296,14,\
1482,347,3,\
1482,406,3,\
1482,1384,3,\
1482,1720,3,\
1483,95,15,\
1483,124,25,\
1483,249,49,\
1483,444,15,\
1483,1133,35,\
1484,60,22,\
1484,116,22,\
1484,142,29,\
1484,173,3,\
1484,195,27,\
1484,434,3,\
1484,597,3,\
1484,671,3,\
1484,730,3,\
1484,818,14,\
1484,851,3,\
1484,955,3,\
1486,0,11,\
1487,0,22,\
1489,844,2,\
1489,833,9,\
1489,941,13,\
1489,992,2,\
1489,1448,2,\
1490,56,17,\
1490,78,14,\
1490,153,3,\
1490,595,36,\
1490,633,4,\
1490,726,14,\
1490,765,14,\
1490,1032,3,\
1491,1,21,\
1491,40,13,\
1491,71,13,\
1491,361,6,\
1491,388,21,\
1491,702,13,\
1491,809,13,\
1491,919,13,\
1491,1022,13,\
1491,1089,13,\
1492,35,19,\
1492,80,19,\
1494,261,22,\
1494,285,3,\
1494,440,22,\
1494,463,3,\
1494,602,3,\
1494,689,3,\
1494,990,3,\
1494,1043,3,\
1495,51,5,\
1495,850,39,\
1495,891,5,\
1496,23,27,\
1496,73,35,\
1496,488,20,\
1497,73,18,\
1497,93,2,\
1497,110,2,\
1497,259,18,\
1497,280,2,\
1497,387,2,\
1497,1295,2,\
1497,1435,2,\
1497,1780,2,\
1497,1887,2,\
1498,17,14,\
1498,126,14,\
1498,241,14,\
1498,461,14,\
1498,1080,13,\
1498,1196,14,\
1498,1308,15,\
1499,69,20,\
1499,108,20,\
1499,154,21,\
1499,423,18,\
1499,443,24,\
1499,469,12,\
1499,483,13,\
1499,535,18,\
1499,719,10,\
1499,1453,20,\
1500,0,31,\
1500,649,12,\
1500,739,12,\
1501,93,13,\
1501,108,13,\
1501,128,35,\
1501,171,35,\
1501,454,13,\
1501,816,13,\
1501,918,20,\
1501,1070,15,\
1501,1670,13,\
1501,1742,36,\
1501,1805,23,\
1502,69,13,\
1502,196,13,\
1502,507,13,\
1502,714,13,\
1502,786,13,\
1502,808,14,\
1502,916,25,\
1502,1024,13,\
1502,1056,14,\
1502,1089,25,\
1502,1289,23,\
1502,1330,14,\
1502,1475,14,\
1502,1534,13,\
1503,107,16,\
1503,167,16,\
1503,185,2,\
1503,292,3,\
1503,265,25,\
1503,420,3,\
1503,444,2,\
1503,523,2,\
1503,1389,3,\
1504,39,24,\
1504,126,24,\
1504,152,3,\
1504,163,17,\
1504,239,16,\
1504,257,6,\
1504,290,16,\
1504,420,3,\
1504,688,3,\
1504,696,25,\
1505,48,8,\
1505,74,8,\
1505,500,3,\
1505,492,6,\
1505,649,3,\
1506,272,17,\
1506,291,3,\
1506,326,20,\
1506,452,24,\
1507,33,22,\
1507,103,22,\
1507,127,3,\
1507,138,44,\
1507,262,17,\
1507,325,17,\
1507,455,52,\
1507,703,3,\
1507,823,3,\
1507,927,3,\
1507,1010,13,\
1507,1144,13,\
1507,1218,13,\
1507,1332,3,\
1507,1605,13,\
1507,1622,2,\
1507,1737,3,\
1508,386,22,\
1509,117,14,\
1510,346,11,\
1510,475,4,\
1511,12,35,\
1512,12,17,\
1512,212,11,\
1512,234,25,\
1512,419,17,\
1512,438,3,\
1512,581,3,\
1512,799,3,\
1512,1137,27,\
1512,2014,10,\
1513,147,14,\
1513,173,49,\
1513,409,6,\
1513,533,15,\
1513,761,26,\
1513,1779,14,\
1514,876,20,\
1514,1229,15,\
1515,53,13,\
1515,158,53,\
1515,356,19,\
1515,504,22,\
1515,759,24,\
1515,1040,18,\
1516,61,8,\
1516,136,8,\
1516,178,9,\
1516,292,21,\
1516,479,8,\
1516,1386,8,\
1516,1503,8,\
1517,91,19,\
1517,143,19,\
1517,164,2,\
1517,187,18,\
1517,387,2,\
1517,502,2,\
1517,643,2,\
1517,784,2,\
1517,1725,2,\
1518,25,23,\
1518,59,23,\
1518,84,3,\
1518,95,36,\
1518,199,3,\
1518,642,3,\
1519,46,19,\
1519,70,11,\
1519,139,19,\
1519,163,11,\
1519,909,8,\
1519,922,11,\
1521,506,3,\
1521,469,35,\
1522,66,29,\
1522,97,29,\
1522,128,3,\
1522,151,26,\
1522,205,26,\
1522,399,5,\
1523,11,16,\
1523,68,16,\
1523,230,16,\
1523,480,30,\
1523,688,16,\
1523,837,16,\
1523,912,16,\
1525,24,14,\
1525,230,4,\
1525,248,14,\
1525,198,30,\
1525,582,10,\
1525,609,26,\
1525,774,14,\
1525,1088,4,\
1525,1488,9,\
1525,1552,4,\
1525,1635,2,\
1525,1768,4,\
1525,1837,24,\
1526,102,21,\
1526,187,32,\
1526,970,3,\
1526,1134,9,\
1527,77,15,\
1528,84,29,\
1528,160,29,\
1528,191,5,\
1528,409,6,\
1528,1272,4,\
1528,1331,4,\
1528,1499,4,\
1528,1638,5,\
1529,22,20,\
1529,46,44,\
1529,101,44,\
1529,147,4,\
1529,157,27,\
1529,372,4,\
1529,405,8,\
1529,894,4,\
1529,1006,8,\
1530,19,13,\
1530,82,13,\
1531,23,29,\
1531,54,29,\
1531,85,5,\
1531,111,20,\
1531,250,5,\
1531,799,5,\
1532,23,20,\
1532,163,20,\
1532,186,3,\
1532,246,3,\
1532,524,3,\
1532,722,18,\
1532,743,2,\
1532,800,53,\
1532,1225,3,\
1532,1443,3,\
1532,1720,21,\
1532,1745,3,\
1534,43,39,\
1534,87,17,\
1534,141,6,\
1534,151,32,\
1534,185,3,\
1534,194,17,\
1534,213,2,\
1534,289,3,\
1534,325,2,\
1534,440,2,\
1534,455,6,\
1534,693,2,\
1534,743,27,\
1534,793,3,\
1534,801,2,\
1534,855,14,\
1534,873,2,\
1534,1104,3,\
1534,1233,3,\
1534,1284,3,\
1534,1386,9,\
1534,1413,23,\
1535,42,8,\
1535,187,8,\
1535,229,8,\
1535,498,8,\
1535,716,8,\
1535,853,8,\
1535,881,7,\
1536,46,7,\
1536,254,7,\
1536,694,7,\
1536,802,30,\
1536,834,11,\
1536,959,15,\
1536,979,13,\
1536,1064,5,\
1537,198,11,\
1538,114,13,\
1538,277,26,\
1538,316,14,\
1538,357,14,\
1538,386,36,\
1538,621,13,\
1538,849,13,\
1538,987,14,\
1538,1098,25,\
1538,1294,25,\
1538,1379,13,\
1538,1549,19,\
1538,1633,13,\
1539,0,14,\
1539,16,14,\
1539,41,15,\
1539,32,2,\
1539,275,5,\
1539,745,18,\
1539,993,5,\
1539,1181,25,\
1539,1438,5,\
1541,441,13,\
1541,584,6,\
1541,734,13,\
1541,995,6,\
1542,80,25,\
1542,113,18,\
1543,198,27,\
1544,18,20,\
1544,342,19,\
1544,570,17,\
1544,888,17,\
1544,1221,6,\
1544,1345,17,\
1545,56,20,\
1545,161,17,\
1545,181,3,\
1545,222,20,\
1545,457,5,\
1545,758,20,\
1545,930,3,\
1545,1193,7,\
1545,1374,6,\
1547,61,18,\
1547,81,18,\
1547,102,2,\
1547,172,2,\
1547,195,2,\
1547,564,18,\
1547,684,2,\
1547,871,2,\
1549,67,29,\
1549,110,29,\
1549,141,5,\
1549,153,33,\
1549,216,48,\
1549,515,5,\
1549,566,5,\
1549,1218,3,\
1549,1486,3,\
1549,1612,5,\
1549,1714,5,\
1550,57,26,\
1551,56,17,\
1551,77,21,\
1551,175,17,\
1551,194,2,\
1551,201,21,\
1551,224,3,\
1551,297,2,\
1551,321,3,\
1551,500,2,\
1551,514,3,\
1551,627,3,\
1551,716,2,\
1551,966,2,\
1551,1061,3,\
1552,26,17,\
1552,128,17,\
1552,147,3,\
1552,254,3,\
1552,467,3,\
1552,770,3,\
1552,812,3,\
1552,1008,3,\
1552,1788,3,\
1553,24,20,\
1553,66,17,\
1553,85,3,\
1553,137,3,\
1553,223,3,\
1553,299,3,\
1553,452,10,\
1553,563,3,\
1553,573,3,\
1553,719,3,\
1554,7,15,\
1554,109,15,\
1554,126,2,\
1554,134,45,\
1554,563,2,\
1554,591,15,\
1554,766,2,\
1554,815,2,\
1554,944,2,\
1554,1252,21,\
1554,1292,6,\
1554,1306,16,\
1554,1412,12,\
1554,1480,28,\
1554,1632,10,\
1554,1646,12,\
1554,1730,2,\
1555,4,23,\
1555,123,31,\
1555,217,19,\
1555,260,30,\
1555,293,3,\
1555,309,17,\
1555,348,3,\
1555,580,6,\
1555,704,3,\
1555,737,23,\
1555,836,33,\
1555,1026,3,\
1555,1120,3,\
1555,1151,15,\
1555,1222,5,\
1555,1293,3,\
1555,1311,3,\
1555,1328,5,\
1556,0,38,\
1556,102,5,\
1556,119,26,\
1556,147,3,\
1556,182,18,\
1556,354,12,\
1556,386,7,\
1556,522,3,\
1556,530,7,\
1556,552,3,\
1556,590,7,\
1556,703,3,\
1556,729,26,\
1556,763,3,\
1556,1022,7,\
1556,1057,3,\
1557,107,18,\
1557,174,27,\
1557,260,37,\
1557,361,32,\
1557,712,2,\
1557,857,22,\
1557,958,17,\
1558,447,15,\
1558,473,21,\
1558,496,35,\
1559,253,30,\
1559,631,20,\
1559,670,18,\
1559,693,29,\
1559,724,3,\
1559,1237,30,\
1559,1473,30,\
1560,315,17,\
1560,334,3,\
1560,418,6,\
1560,537,18,\
1560,921,20,\
1560,968,5,\
1560,1051,17,\
1560,1157,12,\
1560,1227,5,\
1561,0,51,\
1561,107,51,\
1562,145,3,\
1562,170,14,\
1562,1350,15,\
1562,1366,22,\
1562,1541,14,\
1563,71,31,\
1563,182,14,\
1563,198,21,\
1563,335,21,\
1563,402,14,\
1563,497,21,\
1563,592,13,\
1563,727,13,\
1563,766,6,\
1563,853,13,\
1563,1358,30,\
1565,77,14,\
1565,104,14,\
1565,124,26,\
1565,286,14,\
1565,1378,12,\
1567,241,2,\
1567,175,12,\
1568,40,48,\
1568,114,56,\
1568,198,41,\
1568,270,50,\
1568,431,25,\
1568,752,35,\
1568,1193,26,\
1569,47,19,\
1569,173,38,\
1569,267,8,\
1569,308,8,\
1569,1011,8,\
1569,1735,8,\
1570,155,25,\
1570,304,25,\
1570,796,25,\
1571,60,30,\
1571,164,30,\
1571,196,3,\
1571,251,3,\
1571,265,18,\
1571,313,23,\
1571,388,24,\
1571,447,22,\
1571,566,3,\
1571,659,3,\
1571,1011,38,\
1571,1064,3,\
1571,1250,15,\
1571,1452,29,\
1571,1551,3,\
1571,1648,15,\
1571,1724,3,\
1571,1904,3,\
1572,39,19,\
1572,77,19,\
1572,102,21,\
1572,417,65,\
1572,1623,22,\
1573,0,33,\
1574,0,18,\
1574,42,39,\
1574,83,18,\
1574,103,2,\
1574,112,29,\
1574,179,17,\
1574,211,13,\
1574,247,19,\
1574,329,26,\
1574,409,18,\
1574,451,7,\
1574,548,17,\
1574,567,16,\
1574,589,13,\
1574,717,2,\
1576,95,18,\
1576,1416,19,\
1576,1487,20,\
1576,1517,23,\
1576,1548,22,\
1576,1572,17,\
1576,1591,18,\
1576,1611,18,\
1576,1631,21,\
1576,1658,18,\
1576,1845,34,\
1577,204,13,\
1578,0,39,\
1578,43,23,\
1579,12,14,\
1579,244,8,\
1579,256,6,\
1579,339,38,\
1579,423,14,\
1579,461,22,\
1579,485,3,\
1579,507,17,\
1579,533,3,\
1579,562,28,\
1579,707,3,\
1579,1129,3,\
1579,1257,13,\
1579,1423,10,\
1579,1520,3,\
1579,1569,17,\
1580,614,23,\
1581,23,14,\
1581,150,30,\
1581,182,2,\
1581,337,3,\
1581,314,21,\
1581,603,3,\
1581,611,2,\
1581,921,2,\
1581,1009,3,\
1581,1296,2,\
1581,1371,5,\
1582,0,24,\
1582,105,15,\
1582,125,38,\
1582,185,13,\
1582,200,14,\
1582,232,14,\
1582,248,11,\
1582,287,20,\
1582,313,16,\
1582,357,17,\
1582,491,15,\
1582,627,15,\
1582,1008,15,\
1583,71,43,\
1584,0,13,\
1584,51,13,\
1584,69,27,\
1584,98,4,\
1584,169,9,\
1584,183,8,\
1584,202,16,\
1584,232,18,\
1584,256,22,\
1584,319,4,\
1584,337,11,\
1584,453,4,\
1584,613,9,\
1584,682,15,\
1584,738,8,\
1584,748,10,\
1584,809,12,\
1584,823,14,\
1584,839,11,\
1584,852,14,\
1584,868,15,\
1584,885,16,\
1584,925,15,\
1584,1028,9,\
1584,1463,22,\
1585,21,25,\
1585,129,25,\
1585,186,32,\
1585,383,25,\
1585,504,25,\
1585,575,20,\
1585,989,25,\
1585,1417,37,\
1586,51,12,\
1587,0,20,\
1587,22,29,\
1587,53,3,\
1587,436,26,\
1587,466,13,\
1587,481,33,\
1587,520,23,\
1587,692,21,\
1587,788,22,\
1587,812,18,\
1587,942,15,\
1587,987,28,\
1587,1106,22,\
1587,1224,21,\
1587,1382,3,\
1588,25,8,\
1588,35,23,\
1588,63,18,\
1590,27,15,\
1590,47,9,\
1590,251,15,\
1590,271,9,\
1590,962,22,\
1590,1217,9,\
1590,1333,15,\
1590,1353,9,\
1591,17,11,\
1591,33,32,\
1591,83,28,\
1591,140,28,\
1591,170,3,\
1591,197,13,\
1591,360,15,\
1591,473,3,\
1591,619,15,\
1591,677,3,\
1591,698,11,\
1591,770,15,\
1591,836,3,\
1591,857,11,\
1591,968,15,\
1591,1005,36,\
1591,1133,3,\
1591,1185,3,\
1591,1201,3,\
1591,1213,11,\
1591,1337,3,\
1591,1395,11,\
1591,1410,3,\
1591,1448,3,\
1592,0,20,\
1592,77,18,\
1592,255,20,\
1592,277,3,\
1592,285,11,\
1592,375,22,\
1593,16,51,\
1593,119,51,\
1593,172,6,\
1594,0,17,\
1594,68,17,\
1594,87,2,\
1594,96,27,\
1594,221,2,\
1594,309,2,\
1594,385,2,\
1594,650,18,\
1594,670,3,\
1594,745,3,\
1594,1026,3,\
1594,1091,5,\
1594,1198,13,\
1594,1627,2,\
1594,1637,15,\
1594,1715,2,\
1594,1754,3,\
1596,0,19,\
1596,37,20,\
1597,39,16,\
1600,44,25,\
1600,140,13,\
1600,158,14,\
1600,382,13,\
1600,434,13,\
1600,772,25,\
1600,1290,24,\
1600,1418,13,\
1600,1586,13,\
1602,9,17,\
1602,178,26,\
1602,339,25,\
1602,510,8,\
1602,638,17,\
1602,823,26,\
1602,1005,26,\
1603,188,34,\
1603,940,23,\
1604,0,20,\
1605,52,21,\
1605,128,21,\
1605,156,3,\
1605,166,5,\
1605,208,12,\
1605,222,3,\
1605,458,3,\
1605,620,3,\
1605,728,3,\
1605,824,3,\
1605,1100,3,\
1605,1115,3,\
1606,410,18,\
1606,579,15,\
1607,20,30,\
1607,383,19,\
1607,434,7,\
1607,979,42,\
1607,1023,30,\
1607,1055,3,\
1608,86,15,\
1608,103,15,\
1608,186,28,\
1608,256,15,\
1608,590,15,\
1608,1281,15,\
1608,1791,18,\
1608,1841,15,\
1609,97,12,\
1609,122,22,\
1609,150,16,\
1609,479,12,\
1609,642,12,\
1609,770,12,\
1609,1027,12,\
1610,155,23,\
1610,227,39,\
1612,0,42,\
1612,61,56,\
1612,119,63,\
1612,184,3,\
1612,199,36,\
1612,295,18,\
1612,318,17,\
1612,337,21,\
1612,405,13,\
1612,448,13,\
1612,525,13,\
1612,560,3,\
1612,630,12,\
1612,684,3,\
1612,736,30,\
1612,1205,36,\
1612,1300,3,\
1612,1371,12,\
1612,1407,20,\
1612,1431,14,\
1613,43,25,\
1613,103,25,\
1613,130,3,\
1613,145,38,\
1613,194,21,\
1613,456,3,\
1613,558,3,\
1613,1457,3,\
1614,59,15,\
1614,104,15,\
1615,31,14,\
1615,126,14,\
1615,160,33,\
1615,221,10,\
1615,277,3,\
1615,1143,22,\
1615,1334,3,\
1616,21,3,\
1616,101,22,\
1616,155,22,\
1616,179,3,\
1616,190,37,\
1616,276,14,\
1616,397,3,\
1616,542,3,\
1616,649,3,\
1616,1459,3,\
1616,1577,3,\
1617,76,14,\
1617,92,14,\
1617,108,2,\
1617,163,14,\
1617,234,52,\
1617,446,2,\
1617,1643,2,\
1618,1521,18,\
1618,1544,18,\
1619,45,15,\
1619,108,15,\
1619,125,2,\
1619,135,28,\
1619,223,2,\
1619,315,26,\
1619,432,21,\
1619,455,2,\
1619,484,2,\
1619,499,2,\
1619,463,19,\
1619,745,26,\
1619,913,19,\
1619,1028,2,\
1619,1072,2,\
1619,1125,2,\
1619,1176,2,\
1619,1279,2,\
1619,1399,2,\
1619,1570,2,\
1619,1763,2,\
1620,64,14,\
1620,117,22,\
1620,431,14,\
1620,478,22,\
1620,534,14,\
1620,750,14,\
1620,855,22,\
1620,937,14,\
1620,1117,14,\
1620,1204,14,\
1620,1272,14,\
1620,1330,14,\
1620,1372,14,\
1620,1573,14,\
1620,1630,22,\
1620,1710,14,\
1620,1749,22,\
1621,10,8,\
1621,24,25,\
1621,51,8,\
1621,208,14,\
1621,312,8,\
1621,385,29,\
1621,439,8,\
1621,452,14,\
1622,51,17,\
1622,101,38,\
1622,141,3,\
1622,149,3,\
1622,172,17,\
1622,277,3,\
1622,490,18,\
1622,681,3,\
1622,725,3,\
1622,729,3,\
1622,758,17,\
1622,790,3,\
1622,802,3,\
1622,929,16,\
1622,1140,3,\
1622,1331,3,\
1622,1389,18,\
1622,1413,3,\
1622,1460,3,\
1622,1529,28,\
1622,1793,3,\
1623,11,20,\
1623,108,20,\
1623,146,33,\
1623,181,3,\
1623,232,20,\
1623,323,13,\
1623,417,28,\
1623,538,20,\
1623,560,3,\
1623,714,20,\
1623,752,3,\
1623,798,20,\
1623,836,28,\
1623,913,3,\
1623,931,20,\
1624,34,15,\
1624,105,26,\
1624,405,15,\
1624,480,16,\
1624,658,15,\
1624,798,10,\
1624,845,13,\
1624,860,23,\
1624,979,52,\
1624,1064,13,\
1624,1191,15,\
1624,1227,24,\
1625,60,14,\
1626,760,19,\
1626,1470,29,\
1627,45,47,\
1627,125,14,\
1627,153,34,\
1627,216,4,\
1627,737,19,\
1627,797,13,\
1627,1062,19,\
1627,1480,29,\
1628,0,20,\
1628,26,22,\
1628,113,20,\
1628,152,9,\
1628,163,17,\
1628,205,11,\
1628,353,21,\
1628,376,18,\
1628,414,13,\
1628,744,20,\
1629,88,26,\
1629,116,3,\
1629,248,17,\
1629,267,3,\
1629,696,26,\
1629,724,3,\
1629,784,3,\
1629,1091,3,\
1629,1067,22,\
1629,1290,3,\
1629,1503,3,\
1629,1627,6,\
1629,1686,23,\
1629,1725,14,\
1630,0,28,\
1630,220,20,\
1630,242,3,\
1630,694,16,\
1631,96,12,\
1631,208,12,\
1631,297,12,\
1631,423,3,\
1631,938,3,\
1631,960,12,\
1631,1053,12,\
1631,1069,20,\
1632,25,46,\
1632,99,9,\
1632,131,31,\
1632,175,3,\
1632,225,20,\
1632,265,25,\
1632,316,3,\
1632,380,3,\
1632,292,4,\
1632,573,3,\
1632,661,6,\
1632,689,3,\
1632,865,6,\
1632,915,3,\
1632,951,25,\
1632,982,3,\
1632,1092,5,\
1632,1183,7,\
1632,1212,23,\
1632,1268,6,\
1632,1322,3,\
1632,1366,3,\
1632,1600,3,\
1632,1728,18,\
1633,0,13,\
1633,19,20,\
1633,234,20,\
1633,270,11,\
1633,589,29,\
1634,72,8,\
1634,204,8,\
1634,705,20,\
1634,809,8,\
1634,1416,8,\
1635,40,16,\
1635,60,20,\
1635,131,18,\
1635,153,4,\
1635,159,35,\
1635,397,4,\
1635,556,20,\
1635,1184,4,\
1635,1368,14,\
1635,1473,4,\
1636,0,27,\
1636,29,27,\
1636,58,3,\
1636,170,22,\
1636,241,3,\
1636,263,28,\
1636,846,16,\
1636,965,8,\
1636,975,16,\
1636,1109,18,\
1636,1161,3,\
1636,1166,24,\
1636,1238,12,\
1636,1283,8,\
1637,117,28,\
1637,260,18,\
1638,292,15,\
1638,346,14,\
1638,485,3,\
1638,645,15,\
1638,1866,15,\
1639,125,16,\
1639,376,29,\
1640,26,24,\
1640,88,26,\
1640,503,14,\
1640,559,23,\
1640,612,6,\
1640,823,6,\
1641,29,14,\
1641,53,61,\
1641,230,19,\
1641,416,14,\
1641,581,14,\
1642,0,25,\
1642,281,25,\
1642,648,32,\
1642,1361,7,\
1642,1471,26,\
1645,22,35,\
1645,152,35,\
1645,189,3,\
1645,340,26,\
1645,462,21,\
1645,490,3,\
1645,639,3,\
1645,809,3,\
1645,889,8,\
1645,993,5,\
1645,1333,5,\
1645,1362,14,\
1647,35,15,\
1647,52,15,\
1647,69,2,\
1647,129,38,\
1647,284,19,\
1647,305,13,\
1647,347,26,\
1647,479,15,\
1647,900,13,\
1647,918,15,\
1647,1415,15,\
1648,22,20,\
1649,44,6,\
1649,512,7,\
1649,524,6,\
1649,994,13,\
1649,1022,6,\
1649,1036,17,\
1649,1073,14,\
1649,1376,18,\
1650,41,32,\
1650,108,31,\
1650,177,3,\
1650,226,6,\
1650,251,8,\
1650,503,3,\
1650,534,22,\
1650,579,3,\
1650,615,16,\
1650,769,26,\
1650,893,3,\
1650,968,7,\
1650,1068,3,\
1650,1077,15,\
1650,1149,14,\
1650,1204,24,\
1650,1365,25,\
1650,1451,38,\
1650,1508,3,\
1650,1684,25,\
1650,1753,3,\
1650,1868,5,\
1652,291,5,\
1652,340,5,\
1652,367,36,\
1652,648,5,\
1652,856,6,\
1652,872,8,\
1652,969,10,\
1652,1076,5,\
1653,46,18,\
1653,164,18,\
1653,251,5,\
1653,344,5,\
1653,386,18,\
1653,484,18,\
1653,609,17,\
1653,917,7,\
1653,994,5,\
1653,1033,18,\
1653,1114,18,\
1653,1212,5,\
1654,60,24,\
1655,74,22,\
1655,103,46,\
1655,162,57,\
1655,427,22,\
1656,93,26,\
1657,201,11,\
1658,0,43,\
1658,49,13,\
1658,102,20,\
1658,389,22,\
1658,413,29,\
1658,631,20,\
1658,784,40,\
1658,947,13,\
1658,969,20,\
1660,0,36,\
1660,140,36,\
1660,198,18,\
1660,392,18,\
1660,551,15,\
1660,581,18,\
1660,623,35,\
1660,924,39,\
1660,1008,18,\
1660,1130,22,\
1660,1202,18,\
1662,21,22,\
1662,80,22,\
1662,113,27,\
1662,183,21,\
1662,219,22,\
1663,109,23,\
1663,276,30,\
1663,741,23,\
1663,855,30,\
1664,325,6,\
1664,448,3,\
1664,611,3,\
1664,788,13,\
1664,1506,3,\
1665,28,36,\
1665,127,31,\
1665,1109,21,\
1665,1177,14,\
1665,1282,21,\
1665,1409,6,\
1665,1424,5,\
1665,1502,33,\
1665,1540,18,\
1665,1564,17,\
1665,1590,14,\
1665,1682,14,\
1665,1862,24,\
1665,1919,15,\
1666,8,18,\
1667,30,29,\
1667,79,30,\
1668,68,20,\
1668,116,15,\
1668,241,25,\
1668,310,7,\
1668,329,13,\
1668,420,20,\
1668,631,20,\
1669,52,8,\
1669,98,17,\
1669,208,8,\
1669,316,15,\
1669,379,8,\
1669,967,17,\
1670,110,14,\
1670,192,16,\
1670,210,2,\
1670,272,2,\
1670,349,2,\
1670,397,29,\
1670,1014,2,\
1670,1247,27,\
1670,1278,2,\
1670,1352,2,\
1670,1449,2,\
1670,1535,27,\
1670,1566,2,\
1672,37,27,\
1672,66,27,\
1672,95,3,\
1672,106,27,\
1672,161,14,\
1672,180,8,\
1672,237,3,\
1672,403,22,\
1672,482,19,\
1672,578,3,\
1672,667,3,\
1672,805,3,\
1673,0,29,\
1673,64,17,\
1673,112,29,\
1673,144,3,\
1673,155,15,\
1673,187,43,\
1673,233,21,\
1673,333,3,\
1673,480,3,\
1673,554,21,\
1673,578,3,\
1673,658,21,\
1673,682,16,\
1673,731,3,\
1673,738,12,\
1673,798,3,\
1673,839,3,\
1673,861,16,\
1673,1093,39,\
1673,1400,12,\
1673,1423,21,\
1673,1450,20,\
1673,1497,3,\
1673,1540,3,\
1673,1578,16,\
1674,43,20,\
1674,195,30,\
1674,455,20,\
1674,1803,20,\
1675,1,15,\
1675,66,43,\
1675,337,3,\
1675,379,12,\
1675,393,17,\
1675,412,6,\
1675,435,11,\
1675,468,16,\
1675,503,3,\
1675,513,12,\
1675,542,17,\
1675,583,3,\
1675,769,17,\
1675,800,6,\
1675,839,21,\
1675,874,23,\
1675,950,14,\
1675,974,16,\
1675,1351,32,\
1675,1385,4,\
1675,1656,23,\
1675,1754,15,\
1675,2246,24,\
1676,203,2,\
1676,182,19,\
1676,560,2,\
1676,1476,2,\
1677,49,11,\
1677,129,30,\
1677,161,6,\
1677,175,27,\
1677,240,15,\
1677,620,6,\
1677,955,6,\
1678,44,34,\
1678,138,13,\
1678,203,22,\
1678,230,23,\
1678,361,6,\
1678,821,14,\
1678,983,13,\
1678,1047,13,\
1678,1467,22,\
1678,1494,23,\
1679,1,16,\
1679,24,28,\
1679,54,16,\
1679,103,15,\
1679,128,16,\
1679,398,14,\
1679,520,16,\
1679,657,15,\
1680,75,24,\
1680,117,16,\
1680,838,16,\
1680,856,6,\
1680,965,24,\
1680,991,3,\
1680,996,25,\
1680,1192,3,\
1680,1717,18,\
1681,41,19,\
1681,124,22,\
1681,560,19,\
1681,633,22,\
1681,830,12,\
1681,936,22,\
1681,1161,12,\
1681,1233,12,\
1682,52,26,\
1682,166,26,\
1682,194,3,\
1682,681,20,\
1683,76,3,\
1683,111,17,\
1683,130,3,\
1683,160,3,\
1683,233,3,\
1683,898,13,\
1683,1144,8,\
1683,1375,13,\
1683,1400,10,\
1683,1711,17,\
1683,1744,13,\
1684,83,13,\
1685,11,30,\
1685,244,18,\
1685,363,9,\
1685,540,16,\
1685,611,13,\
1685,659,17,\
1685,1023,16,\
1685,1216,17,\
1685,1346,3,\
1686,0,25,\
1686,53,24,\
1686,132,25,\
1686,166,24,\
1686,229,37,\
1686,279,25,\
1686,332,9,\
1686,411,23,\
1686,540,3,\
1686,754,49,\
1686,846,24,\
1686,910,24,\
1686,959,24,\
1686,1148,6,\
1686,1228,5,\
1686,1328,34,\
1686,1548,25,\
1686,1591,25,\
1686,1644,40,\
1686,1701,36,\
1686,1739,52,\
1686,1851,23,\
1687,101,14,\
1687,117,14,\
1687,133,2,\
1687,167,2,\
1687,236,2,\
1687,268,20,\
1687,939,19,\
1687,1878,2,\
1688,1,16,\
1689,12,27,\
1689,41,14,\
1689,128,29,\
1689,215,27,\
1689,245,14,\
1689,262,6,\
1689,507,18,\
1689,1519,6,\
1689,1727,6,\
1689,1834,3,\
1690,28,35,\
1690,69,29,\
1690,256,7,\
1690,288,3,\
1690,341,3,\
1690,397,5,\
1690,492,3,\
1690,580,5,\
1690,726,9,\
1690,778,3,\
1690,821,3,\
1691,404,25,\
1692,109,18,\
1692,134,27,\
1692,170,30,\
1692,239,11,\
1693,0,26,\
1693,28,31,\
1693,189,13,\
1693,222,42,\
1693,310,34,\
1693,843,13,\
1694,205,13,\
1694,246,20,\
1694,268,17,\
1694,291,34,\
1694,491,13,\
1694,1004,34,\
1694,1042,13,\
1695,0,24,\
1695,57,24,\
1695,83,3,\
1695,365,21,\
1695,397,25,\
1696,18,12,\
1696,287,17,\
1696,313,12,\
1696,631,12,\
1696,1936,10,\
1696,2411,10,\
1697,457,6,\
1697,1797,20,\
1697,1939,6,\
1698,15,46,\
1698,118,7,\
1698,461,14,\
1698,644,14,\
1698,815,14,\
1698,842,14,\
1698,1383,14,\
1698,1605,3,\
1698,1700,14,\
1698,1742,13,\
1698,1846,14,\
1698,1954,14,\
1698,1982,14,\
1698,2072,7,\
1698,2084,14,\
1699,162,14,\
1699,405,14,\
1699,727,15,\
1699,760,18,\
1699,806,6,\
1699,817,16,\
1699,1162,14,\
1699,1264,14,\
1700,23,27,\
1701,40,21,\
1701,87,21,\
1701,110,3,\
1701,135,13,\
1701,150,18,\
1701,170,21,\
1701,197,11,\
1701,235,7,\
1701,258,3,\
1701,286,25,\
1701,313,3,\
1701,380,3,\
1701,573,3,\
1701,612,3,\
1701,782,3,\
1701,787,30,\
1701,1451,28,\
1701,1625,25,\
1701,1732,3,\
1702,65,25,\
1702,99,25,\
1702,127,3,\
1702,139,26,\
1702,185,40,\
1702,262,3,\
1702,325,3,\
1702,539,3,\
1702,816,3,\
1703,58,22,\
1703,112,13,\
1703,319,14,\
1703,428,22,\
1703,644,23,\
1703,723,13,\
1703,1045,6,\
1703,1096,23,\
1704,355,28,\
1704,712,28,\
1705,55,36,\
1705,217,26,\
1705,720,6,\
1705,901,6,\
1705,1293,3,\
1705,1334,6,\
1706,89,35,\
1706,126,35,\
1706,179,28,\
1706,245,22,\
1706,723,35,\
1706,1113,7,\
1707,41,33,\
1707,88,33,\
1707,123,3,\
1707,137,62,\
1707,249,33,\
1707,316,19,\
1707,353,14,\
1707,369,3,\
1707,454,9,\
1707,691,3,\
1707,1031,3,\
1707,1204,19,\
1707,1235,14,\
1707,1254,25,\
1707,1343,28,\
1707,1629,19,\
1707,1713,26,\
1707,1750,32,\
1708,44,13,\
1708,118,13,\
1709,64,3,\
1709,196,3,\
1709,339,3,\
1709,367,3,\
1709,484,3,\
1709,748,3,\
1709,768,3,\
1710,14,15,\
1710,184,15,\
1710,218,24,\
1710,324,4,\
1710,291,31,\
1710,587,16,\
1710,652,26,\
1710,680,3,\
1710,917,15,\
1710,1478,3,\
1710,1688,3,\
1711,615,3,\
1712,8,28,\
1714,20,20,\
1714,59,16,\
1714,179,31,\
1714,275,23,\
1714,466,20,\
1714,986,20,\
1714,1604,24,\
1715,17,16,\
1715,105,21,\
1715,140,22,\
1715,659,19,\
1715,882,22,\
1715,933,17,\
1715,960,13,\
1715,975,20,\
1715,1003,13,\
1715,1031,17,\
1715,1134,19,\
1715,1347,16,\
1715,1527,17,\
1715,1574,31,\
1715,1624,16,\
1715,1765,16,\
1716,20,15,\
1716,457,15,\
1716,855,9,\
1716,866,23,\
1716,1140,15,\
1717,27,24,\
1717,53,24,\
1717,94,15,\
1717,178,25,\
1717,1139,27,\
1718,51,21,\
1718,133,21,\
1718,156,3,\
1718,335,3,\
1718,369,30,\
1718,433,3,\
1718,514,3,\
1718,525,3,\
1718,577,3,\
1718,753,19,\
1718,774,3,\
1718,872,3,\
1718,919,3,\
1718,898,19,\
1719,53,28,\
1719,83,29,\
1719,145,28,\
1719,176,3,\
1719,232,3,\
1719,327,3,\
1719,768,3,\
1720,41,23,\
1720,69,21,\
1720,109,21,\
1720,200,17,\
1720,269,21,\
1720,293,3,\
1720,301,23,\
1720,418,23,\
1720,565,3,\
1720,618,3,\
1720,886,3,\
1720,900,3,\
1720,1708,3,\
1721,0,23,\
1722,0,31,\
1722,97,31,\
1722,130,4,\
1722,172,20,\
1722,270,14,\
1722,1476,4,\
1722,1784,4,\
1723,53,29,\
1723,84,29,\
1723,115,5,\
1723,183,21,\
1723,206,3,\
1723,214,22,\
1723,238,5,\
1723,246,5,\
1723,536,5,\
1723,616,5,\
1723,815,3,\
1723,840,5,\
1723,945,3,\
1723,1248,5,\
1723,1457,5,\
1724,208,39,\
1724,249,4,\
1724,1144,3,\
1724,1123,19,\
1725,0,41,\
1725,70,15,\
1725,133,26,\
1725,173,43,\
1725,218,7,\
1725,290,18,\
1725,310,22,\
1725,396,15,\
1725,953,15,\
1725,1000,6,\
1725,1099,41,\
1725,1153,27,\
1727,50,18,\
1727,151,8,\
1727,503,18,\
1727,996,17,\
1728,17,18,\
1728,84,8,\
1728,94,18,\
1728,115,2,\
1728,126,27,\
1728,301,2,\
1728,335,14,\
1728,420,17,\
1728,482,2,\
1728,559,25,\
1728,882,2,\
1729,57,28,\
1729,88,3,\
1729,162,3,\
1729,309,11,\
1729,389,3,\
1729,444,11,\
1729,532,11,\
1729,905,11,\
1730,260,29,\
1730,1020,13,\
1730,1596,12,\
1730,1645,20,\
1731,149,24,\
1731,785,11,\
1732,319,19,\
1732,409,17,\
1733,66,18,\
1733,359,18,\
1733,559,20,\
1735,77,21,\
1735,164,21,\
1735,188,3,\
1736,18,25,\
1736,134,3,\
1736,162,31,\
1736,200,43,\
1736,276,9,\
1736,326,16,\
1736,381,20,\
1736,411,16,\
1736,449,5,\
1736,456,23,\
1736,485,21,\
1736,510,10,\
1736,659,3,\
1736,714,11,\
1736,1184,11,\
1736,1298,11,\
1736,1324,3,\
1736,1419,3,\
1736,1444,11,\
1737,38,18,\
1737,89,18,\
1737,151,45,\
1737,230,18,\
1737,250,2,\
1737,1025,2,\
1738,324,2,\
1738,297,25,\
1738,651,3,\
1738,627,22,\
1738,839,3,\
1738,1395,2,\
1738,1480,3,\
1739,24,49,\
1739,219,15,\
1739,293,17,\
1739,357,15,\
1739,1515,21,\
1739,1653,16,\
1740,31,16,\
1740,71,43,\
1740,145,16,\
1740,163,3,\
1740,433,3,\
1740,552,15,\
1740,609,30,\
1740,941,3,\
1740,994,17,\
1740,1193,3,\
1740,1242,22,\
1740,1282,3,\
1741,175,18,\
1741,226,25,\
1741,253,3,\
1741,527,3,\
1741,532,26,\
1741,564,18,\
1741,647,19,\
1742,72,11,\
1742,94,19,\
1742,115,3,\
1742,142,71,\
1742,430,13,\
1742,935,3,\
1742,1137,11,\
1742,1164,7,\
1742,1265,11,\
1743,12,18,\
1743,67,19,\
1744,180,3,\
1744,152,26,\
1744,370,5,\
1744,428,3,\
1744,502,28,\
1744,773,3,\
1744,890,3,\
1744,1149,6,\
1744,1277,3,\
1745,60,21,\
1745,179,21,\
1745,202,3,\
1745,215,35,\
1745,420,3,\
1746,47,25,\
1746,141,25,\
1746,203,22,\
1746,227,3,\
1746,242,30,\
1746,293,3,\
1746,341,25,\
1746,406,3,\
1746,471,25,\
1746,544,14,\
1746,560,3,\
1746,620,3,\
1746,1279,25,\
1746,1342,3,\
1747,87,20,\
1747,121,50,\
1747,214,20,\
1747,292,30,\
1747,324,3,\
1747,472,12,\
1747,541,15,\
1747,656,12,\
1747,1145,12,\
1747,1312,3,\
1747,1343,12,\
1748,432,8,\
1748,443,14,\
1748,460,28,\
1748,495,25,\
1748,603,8,\
1748,629,26,\
1748,770,8,\
1748,1543,8,\
1749,25,3,\
1749,65,30,\
1749,117,30,\
1749,149,3,\
1749,221,26,\
1749,249,3,\
1749,360,3,\
1749,480,3,\
1749,842,3,\
1750,77,18,\
1750,127,18,\
1750,148,2,\
1750,217,2,\
1750,296,2,\
1750,972,2,\
1750,1063,18,\
1751,65,18,\
1751,271,2,\
1751,330,18,\
1751,351,2,\
1751,664,2,\
1751,683,16,\
1751,898,2,\
1751,1187,2,\
1752,189,17,\
1752,209,4,\
1753,61,25,\
1753,120,25,\
1753,147,4,\
1753,332,50,\
1754,102,14,\
1754,288,14,\
1754,1400,17,\
1754,1460,32,\
1754,1689,6,\
1755,132,25,\
1756,0,28,\
1756,138,28,\
1756,168,3,\
1756,195,23,\
1756,277,10,\
1756,303,3,\
1756,519,19,\
1756,543,10,\
1756,944,3,\
1756,1211,3,\
1757,9,5,\
1757,61,5,\
1757,267,15,\
1757,410,5,\
1758,29,21,\
1759,9,18,\
1759,68,8,\
1759,86,18,\
1759,107,2,\
1759,632,27,\
1759,664,27,\
1759,736,2,\
1759,762,28,\
1759,793,8,\
1759,804,8,\
1759,817,16,\
1760,162,3,\
1760,135,25,\
1760,418,9,\
1760,429,18,\
1760,453,10,\
1760,781,3,\
1760,958,3,\
1760,994,35,\
1760,1171,3,\
1761,25,15,\
1761,54,15,\
1761,71,2,\
1761,80,41,\
1761,171,18,\
1761,191,2,\
1761,308,2,\
1761,631,15,\
1761,705,2,\
1761,1280,2,\
1762,0,18,\
1762,104,18,\
1762,175,15,\
1762,199,26,\
1762,260,15,\
1763,69,13,\
1764,199,21,\
1764,308,10,\
1764,465,16,\
1764,516,16,\
1764,575,17,\
1764,621,5,\
1764,894,8,\
1764,939,8,\
1764,980,30,\
1764,1019,10,\
1764,1044,8,\
1764,1103,6,\
1764,1165,6,\
1764,1180,6,\
1764,1287,6,\
1764,1364,5,\
1764,1409,6,\
1767,0,27,\
1767,72,27,\
1767,101,3,\
1767,116,18,\
1767,188,30,\
1767,360,3,\
1767,386,24,\
1767,434,45,\
1767,518,14,\
1767,638,32,\
1767,672,4,\
1767,701,25,\
1767,773,3,\
1767,1089,25,\
1768,97,14,\
1768,113,2,\
1768,149,28,\
1769,0,24,\
1769,70,24,\
1769,96,3,\
1769,111,38,\
1769,166,8,\
1769,198,3,\
1769,210,13,\
1769,225,16,\
1769,247,18,\
1769,299,22,\
1769,370,15,\
1769,392,12,\
1769,406,11,\
1769,443,12,\
1769,468,21,\
1769,589,3,\
1769,629,22,\
1769,734,3,\
1769,840,63,\
1770,111,22,\
1770,135,3,\
1770,163,22,\
1770,236,3,\
1770,251,21,\
1770,331,16,\
1770,364,13,\
1771,156,18,\
1771,257,18,\
1771,325,7,\
1771,664,7,\
1771,718,15,\
1771,768,18,\
1771,1037,24,\
1771,1195,18,\
1772,22,28,\
1772,183,28,\
1772,694,24,\
1773,40,28,\
1773,350,28,\
1773,1091,28,\
1774,74,56,\
1774,135,23,\
1774,302,24,\
1774,330,18,\
1774,366,24,\
1774,543,32,\
1774,579,19,\
1774,745,23,\
1774,798,24,\
1774,834,22,\
1774,1280,32,\
1774,1737,24,\
1774,2004,32,\
1775,70,3,\
1775,156,21,\
1775,225,20,\
1775,291,34,\
1775,467,24,\
1775,549,24,\
1775,757,30,\
1775,889,10,\
1775,948,30,\
1775,998,15,\
1775,1031,24,\
1775,1108,30,\
1775,1192,20,\
1775,1289,20,\
1775,1381,30,\
1775,1439,31,\
1775,1562,17,\
1776,39,18,\
1776,83,33,\
1776,122,12,\
1776,162,12,\
1776,176,17,\
1776,195,7,\
1776,204,12,\
1776,218,17,\
1776,237,19,\
1776,261,18,\
1776,331,12,\
1776,345,13,\
1776,376,20,\
1776,645,18,\
1778,0,45,\
1778,210,19,\
1778,294,7,\
1778,512,18,\
1778,598,40,\
1778,654,6,\
1778,985,6,\
1779,86,21,\
1779,146,5,\
1779,556,19,\
1779,1262,16,\
1779,1426,19,\
1780,38,17,\
1780,157,17,\
1780,187,6,\
1780,296,5,\
1780,603,17,\
1780,674,5,\
1780,772,5,\
1780,835,17,\
1781,197,26,\
1781,281,15,\
1781,423,15,\
1781,448,16,\
1781,1129,15,\
1781,1621,19,\
1782,0,29,\
1782,44,17,\
1782,115,40,\
1782,205,25,\
1782,437,21,\
1782,711,40,\
1782,753,5,\
1782,861,14,\
1782,1115,21,\
1783,1,24,\
1783,217,27,\
1783,325,38,\
1783,418,39,\
1783,629,30,\
1783,713,43,\
1783,777,16,\
1783,975,30,\
1783,1059,43,\
1783,1123,16,\
1784,0,31,\
1784,87,31,\
1784,120,4,\
1784,1205,6,\
1784,1340,6,\
1785,488,17,\
1786,71,9,\
1786,134,46,\
1786,491,17,\
1786,520,16,\
1786,608,17,\
1786,672,16,\
1786,1661,18,\
1787,0,24,\
1787,29,30,\
1787,117,24,\
1787,143,4,\
1787,153,12,\
1787,329,15,\
1787,349,30,\
1787,381,3,\
1787,585,26,\
1787,613,3,\
1787,729,15,\
1788,94,12,\
1788,143,12,\
1788,244,20,\
1788,365,12,\
1788,395,24,\
1788,547,12,\
1788,867,12,\
1788,1042,13,\
1788,1057,11,\
1788,1070,8,\
1788,1112,18,\
1788,1141,20,\
1788,1172,23,\
1788,1243,17,\
1788,1317,20,\
1788,1396,27,\
1788,1425,3,\
1788,1457,3,\
1788,1488,3,\
1788,1495,15,\
1788,1580,12,\
1788,1754,12,\
1789,88,25,\
1789,245,25,\
1789,272,3,\
1789,337,3,\
1789,484,24,\
1789,723,13,\
1789,805,3,\
1789,1751,3,\
1790,80,13,\
1790,99,34,\
1790,135,4,\
1790,147,17,\
1790,420,13,\
1790,461,20,\
1790,506,18,\
1790,530,25,\
1790,564,16,\
1790,638,16,\
1790,656,15,\
1790,686,18,\
1790,731,13,\
1791,85,31,\
1791,338,31,\
1791,371,4,\
1792,158,30,\
1792,219,27,\
1792,248,3,\
1792,440,24,\
1792,644,15,\
1792,810,31,\
1792,1195,3,\
1792,1375,36,\
1793,85,25,\
1793,196,13,\
1793,584,6,\
1793,759,6,\
1793,935,13,\
1793,1620,24,\
1793,1777,31,\
1793,1873,25,\
1794,81,18,\
1794,126,25,\
1794,204,20,\
1794,226,2,\
1794,231,28,\
1794,279,25,\
1794,261,3,\
1794,521,2,\
1794,685,2,\
1794,746,2,\
1794,813,2,\
1794,905,10,\
1794,1003,4,\
1794,1185,3,\
1794,1384,19,\
1794,1430,2,\
1794,1538,2,\
1794,1897,2,\
1795,1,17,\
1795,20,13,\
1795,38,22,\
1795,62,16,\
1796,16,13,\
1796,351,6,\
1797,12,17,\
1797,118,17,\
1797,493,17,\
1797,596,17,\
1797,623,28,\
1797,669,15,\
1797,759,17,\
1797,886,17,\
1797,1018,17,\
1798,58,28,\
1798,100,28,\
1798,130,3,\
1798,156,21,\
1798,988,3,\
1798,1583,3,\
1800,31,25,\
1800,105,25,\
1800,261,7,\
1800,431,7,\
1800,775,14,\
1800,952,12,\
1800,1073,15,\
1800,1151,7,\
1800,1216,15,\
1800,1262,16,\
1800,1361,20,\
1800,1478,20,\
1800,1548,7,\
1800,1582,12,\
1800,1733,25,\
1800,1847,7,\
1801,77,25,\
1801,590,36,\
1802,0,28,\
1802,376,28,\
1802,425,28,\
1802,602,28,\
1803,28,19,\
1803,142,19,\
1803,200,3,\
1803,219,27,\
1803,571,3,\
1803,711,3,\
1804,63,25,\
1804,102,15,\
1804,131,32,\
1804,165,2,\
1804,214,17,\
1804,463,6,\
1804,577,5,\
1804,925,28,\
1804,1017,16,\
1805,0,16,\
1805,52,28,\
1805,167,25,\
1806,70,29,\
1807,0,24,\
1807,94,9,\
1807,117,24,\
1807,143,3,\
1807,158,25,\
1807,209,3,\
1807,248,3,\
1807,274,22,\
1807,355,3,\
1807,389,12,\
1807,486,9,\
1807,564,33,\
1807,599,4,\
1807,896,6,\
1807,981,3,\
1807,1356,3,\
1807,1398,3,\
1808,220,29,\
1809,36,15,\
1809,99,15,\
1809,133,15,\
1809,373,15,\
1809,574,15,\
1809,676,15,\
1809,747,15,\
1809,828,15,\
1809,973,15,\
1809,1432,7,\
1809,1482,17,\
1809,1580,15,\
1809,1674,15,\
1809,1939,6,\
1810,122,18,\
1810,142,18,\
1810,162,3,\
1810,452,3,\
1810,567,19,\
1810,610,3,\
1812,40,50,\
1812,126,56,\
1813,159,27,\
1813,188,3,\
1813,257,15,\
1813,277,14,\
1813,502,3,\
1813,738,3,\
1813,1195,3,\
1814,1514,11,\
1815,17,44,\
1815,102,14,\
1815,118,2,\
1815,147,24,\
1815,177,19,\
1815,233,30,\
1815,265,3,\
1815,442,32,\
1815,497,3,\
1815,722,3,\
1815,896,20,\
1815,933,3,\
1815,1132,3,\
1815,1224,3,\
1815,1291,3,\
1815,1335,31,\
1815,1429,3,\
1815,1436,2,\
1815,1529,3,\
1815,1581,3,\
1816,0,17,\
1816,136,17,\
1816,159,27,\
1816,204,18,\
1816,231,15,\
1816,248,18,\
1816,278,20,\
1816,746,22,\
1816,1107,17,\
1816,1246,17,\
1816,1384,17,\
1816,1458,21,\
1816,1485,17,\
1816,1778,22,\
1817,76,14,\
1817,186,7,\
1817,248,14,\
1817,264,2,\
1817,526,34,\
1817,928,3,\
1817,1142,7,\
1817,1336,3,\
1817,1389,3,\
1817,1755,3,\
1817,1854,18,\
1818,47,13,\
1818,98,22,\
1818,311,10,\
1818,406,13,\
1818,460,13,\
1818,795,13,\
1818,888,13,\
1818,942,13,\
1818,1644,13,\
1819,62,17,\
1819,178,25,\
1819,318,14,\
1819,341,13,\
1819,362,25,\
1819,404,7,\
1819,497,6,\
1819,605,15,\
1819,638,17,\
1819,793,17,\
1819,913,17,\
1820,18,16,\
1820,84,16,\
1820,209,22,\
1820,233,3,\
1820,1401,13,\
1821,36,25,\
1821,74,14,\
1821,211,14,\
1821,264,14,\
1821,366,14,\
1821,419,25,\
1821,937,14,\
1821,1269,14,\
1821,1570,14,\
1822,28,21,\
1822,54,33,\
1822,112,21,\
1822,188,33,\
1822,300,21,\
1822,365,21,\
1822,391,33,\
1823,80,9,\
1823,153,9,\
1823,291,18,\
1823,345,15,\
1823,639,15,\
1823,968,17,\
1823,1054,9,\
1823,1151,9,\
1823,1336,9,\
1823,1467,9,\
1823,1588,17,\
1824,185,22,\
1824,292,16,\
1824,343,12,\
1824,361,29,\
1824,415,14,\
1824,434,6,\
1824,513,34,\
1825,0,17,\
1825,21,24,\
1825,77,24,\
1825,103,3,\
1825,114,28,\
1825,175,30,\
1825,231,26,\
1825,463,3,\
1825,514,21,\
1825,635,8,\
1825,688,13,\
1825,711,10,\
1825,882,8,\
1825,910,24,\
1825,970,20,\
1825,1052,33,\
1825,1087,30,\
1825,1134,3,\
1825,1374,24,\
1825,1570,13,\
1825,1641,16,\
1825,1662,3,\
1825,1667,13,\
1825,1738,22,\
1825,1767,16,\
1826,100,24,\
1826,179,18,\
1826,273,24,\
1826,299,3,\
1826,326,27,\
1826,386,36,\
1826,433,11,\
1826,446,10,\
1826,458,19,\
1826,483,17,\
1826,518,36,\
1826,556,3,\
1826,634,18,\
1826,708,4,\
1826,756,3,\
1826,870,5,\
1826,941,4,\
1826,1021,4,\
1826,1077,3,\
1826,1561,6,\
1826,1601,4,\
1826,1624,12,\
1826,1641,20,\
1826,1666,11,\
1826,1682,18,\
1826,1719,5,\
1826,1792,6,\
1826,2003,4,\
1826,2040,5,\
1826,2083,6,\
1826,2417,3,\
1826,2616,3,\
1826,2657,4,\
1827,69,8,\
1827,120,14,\
1827,142,20,\
1827,170,28,\
1827,210,26,\
1827,299,8,\
1827,429,3,\
1828,29,26,\
1828,111,26,\
1828,139,3,\
1828,173,6,\
1828,207,3,\
1828,348,8,\
1828,455,8,\
1828,485,36,\
1828,537,17,\
1829,0,16,\
1830,25,24,\
1830,109,24,\
1830,135,2,\
1830,231,13,\
1830,258,30,\
1830,405,13,\
1830,552,2,\
1830,576,13,\
1831,84,17,\
1831,209,15,\
1831,289,15,\
1831,552,15,\
1831,611,13,\
1831,633,17,\
1831,652,3,\
1831,953,15,\
1831,1034,26,\
1831,1303,25,\
1831,1357,27,\
1831,1386,3,\
1831,1584,13,\
1831,1648,6,\
1831,1750,15,\
1831,1891,15,\
1832,24,18,\
1832,60,21,\
1832,148,18,\
1832,193,21,\
1832,216,3,\
1832,420,3,\
1832,705,18,\
1832,783,18,\
1832,922,20,\
1832,1026,18,\
1832,1140,18,\
1832,1164,13,\
1832,1179,26,\
1832,1207,19,\
1832,1228,13,\
1832,1257,42,\
1832,1315,18,\
1832,1387,3,\
1832,1427,24,\
1832,1498,3,\
1832,1653,18,\
1832,1689,3,\
1832,1757,3,\
1832,1806,18,\
1832,1939,3,\
1833,150,33,\
1833,187,15,\
1834,64,9,\
1834,129,17,\
1834,305,49,\
1834,148,3,\
1834,418,35,\
1834,455,3,\
1834,499,9,\
1834,743,3,\
1834,787,3,\
1834,893,3,\
1834,912,5,\
1835,26,18,\
1835,63,6,\
1835,71,6,\
1835,83,18,\
1835,166,18,\
1835,412,6,\
1835,475,18,\
1835,551,20,\
1835,589,8,\
1835,882,18,\
1835,904,27,\
1835,946,17,\
1835,1398,6,\
1836,55,34,\
1836,252,21,\
1836,453,28,\
1836,549,27,\
1836,642,29,\
1836,788,6,\
1836,977,48,\
1836,1330,28,\
1836,1555,17,\
1838,0,3,\
1838,55,20,\
1838,163,11,\
1838,204,19,\
1839,71,18,\
1839,91,8,\
1839,105,18,\
1839,215,8,\
1839,249,64,\
1839,569,8,\
1840,99,6,\
1840,117,3,\
1840,177,6,\
1840,305,7,\
1840,318,6,\
1840,336,25,\
1840,363,3,\
1840,623,3,\
1840,666,7,\
1840,755,3,\
1840,837,3,\
1840,929,3,\
1840,1289,3,\
1840,1340,3,\
1841,46,13,\
1841,73,13,\
1841,97,37,\
1841,271,20,\
1841,293,23,\
1841,322,20,\
1841,419,13,\
1841,462,50,\
1841,1025,13,\
1843,54,19,\
1843,78,27,\
1843,426,14,\
1843,482,13,\
1843,497,9,\
1843,515,20,\
1843,549,26,\
1844,0,10,\
1844,16,8,\
1844,122,10,\
1844,157,22,\
1844,332,18,\
1844,469,10,\
1844,507,22,\
1844,591,10,\
1844,682,10,\
1844,718,20,\
1845,49,17,\
1845,87,2,\
1845,96,27,\
1845,148,19,\
1845,68,17,\
1845,264,2,\
1845,362,18,\
1845,600,2,\
1845,906,2,\
1845,966,19,\
1845,997,2,\
1845,1123,19,\
1846,114,15,\
1847,46,27,\
1847,138,27,\
1847,199,26,\
1847,353,30,\
1847,760,14,\
1848,55,20,\
1848,811,24,\
1848,837,3,\
1848,1019,21,\
1848,1078,3,\
1848,1583,16,\
1848,1603,22,\
1848,2152,51,\
1849,40,21,\
1849,79,25,\
1849,160,20,\
1849,182,3,\
1849,204,31,\
1849,296,6,\
1849,443,3,\
1849,729,11,\
1849,948,3,\
1849,969,11,\
1850,175,20,\
1850,197,3,\
1850,205,11,\
1851,31,29,\
1853,9,26,\
1853,108,26,\
1853,136,3,\
1853,147,54,\
1853,421,3,\
1853,548,3,\
1853,684,3,\
1853,746,3,\
1853,771,21,\
1853,940,3,\
1854,38,20,\
1854,198,20,\
1854,556,20,\
1855,38,10,\
1855,68,17,\
1856,20,15,\
1856,119,14,\
1856,141,24,\
1856,179,6,\
1856,265,14,\
1856,353,22,\
1856,451,22,\
1856,543,7,\
1856,741,8,\
1856,815,33,\
1856,872,7,\
1856,993,7,\
1856,1063,7,\
1856,1178,14,\
1856,1211,7,\
1856,1348,14,\
1856,1364,29,\
1856,1562,7,\
1856,1687,14,\
1856,1769,14,\
1856,1859,15,\
1856,2001,14,\
1856,2068,6,\
1856,2193,14,\
1857,236,24,\
1858,65,19,\
1858,167,19,\
1858,225,3,\
1858,269,28,\
1858,363,22,\
1858,525,3,\
1858,612,3,\
1859,11,14,\
1861,83,28,\
1861,118,71,\
1861,226,32,\
1861,293,28,\
1861,323,3,\
1861,658,3,\
1861,792,3,\
1862,94,16,\
1862,131,51,\
1862,196,18,\
1862,223,16,\
1862,337,15,\
1862,467,15,\
1862,649,25,\
1862,721,6,\
1862,741,15,\
1862,1204,15,\
1862,1221,22,\
1863,37,3,\
1863,109,37,\
1863,148,30,\
1863,181,3,\
1863,221,17,\
1863,284,18,\
1863,370,26,\
1863,399,3,\
1863,444,3,\
1863,476,3,\
1863,559,37,\
1863,599,4,\
1863,737,4,\
1863,1241,4,\
1864,193,19,\
1864,214,4,\
1864,223,25,\
1864,631,26,\
1864,962,4,\
1864,1044,4,\
1865,87,26,\
1865,115,2,\
1865,304,2,\
1865,653,2,\
1866,673,15,\
1866,721,26,\
1866,927,15,\
1867,24,20,\
1867,182,11,\
1867,349,11,\
1867,894,11,\
1869,0,20,\
1869,84,20,\
1869,106,3,\
1869,116,19,\
1869,301,14,\
1869,317,86,\
1869,658,3,\
1870,38,35,\
1871,321,17,\
1871,377,7,\
1871,661,7,\
1871,682,23,\
1871,786,30,\
1872,0,24,\
1872,95,24,\
1872,121,3,\
1872,132,27,\
1872,214,3,\
1872,272,3,\
1872,318,13,\
1872,430,3,\
1872,550,3,\
1872,670,17,\
1872,691,3,\
1872,1312,3,\
1872,1555,3,\
1872,1770,3,\
1874,87,53,\
1874,446,24,\
1874,633,46,\
1874,699,40,\
1874,819,19,\
1874,901,5,\
1874,929,38,\
1875,265,15,\
1875,323,17,\
1876,47,15,\
1876,64,27,\
1876,116,6,\
1876,225,12,\
1876,380,24,\
1876,530,5,\
1876,675,15,\
1876,719,15,\
1876,1081,6,\
1876,1158,24,\
1877,19,14,\
1877,109,13,\
1878,19,26,\
1878,68,14,\
1878,142,14,\
1878,181,18,\
1878,231,27,\
1878,643,14,\
1878,1740,14,\
1878,2060,19,\
1879,1298,6,\
1880,118,13,\
1880,133,12,\
1880,150,27,\
1880,244,20,\
1881,10,35,\
1881,144,35,\
1881,181,3,\
1881,306,59,\
1881,723,31,\
1881,788,19,\
1881,832,3,\
1882,31,15,\
1882,76,15,\
1882,96,13,\
1882,175,23,\
1882,246,13,\
1882,380,15,\
1882,520,15,\
1882,619,15,\
1882,972,15,\
1882,1057,15,\
1882,1143,15,\
1882,1184,15,\
1883,72,33,\
1884,16,13,\
1884,101,13,\
1884,191,13,\
1884,241,20,\
1884,823,13,\
1884,1012,19,\
1884,1415,13,\
1884,1465,19,\
1884,1567,5,\
1885,274,6,\
1885,1011,6,\
1885,1172,6,\
1886,6,15,\
1886,51,39,\
1886,166,41,\
1886,354,41,\
1886,417,24,\
1886,508,17,\
1886,746,17,\
1886,812,8,\
1886,855,17,\
1887,80,16,\
1887,103,16,\
1887,239,11,\
1887,266,11,\
1887,291,16,\
1887,309,3,\
1887,1272,4,\
1888,51,18,\
1888,223,20,\
1888,316,26,\
1888,485,3,\
1888,1279,3,\
1891,26,15,\
1891,46,22,\
1893,157,25,\
1893,183,29,\
1893,684,13,\
1893,801,3,\
1894,13,16,\
1894,33,14,\
1894,349,36,\
1894,860,15,\
1894,991,14,\
1895,60,28,\
1895,235,28,\
1895,265,3,\
1895,356,3,\
1895,416,3,\
1895,910,2,\
1895,1513,3,\
1895,1641,3,\
1896,12,17,\
1896,99,17,\
1896,118,3,\
1896,745,3,\
1896,962,3,\
1897,941,32,\
1897,1359,13,\
1898,101,15,\
1898,179,15,\
1898,196,2,\
1898,326,2,\
1898,456,2,\
1898,1430,2,\
1899,185,14,\
1899,1161,38,\
1899,1959,32,\
1900,36,17,\
1900,149,17,\
1900,168,4,\
1900,439,4,\
1900,558,4,\
1900,705,4,\
1900,859,4,\
1900,1135,4,\
1900,1513,4,\
1900,1650,4,\
1901,0,17,\
1901,88,17,\
1901,107,4,\
1901,134,16,\
1901,192,4,\
1901,522,4,\
1901,963,20,\
1901,1185,17,\
1901,1444,4,\
1902,73,10,\
1902,182,10,\
1902,692,4,\
1902,661,29,\
1903,58,23,\
1903,83,26,\
1903,205,18,\
1903,266,23,\
1903,291,22,\
1903,448,22,\
1903,531,62,\
1903,598,28,\
1903,697,27,\
1904,574,22,\
1905,72,17,\
1905,95,10,\
1905,238,17,\
1905,266,20,\
1905,288,3,\
1905,537,18,\
1905,557,30,\
1905,723,29,\
1905,754,17,\
1905,1234,3,\
1905,1229,3,\
1905,1524,16,\
1905,1545,17,\
1905,2199,17,\
1905,2218,23,\
1906,10,16,\
1906,117,16,\
1906,149,38,\
1906,205,16,\
1906,226,21,\
1906,525,16,\
1906,553,16,\
1906,621,18,\
1906,755,16,\
1906,1336,16,\
1908,1111,24,\
1908,1137,22,\
1908,1164,21,\
1909,22,27,\
1909,66,21,\
1909,126,43,\
1909,294,15,\
1909,353,21,\
1910,128,6,\
1910,214,6,\
1910,367,6,\
1910,441,6,\
1910,1114,3,\
1910,1184,4,\
1910,1351,13,\
1910,1520,13,\
1910,1897,6,\
1910,2055,6,\
1910,2248,6,\
1911,42,13,\
1911,78,13,\
1911,116,6,\
1911,93,2,\
1911,709,2,\
1911,809,23,\
1911,1661,13,\
1913,59,27,\
1913,118,27,\
1913,147,3,\
1913,157,20,\
1913,188,55,\
1913,450,13,\
1913,530,3,\
1913,569,18,\
1913,752,3,\
1914,40,25,\
1915,66,31,\
1915,182,62,\
1915,246,3,\
1915,668,3,\
1915,899,22,\
1915,930,3,\
1915,1117,5,\
1915,1138,21,\
1916,333,5,\
1916,350,26,\
1916,378,3,\
1916,397,13,\
1916,540,3,\
1916,625,3,\
1917,47,6,\
1917,76,30,\
1917,219,14,\
1917,235,2,\
1917,298,25,\
1917,482,34,\
1917,721,6,\
1917,914,4,\
1918,102,11,\
1918,115,14,\
1918,135,13,\
1918,903,10,\
1918,1033,20,\
1918,1091,2,\
1918,1276,2,\
1919,0,15,\
1920,43,28,\
1920,235,28,\
1920,266,3,\
1920,508,3,\
1920,926,23,\
1920,1286,3,\
1921,10,25,\
1922,42,15,\
1922,243,15,\
1922,260,3,\
1922,332,3,\
1922,1463,3,\
1922,1593,3,\
1922,1606,25,\
1922,1852,3,\
1923,95,35,\
1923,249,35,\
1923,662,4,\
1923,731,4,\
1924,0,30,\
1925,18,22,\
1925,199,22,\
1925,224,2,\
1925,251,2,\
1925,819,2,\
1925,1542,2,\
1925,1725,2,\
1926,120,25,\
1926,150,16,\
1926,168,25,\
1926,195,3,\
1926,282,51,\
1926,356,3,\
1926,413,36,\
1926,660,9,\
1926,1144,3,\
1926,1510,16,\
1926,1536,25,\
1926,1614,3,\
1926,1897,11,\
1927,79,14,\
1927,182,14,\
1927,381,35,\
1927,468,24,\
1927,547,30,\
1927,1020,14,\
1927,1068,18,\
1927,1147,46,\
1928,37,25,\
1928,111,21,\
1928,134,21,\
1928,157,3,\
1928,167,16,\
1928,307,18,\
1928,327,13,\
1928,342,20,\
1928,364,12,\
1928,378,11,\
1928,391,14,\
1928,462,27,\
1928,529,3,\
1928,690,3,\
1928,970,3,\
1928,1181,3,\
1929,12,7,\
1929,337,27,\
1929,469,7,\
1929,478,11,\
1929,491,23,\
1929,1446,7,\
1929,1468,23,\
1930,42,14,\
1930,148,17,\
1930,430,17,\
1930,595,17,\
1930,721,5,\
1930,1048,17,\
1930,1257,17,\
1931,20,24,\
1931,95,24,\
1931,121,3,\
1931,486,6,\
1931,529,6,\
1931,598,7,\
1931,770,10,\
1931,927,20,\
1931,1034,3,\
1932,30,25,\
1932,57,3,\
1932,116,25,\
1932,143,3,\
1932,1377,21,\
1932,1450,6,\
1933,0,15,\
1933,24,29,\
1933,62,29,\
1934,73,24,\
1934,324,13,\
1934,511,24,\
1934,707,13,\
1934,989,6,\
1934,1406,6,\
1934,1697,24,\
1936,69,21,\
1936,331,21,\
1936,355,3,\
1936,397,3,\
1936,684,3,\
1936,754,3,\
1937,17,37,\
1937,84,37,\
1938,472,40,\
1939,73,15,\
1939,90,12,\
1939,122,26,\
1939,166,16,\
1939,184,16,\
1939,206,19,\
1939,444,13,\
1939,563,3,\
1939,1049,19,\
1940,56,11,\
1940,70,11,\
1940,93,39,\
1940,146,69,\
1940,433,11,\
1940,519,11,\
1940,636,51,\
1940,833,11,\
1940,1104,27,\
1940,1176,11,\
1941,62,14,\
1941,100,43,\
1941,202,20,\
1941,296,5,\
1941,451,6,\
1941,704,6,\
1941,1029,38,\
1941,1185,21,\
1941,1717,14,\
1942,111,24,\
1942,141,24,\
1942,168,3,\
1942,179,33,\
1942,271,3,\
1942,450,3,\
1942,617,3,\
1942,672,3,\
1942,1309,25,\
1942,1400,3,\
1942,1698,3,\
1943,52,18,\
1943,147,18,\
1943,167,2,\
1943,317,2,\
1944,23,12,\
1945,17,42,\
1945,65,24,\
1945,92,3,\
1945,98,26,\
1945,191,8,\
1945,202,13,\
1945,222,14,\
1945,263,3,\
1945,737,8,\
1945,803,14,\
1945,823,8,\
1945,1066,14,\
1945,1240,14,\
1945,1266,16,\
1945,1347,14,\
1945,1373,12,\
1946,22,29,\
1946,109,29,\
1946,140,3,\
1946,165,21,\
1946,191,22,\
1946,294,3,\
1946,644,3,\
1947,105,13,\
1947,546,5,\
1947,555,13,\
1947,1528,18,\
1948,46,18,\
1948,66,18,\
1948,86,2,\
1948,123,7,\
1948,204,2,\
1948,565,2,\
1948,870,17,\
1948,898,40,\
1948,947,8,\
1948,960,36,\
1948,1196,2,\
1949,19,26,\
1949,70,25,\
1949,97,3,\
1949,191,27,\
1949,232,13,\
1949,307,3,\
1949,436,14,\
1949,454,3,\
1949,583,25,\
1950,0,19,\
1950,24,17,\
1950,120,19,\
1950,141,2,\
1950,230,6,\
1950,241,21,\
1950,344,32,\
1950,400,17,\
1951,47,21,\
1951,181,30,\
1951,438,22,\
1951,849,41,\
1952,18,35,\
1952,67,35,\
1952,104,3,\
1952,169,26,\
1952,274,3,\
1952,334,27,\
1952,363,3,\
1952,397,3,\
1952,424,20,\
1952,485,12,\
1952,510,13,\
1952,1307,14,\
1952,1437,26,\
1953,420,39,\
1953,581,29,\
1954,349,20,\
1955,112,13,\
1955,353,13,\
1955,368,2,\
1955,439,2,\
1955,569,2,\
1955,776,2,\
1955,1001,2,\
1956,62,3,\
1956,74,28,\
1956,105,3,\
1956,254,3,\
1956,656,3,\
1956,916,3,\
1956,989,16,\
1956,1013,3,\
1957,0,12,\
1957,89,18,\
1957,126,12,\
1957,192,36,\
1957,244,31,\
1957,290,13,\
1957,230,4,\
1957,565,27,\
1958,62,17,\
1958,93,17,\
1958,112,4,\
1958,124,28,\
1958,278,4,\
1958,431,4,\
1958,834,4,\
1958,1072,11,\
1958,1088,20,\
1958,1157,40,\
1958,1283,24,\
1958,1534,4,\
1958,1599,4,\
1959,80,35,\
1959,117,35,\
1959,154,3,\
1959,169,28,\
1959,265,26,\
1959,625,3,\
1959,908,23,\
1959,1036,31,\
1959,1367,31,\
1960,22,44,\
1960,160,51,\
1960,278,19,\
1960,329,14,\
1960,463,3,\
1960,662,15,\
1960,767,24,\
1960,824,15,\
1961,15,25,\
1961,91,22,\
1961,115,25,\
1961,142,4,\
1961,218,5,\
1961,340,4,\
1961,640,10,\
1961,741,19,\
1961,762,24,\
1961,904,5,\
1961,932,16,\
1961,967,5,\
1961,1067,5,\
1961,1187,5,\
1961,1290,5,\
1961,1373,4,\
1961,1482,42,\
1962,27,14,\
1962,62,30,\
1962,121,30,\
1962,162,19,\
1962,262,24,\
1962,374,12,\
1962,471,16,\
1962,506,20,\
1962,633,30,\
1962,751,14,\
1962,842,5,\
1962,1015,14,\
1962,1040,7,\
1962,1104,5,\
1962,1339,14,\
1962,1524,14,\
1963,79,21,\
1963,102,3,\
1963,503,21,\
1963,622,13,\
1963,1576,11,\
1964,32,21,\
1964,354,21,\
1965,102,27,\
1965,393,3,\
1965,365,26,\
1965,479,3,\
1965,688,12,\
1965,727,24,\
1965,770,3,\
1965,834,3,\
1966,374,16,\
1966,435,23,\
1966,463,26,\
1967,50,37,\
1967,139,31,\
1967,172,4,\
1967,277,5,\
1967,492,5,\
1967,612,2,\
1967,596,14,\
1967,712,2,\
1967,862,5,\
1968,19,6,\
1968,150,6,\
1968,209,6,\
1968,411,6,\
1968,488,5,\
1968,533,22,\
1968,641,5,\
1968,908,5,\
1968,1105,21,\
1968,1193,6,\
1968,1215,12,\
1968,1396,6,\
1969,49,23,\
1969,74,23,\
1969,126,18,\
1969,270,58,\
1969,372,14,\
1969,416,31,\
1970,45,17,\
1970,206,17,\
1970,557,17,\
1970,1035,29,\
1970,1112,20,\
1970,1138,19,\
1970,1194,21,\
1970,1268,17,\
1970,1356,20,\
1970,1447,15,\
1970,1474,31,\
1971,77,36,\
1971,115,3,\
1971,280,3,\
1971,253,25,\
1971,347,3,\
1971,602,36,\
1971,640,3,\
1971,673,3,\
1971,756,3,\
1971,984,3,\
1971,1105,3,\
1971,1148,7,\
1971,1363,3,\
1971,1458,3,\
1971,1597,3,\
1971,1715,3,\
1972,169,20,\
1972,1419,14,\
1972,1545,3,\
1972,1810,20,\
1973,37,15,\
1973,175,51,\
1973,287,14,\
1973,615,15,\
1973,694,15,\
1973,1738,17,\
1973,2082,14,\
1974,0,16,\
1974,20,16,\
1976,77,20,\
1977,0,22,\
1977,162,22,\
1977,186,3,\
1977,361,3,\
1977,617,3,\
1977,863,3,\
1977,1059,3,\
1977,1230,3,\
1977,1515,3,\
1978,74,24,\
1978,100,3,\
1978,115,28,\
1978,161,28,\
1978,191,22,\
1978,275,36,\
1978,313,4,\
1978,409,4,\
1978,532,3,\
1978,540,4,\
1978,673,18,\
1978,693,3,\
1979,77,51,\
1979,166,14,\
1979,290,13,\
1980,62,22,\
1980,422,22,\
1980,567,29,\
1980,809,18,\
1980,1381,34,\
1981,0,11,\
1981,120,25,\
1981,346,13,\
1981,450,15,\
1981,477,13,\
1981,535,6,\
1981,563,6,\
1981,711,15,\
1981,738,13,\
1981,887,14,\
1981,956,6,\
1981,976,10,\
1981,1016,10,\
1981,1052,6,\
1981,1063,21,\
1981,1088,17,\
1981,1106,15,\
1981,1428,15,\
1981,1562,6,\
1982,67,3,\
1982,107,9,\
1982,313,25,\
1982,411,3,\
1982,438,26,\
1982,691,11,\
1982,911,3,\
1982,1046,21,\
1983,57,22,\
1983,97,5,\
1983,165,7,\
1983,197,5,\
1983,275,6,\
1983,517,6,\
1983,745,13,\
1983,1420,7,\
1984,10,22,\
1984,277,22,\
1984,302,3,\
1984,869,3,\
1984,968,3,\
1984,1041,3,\
1984,1245,3,\
1984,1388,3,\
1984,1420,15,\
1984,1481,3,\
1984,1514,3,\
1985,102,22,\
1985,332,13,\
1986,84,12,\
1986,98,12,\
1986,133,15,\
1986,263,12,\
1986,317,17,\
1986,341,12,\
1986,466,12,\
1986,541,12,\
1986,662,18,\
1986,707,6,\
1986,1256,17,\
1986,1331,18,\
1986,1405,13,\
1986,1494,6,\
1986,1579,12,\
1986,1653,12,\
1986,1680,17,\
1987,247,15,\
1987,784,9,\
1987,825,9,\
1987,2266,15,\
1988,0,11,\
1988,24,11,\
1988,37,2,\
1988,105,17,\
1988,327,2,\
1988,592,6,\
1988,692,21,\
1988,944,20,\
1988,973,4,\
1988,982,21,\
1988,1045,6,\
1988,1258,27,\
1988,1303,20,\
1988,1325,40,\
1988,1382,2,\
1988,1400,3,\
1988,1611,2,\
1988,1711,2,\
1989,140,44,\
1989,186,44,\
1989,233,4,\
1989,246,26,\
1989,283,59,\
1989,471,4,\
1989,1197,4,\
1990,251,20,\
1991,27,20,\
1991,99,45,\
1991,158,27,\
1991,189,25,\
1991,225,39,\
1991,476,22,\
1991,812,24,\
1991,886,31,\
1991,953,22,\
1991,1144,26,\
1992,0,37,\
1992,121,13,\
1992,164,13,\
1992,179,12,\
1992,193,30,\
1992,279,3,\
1992,255,22,\
1992,515,12,\
1992,569,12,\
1992,587,45,\
1992,669,3,\
1992,754,28,\
1993,45,28,\
1993,75,14,\
1993,91,53,\
1993,296,28,\
1993,326,14,\
1993,388,35,\
1994,248,11,\
1994,305,11,\
1994,436,16,\
1994,562,4,\
1995,45,35,\
1995,82,35,\
1995,119,3,\
1995,178,16,\
1995,253,3,\
1995,312,28,\
1995,398,3,\
1996,166,18,\
1996,732,16,\
1996,991,31,\
1996,1274,16,\
1997,55,17,\
1997,122,30,\
1997,155,3,\
1997,254,19,\
1997,278,9,\
1997,338,3,\
1997,362,3,\
1997,369,37,\
1997,478,3,\
1997,500,26,\
1997,554,19,\
1997,645,3,\
1997,691,8,\
1997,724,3,\
1997,942,17,\
1997,981,8,\
1997,1060,19,\
1997,1089,9,\
1997,1149,3,\
1997,1322,3,\
1997,1419,36,\
1997,1530,3,\
1997,1586,19,\
1997,1610,9,\
1997,1668,3,\
1997,1707,17,\
1999,36,21,\
1999,89,33,\
1999,124,21,\
1999,158,16,\
1999,185,23,\
1999,336,18,\
1999,356,29,\
1999,387,24,\
1999,501,27,\
1999,542,38,\
1999,625,29,\
1999,656,93,\
1999,759,14,\
1999,775,20,\
1999,800,13,\
1999,861,17,\
1999,909,14,\
1999,939,38,\
1999,1050,21,\
1999,1235,27,\
1999,1282,29,\
1999,1336,26,\
2000,5,19,\
2001,34,20,\
2001,62,23,\
2001,156,25,\
2001,200,23,\
2001,225,3,\
2001,400,21,\
2001,501,30,\
2001,539,3,\
2002,197,50,\
2002,1183,50,\
2003,0,16,\
2004,54,21,\
2004,184,21,\
2004,207,3,\
2004,357,3,\
2004,605,3,\
2004,780,3,\
2004,795,3,\
2004,996,3,\
2004,1155,3,\
2005,60,26,\
2005,88,26,\
2005,116,3,\
2005,124,29,\
2005,300,9,\
2005,508,9,\
2005,730,3,\
2008,23,26,\
2008,51,26,\
2008,79,4,\
2009,106,18,\
2009,139,18,\
2009,159,2,\
2009,194,18,\
2009,329,2,\
2009,459,2,\
2009,594,2,\
2009,819,28,\
2009,993,2,\
2009,1156,2,\
2010,0,14,\
2010,167,14,\
2010,483,14,\
2010,1054,14,\
2011,110,21,\
2011,656,21,\
2011,679,3,\
2011,691,17,\
2011,855,3,\
2011,947,3,\
2011,1008,3,\
2011,1275,22,\
2011,1350,3,\
2011,1685,3,\
2012,25,27,\
2013,0,28,\
2014,17,14,\
2014,52,16,\
2014,142,14,\
2014,177,40,\
2014,580,13,\
2014,896,34,\
2014,975,16,\
2014,1043,14,\
2014,1418,14,\
2014,1487,35,\
2014,1527,17,\
2015,39,26,\
2015,67,26,\
2015,139,26,\
2015,167,3,\
2015,204,30,\
2015,267,3,\
2015,359,8,\
2015,462,3,\
2015,547,22,\
2015,665,8,\
2015,689,8,\
2015,800,7,\
2015,840,22,\
2015,930,21,\
2015,978,18,\
2016,48,17,\
2017,22,35,\
2017,160,35,\
2017,451,35,\
2017,1213,23,\
2017,1255,17,\
2018,738,2,\
2020,0,18,\
2020,26,6,\
2020,78,6,\
2020,478,6,\
2020,558,18,\
2020,581,14,\
2020,668,6,\
2020,725,6,\
2023,124,29,\
2023,289,17,\
2023,309,3,\
2023,1504,3,\
2023,1956,16,\
2024,40,27,\
2024,69,3,\
2025,87,26,\
2025,304,26,\
2026,0,23,\
2026,25,33,\
2026,102,70,\
2026,197,14,\
2026,291,14,\
2026,420,17,\
2026,805,23,\
2027,40,21,\
2027,63,3,\
2027,72,18,\
2027,124,16,\
2027,343,3,\
2027,395,3,\
2027,437,6,\
2027,595,6,\
2028,0,20,\
2028,123,20,\
2028,206,23,\
2028,231,21,\
2028,538,35,\
2028,599,25,\
2028,822,33,\
2030,0,48,\
2030,151,56,\
2030,209,5,\
2030,368,5,\
2030,642,29,\
2030,789,5,\
2030,1075,5,\
2031,11,16,\
2031,102,7,\
2031,125,16,\
2031,159,18,\
2031,179,17,\
2031,198,13,\
2031,216,8,\
2031,260,17,\
2031,290,13,\
2031,500,16,\
2032,13,14,\
2033,0,25,\
2033,177,18,\
2033,367,34,\
2033,405,50,\
2034,61,14,\
2034,115,14,\
2034,147,6,\
2034,173,14,\
2034,191,12,\
2034,435,6,\
2035,58,22,\
2036,223,17,\
2037,4,28,\
2037,153,59,\
2037,214,3,\
2037,295,3,\
2037,309,8,\
2037,319,46,\
2037,382,18,\
2037,430,24,\
2037,639,8,\
2037,681,3,\
2037,750,8,\
2039,23,25,\
2039,50,3,\
2039,107,25,\
2039,134,3,\
2039,183,15,\
2039,718,3,\
2039,811,3,\
2039,815,3,\
2040,39,27,\
2041,14,14,\
2042,0,22,\
2042,168,22,\
2042,192,2,\
2042,490,11,\
2042,655,20,\
2042,683,12,\
2042,967,6,\
2042,978,22,\
2042,1123,2,\
2043,14,45,\
2043,846,39,\
2043,1128,22,\
2044,22,18,\
2044,56,6,\
2044,101,18,\
2044,134,6,\
2044,193,18,\
2044,625,18,\
2044,712,6,\
2044,783,18,\
2044,851,15,\
2044,893,100,\
2044,998,8,\
2044,1008,8,\
2044,1033,20,\
2044,1144,8,\
2044,1670,18,\
2044,1751,6,\
2045,80,30,\
2045,199,6,\
2045,276,17,\
2045,451,6,\
2045,524,5,\
2045,612,5,\
2045,693,3,\
2045,770,17,\
2045,925,3,\
2047,18,12,\
2047,221,12,\
2048,116,8,\
2048,126,26,\
2048,224,6,\
2048,432,6,\
2048,447,6,\
2049,43,12,\
2049,60,8,\
2049,94,12,\
2049,112,8,\
2049,163,25,\
2049,543,17,\
2049,570,12,\
2049,689,8,\
2049,699,12,\
2049,717,12,\
2049,819,8,\
2049,842,12,\
2049,878,8,\
2049,952,12,\
2049,969,8,\
2049,1036,4,\
2049,1067,8,\
2050,33,18,\
2050,91,37,\
2050,637,18,\
2051,367,14,\
2052,71,19,\
2054,52,27,\
2055,129,27,\
2055,265,27,\
2055,508,37,\
2055,626,3,\
2055,803,3,\
2055,811,3,\
2055,983,3,\
2055,1082,3,\
2056,544,27,\
2056,727,27,\
2056,756,3,\
2056,847,33,\
2057,12,37,\
2057,55,37,\
2057,109,28,\
2057,153,19,\
2057,174,16,\
2057,196,29,\
2057,278,37,\
2057,657,37,\
2057,826,37,\
2061,13,40,\
2062,22,25,\
2063,15,27,\
2063,44,27,\
2063,73,3,\
2063,90,26,\
2063,411,3,\
2063,752,3,\
2063,1077,3,\
2064,85,20,\
2064,107,20,\
2064,129,3,\
2064,139,18,\
2064,735,30,\
2065,150,20,\
2065,172,2,\
2065,334,2,\
2065,920,2,\
2065,1112,2,\
2066,10,10,\
2066,85,20,\
2066,116,22,\
2066,178,16,\
2066,572,20,\
2066,809,21,\
2066,898,19,\
2067,44,14,\
2067,172,6,\
2067,274,18,\
2067,354,14,\
2067,449,6,\
2067,668,14,\
2069,38,34,\
2069,234,20,\
2069,256,3,\
2069,342,5,\
2069,807,13,\
2071,35,15,\
2071,83,14,\
2071,190,29,\
2071,248,14,\
2071,343,26,\
2072,206,15,\
2072,223,3,\
2073,545,3,\
2073,661,22,\
2074,108,9,\
2075,19,18,\
2075,73,18,\
2075,93,2,\
2075,113,40,\
2075,201,2,\
2075,310,2,\
2075,385,2,\
2075,549,2,\
2075,591,2,\
2076,81,17,\
2076,240,16,\
2077,51,20,\
2077,99,20,\
2077,121,2,\
2077,463,2,\
2077,729,2,\
2077,913,2,\
2077,973,2,\
2078,50,20,\
2078,75,19,\
2078,207,2,\
2078,452,10,\
2078,543,2,\
2078,752,6,\
2078,801,2,\
2078,884,8,\
2078,977,2,\
2078,984,2,\
2078,1425,2,\
2078,1432,2,\
2079,106,18,\
2080,7,52,\
2080,917,21,\
2080,940,4,\
2080,1070,16,\
2080,1202,6,\
2080,1503,4,\
2080,1519,6,\
2081,78,24,\
2081,172,24,\
2082,0,43,\
2087,0,26,\
2087,32,17,\
2087,159,21,\
2087,365,20,\
2087,389,26,\
2088,770,18,\
2090,20,15,\
2090,108,15,\
2090,125,2,\
2090,508,2,\
2090,605,2,\
2090,1009,2,\
2093,0,17,\
2094,8,27,\
2095,1,22,\
2095,72,24,\
2095,257,32,\
2095,305,16,\
2095,556,12,\
2095,584,14,\
2095,660,34,\
2096,52,41,\
2097,38,13,\
2097,84,6,\
2097,149,13,\
2097,362,6,\
2097,504,6,\
2097,698,6,\
2097,753,13,\
2098,90,31,\
2098,165,31,\
2098,318,14,\
2098,334,18,\
2098,358,14,\
2098,468,17,\
2098,496,10,\
2098,640,17,\
2098,908,28,\
2100,0,23,\
2101,9,25,\
2101,75,14,\
2101,102,23,\
2101,151,25,\
2101,193,12,\
2101,1081,14,\
2102,35,14,\
2102,97,30,\
2102,212,14,\
2102,282,30,\
2102,314,3,\
2102,566,3,\
2102,620,3,\
2103,0,29,\
2103,47,21,\
2103,94,29,\
2103,334,6,\
2103,342,18,\
2103,362,6,\
2103,370,8,\
2103,402,13,\
2103,421,19,\
2104,1,28,\
2104,59,17,\
2105,63,26,\
2105,91,26,\
2105,124,27,\
2105,191,27,\
2105,233,26,\
2105,316,28,\
2105,479,26,\
2105,595,33,\
2105,839,26,\
2105,1144,5,\
2105,1163,26,\
2106,5,15,\
2106,22,13,\
2106,46,15,\
2106,137,38,\
2106,207,14,\
2106,361,14,\
2106,389,13,\
2106,513,16,\
2106,541,16,\
2106,787,13,\
2106,810,15,\
2106,837,15,\
2107,92,12,\
2107,525,14,\
2108,703,2,\
2109,22,15,\
2109,142,15,\
2109,251,25,\
2109,534,25,\
2109,805,15,\
2109,908,62,\
2112,36,5,\
2112,88,14,\
2112,125,14,\
2112,274,5,\
2112,501,14,\
2112,558,15,\
2112,600,16,\
2112,844,14,\
2112,941,14,\
2112,1019,9,\
2112,1070,9,\
2112,1257,5,\
2114,0,19,\
2114,41,10,\
2114,239,10,\
2114,251,3,\
2114,291,26,\
2114,613,30,\
2114,725,17,\
2114,800,19,\
2114,844,3,\
2114,898,20,\
2114,1044,3,\
2116,0,27,\
2117,1,30,\
2118,41,20,\
2118,63,3,\
2118,69,30,\
2118,101,3,\
2118,125,10,\
2118,137,3,\
2121,30,21,\
2121,144,35,\
2122,0,20,\
2122,32,20,\
2122,245,16,\
2122,281,20,\
2122,414,17,\
2122,500,20,\
2123,48,12,\
2124,0,21,\
2124,201,21,\
2124,224,3,\
2124,460,3,\
2125,14,27,\
2126,142,3,\
2126,2243,14,\
2127,50,22,\
2127,121,6,\
2127,195,5,\
2127,238,17,\
2127,313,12,\
2127,535,6,\
2127,590,14,\
2127,609,12,\
2127,732,5,\
2127,949,5,\
2128,0,33,\
2128,76,24,\
2128,102,3,\
2128,426,21,\
2128,449,12,\
2128,472,18,\
2128,529,19,\
2128,565,9,\
2128,637,20,\
2128,684,29,\
2128,715,4,\
2128,1294,4,\
2129,0,13,\
2129,18,10,\
2130,23,32,\
2130,104,6,\
2130,333,25,\
2130,391,9,\
2130,701,51,\
2130,823,38,\
2130,868,68,\
2130,947,33,\
2131,0,25,\
2131,73,15,\
2131,225,28,\
2131,255,15,\
2131,625,32,\
2131,817,15,\
2131,1010,15,\
2132,31,24,\
2132,71,34,\
2132,156,24,\
2132,359,34,\
2132,530,17,\
2133,104,41,\
2133,150,17,\
2134,0,71,\
2134,97,8,\
2135,158,22,\
2135,182,11,\
2135,195,14,\
2135,211,21,\
2135,234,16,\
2135,267,8,\
2135,1118,37,\
2135,1246,20,\
2135,1394,16,\
2135,1595,25,\
2136,38,11,\
2136,66,43,\
2136,333,14,\
2136,414,7,\
2136,485,14,\
2137,29,18,\
2137,110,18,\
2137,130,2,\
2137,397,13,\
2137,414,18,\
2137,437,16,\
2137,776,2,\
2138,32,15,\
2138,146,15,\
2138,362,9,\
2138,777,9,\
2138,954,12,\
2138,1005,14,\
2138,1114,17,\
2138,1301,6,\
2138,1373,15,\
2138,1562,12,\
2140,550,17,\
2140,579,16,\
2140,597,3,\
2140,1543,3,\
2141,55,38,\
2141,190,38,\
2141,587,18,\
2142,197,39,\
2143,329,2,\
2143,811,2,\
2144,4,27,\
2144,43,27,\
2144,72,27,\
2144,112,21,\
2144,174,21,\
2144,197,16,\
2144,228,26,\
2144,622,27,\
2144,667,27,\
2144,763,16,\
2144,783,18,\
2144,984,16,\
2145,19,18,\
2146,53,21,\
2146,150,13,\
2146,197,13,\
2146,337,13,\
2147,307,22,\
2147,1082,12,\
2149,22,26,\
2149,113,37,\
2149,262,26,\
2149,689,15,\
2149,775,15,\
2149,805,15,\
2149,952,26,\
2149,991,15,\
2149,1256,15,\
2149,1355,15,\
2151,0,22,\
2152,1,44,\
2152,320,16,\
2152,368,51,\
2153,20,13,\
2154,15,24,\
2154,104,24,\
2154,130,3,\
2154,159,25,\
2154,233,3,\
2154,313,3,\
2154,416,3,\
2154,549,3,\
2154,663,3,\
2154,796,3,\
2154,912,3,\
2154,961,3,\
2155,37,24,\
2155,103,24,\
2155,372,6,\
2155,408,21,\
2157,28,7,\
2158,69,22,\
2158,148,22,\
2158,172,2,\
2158,408,2,\
2158,557,2,\
2158,624,2,\
2159,48,16,\
2159,275,23,\
2159,892,7,\
2159,1483,6,\
2160,12,22,\
2160,36,19,\
2160,60,17,\
2161,4,22,\
2161,31,14,\
2161,65,22,\
2161,237,24,\
2161,263,3,\
2161,320,3,\
2161,298,20,\
2161,652,3,\
2161,796,33,\
2161,923,14,\
2161,997,22,\
2161,1232,14,\
2161,1286,22,\
2161,1341,14,\
2161,1405,22,\
2162,10,49,\
2162,76,9,\
2162,177,22,\
2162,1310,12,\
2162,1757,17,\
2162,1987,12,\
2164,13,47,\
2164,82,58,\
2164,181,13,\
2164,221,20,\
2164,360,24,\
2164,784,12,\
2164,1075,9,\
2164,1176,23,\
2164,2053,12,\
2164,2171,20,\
2164,2335,12,\
2165,31,19,\
2166,277,30,\
2166,482,51,\
2166,602,7,\
2166,704,21,\
2167,10,14,\
2167,221,14,\
2167,296,16,\
2167,380,12,\
2167,426,32,\
2167,494,30,\
2167,593,37,\
2168,56,10,\
2168,183,10,\
2168,245,24,\
2168,286,14,\
2168,332,16,\
2168,629,21,\
2168,708,10,\
2168,833,14,\
2169,26,13,\
2170,16,44,\
2170,147,44,\
2170,273,25,\
2170,364,44,\
2171,80,28,\
2172,97,14,\
2172,128,26,\
2172,173,46,\
2172,458,14,\
2172,740,39,\
2173,0,20,\
2173,60,26,\
2173,229,17,\
2173,337,12,\
2173,372,12,\
2173,495,12,\
2173,671,12,\
2173,911,12,\
2174,238,15,\
2175,428,21,\
2175,1241,18,\
2176,70,28,\
2176,202,28,\
2177,24,18,\
2177,63,18,\
2177,83,2,\
2177,268,17,\
2177,326,2,\
2177,393,2,\
2177,557,2,\
2177,642,2,\
2177,672,32,\
2178,26,17,\
2179,41,33,\
2179,77,33,\
2179,117,28,\
2179,168,17,\
2179,248,18,\
2179,315,8,\
2179,362,19,\
2179,395,16,\
2179,443,12,\
2179,536,12,\
2179,895,28,\
2179,954,16,\
2179,1040,26,\
2179,1087,17,\
2179,1198,20,\
2179,1369,16,\
2179,1423,33,\
2179,1581,12,\
2179,1701,16,\
2180,0,47,\
2180,70,22,\
2180,153,34,\
2180,268,23,\
2180,791,23,\
2180,880,32,\
2180,1099,18,\
2180,1230,23,\
2181,0,44,\
2181,46,22,\
2182,14,31,\
2182,68,14,\
2182,169,14,\
2182,225,32,\
2182,279,12,\
2182,293,8,\
2182,369,14,\
2182,564,14,\
2183,1080,27,\
2185,48,18,\
2185,179,27,\
2186,0,36,\
2186,96,20,\
2187,0,20,\
2187,48,14,\
2187,69,20,\
2187,179,16,\
2187,236,18,\
2188,0,42,\
2188,44,33,\
2188,130,51,\
2188,207,8,\
2188,241,26,\
2188,450,17,\
2189,0,22,\
2189,54,22,\
2189,78,3,\
2189,320,14,\
2189,450,15,\
2189,523,24,\
2189,674,22,\
2189,712,22,\
2189,799,24,\
2189,851,10,\
2189,865,10,\
2189,917,3,\
2190,46,26,\
2190,135,22,\
2190,171,19,\
2190,675,24,\
2190,1070,27,\
2191,76,14,\
2191,278,14,\
2191,879,14,\
2192,67,15,\
2192,265,15,\
2192,282,2,\
2192,987,2,\
2192,1108,27,\
2193,1,18,\
2194,12,10,\
2194,263,20,\
2194,310,21,\
2194,345,17,\
2194,376,20,\
2194,947,21,\
2194,991,13,\
2195,0,43,\
2195,123,8,\
2195,176,50,\
2195,521,18,\
2195,597,8,\
2196,1,22,\
2196,27,22,\
2196,61,36,\
2196,101,18,\
2196,145,23,\
2196,214,19,\
2196,235,56,\
2196,303,28,\
2196,339,17,\
2196,646,16,\
2196,708,19,\
2196,739,59,\
2197,20,33,\
2197,163,25,\
2197,236,14,\
2197,340,33,\
2197,375,4,\
2197,505,15,\
2197,1132,19,\
2197,1542,26,\
2198,0,21,\
2198,26,39,\
2199,69,17,\
2199,164,17,\
2199,183,42,\
2199,231,16,\
2199,860,17,\
2200,0,26,\
2200,83,26,\
2200,203,31,\
2200,323,30,\
2200,606,26,\
2201,129,12,\
2201,552,12,\
2202,712,15,\
2204,143,21,\
2205,1,22,\
2206,1,22,\
2208,41,20,\
2208,63,22,\
2208,435,22,\
2208,459,2,\
2208,711,30,\
2208,1251,27,\
2208,1464,45,\
2208,1581,9,\
2208,1594,22,\
2208,1627,13,\
2208,1836,13,\
2208,1882,9,\
2208,1942,37,\
2208,2029,2,\
2208,2102,2,\
2208,2200,2,\
2209,16,22,\
2209,40,22,\
2209,64,3,\
2209,205,3,\
2209,340,22,\
2210,0,16,\
2210,20,18,\
2210,93,27,\
2210,188,17,\
2210,230,18,\
2210,339,18,\
2210,530,34,\
2210,592,18,\
2210,686,18,\
2210,843,18,\
2210,1022,18,\
2211,32,24,\
2211,590,2,\
2211,807,23,\
2211,852,49,\
2211,963,22,\
2213,27,21,\
2213,118,21,\
2214,0,19,\
2214,42,22,\
2214,231,22,\
2214,255,3,\
2214,300,18,\
2214,320,3,\
2214,575,3,\
2214,740,3,\
2214,1115,3,\
2214,1123,17,\
2214,1185,3,\
2214,1220,3,\
2214,1277,7,\
2214,1441,3,\
2214,1592,3,\
2214,1719,3,\
2214,1748,3,\
2214,1796,3,\
2214,2036,3,\
2215,40,25,\
2217,18,30,\
2217,62,35,\
2217,122,35,\
2217,159,3,\
2217,823,3,\
2218,51,22,\
2218,78,9,\
2218,197,9,\
2218,332,22,\
2218,356,2,\
2218,458,9,\
2218,496,2,\
2218,503,9,\
2218,542,2,\
2218,641,26,\
2218,1306,21,\
2218,1348,9,\
2218,1456,22,\
2218,1570,2,\
2218,1598,22,\
2218,1654,2,\
2218,1709,13,\
2218,1836,13,\
2218,1904,13,\
2218,1922,37,\
2218,1965,2,\
2219,637,2,\
2220,209,28,\
2220,239,3,\
2220,271,25,\
2220,414,21,\
2221,27,14,\
2221,117,14,\
2221,224,6,\
2221,900,5,\
2222,0,21,\
2222,113,21,\
2222,136,3,\
2222,207,25,\
2222,234,31,\
2222,416,3,\
2223,11,20,\
2223,35,26,\
2224,0,20,\
2224,25,20,\
2225,19,22,\
2225,59,45,\
2225,147,22,\
2225,188,45,\
2225,338,19,\
2225,385,19,\
2225,558,6,\
2225,791,55,\
2225,899,15,\
2225,932,45,\
2225,1215,22,\
2226,0,13,\
2226,17,56,\
2226,75,56,\
2226,137,39,\
2226,364,11,\
2226,523,22,\
2227,0,51,\
2227,69,19,\
2227,627,21,\
2227,701,39,\
2227,807,31,\
2227,882,29,\
2228,23,13,\
2228,171,13,\
2228,254,21,\
2228,1175,13,\
2228,1239,13,\
2228,1376,13,\
2232,0,35,\
2232,153,18,\
2232,388,26,\
2232,1129,26,\
2234,0,13,\
2235,18,21,\
2236,51,18,\
2236,73,45,\
2237,32,21,\
2238,42,38,\
2238,97,11,\
2238,113,11,\
2239,0,43,\
2239,60,32,\
2240,0,6,\
2240,19,26,\
2241,0,23,\
2242,0,24,\
2243,1,42,\
2243,45,8,\
2243,58,44,\
2244,15,16,\
2245,11,8,\
2246,1099,20,\
2248,68,27,\
2249,23,28,\
2252,15,28,\
2253,31,24,\
2253,133,24,\
2253,974,24,\
2254,14,13,\
2255,24,17,\
2255,140,17,\
2255,177,14,\
2255,210,14,\
2255,274,13,\
2255,311,26,\
2255,564,14,\
2255,611,17,\
2255,759,14,\
2255,793,17,\
2255,847,14,\
2255,868,8,\
2255,929,17,\
2256,0,43,\
2257,15,14,\
2257,75,14,\
2257,237,14,\
2258,107,20,\
2258,169,20,\
2258,233,75,\
2259,1,14,\
2261,80,14,\
2262,27,14,\
2264,42,35,\
2265,72,16,\
2268,75,14,\
2269,13,56,\
2269,73,7,\
2270,40,38,\
2270,84,42,\
2272,14,12,\
2274,0,66,\
2274,79,51,\
2274,768,14,\
2274,958,17,\
2274,1108,22,\
2274,1178,14,\
2274,1242,15,\
2275,12,18,\
2276,26,44,\
2278,17,22,\
2278,152,22,\
2278,350,22,\
2279,13,6,\
2279,42,14,\
2279,97,14,\
2279,168,14,\
2279,188,6,\
2279,466,15,\
2279,649,5,\
2279,679,17,\
2279,720,16,\
2279,773,15,\
2279,835,6,\
2280,0,30,\
2281,0,22,\
2281,104,10,\
2284,244,28,\
2285,1245,24,\
2285,1358,12,\
2285,1494,12,\
2286,93,43,\
2287,20,17,\
2288,4,28,\
2288,34,29,\
2289,1,16,\
2289,28,44,\
2290,0,23,\
2290,25,35,\
2291,11,16,\
2291,48,65,\
2292,1,12,\
2293,0,31,\
2293,93,10,\
2293,129,19,\
2293,187,23,\
2293,212,14,\
2293,228,14,\
2293,244,20,\
2293,293,16,\
2293,337,11,\
2295,9,16,\
2296,55,18,\
2298,23,57,\
2298,238,64,\
2299,79,22,\
2299,128,22,\
2299,152,2,\
2299,314,19,\
2299,510,2,\
2299,739,16,\
2299,857,2,\
2299,965,2)


In [ ]:
a = BannerAnnotate()

In [ ]:
len(a.annotate())/3

In [ ]:
dfmt1[dfmt1.doc_id == 22]

In [ ]:
len(dfout)

In [ ]:
dfout[['doc_id','annotation_offset','annotation_len']].drop_duplicates()